In [1]:
import random

In [2]:
# Failed v17

import inspect

def print_variables(*args):
    frame = inspect.currentframe()
    frame = inspect.getouterframes(frame)[1]
    string = inspect.getframeinfo(frame[0]).code_context[0].strip()
    # print('inspect.getframeinfo(frame[0]).code_context',inspect.getframeinfo(frame[0]).code_context)
    # print('string:',string)
    args_base = string[string.find('(') + 1:-1].replace('\n','').split(',')
   
    names = []
    for i in args_base:
        if i.find('=') != -1:
            names.append(i.split('=')[1].strip())
       
        else:
            names.append(i)

    variable_dict = {name:arg for name,arg in zip(names,args)}
    
    ind = names[0]
    print(f'\ntata{ind} start')
    for k,v in variable_dict.items():
        if k==ind:
            continue
        print(f'{k}: {v}')

    print(f'tata{ind} end\n')
    
    
def print_variables(*args):
    pass
    
pp = print_variables
p = lambda x: print(f'pointer {x}')


from collections import deque,Counter

class Node:
    def __init__(self, val):
        self.val = val
        self.well_cost = None
        self.pipes = dict()
        self.is_outer = True
        self.info = {
            'connected_min_well_node':{},
            'connected_min_well_cost':{},
            'connected_pipe_cost':{},
            'connected_pipe_set':{}
        }
        
    def check_outer(self):
        if len(self.pipes)>1:
            self.is_outer = False
        else:
            self.is_outer = True
            
    def add_pipe(self, other_node,cost):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        self.pipes[other_node_val] = cost
        self.check_outer()
    
    def remove_pipe(self, other_node):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        del self.pipes[other_node_val]
        self.check_outer()
        
    def __eq__(self, other):
        return self.val==other.val
    
    def __gt__(self,other):
        return self.val>other.val
        
    def __lt__(self, other):
        return self.val<other.val
    
    def __hash__(self):
        return hash(self.val)
    
        

class Solution:
    
    def minCostToSupplyWater_helper(self, n, wells, pipes,testing=False):
        
        BASE_NODE = 77#65#-1#46# 64 # 33 # 25 # 326
        
        
        def packupNode(n, well_cost, pipes):
            # Initial Node
            

            if type(well_cost)==dict:
                node_dict = {i:Node(i) for i in well_cost.keys()}
                
                for k,v in well_cost.items():
                    node_dict[k].well_cost = v
                
            elif type(well_cost)==list:
                node_dict = {i:Node(i) for i in range(1,n+1)}
                for i in range(len(well_cost)):
                    node_dict[i+1].well_cost = well_cost[i]
            else:
                raise

            # Add Edges  
            for edge in pipes:
                node_a = node_dict[edge[0]]
                node_b = node_dict[edge[1]]
                cost = edge[2]
                
                if node_a.val in node_b.pipes:
                    if  node_b.pipes[node_a.val]>cost:
                        node_b.add_pipe(node_a.val,cost)
                        node_a.add_pipe(node_b.val,cost)
                else:
                    node_b.add_pipe(node_a.val,cost)
                    node_a.add_pipe(node_b.val,cost)     
                    
            return node_dict
    

        def getGroupCost(group_set, node_dict):

#             well_status = {}
            well_status = {x.val:0 for  x in group_set}
            pipe_status = {}


            connected_node_set = set()
            connected_pipe_set = set()
            
            # Round1 Edge to Fork (or edge)
            def get_pipe_key(node1,node2):
                if (type(node1)==str) or (type(node1)==int):
                    key = tuple(sorted([node1, node2]))
                else:
                    key = tuple(sorted([node1.val, node2.val]))
                return key
            
            
            
            def update_pipe_status(node1,node2,status):
                key = get_pipe_key(node1,node2)
                pipe_status[key] = status 
                
                
                
            def disconnect(node1,node2):
                # pp(' Disconnect',node1.val,node2.val)
                if (type(node1) in (str,int)):
                    node1 = node_dict[node1]
                if (type(node2) in (str,int)):
                    node2 = node_dict[node2]
                node1.remove_pipe(node2)
                node2.remove_pipe(node1)
                
            def clean_edge(node):
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if (cost> neighbor_node.well_cost) and (cost>node.well_cost):
                        disconnect(node,neighbor_node)
                    

            def check_t1(node):# Center Smallest
                if node.val==BASE_NODE:
                    pp(1.01,well_status)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                # pp(' t1',node.val)
                min_node = node.val
                well_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if cost < well_cost:
                        return False,None,None
                return True,min_node,well_cost
                
                
            def check_t2(node): # All neighbor connected, choose the min pipe
                # pp(' t2',node.val)
                if node.val==BASE_NODE:
                    pp(2.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                min_node = node.val
                min_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if not neighbor in connected_node_set:
                        return False,None,None
                    if cost<min_cost:
                        min_cost = cost
                        min_node = neighbor
                        
                return True, min_node, min_cost
                    
                
#             def check_t3(node): # Short version
#                 min_node = node.val
#                 min_cost = node.well_cost
#                 for neighbor,cost in node.pipes.items():
#                     if not neighbor in connected_node_set:
#                         neighbor_node = node_dict[neighbor]
#                         for n,_ in neighbor_node.pipes.items():
#                             if n==node.val:
#                                 continue
#                             else:
#                                 if not n in connected_node_set:
#                                     return False,None,None
#                         if (cost+neighbor_node.well_cost)<min_cost:
#                             min_cost = cost+neighbor_node.well_cost
#                             min_node = neighbor_node.val
                            
#                     else:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
                
#                 return True,min_node,min_cost
                
                
            def get_connected_group(node, connected_set):
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    group_set.add(cur)
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if (not adj_node in group_set) and (not adj_node in connected_set):
                            stack.append(adj_node)   
                            if adj_node.well_cost<min_cost:
                                min_node = adj_node.val
                                min_cost = adj_node.well_cost
                return min_node,min_cost
                
                
            # Unknown
            def check_t4(node): 
                min_node,min_cost = get_connected_group(node, connected_node_set)
                return True, min_node,min_cost
                
            # Not sure
            def check_t5(node):
                pass
                
            # 
            def check_t6(node):  # is outer or connected 
                # pp(6)
                if node.val==BASE_NODE:
                    pp(6.01)
                    p = pp
                    import pdb
                    bp = pdb.set_trace
                else:
                    def p(*args):
                        pass
                    def bp():
                        pass
                
                
                min_node = node.val
                min_node_cost = node.well_cost
                min_pipe_cost = 0
                min_cost = (min_node_cost+min_pipe_cost)
                
                with_additional_node = True
                with_additional_pipe = False
                for neighbor,cost in node.pipes.items():
                    
                    p(6.010,neighbor)
                    
                    neighbor_node = node_dict[neighbor]

                    if (not neighbor in connected_node_set) and neighbor_node.is_outer:
                        if neighbor_node.well_cost<cost:
                            with_additional_node = True
                            with_additional_pipe = False
                            min_node = neighbor
                            return True,min_node,neighbor_node.well_cost, with_additional_node,with_additional_pipe
                    
                    
                    p(6.010,node.val,connected_node_set,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,neighbor_node.is_outer,min_cost,neighbor_node.pipes.keys(),well_status,pipe_status)
                    
                    min_cost = (min_node_cost+min_pipe_cost)
                    
                    
                    if neighbor in connected_node_set:
                        p(6.0101)
                        if min_node_cost==0: # pre is pipe
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                        else:
                            if cost<min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                    
                    elif neighbor_node.is_outer:
                        p(6.0102)
                        
                        if min_node_cost==0: # pre is pipe
                            if neighbor_node.well_cost <min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                        else:
#                             bp()
                            if neighbor_node.well_cost <min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                            
                    elif cost>min_cost:
                        continue
                    else:
                        p(6.0103)
                        return False, None, None, None, None 
                        
                p(6.011,node.val,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,min_node,well_status,pipe_status)
                
                return True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                
                
            def check_t7(node):  # Disconnected Group Min
                # pp(7)
                if node.val==BASE_NODE:
                    pp(7.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    if cur.val in connected_node_set:
                        return False, None, None
                    group_set.add(cur)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                    
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if adj_node.val in connected_node_set:
                            return False, None, None
                        elif not adj_node in group_set:
                            stack.append(adj_node)
                p(7.01,'Success')
                return True, min_node,min_cost
                
              
            
#             def check_t8(node):  # is outer min_node in surounding connected edges or surounding not connected edges 
#                                  # but with larger cost 
#                 if node.val==BASE_NODE:
#                     pp(8.01)
#                     p = pp
#                 else:
#                     def p(*args):
#                         pass
                
                
#                 min_node = node.val
#                 min_node_cost = node.well_cost
#                 min_pipe_cost = 0
                
#                 min_cost = min_node_cost+min_pipe_cost
                
#                 with_additional_node = False
#                 not_connected_node_list = []
#                 for neighbor,cost in node.pipes.items():
#                     neighbor_node = node_dict[neighbor]
                    
#                     min_cost = min_node_cost+min_pipe_cost
                    
#                     if neighbor in connected_node_set:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
#                             with_additional_node=False
                        
#                     elif neighbor_node.is_outer:
#                         if (neighbor_node.well_cost+cost)<min_cost:
#                             min_cost = neighbor_node.well_cost+cost
#                             min_node = neighbor
#                             with_additional_node=True
                            
#                     else: # not connected not outer
#                         not_connected_node_list.append((neighbor, cost))
                        
#                 for neighbor,cost in not_connected_node_list:
#                     if cost<min_cost:
#                         return False, None, None, None
 
#                 p(8.01,'Success')
#                 return True, min_node,min_cost,with_additional_node


                    
            def check_t9(node):    # Min neighbor node
                if node.val==BASE_NODE:
                    pp(9.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                min_node = node.val
                min_well_cost = node.well_cost
                min_pipe_cost = 0
                
                with_additional_node = False
                for neighbor,cost in node.pipes.items():
                    p(9.011,neighbor,cost,min_node,min_well_cost,min_pipe_cost,with_additional_node)
                    neighbor_node = node_dict[neighbor]
                    
                    
                    if neighbor in connected_node_set:
                        if min_well_cost==0: 
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False

                        else:
                            if cost<min_well_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False
                        # pass
                    
                    else:
                        for nn,cc in neighbor_node.pipes.items():
                            if nn==node.val:
                                continue 
                                
                            nn_node = node_dict[nn]
                            if cc<neighbor_node.well_cost:
                                p(9.011,node.val,neighbor,nn,cc,neighbor_node.well_cost)
                                return False, None, None, None
                            elif (neighbor_node.well_cost>min_well_cost) and (cost<min_pipe_cost):
                                return False, None, None, None
                            

#                         bp()
                        if min_well_cost==0: 
                            
                            if (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                            
                            elif neighbor_node.well_cost < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        else:
                            
                            if cost > (min_pipe_cost+min_well_cost):
                                continue
                            
                            elif (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                                
                            elif neighbor_node.well_cost < min_well_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        # pass
                    
                p(9.012,node.val,min_node,min_well_cost,min_pipe_cost,with_additional_node)
                p(9.01,'Success')
                return True,min_node,min_well_cost+min_pipe_cost,with_additional_node
            
            
            
            def check_t10(node):  # Min Node or Already Connected Pipe in a Group
                # pp(10)
                if node.val==BASE_NODE:
                    pp(10.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                stack = deque()
                checked_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                pre_node = None
                
                with_additional_node=True
                while(stack):
                    cur = stack.pop()
                    checked_set.add(cur.val)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                        with_additional_node=True
                        pre_node = None
                    
                    for adj, cc in cur.pipes.items():
                        adj_node = node_dict[adj]
                        if adj in connected_node_set:
                            checked_set.add(adj)
                            if cc<min_cost:
                                min_node = adj
                                min_cost = cc
                                with_additional_node = False
                                pre_node = cur.val
    
                        elif not adj in checked_set:
                            stack.append(adj_node)   
                
                
                p(10.011,node.val,min_node,min_cost,with_additional_node,pre_node)
                p(10.01,'Success')
                return True,min_node,min_cost,with_additional_node,pre_node
                
            
            def clean_edge_II(node): # Pruning
                min_node = node.val
                min_cost = node.well_cost
                changed_ind=False
                
                with_additional_node=False
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost<min_cost:
                            min_cost = cost
                            min_node = neighbor
                            with_additional_node = False
                    # elif neighbor_node.is_outer:
                    elif (neighbor_node.well_cost+cost)<min_cost:
                            min_cost = neighbor_node.well_cost+cost
                            min_node = neighbor
                            with_additional_node = True
                            
                    else:
                        pass
                    
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost>min_cost:
                            disconnect(node,neighbor)
                            changed_ind=True
            
                return changed_ind
            
            
            
            for node in group_set:
                clean_edge(node)
                
            # print()
            # for x in group_set:
            #     print('node:',x.val)
            #     # print('info:',x.info)
            #     print('pipes:',x.pipes.keys())
            #     print('is_outer:',x.is_outer)
            #     print()
                
            stack = deque(group_set)
            
            
            import copy
            pre = copy.copy(stack)
            pre_pipe_status = copy.copy(pipe_status)
            pre_well_status = copy.copy(well_status)
            while(stack):
                for i in range(len(stack)):
                    if not stack:
                        break
#                     print('stack:',[x.val for x in stack])
                        
                    cur = stack.popleft()
                    
                    pp('#####################################################',cur.val,well_status)
                    
                    # if cur.val in connected_node_set:
                    #     continue
                    
                    ind, min_node, min_cost = check_t1(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        well_status[cur.val] = 1
                        continue

                    ind, min_node, min_cost = check_t2(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                        continue  

                    # ind, min_node, min_cost = check_t3(cur)
                    # if ind:
                    #     connected_node_set.add(min_node)
                    #     connected_node_set.add(cur.val)
                    #     if cur.val!=min_node:
                    #         key = get_pipe_key(min_node,cur.val)
                    #         connected_pipe_set.add(key)
                    #     continue

                    ind, min_node, min_cost,with_additional_node,with_additional_pipe = check_t6(cur)
                    if ind:
                        pp(6)
                        if with_additional_node and (not with_additional_pipe):
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if min_node!=cur.val:
                                stack.append(cur)
                            
                        elif with_additional_node and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        
                        elif (not with_additional_node) and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        else: # (not with_additional_node) and (not with_additional_pipe):
                            pass
                        

                        continue
                              
                    """
                    True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                    
                    """        
                    
                            
                            
                            
                    ind, min_node, min_cost = check_t7(cur)
                    if ind:
                        pp(7,min_node,well_status)
                        connected_node_set.add(min_node)
                        well_status[min_node] = 1
                        
                        if min_node!=cur.val:
                            stack.append(cur)
                        # pp('t7',min_node)
                        continue 
                        

                        
#                     ind, min_node, min_cost,with_additional_node = check_t8(cur)
#                     if ind:
#                         pp(8)
#                         connected_node_set.add(cur.val)
#                         connected_node_set.add(min_node)
#                         if cur.val!=min_node:
#                             pipe_key = get_pipe_key(min_node,cur.val)
#                             connected_pipe_set.add(pipe_key)
#                             pipe_status[pipe_key] = 1
                    
#                         if cur.val==min_node:
#                             well_status[cur.val] = 1
#                         elif with_additional_node:
#                             well_status[min_node] = 1

#                         continue
                          
                            
                    ind, min_node, min_cost,with_additional_node = check_t9(cur) # Inner Net
                    if ind:
                        pp(9)
                        connected_node_set.add(cur.val)
                        connected_node_set.add(min_node)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                    
                        if cur.val==min_node:
                            well_status[cur.val] = 1
                        elif with_additional_node:
                            well_status[min_node] = 1

                        pp(9.1,well_status,pipe_status)
                        continue


                    ind, min_node, min_cost,with_additional_node,pre_node = check_t10(cur)
                    if ind:
                        pp(10,min_node,well_status)
                            
                        if with_additional_node:
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            
                            if min_node!=cur.val:
                                stack.append(cur)
                        else:
                            # if (not min_node) or (not(pre_node))
                            pipe_key = get_pipe_key(min_node,pre_node)
                            connected_pipe_set.add(pipe_key)
                            connected_node_set.add(pre_node)
                            pipe_status[pipe_key] = 1

                        
                            if pre_node!=cur.val:
                                stack.append(cur)

                        # pp('t7',min_node)
                        continue 


                            
                    stack.append(cur)
                    

                    
                # pp(9,[x.val for x in stack],[x.val for x in pre])
                    
                if (stack==pre) and (pre_pipe_status==pipe_status) and (pre_well_status==well_status):
                    
                    changed=False
                    for node in stack:
                        ind = clean_edge_II(node)
                        if ind:
                            changed=True
                    
                    if not changed:
                        break
                else:
                    pre = copy.copy(stack)
                    pre_pipe_status = copy.copy(pipe_status)
                    pre_well_status = copy.copy(well_status)

                    
            print('##########################################################################')
                    
            print("stack:",[x.val for x in stack])
#             print("connected_node_set:", connected_node_set)
#             print("connected_pipe_set:", connected_pipe_set)
                    
                       
            pipe_status = {k:v for k,v in pipe_status.items() if v}
            well_status = {k:v for k,v in well_status.items() if v}
            

#             print()
#             for x in node_dict.values():
#                 print('node:',x.val)
#                 # print('info:',x.info)
#                 print('pipes:',[x for x in x.pipes.keys()])
#                 # print('is_outer:',x.is_outer)
#                 print()


            print("well_status=",sorted([x for x in well_status.keys()]))
            print("pipe_status=",sorted([x for x in pipe_status.keys()]))
            
            
            
            pipe_cost = sum([pipe_cost_dict[k] for k,v in pipe_status.items() if v])
            node_cost = sum([well_cost_dict[k] for k,v in well_status.items() if v])

            print('node_cost:',node_cost)
            print('pipe_cost:',pipe_cost)
            return pipe_cost, node_cost,well_status,pipe_status


        
        
        
        pipe_cost_dict = {}
        for x in pipes:
            key = tuple(sorted((x[0],x[1])))
            val = x[2]
            
            if key in pipe_cost_dict:
                if pipe_cost_dict[key]>val:
                    pipe_cost_dict[key]=val
            else:
                pipe_cost_dict[key]=val

        if type(wells)==list:
            well_cost_dict = {(i+1):x for i,x in enumerate(wells)}
        elif type(wells)==dict:
            well_cost_dict = {k:v for k,v in wells.items()}
        # Base
        checked_set = set()
        node_dict = packupNode(n, wells, pipes)
        node_list = list(node_dict.values())

        pipe_cost, node_cost,well_status,pipe_status = getGroupCost(node_list, node_dict)
        group_cost = pipe_cost + node_cost
            
            
        if testing: 
            return group_cost,sorted([x for x in well_status.keys()]),sorted([x for x in pipe_status.keys()])
        else:
            return group_cost


    
    def minCostToSupplyWater(self, n, wells, pipes):
    



        def packupNode(n, well_cost, pipes):
            # Initial Node
            node_dict = {i:Node(i) for i in range(1,n+1)}

            for i in range(len(well_cost)):
                node_dict[i+1].well_cost = well_cost[i]

            # Add Edges  
            for edge in pipes:
                node_a = node_dict[edge[0]]
                node_b = node_dict[edge[1]]
                cost = edge[2]

                if node_a.val in node_b.pipes:
                    if  node_b.pipes[node_a.val]>cost:
                        node_b.add_pipe(node_a.val,cost)
                        node_a.add_pipe(node_b.val,cost)
                else:
                    node_b.add_pipe(node_a.val,cost)
                    node_a.add_pipe(node_b.val,cost)     

            return node_dict


        def disconnect(node1,node2):
            # pp(' Disconnect',node1.val,node2.val)
            if (type(node1) in (str,int)):
                node1 = node_dict[node1]
            if (type(node2) in (str,int)):
                node2 = node_dict[node2]
            node1.remove_pipe(node2)
            node2.remove_pipe(node1)

        def get_connected_group(node,checked_node_set):
            stack = deque()
            group_set = set()
            stack.append(node)

            while(stack):
                cur = stack.popleft()
                group_set.add(cur)
                checked_node_set.add(cur)
                for pipe, cost in list(cur.pipes.items()):
                    adj_node = node_dict[pipe]
                    if (cost>adj_node.well_cost and cost>cur.well_cost):
                        disconnect(adj_node,cur)
                    else:
                        if (not adj_node in group_set):
                            stack.append(adj_node)   
            return group_set


        def get_groups(node_list):
            group_list = []
            checked_node_set = set()
            for node in node_list:
                if node in checked_node_set:
                    continue
                else:
                    group = get_connected_group(node,checked_node_set)
                    group_list.append(group)
            return group_list


        # Round1 Edge to Fork (or edge)
        def get_pipe_key(node1,node2):
            if (type(node1)==str) or (type(node1)==int):
                key = tuple(sorted([node1, node2]))
            else:
                key = tuple(sorted([node1.val, node2.val]))
            return key

        def nodeGroup2dict(group,output_format='dict'):
            well_dict = {}
            pipe_dict = {}
            node_dict = {node.val:node for node in group}
            for node in group:
                well_dict[node.val]=node.well_cost
                for adj,cost in node.pipes.items():
                    key = get_pipe_key(node.val,adj)
                    pipe_dict[key]=cost

            if output_format=='dict':
                return node_dict, pipe_dict,None
            elif output_format=='list':
                wells_key_mapping = {x:i+1 for i,x in enumerate(well_dict.keys())}
                wells_key_mapping_reverse = {i:x for x,i in wells_key_mapping.items()}
                wells_list = [well_dict[wells_key_mapping_reverse[i+1]] for i in range(len(wells_key_mapping))]

                pipes = [[k[0],k[1],v] for k,v in pipe_dict.items()]
                pipe_list = [[wells_key_mapping[i],wells_key_mapping[j],x] for i,j,x in pipes]

                return wells_list,pipe_list,wells_key_mapping_reverse

            else:
                return None,None,None

            
        # Main
        node_dict = packupNode(n, wells, pipes)
#         node_list = sorted(node_dict.values())

        node_list = list(node_dict.values())
        # import random
        random.shuffle(node_list)

        group_list = get_groups(node_list)

#         return group_list
#         group_list = [x for x in group_list if len(x)>100]
    
        tmp_group = group_list
        new_group = []
        for x in tmp_group:
            new_group+=x            
            
            
        base = new_group

        wells_list,pipe_list,wells_mapping = nodeGroup2dict(base,output_format='list')
        n = len(wells_list)

        print('n=',n)
        print('wells_list=',wells_list)
        print('pipe_list=',pipe_list)
    
        res,well_status,pipe_status = self.minCostToSupplyWater_helper(n, wells_list, pipe_list,True)
    
        print('wells_mapping=',wells_mapping)
        return res
    
    

In [3]:


def packupNode(n, well_cost, pipes):
    # Initial Node
    node_dict = {i:Node(i) for i in range(1,n+1)}

    for i in range(len(well_cost)):
        node_dict[i+1].well_cost = well_cost[i]

    # Add Edges  
    for edge in pipes:
        node_a = node_dict[edge[0]]
        node_b = node_dict[edge[1]]
        cost = edge[2]

        if node_a.val in node_b.pipes:
            if  node_b.pipes[node_a.val]>cost:
                node_b.add_pipe(node_a.val,cost)
                node_a.add_pipe(node_b.val,cost)
        else:
            node_b.add_pipe(node_a.val,cost)
            node_a.add_pipe(node_b.val,cost)     

    return node_dict


def disconnect(node1,node2):
    # pp(' Disconnect',node1.val,node2.val)
    if (type(node1) in (str,int)):
        node1 = node_dict[node1]
    if (type(node2) in (str,int)):
        node2 = node_dict[node2]
    node1.remove_pipe(node2)
    node2.remove_pipe(node1)

def get_connected_group(node,checked_node_set):
    stack = deque()
    group_set = set()
    stack.append(node)

    while(stack):
        cur = stack.popleft()
        group_set.add(cur)
        checked_node_set.add(cur)
        for pipe, cost in list(cur.pipes.items()):
            adj_node = node_dict[pipe]
            if (cost>adj_node.well_cost and cost>cur.well_cost):
                disconnect(adj_node,cur)
            else:
                if (not adj_node in group_set):
                    stack.append(adj_node)   
    return group_set


def get_groups(node_list):
    group_list = []
    checked_node_set = set()
    for node in node_list:
        if node in checked_node_set:
            continue
        else:
            group = get_connected_group(node,checked_node_set)
            group_list.append(group)
    return group_list




# n = 2472
# wells = [40712, 22902, 11267, 98706, 44691, 30785, 89097, 2807, 74834, 44882, 38200, 65030, 22725, 17906, 91949, 98031, 30852, 85656, 53943, 1398, 77320, 81493, 58511, 88049, 11941, 18102, 89282, 67220, 81739, 89253, 34886, 88922, 26551, 11679, 53608, 51266, 57722, 62836, 33376, 99341, 69716, 44508, 47020, 71926, 4970, 92543, 29286, 70234, 77130, 21499, 42814, 9960, 18351, 28979, 40416, 91159, 78138, 59823, 25819, 65029, 35944, 17872, 79905, 71808, 26375, 17276, 25131, 10308, 38148, 99185, 21280, 47968, 95759, 91033, 73978, 84684, 38677, 15292, 81099, 78241, 43650, 72865, 52195, 27022, 11291, 47521, 61757, 50266, 20198, 34755, 8410, 89006, 75102, 86680, 82116, 66532, 76855, 13902, 41026, 23841, 9406, 277, 64458, 10399, 84146, 33661, 83161, 25991, 69578, 15438, 55012, 14395, 95540, 50662, 63248, 45513, 40601, 31882, 99787, 34931, 92410, 49481, 89566, 95936, 29562, 83667, 76998, 79124, 40570, 76340, 70337, 31812, 94026, 39768, 66346, 41237, 12493, 94859, 55147, 75458, 87254, 94647, 14240, 22771, 22430, 84601, 50340, 47131, 61434, 19941, 47215, 84766, 40706, 88971, 47216, 72856, 42401, 50583, 59464, 11235, 14075, 68583, 77660, 31150, 25984, 19666, 43773, 6201, 64481, 52530, 7894, 54636, 81783, 81166, 19161, 47312, 41076, 67013, 28505, 62157, 71734, 25615, 38168, 66862, 4594, 60831, 18034, 43613, 6001, 68877, 46319, 77015, 1750, 9818, 9785, 39213, 65659, 43927, 82036, 83036, 55433, 72458, 17058, 15378, 61206, 35876, 63846, 10002, 94911, 16566, 30490, 4748, 34494, 27446, 29107, 56556, 88561, 55869, 94718, 98330, 4797, 53320, 17673, 81259, 26108, 68218, 15092, 24730, 62171, 51813, 90780, 84949, 87346, 11641, 56197, 36489, 65478, 95540, 76282, 64025, 74797, 32553, 69489, 8659, 91457, 69425, 90254, 98782, 20608, 14856, 72358, 41450, 64339, 55695, 30762, 3030, 9553, 93521, 61207, 75424, 97238, 95582, 81424, 34410, 87433, 3721, 91728, 21955, 41679, 9944, 73049, 97187, 6882, 92234, 27172, 69358, 8729, 82249, 44667, 67526, 73009, 36219, 34359, 87054, 92724, 9910, 51020, 25081, 89519, 57695, 21122, 19429, 43854, 8018, 46685, 14557, 37020, 94748, 1177, 92358, 54737, 51417, 50616, 84352, 52697, 24035, 52505, 99671, 80639, 92986, 73818, 39093, 59256, 72362, 42083, 9524, 65684, 58, 12962, 78500, 98921, 77535, 60926, 17090, 95071, 5195, 38826, 64260, 29977, 39146, 66052, 27084, 31630, 64562, 94339, 8309, 82313, 7069, 98921, 72160, 78632, 8377, 45221, 25866, 20515, 45314, 64962, 3121, 48983, 76657, 2166, 94788, 20396, 97398, 52970, 67904, 49047, 47257, 62933, 98093, 73948, 97709, 57516, 85603, 99462, 18497, 50701, 70696, 79519, 92357, 56986, 24836, 87552, 63148, 15538, 8925, 97919, 10155, 99664, 52684, 20792, 13210, 82734, 63307, 793, 89247, 18524, 37339, 14194, 88599, 98492, 65479, 82518, 32636, 33523, 99719, 53624, 96550, 84152, 65897, 96070, 93971, 42961, 63391, 75927, 22015, 90071, 94324, 97038, 93437, 3667, 78000, 52344, 35605, 85302, 36006, 96446, 84270, 83502, 99343, 80421, 28173, 35611, 35849, 52575, 80954, 39100, 67543, 3805, 37641, 98317, 36326, 4152, 78724, 24681, 39430, 46754, 99970, 76166, 78790, 30927, 64210, 46159, 1426, 93204, 41627, 3539, 29383, 40563, 60377, 82316, 98449, 1383, 24205, 31139, 92270, 84217, 67308, 24119, 75540, 86309, 26315, 54, 61990, 45709, 28084, 10297, 49367, 39804, 4646, 14159, 24397, 90111, 29023, 53494, 71221, 67636, 55676, 39825, 83066, 82011, 36722, 87936, 23003, 44492, 78173, 81928, 83113, 53638, 30388, 8438, 66445, 92082, 74201, 17631, 28501, 37950, 1118, 9973, 11722, 57453, 27952, 87657, 30387, 69458, 44642, 52334, 58705, 27279, 87634, 63675, 37504, 69584, 35679, 69084, 32431, 32362, 71064, 50096, 1347, 24999, 99799, 9314, 38935, 79758, 2035, 61399, 74602, 84902, 14838, 83917, 50023, 8962, 92789, 16163, 12504, 70052, 6379, 50228, 30072, 42981, 65339, 51047, 63378, 19141, 90039, 90728, 69677, 33765, 81421, 96435, 3440, 978, 55927, 56387, 78297, 31749, 34688, 96989, 65029, 61789, 50140, 37905, 38235, 90161, 58607, 79889, 7751, 54661, 33350, 37020, 44260, 58086, 14849, 43710, 46184, 52378, 32704, 66760, 1787, 91898, 27094, 93954, 38755, 81623, 62081, 31545, 62614, 82269, 79952, 40644, 61834, 19287, 57518, 24177, 61926, 75732, 98821, 37259, 33631, 32829, 28044, 28595, 54452, 14811, 51572, 37339, 35096, 74973, 38967, 53729, 94174, 83119, 73311, 856, 69276, 22487, 33301, 85187, 43699, 98773, 59384, 33159, 8159, 30661, 55036, 27791, 87780, 20470, 95229, 34503, 84115, 86690, 17591, 88481, 22643, 77779, 24961, 88724, 1532, 62902, 63291, 35701, 84807, 59296, 27431, 26148, 11969, 61941, 26227, 14462, 91066, 68259, 28572, 47884, 59505, 87971, 36815, 54225, 45356, 50450, 19938, 91481, 81159, 34636, 90696, 85565, 85268, 65125, 4620, 93577, 86753, 7315, 56141, 59549, 74785, 91024, 90781, 15739, 76943, 15950, 2596, 13574, 85610, 98729, 40850, 33746, 59131, 31813, 1478, 7372, 21209, 33646, 54178, 88853, 63859, 16076, 23587, 22946, 58952, 4592, 11521, 20140, 11136, 10240, 87551, 22960, 60684, 54014, 31816, 23953, 78000, 52123, 87986, 67927, 77553, 91248, 50755, 34507, 13442, 13618, 50999, 85198, 65892, 75722, 9908, 34306, 39427, 67481, 8376, 88886, 17027, 69549, 5704, 80787, 94114, 14611, 8692, 13998, 57844, 71866, 46412, 55411, 73825, 16550, 47923, 42192, 25393, 93465, 94352, 35829, 5753, 20259, 89442, 92604, 52408, 27019, 69270, 95524, 88566, 77153, 89877, 37255, 94204, 96751, 24805, 66885, 36114, 60016, 29500, 77226, 49557, 27349, 14864, 7184, 47424, 97663, 19982, 91529, 14965, 69868, 52117, 29235, 46688, 52010, 53707, 46680, 97762, 76670, 95181, 49353, 90483, 31229, 97916, 86153, 48189, 23737, 48324, 71562, 21931, 99922, 86704, 83808, 38909, 88789, 20333, 71809, 38625, 91458, 25599, 14797, 72233, 430, 62557, 774, 81473, 45391, 554, 92055, 14714, 27904, 70729, 2129, 89012, 40285, 33575, 44967, 46905, 69654, 9988, 74765, 69902, 43740, 68788, 31959, 30139, 46554, 10298, 44132, 75580, 87784, 52954, 11622, 54386, 18273, 24828, 60298, 51533, 43476, 7332, 90492, 83198, 40778, 31195, 55851, 4641, 31850, 97848, 38826, 44604, 50244, 48737, 14454, 15430, 56542, 32362, 20660, 5272, 4262, 9553, 90919, 32389, 65877, 87667, 46518, 41753, 15451, 29209, 48611, 4825, 47633, 4005, 29540, 42968, 68661, 50153, 20981, 69822, 98579, 43859, 21454, 52483, 51971, 74183, 49424, 59536, 48421, 77314, 77723, 55879, 92658, 52241, 75458, 41438, 1682, 12363, 66222, 71925, 5910, 13669, 87090, 11056, 17887, 1658, 70196, 50866, 74383, 60402, 82654, 85048, 9901, 33170, 66947, 57160, 91487, 67010, 94244, 78857, 44048, 73565, 21539, 40285, 24519, 33131, 61060, 84186, 24208, 78222, 86944, 95820, 39761, 68019, 88436, 2648, 70823, 45074, 70072, 69463, 91147, 74265, 122, 30498, 31743, 95231, 19103, 81906, 98921, 83777, 66478, 26702, 81944, 12944, 58802, 36798, 71281, 40212, 24387, 76128, 87314, 3679, 42400, 72259, 45613, 35436, 80008, 37106, 529, 87612, 62451, 56911, 18276, 98352, 82128, 66407, 61428, 90216, 61927, 9629, 65609, 43420, 87533, 33281, 94922, 2130, 97250, 68144, 59749, 36830, 95527, 88862, 46343, 76967, 84425, 55092, 53621, 50871, 99525, 45190, 15291, 42738, 70605, 84152, 40746, 89850, 93935, 921, 16621, 34148, 87178, 13938, 5398, 21312, 74387, 23142, 72638, 66761, 83340, 61227, 63868, 63887, 80281, 22089, 17201, 57833, 63819, 35626, 18925, 72992, 11375, 32311, 87302, 71411, 18057, 70844, 1885, 18594, 42434, 30610, 52292, 7753, 20551, 15594, 81253, 32760, 7279, 36600, 9732, 86037, 21724, 60272, 78818, 21543, 94848, 76602, 56808, 7252, 45058, 5949, 53760, 20090, 71683, 87047, 90690, 37126, 21760, 17198, 140, 51568, 14935, 37277, 18477, 84684, 50209, 38500, 29587, 48009, 85144, 13611, 70229, 67561, 77011, 83182, 83699, 68421, 57229, 58897, 97716, 79676, 88468, 73538, 72434, 56735, 27652, 87346, 44389, 47134, 61167, 88011, 93215, 215, 4713, 95421, 53499, 77081, 78506, 94974, 60334, 37356, 75527, 82670, 38617, 86181, 55787, 49424, 99730, 25092, 44092, 98290, 11583, 49499, 13525, 22146, 5449, 41696, 88900, 51856, 62152, 30789, 54031, 50777, 28841, 92693, 64092, 60279, 20657, 44645, 81428, 62298, 56050, 37415, 35692, 80967, 54956, 78232, 69627, 26235, 32838, 25953, 99226, 22827, 73993, 64666, 98763, 80755, 2629, 6658, 87030, 36088, 62940, 91566, 76624, 31437, 35470, 65819, 60734, 63386, 66919, 98327, 81989, 93350, 51011, 45446, 42079, 96684, 23651, 11598, 24666, 16608, 50028, 51941, 91889, 55723, 40880, 66182, 98242, 73631, 59182, 30742, 49379, 69237, 1077, 75573, 5435, 15350, 6553, 88730, 55425, 80424, 26879, 18786, 79195, 30710, 31902, 53447, 63534, 46751, 76838, 99446, 47889, 98216, 50241, 42368, 78240, 50520, 65414, 20384, 75962, 33421, 83767, 15602, 8722, 46000, 687, 63768, 48971, 81328, 23130, 14124, 68346, 77455, 15604, 7077, 86739, 94755, 15025, 18133, 85475, 69255, 49087, 31670, 96087, 12508, 55169, 58037, 97734, 20409, 31215, 24721, 8917, 68532, 99042, 99862, 97478, 9226, 44608, 87163, 82161, 55482, 92752, 44214, 69627, 53772, 36116, 11373, 57425, 31803, 52652, 72430, 83571, 92202, 25451, 23, 38708, 41658, 25678, 76876, 28944, 94860, 76740, 68953, 52470, 59837, 2527, 50705, 26949, 57401, 62238, 24732, 22891, 10804, 22407, 64614, 51416, 13422, 14228, 39724, 30084, 98739, 91103, 20326, 16658, 34322, 80138, 84553, 12836, 35536, 78277, 25252, 22687, 59164, 1727, 85495, 6751, 59169, 92302, 94645, 32160, 95691, 18063, 75726, 80975, 84790, 90064, 38436, 92475, 73473, 74702, 69822, 8899, 98447, 70940, 57847, 59190, 92867, 51211, 5556, 65040, 15898, 5846, 39531, 74169, 5148, 7236, 72514, 99021, 44461, 62058, 65214, 75191, 91044, 20583, 54000, 78327, 66248, 58469, 71837, 4555, 60112, 22749, 50703, 15324, 44766, 56252, 80137, 58428, 9631, 57736, 1193, 13937, 93148, 45593, 58416, 91423, 69858, 66485, 64649, 29503, 4641, 17591, 88302, 88067, 16807, 70778, 15108, 72962, 17678, 4202, 34248, 22933, 288, 52260, 74490, 2565, 68523, 90093, 86616, 38298, 53443, 6615, 4562, 34793, 90425, 82910, 76968, 42735, 25006, 3263, 3623, 9190, 65868, 87268, 33897, 77893, 67461, 70152, 95719, 72771, 90663, 75753, 38083, 8540, 61650, 26666, 91612, 60781, 79414, 50402, 85665, 35399, 95930, 22741, 54391, 91829, 90009, 62969, 77304, 94381, 21799, 90440, 80003, 23874, 48586, 18308, 86685, 63988, 16946, 66010, 40739, 73936, 92452, 10929, 79635, 12792, 1807, 44614, 81274, 19972, 78015, 36336, 41726, 21093, 63597, 45101, 14534, 1574, 63704, 73330, 46150, 3170, 6526, 40782, 29723, 62880, 85538, 56867, 84701, 90467, 33448, 9895, 23512, 88706, 39320, 89394, 58607, 59362, 35386, 39305, 78528, 31713, 43147, 13270, 71241, 1648, 85629, 98801, 79980, 99079, 73186, 38503, 15725, 81195, 53649, 65863, 26006, 49342, 43418, 19105, 14401, 42643, 96315, 80262, 26684, 84442, 60240, 82835, 34614, 62783, 53399, 80425, 63896, 32583, 69325, 75920, 13078, 24545, 42650, 54215, 29118, 27159, 87139, 90297, 65547, 41812, 33395, 61922, 38213, 92481, 6357, 90063, 94253, 52616, 43500, 28277, 79362, 96151, 40680, 37659, 76487, 39806, 31414, 50793, 85396, 10036, 65131, 90267, 83146, 12725, 14443, 95961, 78661, 70908, 66123, 76319, 59056, 19274, 22856, 19381, 57222, 2817, 40276, 48755, 39552, 88423, 37858, 59497, 14057, 15928, 42876, 58897, 72259, 19876, 10692, 6332, 15866, 55420, 5576, 6616, 2381, 95050, 64022, 89923, 86074, 34049, 30648, 84734, 68372, 27765, 97336, 17750, 99561, 15802, 77872, 36675, 50059, 80115, 12108, 36492, 48263, 92333, 78876, 26093, 50987, 8516, 39464, 23696, 88035, 35630, 33460, 28200, 12238, 55671, 90260, 90383, 55202, 97190, 28523, 50330, 56396, 58968, 15412, 22128, 21031, 33821, 79102, 42485, 87726, 56796, 50990, 76496, 52628, 34162, 75631, 70998, 87913, 38304, 35820, 2344, 62213, 68674, 70444, 78422, 92693, 945, 92034, 53193, 10157, 59397, 40456, 98721, 87639, 86826, 3077, 23965, 22750, 58299, 96721, 9188, 45680, 57558, 15201, 34448, 40894, 45456, 68761, 97737, 72131, 38197, 96895, 79019, 87247, 46525, 70111, 99604, 57332, 69473, 82783, 44742, 15685, 19003, 67272, 65309, 16442, 46233, 12507, 48544, 54357, 39895, 53987, 93873, 34658, 81902, 79506, 18364, 79885, 50861, 3856, 40309, 5245, 70315, 90387, 33107, 89444, 89772, 99643, 31030, 28710, 99409, 40562, 87459, 62866, 6811, 4253, 74321, 95938, 56314, 10863, 54205, 60381, 97943, 75193, 8045, 62846, 18073, 80765, 19167, 9201, 76643, 75620, 98522, 2134, 15733, 20970, 52441, 79605, 30010, 21646, 64461, 45526, 86268, 95181, 91699, 2481, 79532, 83196, 16273, 16500, 87286, 12402, 74467, 56288, 99259, 52970, 36175, 36348, 56184, 45498, 37304, 18758, 21394, 65022, 59872, 27488, 23097, 41991, 68217, 52300, 81514, 90096, 66011, 76208, 42706, 26968, 57962, 47844, 93834, 64749, 35261, 43140, 2369, 98339, 13432, 64939, 89034, 80337, 45818, 16143, 50399, 69796, 99013, 34972, 13053, 46756, 58955, 44928, 65582, 21872, 47486, 20177, 23945, 82102, 60222, 15986, 33022, 30067, 38516, 28810, 70666, 39087, 43468, 1315, 91277, 37091, 53608, 89688, 49344, 3277, 6151, 6924, 64854, 51834, 4024, 94425, 99790, 26054, 41342, 21459, 37240, 5697, 39516, 820, 18716, 33199, 65571, 29438, 52004, 49557, 49903, 73718, 70650, 6089, 13090, 9842, 39495, 53489, 1422, 43963, 82471, 66087, 33059, 40637, 38188, 30762, 67693, 26635, 24029, 4310, 55902, 78630, 99070, 13711, 49745, 15189, 64121, 42001, 14764, 76064, 47271, 38983, 60199, 75328, 24699, 72747, 59823, 85938, 46409, 35409, 78426, 57680, 96231, 51161, 48437, 94048, 8730, 69743, 96235, 79483, 13517, 84361, 66673, 65548, 75197, 25325, 47088, 79309, 37641, 21241, 94887, 40025, 70506, 11977, 79021, 76536, 29146, 23765, 96417, 24238, 57108, 94972, 79752, 32493, 3696, 73768, 64115, 93996, 22647, 88189, 77611, 81269, 79403, 12206, 5864, 31319, 2788, 85671, 11092, 6275, 77655, 91767, 61774, 67572, 7373, 91174, 65833, 93411, 17100, 36027, 278, 50241, 61366, 20502, 63746, 86711, 37840, 42480, 49837, 39348, 74682, 42682, 69582, 11165, 90124, 89595, 89085, 34859, 39882, 69754, 54635, 60009, 34452, 97901, 65104, 95021, 17384, 34016, 93094, 45094, 40857, 48132, 95910, 98905, 41865, 38323, 98780, 1276, 37166, 58531, 72750, 31258, 98099, 19807, 90624, 85466, 55655, 49063, 88600, 826, 85963, 77967, 86067, 43234, 67603, 88858, 63372, 98162, 96141, 12848, 48195, 90461, 78580, 45065, 72920, 6596, 67388, 56109, 71460, 46266, 958, 64485, 32320, 91132, 84913, 93082, 27775, 95604, 46317, 58154, 62604, 51633, 25745, 1163, 89843, 77215, 16603, 98929, 77738, 31936, 9359, 52940, 57997, 62495, 97406, 12414, 98598, 82921, 39363, 1264, 603, 57163, 12925, 62529, 40314, 56186, 44394, 66735, 23777, 65341, 702, 80380, 38978, 29204, 77265, 71468, 54602, 18491, 99314, 36569, 98117, 87421, 95972, 943, 36086, 63435, 84833, 61390, 86247, 4581, 56697, 13371, 22197, 61664, 66901, 61311, 61686, 87536, 78146, 75892, 46526, 16438, 96154, 56005, 89325, 22491, 36357, 29131, 94445, 19969, 35152, 91667, 75500, 26311, 51577, 34488, 71395, 67288, 57825, 50095, 83237, 34209, 95007, 26853, 31621, 58666, 42780, 92055, 71525, 30231, 23491, 71769, 51608, 5679, 70456, 32506, 44213, 6369, 83884, 14501, 1181, 29909, 4958, 42337, 63198, 5726, 66244, 21551, 67297, 18915, 35473, 29887, 9810, 57135, 41926, 81496, 84042, 33265, 49806, 3340, 94833, 26189, 25161, 32879, 48969, 42073, 71557, 21134, 71560, 89008, 2250, 31628, 33030, 64432, 89462, 79576, 62928, 28477, 59357, 2841, 10605, 95428, 95909, 42797, 75777, 97963, 54349, 61896, 14516, 76246, 54867, 11646, 35155, 50719, 83856, 41951, 84068, 93703, 80073, 77785, 67725, 4559, 98179, 67319, 94933, 20242, 79944, 63892, 55772, 99094, 97233, 51712, 30451, 56685, 53338, 54910, 56229, 90568, 52094, 56661, 70559, 37426, 99509, 11947, 91704, 98741, 59013, 37972, 28010, 3789, 79132, 28147, 63110, 92828, 37544, 83482, 21449, 32852, 67874, 56253, 59038, 13322, 81532, 9693, 32122, 62730, 20437, 64223, 90921, 19100, 29838, 66231, 3560, 94138, 63795, 44145, 39657, 16469, 84037, 4209, 27956, 29422, 52934, 48873, 14472, 10905, 61335, 56499, 28242, 75748, 75839, 75987, 28644, 31192, 80590, 67303, 26464, 32253, 87318, 37462, 92000, 6603, 24584, 3271, 36874, 59879, 83417, 79809, 86997, 12956, 24564, 4388, 89426, 52079, 63207, 90294, 70248, 68756, 68938, 97569, 83634, 22156, 78601, 54262, 90408, 37616, 91527, 19389, 14541, 64577, 85896, 54069, 6167, 44909, 8466, 83941, 35168, 47906, 22061, 43641, 95235, 33930, 82119, 63457, 79995, 61137, 68183, 39971, 97977, 13572, 35873, 52262, 70646, 30083, 66253, 92112, 55239, 68141, 79481, 14277, 28734, 63869, 86450, 10327, 10241, 88266, 58745, 17247, 17705, 40882, 1851, 51908, 10750, 89919, 38091, 66541, 48673, 42953, 51372, 75528, 7425, 14975, 64666, 307, 30662, 44248, 65762, 5726, 7575, 43645, 47989, 15461, 63575, 36609, 66733, 71138, 45577, 90119, 17145, 61939, 93497, 93626, 17181, 47836, 50667, 80714, 45248, 83783, 43018, 37307, 74373, 98844, 65416, 44134, 8325, 42924, 92407, 69330, 99304, 38357, 35290, 23631, 46721, 90398, 76206, 11773, 52676, 36490, 20168, 69841, 62552, 51811, 35592, 55755, 23078, 31588, 23451, 43399, 90476, 80181, 91728, 6884, 93036, 16750, 68985, 97804, 60362, 71565, 67520, 27969, 29015, 51390, 99093, 6501, 62259, 93394, 23618, 40917, 78859, 16154, 724, 85833, 80988, 33121, 68698, 9972, 71865, 77695, 73574, 34276, 3336, 19831, 22218, 69012, 93290, 67396, 20466, 84536, 59, 86352, 69783, 9081, 39514, 22574, 48044, 42270, 14472, 11178, 75683, 97551, 50814, 60304, 22662, 76122, 23190, 57266]
# pipes = [[2325, 2356, 29701], [2328, 2325, 97954], [577, 2328, 14600], [375, 2325, 22110], [337, 2328, 14412], [364, 577, 2560], [363, 577, 30822], [2131, 375, 24812], [2156, 2356, 30583], [2136, 363, 51992], [2140, 2131, 68435], [2139, 577, 73049], [2150, 577, 96887], [2067, 364, 7896], [671, 2067, 87635], [857, 577, 65804], [754, 671, 38626], [755, 2356, 23842], [765, 2156, 13943], [758, 364, 86959], [781, 857, 56993], [785, 671, 65478], [829, 781, 1745], [834, 2150, 87906], [835, 755, 95020], [812, 2356, 32630], [816, 2139, 92967], [824, 816, 82313], [1389, 2356, 99891], [1391, 2139, 34162], [905, 2136, 38087], [907, 671, 57011], [944, 337, 18719], [941, 812, 77119], [939, 944, 21348], [937, 2325, 47063], [935, 907, 49034], [925, 834, 66432], [923, 2328, 84395], [916, 364, 47710], [910, 755, 91789], [2059, 824, 30519], [2065, 781, 43347], [2074, 2139, 67496], [2077, 765, 74982], [2083, 916, 29080], [2084, 944, 88674], [2087, 2074, 11198], [2093, 758, 25532], [2098, 2083, 43847], [2099, 337, 50451], [2101, 910, 65665], [2106, 925, 39159], [2112, 2093, 52212], [2113, 910, 83462], [2123, 2098, 33747], [2125, 905, 89431], [2364, 765, 79585], [2367, 835, 28242], [2370, 907, 62811], [2371, 2140, 48256], [2374, 937, 74903], [2376, 2084, 55040], [2378, 2098, 38146], [2381, 2065, 77445], [692, 1391, 93110], [697, 758, 9519], [698, 829, 34595], [703, 2378, 87776], [705, 835, 37237], [707, 765, 90199], [709, 916, 74313], [713, 2093, 33433], [714, 781, 61179], [715, 2084, 84718], [718, 2106, 13810], [723, 715, 86910], [726, 765, 29483], [730, 2378, 91981], [733, 337, 73538], [735, 2356, 23474], [737, 754, 60080], [739, 2087, 75982], [740, 925, 81311], [741, 715, 74974], [1436, 735, 25868], [1439, 707, 19432], [263, 741, 86544], [268, 816, 22154], [269, 2113, 31854], [271, 2371, 10505], [276, 910, 60703], [282, 2098, 91572], [285, 671, 58132], [287, 2112, 77145], [289, 2093, 48422], [293, 1391, 95248], [294, 2077, 61978], [299, 2084, 76738], [303, 925, 73090], [306, 944, 25304], [308, 941, 79013], [316, 287, 19431], [318, 2370, 38563], [323, 910, 31287], [325, 2084, 76222], [150, 2370, 24477], [153, 364, 76610], [155, 153, 89761], [160, 829, 25753], [165, 160, 67935], [170, 364, 7909], [172, 944, 67848], [181, 2376, 82491], [188, 916, 5625], [190, 2099, 69468], [193, 2376, 9665], [199, 172, 62761], [2086, 2065, 44662], [2092, 160, 40719], [2096, 2325, 35724], [2102, 2087, 35502], [2105, 698, 40461], [2115, 2370, 48251], [2117, 199, 31468], [2120, 816, 28268], [2122, 740, 92398], [2124, 2371, 93630], [2127, 2074, 32983], [2130, 737, 24667], [2134, 316, 83564], [2142, 308, 5146], [2148, 705, 81634], [2153, 263, 49365], [2158, 2325, 2743], [543, 2381, 70265], [549, 318, 93597], [552, 549, 82025], [557, 715, 17511], [561, 293, 43398], [563, 2150, 39531], [565, 735, 98386], [569, 318, 2478], [573, 737, 54275], [575, 2099, 57178], [580, 190, 7814], [588, 172, 77445], [592, 2117, 58732], [595, 2142, 40261], [599, 2381, 36161], [602, 713, 98426], [604, 1391, 5950], [609, 730, 93483], [1680, 2067, 24114], [1681, 2140, 56866], [1684, 2381, 83242], [1696, 785, 70301], [1701, 1391, 12339], [1706, 925, 66095], [1710, 190, 63542], [1715, 916, 80572], [1717, 1710, 49480], [1724, 2130, 28620], [1730, 2130, 15336], [1735, 282, 288], [1737, 2120, 40005], [1748, 2371, 22347], [1754, 709, 30440], [1764, 153, 82039], [1768, 337, 84935], [1774, 263, 26433], [1781, 2153, 1272], [768, 2127, 46568], [776, 2101, 17273], [779, 2112, 16543], [783, 363, 22913], [786, 835, 94387], [793, 2127, 39029], [797, 939, 38436], [800, 563, 55232], [801, 857, 21950], [807, 1774, 29748], [809, 2131, 79111], [815, 602, 1199], [820, 316, 58695], [823, 709, 95981], [828, 2136, 27236], [833, 741, 9606], [838, 153, 91230], [842, 692, 41108], [847, 1735, 57700], [848, 829, 29689], [852, 2374, 85869], [855, 1684, 27571], [861, 2123, 41434], [863, 714, 39683], [867, 857, 81282], [1211, 857, 18774], [1214, 783, 49499], [1217, 834, 35476], [1229, 1724, 74891], [1687, 1229, 47060], [1690, 730, 7734], [1691, 1774, 48920], [1695, 602, 97516], [1703, 1735, 52246], [1707, 268, 43065], [1716, 820, 89573], [1718, 289, 58966], [1728, 2148, 52119], [1733, 939, 62983], [1743, 188, 78837], [1753, 2150, 9200], [1763, 2125, 46561], [1770, 2136, 12250], [1778, 718, 33641], [1212, 1764, 15815], [1218, 740, 24866], [1222, 599, 64697], [1225, 1770, 55239], [1228, 1743, 9797], [1241, 549, 39615], [1243, 1774, 73084], [1246, 2099, 95143], [1249, 323, 43237], [1252, 2124, 60149], [1260, 2153, 87257], [1265, 1214, 91407], [1273, 563, 74371], [2225, 705, 7549], [2234, 2148, 31408], [2236, 2113, 10431], [1443, 2113, 62655], [1446, 2122, 73434], [1458, 316, 90573], [1461, 199, 86501], [1467, 2153, 65834], [1469, 2136, 10629], [1472, 2115, 58288], [1477, 857, 39544], [1479, 1684, 33084], [1483, 1225, 19688], [1487, 588, 58833], [1491, 793, 40730], [1495, 823, 26525], [1498, 363, 91271], [1502, 2140, 92204], [1510, 783, 39725], [1514, 1778, 10054], [1518, 848, 93549], [1521, 268, 79149], [1527, 1724, 49358], [1530, 1225, 68552], [1533, 2234, 91335], [1539, 1467, 16607], [1660, 785, 8117], [1662, 303, 9942], [1667, 2131, 5518], [1671, 754, 81096], [1674, 937, 77858], [1678, 730, 85794], [1686, 1443, 63758], [1688, 2158, 10604], [1699, 714, 32353], [1702, 2136, 49598], [1705, 867, 63931], [1709, 786, 54049], [1712, 707, 30348], [1720, 318, 17723], [1723, 2093, 14597], [1726, 2098, 69424], [1729, 1691, 86389], [1732, 299, 81918], [1742, 2065, 19960], [1745, 2140, 28629], [1749, 599, 93187], [1752, 923, 68123], [1756, 592, 56918], [1759, 1472, 76694], [1760, 294, 63931], [1765, 2074, 442], [1769, 925, 52931], [1776, 1533, 4396], [1779, 2105, 11191], [1782, 293, 43474], [1987, 165, 6269], [1990, 2098, 33667], [1992, 1214, 6513], [1995, 1483, 22053], [1996, 726, 40227], [2000, 1479, 19227], [2001, 2364, 81886], [2002, 1667, 67585], [2006, 1687, 57839], [2011, 577, 30096], [2012, 707, 47372], [2018, 1518, 23678], [2021, 2136, 670], [2026, 1760, 13894], [2029, 1720, 64933], [2031, 833, 93614], [2033, 754, 53580], [2035, 741, 39323], [2037, 1439, 41593], [2040, 2112, 79360], [2046, 2099, 49864], [2047, 1732, 71371], [2049, 2225, 50407], [2241, 1763, 99842], [2244, 1735, 89522], [2248, 713, 47359], [2250, 2120, 10440], [2251, 800, 69916], [2259, 306, 82127], [2262, 718, 16063], [2264, 323, 22127], [2266, 2101, 80682], [2268, 1774, 89040], [2269, 2026, 80592], [2276, 1696, 96631], [2279, 834, 61883], [2281, 2364, 27086], [2287, 1726, 24856], [2288, 2099, 31190], [2289, 2156, 6332], [2293, 1483, 37086], [2296, 820, 66342], [2297, 1477, 63657], [2299, 1781, 66852], [2301, 671, 3203], [2390, 1241, 23276], [2393, 2370, 24924], [2395, 588, 4120], [2403, 2047, 1127], [2404, 852, 86892], [2408, 287, 13695], [2411, 2124, 90507], [2413, 1701, 74287], [2417, 323, 92535], [2422, 740, 27284], [2427, 1756, 48670], [2430, 2376, 90645], [2432, 1781, 85273], [2438, 172, 49536], [2440, 707, 21260], [2444, 1472, 2781], [2451, 1443, 269], [2454, 1776, 60842], [2459, 698, 22681], [2462, 2099, 85785], [2468, 575, 89197], [2471, 2105, 69847], [46, 785, 39904], [144, 697, 19893], [147, 907, 56127], [157, 1709, 48189], [161, 1265, 89934], [166, 698, 66369], [168, 1756, 29590], [176, 2026, 3754], [186, 1765, 9509], [195, 2279, 79715], [198, 2087, 47434], [200, 303, 47336], [202, 809, 69146], [204, 2127, 40688], [211, 1781, 28761], [214, 2099, 40084], [216, 2276, 82437], [218, 916, 42658], [222, 2117, 80995], [224, 2367, 34338], [226, 2124, 40008], [230, 364, 63149], [381, 2096, 31739], [385, 867, 89392], [388, 2122, 63529], [399, 575, 36936], [404, 835, 63126], [411, 293, 31083], [413, 755, 53705], [416, 2156, 56687], [423, 153, 88308], [430, 2451, 22839], [433, 1211, 89481], [439, 168, 6691], [443, 147, 31223], [448, 1510, 23396], [449, 1518, 69334], [453, 448, 91478], [459, 2390, 16589], [460, 707, 8928], [463, 2281, 143], [467, 580, 16663], [471, 939, 81576], [472, 430, 70081], [630, 443, 24489], [631, 2299, 2672], [634, 271, 69484], [636, 2381, 59446], [641, 1469, 52696], [648, 1720, 84846], [652, 2422, 94453], [654, 2106, 46202], [661, 2117, 43707], [664, 754, 80290], [665, 718, 46972], [668, 543, 49175], [669, 1710, 96266], [674, 569, 95130], [676, 797, 35117], [679, 2140, 94893], [682, 1707, 13503], [686, 2462, 90696], [690, 2156, 39992], [874, 2413, 26948], [877, 1229, 27388], [881, 2259, 12535], [883, 2158, 12371], [885, 1265, 39834], [886, 1732, 75407], [888, 1389, 70914], [892, 2378, 54875], [893, 577, 20308], [896, 2408, 85175], [897, 730, 60506], [899, 218, 27167], [900, 1776, 61452], [903, 1688, 35717], [906, 648, 29093], [911, 433, 31228], [913, 195, 19151], [915, 1681, 97610], [919, 459, 21279], [920, 1443, 46233], [924, 848, 69736], [927, 1724, 31508], [931, 834, 44004], [936, 224, 3962], [940, 939, 79612], [943, 289, 4250], [946, 609, 97829], [1058, 1527, 1066], [1059, 2444, 7403], [1060, 543, 37692], [1063, 669, 22687], [1067, 449, 56506], [1070, 282, 52921], [1072, 472, 96586], [1078, 835, 5731], [1083, 1067, 57131], [1085, 848, 96517], [1086, 2269, 99809], [1088, 198, 68589], [1091, 1467, 71272], [1093, 1662, 57985], [1094, 737, 27813], [1096, 1252, 70912], [1100, 2371, 86659], [1106, 2370, 67407], [1107, 2000, 37718], [1112, 1083, 12823], [1115, 2131, 26811], [1117, 2404, 91098], [1119, 155, 52001], [1210, 1748, 21530], [1221, 193, 37370], [1224, 308, 37112], [1235, 1094, 74920], [1239, 881, 91805], [1245, 308, 80731], [1254, 1770, 44481], [1257, 2150, 7873], [1259, 741, 21003], [1270, 867, 47200], [1272, 707, 74453], [1276, 157, 14129], [1287, 935, 58054], [1288, 669, 49276], [1290, 1779, 56280], [1295, 800, 9619], [1298, 665, 78638], [1311, 2262, 10712], [1315, 2454, 66020], [1319, 1720, 80295], [1321, 2297, 28690], [1325, 198, 83992], [1444, 1259, 55730], [1452, 935, 7169], [1456, 1996, 56690], [1460, 202, 85799], [1468, 1479, 3931], [1488, 779, 24810], [1490, 2142, 77452], [1494, 2001, 32911], [1501, 1680, 42964], [1507, 735, 31069], [1516, 1212, 41219], [1517, 1745, 2034], [1519, 676, 40486], [1523, 1456, 62129], [1528, 1321, 7965], [1529, 1239, 91508], [1531, 2364, 47778], [1534, 1754, 10275], [1663, 2413, 94898], [1668, 1732, 60062], [1673, 1688, 93689], [1677, 2124, 13272], [1682, 1100, 78544], [1689, 1100, 47387], [1694, 807, 4860], [1698, 2281, 6421], [1708, 776, 76139], [1719, 1501, 2364], [1721, 1315, 78385], [1725, 1990, 29835], [1736, 2328, 41852], [1741, 2241, 63462], [1751, 781, 93519], [1757, 580, 37296], [1761, 1117, 10557], [1766, 2098, 65851], [1771, 703, 99995], [1772, 472, 12483], [1777, 2096, 37164], [1783, 1495, 14026], [1931, 1702, 29444], [1934, 1228, 7819], [1936, 1218, 82004], [1939, 1458, 47984], [1945, 1469, 29448], [1946, 416, 31385], [1949, 679, 63095], [1951, 905, 98741], [1954, 703, 54784], [1958, 1444, 10447], [1960, 1995, 72751], [1962, 2049, 45488], [1964, 2037, 75688], [1966, 153, 10869], [1968, 1088, 81730], [1970, 1757, 29497], [1971, 1523, 37332], [1975, 931, 42589], [1978, 1273, 53733], [1983, 460, 52573], [2162, 943, 7587], [2166, 682, 19826], [2170, 931, 41327], [2171, 1756, 70389], [2175, 936, 26277], [2181, 765, 39546], [2183, 147, 66509], [2193, 1681, 98920], [2197, 941, 19089], [2202, 2236, 35303], [2208, 881, 4232], [2211, 1741, 54115], [2213, 226, 45857], [2217, 1710, 49621], [2221, 714, 349], [2224, 1443, 63796], [2227, 595, 65490], [2229, 809, 40970], [2237, 144, 52376], [2382, 2289, 34623], [2392, 1954, 98601], [2397, 573, 1562], [2399, 186, 9795], [2414, 1663, 69249], [2416, 943, 46677], [2419, 1260, 78156], [2423, 823, 99773], [2426, 172, 10140], [2436, 453, 15448], [2441, 2000, 40427], [2450, 1468, 43109], [2452, 1677, 19523], [2457, 2404, 10769], [2466, 2374, 37047], [2472, 740, 17012], [56, 741, 60120], [59, 668, 6555], [61, 907, 30469], [63, 2436, 6630], [68, 1452, 76908], [72, 337, 24572], [78, 147, 71619], [79, 1443, 76514], [82, 2356, 32379], [84, 2462, 16104], [90, 1698, 30853], [96, 1721, 31911], [99, 549, 59986], [105, 1246, 91902], [110, 1694, 32858], [115, 282, 49924], [118, 676, 79879], [121, 160, 54271], [126, 877, 74548], [130, 1703, 92122], [131, 2382, 5796], [132, 735, 27394], [327, 2115, 64291], [328, 1067, 92395], [332, 155, 54614], [335, 2142, 8970], [338, 199, 88394], [339, 1249, 77577], [341, 222, 97694], [345, 1975, 84105], [346, 2404, 47890], [352, 781, 8476], [355, 1662, 34665], [356, 160, 93412], [361, 2276, 29571], [362, 575, 81957], [367, 2120, 52023], [368, 82, 16762], [373, 1958, 21122], [377, 692, 63619], [380, 1782, 65787], [542, 364, 31441], [546, 2136, 986], [548, 758, 54773], [550, 1964, 70162], [553, 781, 57184], [556, 72, 90143], [559, 2131, 91283], [564, 367, 13805], [567, 2217, 7617], [572, 2408, 87674], [576, 807, 23108], [578, 703, 98725], [581, 907, 50697], [583, 1753, 48048], [584, 121, 60324], [587, 1719, 93321], [591, 1966, 38489], [596, 204, 5394], [601, 2183, 12787], [603, 2127, 23866], [606, 2120, 90217], [607, 2117, 82553], [610, 581, 7973], [612, 200, 76527], [614, 226, 56597], [2349, 2472, 47951], [92, 1757, 83025], [638, 559, 41159], [616, 1112, 18057], [618, 1684, 30510], [620, 1992, 57167], [621, 460, 7280], [622, 1710, 95340], [625, 153, 42062], [756, 741, 20734], [766, 781, 83850], [770, 1753, 64698], [774, 636, 68193], [789, 299, 75048], [791, 913, 39739], [792, 765, 22937], [796, 2217, 27970], [805, 380, 75522], [808, 801, 43061], [811, 604, 14018], [819, 1741, 96330], [821, 2459, 44728], [830, 1718, 23323], [831, 927, 5042], [836, 709, 54984], [851, 2417, 34668], [858, 1528, 52639], [869, 834, 5296], [871, 1674, 80879], [1007, 1696, 73533], [1008, 1222, 83354], [1011, 1725, 50680], [1017, 2259, 48772], [1020, 1534, 73895], [1021, 195, 24064], [1027, 1778, 14790], [1028, 399, 85873], [1031, 199, 28579], [1035, 1971, 32647], [1043, 1490, 12964], [1050, 2026, 2904], [1054, 2021, 50018], [1120, 1243, 82885], [1124, 306, 99206], [1126, 328, 63921], [1128, 1688, 66754], [1131, 362, 66462], [1134, 294, 96234], [1137, 559, 26666], [1139, 1748, 57581], [1144, 1735, 30112], [1145, 625, 10459], [1146, 2046, 44090], [1150, 2018, 60197], [1159, 1311, 71198], [1161, 299, 54976], [1163, 1252, 55682], [1166, 2416, 59409], [1167, 1146, 89738], [1172, 900, 73013], [1183, 1115, 93100], [1188, 906, 79377], [1190, 811, 69952], [1193, 1771, 23610], [1199, 703, 16640], [1201, 1753, 59724], [1204, 616, 84764], [1387, 1951, 45024], [1393, 2419, 62081], [1398, 584, 79489], [1401, 915, 39178], [1402, 1752, 74206], [1405, 877, 63863], [1410, 559, 66011], [1411, 835, 82499], [1416, 2395, 60177], [1417, 768, 48320], [1420, 335, 58341], [1423, 2125, 56620], [1424, 1199, 82414], [1427, 2065, 10017], [1432, 1126, 23275], [1433, 1996, 34075], [1438, 612, 62803], [1440, 118, 43520], [1442, 285, 52854], [1579, 1741, 51614], [1580, 1433, 89488], [1585, 816, 66757], [1587, 553, 44073], [1589, 1996, 85687], [1595, 2390, 29730], [1597, 2139, 90839], [1600, 847, 41194], [1602, 352, 24041], [1606, 630, 43968], [1607, 1585, 65842], [1610, 2392, 43786], [1613, 614, 18594], [1617, 923, 74456], [1618, 851, 41188], [1622, 2037, 84327], [1628, 1287, 36410], [1630, 147, 35702], [1631, 1060, 34017], [1638, 2153, 85347], [1640, 888, 29166], [1645, 82, 96409], [1647, 2299, 8271], [1657, 1681, 8017], [1845, 467, 23508], [1851, 2124, 32942], [1854, 923, 33343], [1858, 126, 17269], [1859, 899, 976], [1860, 1949, 83076], [1868, 576, 43856], [1870, 1246, 96287], [1874, 634, 2662], [1879, 861, 84453], [1882, 924, 29772], [1886, 587, 33370], [1894, 2293, 75637], [1903, 1131, 13487], [1905, 2411, 48949], [1907, 1210, 47795], [1908, 1433, 93754], [1912, 1490, 28897], [1914, 576, 12717], [1919, 1723, 68809], [1921, 1479, 16171], [1922, 2374, 88933], [1924, 638, 87030], [2054, 2390, 4278], [2055, 1778, 25616], [2057, 1389, 61937], [2061, 1589, 56804], [2063, 147, 80587], [2070, 935, 38880], [2072, 1521, 57168], [2076, 789, 61736], [2079, 144, 84984], [2080, 2224, 52546], [2085, 118, 5940], [2090, 1667, 29613], [2094, 1962, 39580], [2100, 572, 27998], [2103, 1725, 61685], [2108, 230, 74288], [2109, 1756, 20328], [2114, 285, 2969], [2119, 1438, 23636], [2126, 1735, 5032], [2132, 1733, 79476], [2138, 1716, 79966], [2146, 714, 61231], [2155, 1761, 32449], [2306, 1894, 32972], [2309, 1845, 6861], [2310, 463, 90408], [2312, 1707, 29763], [2316, 1494, 30081], [2319, 165, 19984], [2320, 131, 57495], [2322, 621, 63029], [2329, 1290, 10344], [2334, 833, 20088], [2335, 1502, 53171], [2341, 287, 38445], [2344, 2237, 69892], [2348, 1438, 70080], [2351, 170, 88174], [2353, 2221, 27401], [2359, 439, 43755], [2366, 1782, 30755], [2372, 842, 42015], [2377, 823, 49610], [2379, 2356, 69359], [1, 1606, 46559], [2, 1766, 76517], [4, 1736, 16816], [7, 829, 63889], [8, 831, 39489], [9, 690, 17797], [10, 2312, 68888], [12, 631, 52521], [13, 1699, 44975], [17, 411, 9503], [21, 1771, 14523], [22, 885, 77685], [23, 595, 28309], [26, 13, 17487], [28, 193, 60064], [30, 1958, 76168], [36, 2105, 83440], [38, 2001, 82745], [41, 906, 75405], [45, 2102, 82402], [48, 168, 66823], [235, 1606, 48085], [245, 289, 90079], [248, 2250, 5616], [249, 2156, 68623], [251, 2175, 19385], [254, 871, 46757], [261, 82, 55296], [274, 546, 30375], [280, 1483, 91717], [284, 766, 67665], [288, 1607, 52548], [298, 835, 63235], [311, 634, 96433], [315, 1882, 99651], [320, 2124, 76994], [324, 1510, 42643], [477, 1782, 12815], [478, 730, 25495], [480, 2349, 57217], [481, 863, 42236], [484, 1757, 75626], [487, 1257, 26946], [493, 2471, 56242], [496, 1695, 79397], [498, 1628, 53379], [500, 588, 23336], [503, 153, 6950], [505, 230, 696], [510, 1759, 65835], [512, 2211, 61351], [514, 1458, 89541], [515, 2251, 68950], [519, 188, 32748], [520, 2229, 81843], [522, 686, 4490], [525, 587, 72524], [526, 1495, 70036], [528, 214, 52589], [531, 899, 90447], [532, 791, 47774], [533, 1393, 54053], [536, 559, 40525], [538, 2225, 46144], [693, 99, 62853], [694, 559, 54067], [696, 596, 67504], [706, 1868, 80567], [722, 789, 22963], [725, 362, 56242], [727, 607, 6574], [736, 592, 70722], [738, 1647, 45292], [744, 588, 52359], [746, 1613, 39501], [748, 520, 32544], [750, 204, 56884], [947, 315, 41796], [953, 664, 49328], [956, 1970, 11891], [961, 335, 83820], [966, 692, 21744], [967, 1602, 87699], [972, 565, 78040], [976, 830, 71910], [978, 1519, 74307], [981, 1720, 17008], [990, 897, 63190], [993, 573, 41854], [998, 41, 19567], [999, 1393, 96123], [1002, 1060, 36096], [1130, 896, 81426], [1132, 2065, 90421], [1135, 1970, 16556], [1140, 268, 81131], [1141, 1983, 31427], [1143, 22, 89625], [1149, 820, 12035], [1151, 1211, 79609], [1156, 1188, 5052], [1157, 1389, 65741], [1162, 2070, 71008], [1165, 1585, 7224], [1171, 1939, 78199], [1176, 188, 29946], [1177, 30, 8212], [1180, 325, 41003], [1181, 2356, 90961], [1186, 2417, 80313], [1196, 1699, 89332], [1198, 7, 48982], [1200, 1631, 39129], [1203, 1134, 86112], [1207, 2419, 27089], [1328, 2079, 34766], [1331, 2139, 61973], [1335, 1270, 84557], [1336, 1172, 16913], [1340, 828, 59548], [1341, 1134, 32759], [1343, 121, 38234], [1344, 2227, 39254], [1346, 1063, 43711], [1352, 578, 28940], [1355, 796, 33981], [1363, 503, 21487], [1365, 2148, 84244], [1369, 1507, 57088], [1371, 2306, 83288], [1372, 1440, 56168], [1374, 1091, 17644], [1377, 131, 42801], [1380, 828, 57027], [1381, 1945, 96016], [1382, 2112, 26996], [1540, 1446, 75056], [1541, 2079, 13665], [1542, 2367, 88854], [1546, 2269, 23074], [1548, 2250, 2301], [1550, 1934, 12625], [1551, 692, 84195], [1552, 1200, 16484], [1553, 121, 31998], [1555, 2197, 18504], [1557, 254, 99762], [1558, 543, 39921], [1559, 481, 87827], [1561, 855, 27048], [1563, 1745, 94118], [1565, 2115, 87086], [1566, 1962, 72420], [1568, 1945, 44397], [1570, 755, 15100], [1572, 709, 80135], [1577, 248, 58950], [1785, 289, 81901], [1789, 1377, 4709], [1792, 254, 39726], [1793, 2268, 7736], [1794, 1870, 54710], [1797, 288, 62001], [1798, 2181, 29773], [1800, 1141, 65858], [1803, 2417, 95922], [1811, 2452, 2412], [1813, 2348, 68224], [1815, 1021, 83889], [1817, 1501, 79657], [1818, 936, 93207], [1827, 453, 40377], [1833, 883, 58237], [1839, 1241, 29188], [1840, 1617, 25838], [1988, 2468, 56007], [1993, 990, 60140], [1997, 263, 79481], [2004, 783, 27911], [2005, 1199, 3582], [2007, 1420, 49154], [2010, 1050, 87652], [2014, 1017, 73281], [2016, 10, 36274], [2020, 1440, 61660], [2023, 1145, 32181], [2025, 195, 29200], [2030, 2090, 86665], [2032, 2289, 30738], [2034, 335, 70415], [2038, 638, 3994], [2041, 377, 80126], [2043, 214, 71618], [2048, 200, 45588], [2051, 2010, 55626], [2053, 1140, 31588], [2243, 722, 31789], [2247, 1703, 17302], [2249, 1444, 12746], [2252, 1886, 37508], [2253, 2382, 4480], [2255, 1257, 58719], [2256, 2010, 65512], [2257, 1416, 28926], [2260, 953, 45187], [2261, 1860, 78056], [2263, 323, 97799], [2265, 1198, 76020], [2267, 893, 75005], [2270, 885, 1316], [2271, 2243, 81828], [2273, 781, 52498], [2278, 1732, 33525], [2280, 2051, 31623], [2283, 2063, 43117], [2284, 696, 5354], [2285, 413, 7771], [2291, 768, 88131], [2292, 2146, 35519], [2294, 90, 64864], [2298, 805, 25739], [2300, 2005, 13813], [2302, 2260, 42627], [3, 2253, 40231], [6, 45, 87466], [11, 251, 13624], [14, 368, 79724], [16, 1145, 67808], [18, 1145, 52411], [19, 1131, 43981], [24, 311, 68647], [25, 792, 65029], [27, 222, 9544], [29, 90, 38297], [34, 602, 63983], [35, 727, 34481], [40, 1295, 55087], [44, 254, 11747], [47, 1523, 79129], [49, 2127, 32580], [50, 2393, 20877], [51, 328, 71548], [145, 1548, 46879], [156, 953, 5783], [179, 150, 3442], [184, 46, 71950], [187, 2211, 94219], [201, 423, 98335], [203, 1589, 77875], [207, 531, 89787], [208, 1100, 5089], [212, 235, 1950], [215, 2093, 16928], [225, 883, 55387], [228, 2310, 938], [382, 105, 26394], [384, 1874, 60967], [387, 2450, 27009], [391, 72, 82089], [393, 546, 46433], [394, 1107, 24708], [397, 1439, 3945], [401, 2109, 56941], [405, 2349, 64853], [408, 774, 94888], [409, 781, 70744], [417, 1490, 21963], [419, 706, 45436], [422, 905, 64359], [425, 789, 19091], [427, 1566, 87745], [431, 1886, 79986], [432, 1377, 71054], [436, 2085, 1784], [438, 168, 305], [450, 40, 35470], [454, 510, 19927], [456, 10, 34071], [461, 463, 25875], [465, 1382, 33904], [468, 2422, 35240], [469, 900, 96757], [633, 993, 99260], [635, 520, 21105], [643, 1063, 56914], [645, 68, 645], [647, 2083, 96687], [649, 1745, 43050], [651, 9, 99373], [655, 1171, 83226], [657, 2076, 56519], [659, 1124, 47035], [660, 2049, 75986], [663, 1903, 81904], [670, 944, 99363], [678, 2025, 72480], [680, 2113, 45907], [683, 448, 84406], [685, 2012, 70180], [688, 848, 19553], [872, 2072, 8386], [875, 2083, 252], [876, 1162, 97365], [878, 2014, 37848], [884, 2247, 37118], [894, 195, 39041], [901, 375, 99946], [908, 99, 41996], [918, 380, 27106], [921, 2432, 97036], [926, 1070, 77664], [929, 449, 4037], [932, 885, 32401], [934, 510, 85459], [942, 367, 27207], [1057, 1845, 57134], [1062, 2076, 28041], [1066, 2370, 77491], [1069, 1551, 65580], [1071, 920, 87864], [1075, 1988, 15442], [1079, 224, 5407], [1081, 1096, 73219], [1092, 1833, 57141], [1095, 1949, 52713], [1099, 1690, 43624], [1101, 315, 32118], [1103, 838, 47822], [1104, 1514, 29408], [1108, 280, 78049], [1111, 823, 50714], [1223, 815, 86472], [1231, 387, 68476], [1234, 531, 38085], [1236, 1534, 81822], [1244, 1521, 14775], [1250, 1553, 67989], [1258, 683, 52152], [1262, 2366, 92443], [1269, 1410, 37819], [1278, 676, 85861], [1280, 1689, 1976], [1282, 1772, 59940], [1294, 1519, 43078], [1297, 1983, 9528], [1299, 1328, 5966], [1302, 1094, 28826], [1303, 1555, 77664], [1312, 1352, 24149], [1320, 2123, 72484], [1323, 881, 37842], [1447, 1501, 91619], [1450, 34, 67223], [1457, 1741, 78596], [1462, 779, 74695], [1464, 2459, 33923], [1465, 2020, 43447], [1470, 1276, 36399], [1474, 2150, 49664], [1475, 427, 69054], [1481, 1236, 61936], [1492, 1726, 49966], [1496, 235, 42409], [1505, 526, 50699], [1508, 1172, 13183], [1524, 1211, 50778], [1532, 1757, 92462], [1536, 1363, 10492], [1659, 90, 35982], [1661, 2131, 15974], [1666, 828, 67054], [1670, 1141, 86577], [1672, 1249, 7191], [1676, 572, 98932], [1683, 577, 28736], [1692, 2018, 13772], [1697, 792, 7096], [1704, 2007, 54452], [1711, 1958, 19943], [1714, 2053, 35404], [1722, 1717, 96687], [1727, 1302, 26920], [1734, 1108, 458], [1740, 6, 39277], [1744, 216, 57936], [1747, 1140, 3756], [1750, 1229, 52759], [1758, 2471, 38159], [1762, 1677, 69176], [1773, 900, 86148], [1927, 1369, 19519], [1930, 79, 29015], [1932, 1460, 63642], [1935, 2083, 4870], [1937, 655, 43005], [1938, 2134, 57183], [1942, 897, 16655], [1943, 453, 1151], [1944, 2266, 98617], [1948, 303, 89028], [1952, 1678, 79083], [1956, 601, 13599], [1957, 671, 94755], [1959, 480, 1253], [1963, 96, 42950], [1967, 144, 99864], [1969, 2344, 60237], [1972, 789, 44646], [1976, 2122, 27747], [1980, 738, 89232], [1981, 2113, 48528], [1985, 2093, 98831], [2161, 1099, 37847], [2168, 362, 74462], [2172, 1262, 97605], [2173, 2227, 45431], [2186, 993, 38924], [2189, 2441, 73172], [2198, 335, 55698], [2206, 1978, 86420], [2209, 858, 33124], [2215, 1111, 27023], [2216, 1043, 41818], [2219, 575, 22051], [2222, 1647, 91198], [2231, 2126, 55308], [2384, 1320, 8405], [2387, 78, 65093], [2394, 2393, 54963], [2398, 2132, 95585], [2401, 774, 27071], [2406, 1096, 49396], [2410, 1507, 29232], [2421, 160, 90311], [2429, 1761, 14562], [2437, 1745, 77705], [2442, 166, 57376], [2445, 2441, 58445], [2448, 41, 97281], [2449, 649, 21227], [2455, 1756, 25219], [2463, 2040, 26123], [2465, 1433, 19269], [2469, 1365, 36245], [55, 713, 55653], [57, 2278, 72465], [62, 1235, 46922], [66, 2263, 9719], [70, 774, 29885], [76, 961, 47172], [81, 591, 39632], [86, 235, 69567], [88, 1469, 17907], [102, 1103, 21517], [104, 1565, 32961], [108, 269, 50629], [112, 126, 88768], [114, 1680, 61], [119, 630, 3293], [123, 2374, 17187], [127, 1870, 81262], [137, 46, 79260], [138, 449, 29515], [140, 563, 47097], [329, 86, 33223], [330, 2288, 86813], [334, 943, 42502], [336, 51, 75460], [340, 1377, 74267], [343, 1124, 56370], [344, 657, 41896], [347, 1732, 92434], [349, 1502, 8121], [351, 1734, 28006], [353, 268, 19345], [354, 1797, 44573], [359, 299, 34942], [360, 23, 5950], [365, 1962, 2090], [369, 1980, 88550], [371, 1565, 48699], [372, 1436, 4514], [376, 2251, 3184], [378, 349, 77394], [379, 1062, 7405], [545, 2348, 29788], [551, 1432, 36518], [554, 1729, 88711], [560, 894, 37324], [562, 1249, 73937], [568, 756, 19147], [570, 2153, 57215], [586, 2298, 24217], [593, 422, 26681], [600, 328, 37182], [605, 1981, 82393], [613, 2270, 32192], [615, 1221, 39154], [619, 505, 70901], [629, 1257, 77526], [760, 1964, 12651], [763, 1750, 82223], [764, 1017, 79693], [773, 796, 42181], [782, 652, 52425], [784, 981, 66822], [788, 168, 1686], [795, 1442, 65306], [799, 1502, 37170], [804, 1295, 77594], [810, 2219, 89385], [818, 90, 15455], [832, 663, 99751], [841, 1815, 62729], [844, 1754, 92741], [846, 2270, 98670], [856, 831, 73786], [862, 1580, 6316], [866, 999, 52349], [1004, 2419, 97209], [1005, 816, 67989], [1010, 1617, 35613], [1012, 919, 83583], [1013, 2280, 13897], [1016, 1295, 64767], [1019, 1199, 42027], [1022, 2048, 74106], [1023, 768, 8917], [1024, 349, 97126], [1026, 1970, 21924], [1030, 789, 78273], [1032, 1387, 56058], [1034, 770, 44013], [1037, 1259, 38845], [1039, 919, 27218], [1040, 2292, 91012], [1044, 2229, 65079], [1046, 2270, 13174], [1048, 2234, 91914], [1052, 690, 70268], [1055, 138, 43547], [1215, 1770, 95441], [1219, 1922, 18636], [1230, 2090, 90762], [1232, 1728, 58776], [1238, 381, 22743], [1242, 1135, 64933], [1251, 1058, 83847], [1253, 1975, 11468], [1267, 156, 39535], [1268, 2379, 25968], [1274, 2316, 66997], [1279, 2206, 98726], [1284, 2316, 79867], [1291, 2312, 81058], [1296, 613, 12658], [1304, 2108, 71667], [1309, 1483, 94781], [1310, 863, 37981], [1314, 543, 18786], [1316, 344, 88759], [1322, 1963, 78111], [1388, 2406, 13819], [1390, 1563, 43111], [1394, 459, 20474], [1396, 807, 3246], [1399, 1765, 45958], [1400, 176, 53648], [1404, 1143, 33187], [1407, 2215, 83810], [1408, 676, 87228], [1412, 1540, 45312], [1413, 2278, 63390], [1414, 355, 92916], [1415, 12, 47883], [1422, 1461, 53860], [1426, 915, 8672], [1429, 1839, 86785], [1430, 487, 65901], [1431, 481, 51262], [1434, 614, 80207], [1582, 785, 75222], [1583, 2322, 27666], [1586, 2229, 57802], [1590, 1765, 33633], [1596, 459, 50388], [1598, 2414, 23704], [1599, 2371, 74370], [1605, 1936, 77667], [1609, 1234, 5221], [1612, 1166, 43471], [1621, 1492, 61996], [1625, 823, 76535], [1627, 1162, 26831], [1632, 1022, 8445], [1636, 2103, 29489], [1637, 168, 20237], [1639, 453, 44931], [1643, 674, 92900], [1653, 2382, 24770], [1656, 1331, 77359], [1844, 1267, 74314], [1848, 1446, 17531], [1852, 2010, 77686], [1855, 1970, 16228], [1863, 956, 19946], [1869, 1012, 94200], [1871, 1103, 3481], [1876, 1401, 17914], [1883, 505, 1114], [1888, 1631, 61869], [1890, 1321, 98166], [1893, 155, 80127], [1896, 1069, 47055], [1902, 1696, 89446], [1909, 1272, 88789], [1915, 1162, 83878], [1916, 595, 67068], [1923, 1510, 64810], [2060, 1555, 80527], [2064, 38, 88069], [2069, 2423, 52479], [2071, 2376, 50371], [2073, 211, 23137], [2088, 1012, 32526], [2091, 944, 34004], [2095, 1048, 64978], [2104, 1668, 93286], [2110, 1637, 34363], [2116, 705, 37193], [2128, 981, 9181], [2133, 1968, 8918], [2135, 168, 32415], [2145, 2250, 44129], [2147, 2021, 45003], [2149, 805, 6625], [2151, 251, 59018], [2157, 910, 57721], [2303, 528, 85166], [2305, 1010, 65677], [2308, 127, 41424], [2314, 2322, 86664], [2317, 361, 57113], [2318, 863, 85420], [2324, 28, 21810], [2326, 2157, 52942], [2330, 365, 51677], [2336, 2021, 77734], [2339, 153, 61294], [2342, 1531, 56154], [2345, 1815, 29865], [2346, 1800, 907], [2354, 916, 7889], [2360, 1914, 58732], [2363, 693, 79242], [2369, 1246, 75829], [2375, 1742, 67049], [2380, 11, 1592], [54, 1265, 95786], [58, 1278, 24490], [65, 610, 85484], [67, 665, 77635], [69, 2031, 37718], [74, 2234, 4761], [77, 176, 12374], [83, 2451, 11572], [85, 1863, 1816], [87, 1054, 82955], [89, 792, 43306], [94, 1969, 72473], [97, 77, 87446], [98, 193, 27840], [100, 1570, 37426], [107, 1766, 23912], [109, 686, 82520], [113, 727, 96068], [117, 35, 55136], [122, 1687, 86300], [125, 1246, 69768], [128, 765, 28291], [134, 1527, 53655], [135, 573, 32168], [136, 109, 38422], [139, 2257, 96425], [233, 1833, 76632], [236, 202, 96901], [238, 832, 48295], [241, 915, 11112], [243, 1586, 277], [247, 554, 91888], [250, 2339, 2254], [256, 1115, 52498], [260, 906, 50834], [266, 1210, 54532], [270, 791, 84466], [272, 1934, 14627], [281, 651, 14495], [291, 592, 27442], [296, 1770, 7948], [297, 413, 49316], [304, 1267, 79626], [305, 1583, 60102], [310, 1536, 42394], [322, 1424, 27468], [475, 2455, 42013], [483, 1557, 64794], [489, 976, 28827], [490, 1031, 6090], [491, 2335, 66047], [495, 1815, 54962], [497, 1597, 96608], [499, 14, 46048], [501, 1529, 37266], [504, 55, 30408], [506, 671, 26769], [507, 924, 74818], [518, 1692, 68974], [523, 797, 51877], [527, 266, 17931], [530, 405, 74916], [535, 2384, 16680], [537, 1328, 35729], [541, 907, 98928], [699, 1687, 85916], [701, 1432, 38150], [711, 1315, 29668], [717, 2132, 56368], [719, 961, 58075], [721, 916, 69253], [724, 972, 6161], [729, 2411, 51819], [732, 118, 46093], [734, 198, 54299], [742, 110, 14478], [743, 11, 5109], [745, 1112, 71181], [747, 2252, 15817], [749, 841, 80072], [751, 338, 74711], [752, 193, 64622], [753, 2229, 93265], [949, 1632, 60543], [952, 1636, 72127], [954, 2011, 27712], [957, 670, 1410], [959, 260, 16310], [962, 1531, 4071], [963, 872, 3099], [964, 1811, 73169], [969, 327, 78551], [971, 324, 62568], [974, 1229, 21116], [977, 785, 58317], [980, 280, 57855], [982, 829, 65994], [984, 1244, 91291], [985, 568, 47920], [986, 1792, 3364], [987, 2085, 39836], [988, 1740, 24790], [991, 789, 61865], [994, 3, 6234], [996, 2136, 33000], [997, 967, 43408], [1121, 2381, 32464], [1125, 327, 86602], [1129, 855, 61679], [1133, 709, 10910], [1142, 2263, 7689], [1154, 436, 32927], [1155, 2257, 77445], [1158, 562, 64029], [1168, 1719, 7387], [1170, 105, 63811], [1174, 2161, 29992], [1179, 2377, 54374], [1182, 2244, 78783], [1185, 908, 77763], [1189, 1811, 64587], [1191, 682, 36166], [1192, 1548, 41809], [1197, 1676, 20745], [1202, 709, 23467], [1206, 2260, 68460], [1208, 1703, 81702], [1327, 1694, 21373], [1330, 1299, 47377], [1332, 671, 63391], [1333, 939, 70491], [1334, 1674, 85392], [1339, 2085, 23659], [1347, 2273, 70857], [1350, 1502, 12881], [1351, 2322, 13195], [1354, 1320, 73572], [1357, 1214, 4191], [1358, 2206, 88798], [1360, 962, 22906], [1362, 2181, 60074], [1364, 735, 53898], [1368, 1192, 8944], [1370, 184, 18707], [1373, 706, 27129], [1376, 1251, 95229], [1379, 369, 89509], [1383, 2231, 32195], [1384, 746, 27214], [1543, 1871, 81072], [1544, 2138, 73489], [1545, 1844, 25397], [1547, 241, 82341], [1549, 2202, 80009], [1554, 2227, 19999], [1556, 330, 41243], [1560, 2237, 53675], [1562, 495, 84818], [1564, 829, 84877], [1567, 1450, 42945], [1569, 361, 38094], [1571, 1967, 74968], [1573, 280, 76348], [1574, 27, 41404], [1575, 1190, 59921], [1576, 1996, 92885], [1786, 2079, 44622], [1788, 1995, 92351], [1790, 1756, 28089], [1791, 1507, 99775], [1796, 1852, 67022], [1801, 226, 33894], [1805, 88, 83421], [1807, 215, 37277], [1808, 328, 89040], [1809, 838, 56072], [1814, 688, 19675], [1819, 947, 15443], [1821, 1383, 6588], [1825, 1621, 13194], [1829, 779, 37876], [1830, 591, 71611], [1832, 7, 98562], [1834, 233, 25398], [1835, 1368, 54325], [1836, 70, 93158], [1838, 1492, 81026], [1841, 1777, 53355], [1986, 1516, 1611], [1989, 207, 99311], [1991, 2406, 89435], [1994, 123, 69074], [1998, 364, 267], [1999, 2035, 95869], [2003, 2459, 79487], [2008, 1288, 73326], [2009, 1268, 86118], [2013, 1380, 95800], [2015, 1474, 33715], [2017, 2296, 51936], [2019, 1638, 53442], [2022, 1710, 54306], [2024, 1282, 70621], [2027, 1094, 32138], [2028, 2151, 9399], [2036, 1394, 8402], [2039, 1020, 32204], [2042, 2276, 71799], [2044, 467, 26041], [2045, 1203, 94055], [2050, 404, 74525], [2052, 2330, 76596], [2240, 876, 82944], [2242, 144, 32981], [2245, 1839, 6457], [2246, 952, 83851], [2254, 715, 8550], [2258, 1939, 6232], [2272, 2139, 9121], [2274, 1099, 20122], [2275, 1859, 45595], [2277, 2374, 6340], [2282, 2325, 26164], [2286, 1510, 94845], [2290, 2452, 68855], [2295, 1919, 169], [5, 738, 81617], [15, 1155, 29846], [20, 654, 77306], [31, 2023, 65562], [32, 848, 27441], [33, 570, 2756], [37, 1971, 31125], [39, 1295, 64202], [42, 1219, 78990], [43, 1983, 55293], [52, 1989, 20818], [143, 750, 56323], [148, 549, 20705], [149, 504, 95923], [151, 1572, 24855], [159, 1818, 52017], [162, 1533, 58914], [167, 1311, 11630], [171, 212, 39960], [173, 1706, 82356], [175, 1698, 11807], [177, 2076, 41374], [180, 1151, 96646], [182, 2106, 45674], [185, 2367, 80440], [191, 1467, 32028], [196, 861, 83916], [205, 2237, 79739], [209, 1762, 1547], [210, 2316, 94539], [217, 114, 82962], [219, 371, 89919], [220, 323, 41101], [383, 1809, 88221], [386, 2100, 64053], [392, 1590, 14696], [395, 1002, 27753], [400, 13, 99994], [402, 247, 11211], [410, 303, 47657], [415, 2161, 88755], [418, 2367, 4378], [420, 1523, 77262], [424, 1404, 67334], [426, 1394, 91988], [428, 833, 3505], [434, 919, 80341], [437, 2110, 76686], [440, 2, 35797], [442, 25, 60716], [444, 439, 76699], [447, 1751, 53458], [452, 501, 61650], [455, 962, 10714], [457, 433, 42608], [464, 727, 14340], [470, 1388, 94547], [474, 423, 26238], [632, 1758, 16938], [639, 939, 26568], [640, 218, 27499], [673, 874, 47533], [677, 352, 46300], [687, 1295, 41583], [691, 1569, 35417], [873, 2023, 67201], [879, 542, 51183], [880, 1517, 62572], [882, 2076, 29894], [887, 477, 74715], [889, 490, 93557], [890, 1876, 61101], [891, 108, 52269], [895, 942, 1983], [898, 1402, 26434], [902, 1764, 2485], [904, 1704, 65894], [909, 434, 25760], [912, 1705, 53921], [914, 2007, 96371], [917, 2305, 3169], [922, 1995, 23248], [928, 1372, 24471], [930, 1432, 31078], [933, 82, 97707], [938, 381, 48482], [945, 405, 11967], [1061, 2015, 30939], [1064, 2073, 11228], [1065, 654, 9713], [1068, 1796, 12169], [1073, 2442, 67465], [1074, 2014, 60172], [1076, 456, 36092], [1077, 1753, 59082], [1080, 746, 84495], [1082, 1691, 63137], [1084, 2061, 47601], [1087, 1995, 81389], [1089, 361, 44549], [1090, 528, 75267], [1097, 2054, 30874], [1098, 2231, 78303], [1102, 1472, 9274], [1105, 902, 97570], [1109, 1319, 20953], [1110, 938, 41873], [1113, 971, 645], [1114, 910, 12841], [1116, 932, 77592], [1118, 799, 25201], [1329, 1625, 34839], [1337, 895, 71954], [1338, 108, 57695], [1342, 2359, 10125], [1345, 339, 98285], [1348, 549, 27341], [1349, 1050, 20797], [1353, 1075, 24027], [1356, 661, 88213], [1359, 1708, 27202], [1361, 1166, 86582], [1366, 1129, 28903], [1367, 2348, 99950], [1375, 1602, 65086], [1378, 2269, 90723], [1385, 682, 36026], [1448, 1367, 27630], [1454, 13, 6199], [1459, 680, 14751], [1463, 203, 10932], [1478, 1628, 82359], [1480, 1632, 73194], [1482, 1085, 65840], [1486, 2008, 93516], [1493, 2027, 14558], [1497, 1724, 6309], [1500, 841, 72131], [1504, 1579, 78534], [1511, 1074, 84288], [1513, 945, 23200], [1522, 474, 27198], [1526, 2067, 32754], [1537, 1494, 62701], [1658, 276, 63473], [1664, 705, 55910], [1665, 1093, 40240], [1669, 1091, 88703], [1675, 1636, 51063], [1679, 602, 28130], [1685, 523, 32079], [1693, 2296, 58368], [1700, 612, 74768], [1713, 182, 42853], [1731, 378, 29772], [1738, 1998, 59751], [1739, 337, 54263], [1746, 1242, 24962], [1755, 1149, 75858], [1767, 2465, 20840], [1775, 1992, 82318], [1780, 21, 64043], [1784, 2301, 47710], [1925, 1228, 56882], [1926, 1461, 52948], [1928, 1217, 17413], [1929, 2009, 64597], [1933, 1225, 51171], [1940, 2240, 50687], [1941, 2426, 19710], [1947, 2022, 27955], [1950, 764, 72158], [1953, 976, 1948], [1955, 82, 39565], [1961, 1270, 41128], [1965, 2440, 57496], [1973, 1951, 30078], [1974, 1183, 717], [1977, 8, 90742], [1979, 1486, 49075], [1982, 483, 15119], [1984, 815, 34513], [2160, 478, 85313], [2167, 908, 54983], [2174, 112, 27552], [2179, 2266, 31867], [2184, 855, 76117], [2191, 1869, 91546], [2192, 851, 39352], [2195, 1020, 49458], [2207, 1358, 59254], [2212, 753, 83935], [2223, 67, 33895], [2230, 615, 79133], [2233, 2231, 6384], [2235, 363, 59020], [2386, 2102, 59876], [2389, 952, 27732], [2396, 1325, 90021], [2412, 1402, 49483], [2418, 177, 92981], [2424, 465, 25787], [2425, 1953, 84024], [2431, 27, 64581], [2434, 2225, 30676], [2439, 2469, 61083], [2446, 2212, 89809], [2458, 83, 18571], [2464, 2007, 18000], [462, 945, 3788], [142, 742, 64471], [146, 1481, 99976], [152, 1737, 47288], [154, 900, 55572], [158, 2299, 28661], [163, 2276, 46241], [164, 1689, 6963], [169, 1705, 26209], [174, 685, 78891], [178, 2063, 83454], [183, 1764, 89172], [189, 1723, 31510], [192, 2344, 11533], [194, 1519, 43805], [197, 1129, 7418], [206, 698, 87236], [213, 2367, 72612], [221, 841, 14805], [223, 922, 72674], [227, 137, 56694], [229, 480, 84258], [331, 721, 63828], [333, 203, 74218], [342, 734, 71140], [348, 1888, 22537], [350, 2171, 81241], [357, 2298, 2722], [358, 788, 95254], [366, 1083, 13924], [370, 913, 86848], [374, 2140, 29485], [544, 2112, 28498], [547, 282, 57668], [555, 150, 72312], [558, 1576, 8353], [566, 2148, 80094], [571, 2267, 19010], [574, 2281, 23261], [579, 1586, 91806], [582, 1282, 88329], [585, 1470, 67849], [589, 1162, 79265], [590, 1413, 1719], [594, 2363, 40343], [597, 122, 54007], [598, 401, 80786], [608, 580, 86004], [611, 1768, 80257], [617, 110, 42615], [623, 2231, 99852], [626, 2103, 35892], [627, 249, 90744], [628, 1069, 13005], [761, 1696, 94129], [767, 324, 9129], [771, 57, 9557], [775, 468, 11722], [777, 2014, 59533], [778, 415, 95802], [790, 1518, 47303], [794, 546, 46280], [798, 2229, 55986], [802, 766, 71946], [806, 3, 92055], [814, 2410, 40894], [825, 260, 59117], [826, 1272, 96183], [839, 496, 8145], [840, 1868, 66958], [845, 304, 18473], [850, 2126, 98455], [853, 164, 5149], [859, 130, 28644], [864, 2396, 24248], [868, 1684, 91417], [1003, 472, 90468], [1006, 842, 76866], [1009, 1244, 32997], [1014, 328, 55792], [1015, 325, 74325], [1018, 2329, 79094], [1025, 522, 74281], [1029, 1391, 91715], [1033, 447, 46284], [1036, 1688, 50351], [1038, 413, 73798], [1041, 783, 31886], [1042, 2421, 10462], [1045, 551, 36172], [1047, 2173, 19125], [1049, 790, 75147], [1051, 935, 69626], [1053, 1632, 69490], [1056, 161, 65538], [1209, 241, 20816], [1216, 1360, 91627], [1226, 1137, 91731], [1248, 606, 10884], [1255, 1665, 35818], [1256, 1677, 98429], [1261, 1631, 63750], [1264, 972, 64894], [1277, 1327, 62455], [1281, 751, 47468], [1283, 205, 7799], [1285, 2010, 80504], [1289, 146, 70371], [1292, 178, 11071], [1293, 562, 37746], [1300, 838, 63913], [1305, 1821, 40383], [1308, 2464, 23962], [1313, 615, 70578], [1317, 1690, 77323], [1318, 1019, 76244], [1326, 2298, 58208], [1386, 1811, 51251], [1392, 2235, 49326], [1395, 2155, 48923], [1397, 1669, 9396], [1403, 336, 63084], [1406, 1015, 73369], [1409, 2248, 5356], [1418, 1707, 98463], [1419, 1444, 26962], [1421, 1288, 88233], [1425, 1130, 34702], [1428, 685, 8586], [1435, 136, 50488], [1437, 1353, 33924], [1441, 1040, 24917], [1584, 1514, 69141], [1591, 1399, 11384], [1593, 416, 73630], [1604, 1742, 12843], [1615, 2397, 48977], [1616, 1682, 92103], [1620, 39, 92134], [1623, 1140, 43147], [1624, 1929, 80709], [1629, 13, 13879], [1633, 1555, 84656], [1635, 2044, 86421], [1642, 1943, 84806], [1648, 676, 14770], [1649, 186, 75906], [1651, 1384, 1610], [1655, 687, 65871], [1847, 1110, 48053], [1853, 464, 23148], [1857, 1111, 965], [1862, 1791, 94178], [1864, 1009, 42139], [1865, 2193, 13692], [1866, 299, 5828], [1873, 42, 98644], [1877, 893, 54255], [1881, 1697, 56078], [1884, 2172, 27111], [1887, 1252, 76528], [1889, 1981, 31703], [1892, 903, 65212], [1895, 1720, 76874], [1897, 2147, 48893], [1899, 1049, 29691], [1900, 2030, 45404], [1911, 1245, 30317], [1917, 682, 97696], [2056, 603, 72489], [2058, 2116, 98625], [2062, 296, 57956], [2066, 345, 38444], [2068, 1827, 86128], [2075, 1793, 60886], [2078, 2149, 45791], [2081, 266, 33909], [2082, 153, 46457], [2089, 812, 10420], [2097, 1815, 25051], [2107, 988, 46145], [2111, 1265, 37194], [2118, 912, 81452], [2121, 1887, 82490], [2129, 216, 75564], [2137, 2425, 78807], [2141, 583, 462], [2143, 1167, 7849], [2144, 668, 42025], [2152, 219, 67515], [2154, 595, 59048], [2304, 1680, 36689], [2307, 754, 45305], [2311, 2328, 30762], [2313, 1640, 55454], [2315, 333, 70198], [2321, 433, 85324], [2323, 1013, 11811], [2327, 1388, 34715], [2331, 1996, 22683], [2332, 1478, 99073], [2333, 904, 47495], [2337, 2344, 16395], [2338, 1181, 98293], [2340, 2072, 18170], [2343, 468, 44600], [2347, 832, 50151], [2350, 2145, 60744], [2352, 2249, 62076], [2355, 2327, 74], [2357, 1378, 9137], [2358, 366, 29697], [2361, 1666, 17369], [2362, 2013, 33481], [2365, 341, 36569], [2368, 2219, 17384], [2373, 2351, 19927], [231, 117, 75941], [240, 2466, 77574], [246, 404, 97439], [252, 1854, 53697], [253, 1964, 11910], [257, 1006, 88202], [259, 1751, 19617], [267, 2329, 4610], [273, 632, 22563], [278, 1126, 10155], [279, 1427, 79133], [295, 2155, 8935], [302, 751, 13737], [309, 1721, 48976], [312, 1244, 35040], [314, 1688, 38542], [317, 1358, 4062], [326, 1869, 98129], [476, 1638, 45685], [479, 856, 34488], [482, 1452, 19412], [485, 1857, 11512], [486, 1058, 59319], [488, 1972, 29391], [492, 1574, 42870], [494, 484, 57712], [502, 2418, 32478], [508, 2083, 90521], [509, 330, 10122], [511, 419, 15752], [513, 1973, 6372], [516, 1165, 14700], [517, 1270, 4921], [521, 2307, 65553], [524, 981, 65415], [529, 1180, 858], [534, 806, 54238], [539, 179, 67282], [540, 2380, 1348], [757, 947, 97941], [759, 1038, 95062], [762, 61, 41201], [769, 1075, 955], [772, 668, 74274], [780, 325, 53493], [787, 1531, 14260], [803, 2144, 58308], [813, 1960, 64378], [817, 2346, 86495], [822, 67, 78630], [827, 1994, 68705], [837, 609, 21107], [843, 1707, 1146], [849, 2021, 10622], [854, 1133, 78832], [860, 2384, 21823], [865, 287, 26654], [870, 818, 19818], [1122, 410, 37214], [1123, 1264, 23850], [1127, 207, 29667], [1136, 1692, 56643], [1138, 779, 262], [1147, 1444, 6909], [1148, 830, 12657], [1152, 424, 66584], [1153, 2095, 61987], [1160, 1272, 24585], [1164, 963, 24894], [1169, 1768, 69014], [1173, 254, 24352], [1175, 127, 42459], [1178, 263, 88046], [1184, 1196, 24880], [1187, 2020, 36163], [1194, 260, 78577], [1195, 888, 10204], [1205, 1262, 75487], [1445, 515, 4979], [1449, 1551, 10694], [1451, 1887, 10740], [1453, 528, 63955], [1455, 96, 40295], [1466, 557, 70913], [1471, 777, 80725], [1473, 962, 70490], [1476, 568, 55600], [1484, 1703, 15106], [1485, 392, 85057], [1489, 1389, 74390], [1499, 2264, 56403], [1503, 57, 62586], [1506, 509, 44459], [1509, 806, 994], [1512, 190, 6535], [1515, 1668, 2264], [1520, 169, 82741], [1525, 2301, 62338], [1535, 1387, 92930], [1538, 1366, 32923], [1787, 629, 52330], [1795, 2379, 45963], [1799, 639, 54814], [1802, 832, 23890], [1804, 628, 24145], [1806, 183, 69], [1810, 552, 85944], [1812, 595, 22376], [1816, 310, 76077], [1820, 229, 75607], [1822, 278, 53632], [1823, 2085, 44180], [1824, 1060, 82577], [1826, 917, 73591], [1828, 2195, 96113], [1831, 1135, 89687], [1837, 729, 88376], [1842, 393, 16920], [2159, 1517, 89834], [2177, 1756, 7785], [2190, 603, 59707], [2194, 1383, 67881], [2199, 6, 19134], [2205, 1060, 31303], [2214, 766, 20726], [2220, 123, 53987], [2226, 2235, 71449], [2239, 1616, 48122], [2383, 1727, 57119], [2385, 1763, 42413], [2388, 1157, 20605], [2391, 1277, 53813], [2400, 1236, 18798], [2402, 1748, 10562], [2405, 33, 26879], [2407, 169, 40546], [2409, 256, 71918], [2415, 171, 64514], [2420, 1702, 54744], [2428, 198, 90505], [2433, 2292, 60800], [2435, 1649, 6593], [2443, 166, 88097], [2447, 507, 5260], [2453, 2472, 29955], [2456, 2208, 420], [2460, 2018, 11745], [2461, 923, 1527], [2467, 876, 51468], [2470, 226, 85617], [624, 2317, 5106], [232, 2004, 1617], [234, 966, 87332], [237, 758, 92812], [239, 496, 69959], [242, 310, 28961], [244, 2213, 29637], [255, 1005, 68302], [258, 84, 98515], [262, 491, 43250], [264, 971, 22302], [265, 724, 65452], [275, 1130, 39670], [277, 1151, 12257], [283, 252, 9888], [286, 1438, 69990], [290, 1957, 45075], [292, 1491, 50863], [300, 868, 63782], [301, 2368, 93868], [307, 534, 89526], [313, 2175, 28540], [319, 431, 55854], [321, 2436, 86123], [637, 454, 89859], [642, 2371, 54933], [644, 1197, 28480], [646, 867, 78300], [650, 289, 86898], [653, 285, 10762], [656, 84, 49926], [658, 757, 22168], [662, 1141, 56400], [666, 1260, 71228], [667, 1576, 90011], [672, 574, 51614], [675, 33, 9524], [681, 1466, 18133], [684, 667, 52396], [689, 2013, 45613], [948, 195, 89525], [950, 1680, 72605], [951, 868, 62971], [955, 1454, 48598], [958, 2074, 62882], [960, 97, 85141], [965, 1461, 48508], [968, 2125, 66773], [970, 480, 46200], [973, 2005, 85145], [975, 926, 57368], [979, 2027, 16635], [983, 1753, 48639], [989, 508, 75460], [992, 2242, 14580], [995, 1533, 72083], [1000, 404, 22635], [1001, 1658, 38523], [1213, 1569, 86232], [1220, 208, 71088], [1227, 399, 6423], [1233, 495, 59313], [1237, 1223, 3898], [1240, 69, 23283], [1247, 1700, 47780], [1263, 1089, 9451], [1266, 674, 88792], [1271, 2194, 55572], [1275, 1921, 98863], [1286, 959, 1877], [1301, 2103, 22218], [1306, 762, 50945], [1307, 1264, 48824], [1324, 303, 64406], [1578, 183, 12204], [1581, 2106, 82447], [1588, 2433, 16978], [1592, 1381, 20708], [1594, 1480, 73424], [1601, 900, 38291], [1603, 562, 63022], [1608, 1234, 27226], [1611, 334, 15981], [1614, 151, 80179], [1619, 1631, 38442], [1626, 1149, 2983], [1634, 2323, 71546], [1641, 1499, 26653], [1644, 1418, 30293], [1646, 883, 73331], [1650, 974, 89370], [1652, 2419, 46090], [1654, 315, 2204], [1843, 1426, 60368], [1846, 1999, 31890], [1849, 2284, 68316], [1850, 1436, 71315], [1856, 281, 89794], [1861, 223, 92873], [1867, 1595, 76376], [1872, 691, 47405], [1875, 467, 98918], [1878, 1077, 74773], [1880, 567, 20622], [1885, 1395, 89776], [1891, 759, 63616], [1898, 322, 10149], [1901, 495, 13547], [1904, 1914, 97768], [1906, 1398, 99894], [1910, 422, 53156], [1913, 2276, 82698], [1918, 1312, 21276], [1920, 92, 59162], [2163, 2297, 32485], [2164, 1213, 24576], [2165, 1355, 90547], [2169, 467, 44605], [2176, 2156, 98743], [2178, 2158, 22124], [2180, 172, 40306], [2182, 1951, 1863], [2185, 2231, 39103], [2187, 2384, 74467], [2188, 1136, 49190], [2196, 927, 1916], [2200, 1143, 44967], [2201, 199, 15327], [2203, 1524, 7855], [2204, 66, 68184], [2210, 1036, 61384], [2218, 965, 2714], [2228, 1265, 24479], [2232, 1008, 56902], [2238, 1287, 86052], [53, 886, 44549], [60, 506, 43712], [64, 1817, 91180], [71, 2304, 56613], [73, 1467, 30292], [75, 783, 78828], [80, 2427, 2970], [91, 827, 44285], [93, 2047, 57283], [95, 1704, 98032], [101, 1641, 48916], [103, 1190, 1113], [106, 1979, 48818], [111, 1234, 39021], [116, 2143, 6059], [120, 666, 11116], [124, 1865, 41184], [129, 1529, 83205], [133, 49, 41878], [141, 6, 6962], [389, 537, 79013], [390, 1982, 64991], [396, 115, 92433], [398, 2096, 68136], [403, 1291, 46290], [406, 68, 9715], [407, 1756, 92878], [412, 1396, 95270], [414, 1349, 34381], [421, 399, 12664], [429, 2445, 16395], [435, 1333, 93618], [441, 1571, 13123], [445, 1598, 57486], [446, 1406, 28623], [451, 2387, 71868], [458, 2273, 89319], [466, 936, 36574], [473, 2419, 79605], [695, 1985, 63608], [700, 1866, 93517], [702, 2197, 5688], [704, 2361, 69525], [708, 1805, 79888], [710, 820, 63307], [712, 588, 72376], [716, 143, 56577], [720, 469, 49804], [728, 2375, 20200], [731, 2406, 88193]]



# node_dict = packupNode(n, wells, pipes)
# node_list = list(node_dict.values())

# group_list = get_groups(node_list)



# Round1 Edge to Fork (or edge)
def get_pipe_key(node1,node2):
    if (type(node1)==str) or (type(node1)==int):
        key = tuple(sorted([node1, node2]))
    else:
        key = tuple(sorted([node1.val, node2.val]))
    return key

def nodeGroup2dict(group,output_format='dict'):
    well_dict = {}
    pipe_dict = {}
    node_dict = {node.val:node for node in group}
    for node in group:
        well_dict[node.val]=node.well_cost
        for adj,cost in node.pipes.items():
            key = get_pipe_key(node.val,adj)
            pipe_dict[key]=cost
    
    if output_format=='dict':
        return node_dict, pipe_dict,None
    elif output_format=='list':
        wells_key_mapping = {x:i+1 for i,x in enumerate(well_dict.keys())}
        wells_key_mapping_reverse = {i:x for x,i in wells_key_mapping.items()}
        wells_list = [well_dict[wells_key_mapping_reverse[i+1]] for i in range(len(wells_key_mapping))]
        
        pipes = [[k[0],k[1],v] for k,v in pipe_dict.items()]
        pipe_list = [[wells_key_mapping[i],wells_key_mapping[j],x] for i,j,x in pipes]
    
        return wells_list,pipe_list,wells_key_mapping_reverse
    
    else:
        return None,None,None
    



In [95]:
n = 2160
wells_list = [16164,30030,19820,27644,84098,28288,66511,70326,16298,44697,23103,62955,11159,14602,29307,98398,37787,66430,30488,48570,73689,8153,34386,80699,33081,86182,91765,56117,99587,47926,64484,45055,70713,42689,46584,86633,88888,86696,89505,98661,53309,64292,15654,53562,5412,83789,76066,60577,98475,94666,60603,6301,70634,15731,91207,64378,90525,71587,42723,56647,65879,36201,20262,97595,39994,63029,98072,6978,53534,76282,50531,94408,25085,8957,23328,85237,19580,65626,41791,97149,85964,3052,82722,3114,10641,3542,9796,87078,31566,68901,9968,22758,59683,23465,69322,47732,71363,57455,85462,22508,9150,8204,74709,84196,2232,60748,86337,12960,35799,40488,63698,52285,67282,17252,28614,71747,75234,30524,81074,58572,77321,4152,49363,21735,70040,83031,45271,50979,50762,34059,52108,19311,16868,4664,47081,95570,1349,46009,20517,98740,86350,6993,47560,16713,61317,74923,86179,66612,36128,5027,67512,9403,21970,31534,31348,93175,39107,4026,89929,78887,20502,74687,56680,67724,83518,93561,19372,47740,83802,39243,68396,52863,64335,61468,63690,1264,77471,68742,69783,56203,23442,95657,74823,6649,30094,91865,87066,27170,49212,41526,82706,53923,32140,31037,80081,12341,98095,64620,47209,33917,98318,42496,49192,25761,88103,82947,36626,75537,88970,54675,54768,14210,81455,28588,73898,47871,15272,90124,46125,88501,5520,48391,92560,35980,26188,38325,97363,11507,77184,88176,26673,91150,1714,62043,45453,33413,255,77675,99874,21939,50557,79315,85479,5434,30923,63993,5392,4715,49549,45366,39422,40675,15632,25622,15429,3609,15280,97656,65743,3751,19773,65308,83107,9845,82593,77154,68741,5907,45311,75216,29384,13027,20779,82099,1100,1026,76317,87359,88375,10870,10678,31226,52699,18537,21275,67934,23362,85891,49656,35546,28982,17491,58730,9585,20839,48913,68340,94769,456,12630,55974,12601,19817,3082,89736,60727,60579,6078,95367,88804,73448,11061,72736,46787,21353,7000,22608,10771,56033,6581,44070,40441,24195,75038,56244,69479,60964,28948,54264,78624,3944,59794,13952,28649,12016,56472,31165,25008,68237,41472,97386,60336,69741,16651,49932,51650,95116,5096,92109,84940,21503,37006,51340,35958,62205,85963,84551,16101,26081,94333,10330,57023,64485,50586,90437,69623,5336,74743,71432,64569,20105,30774,29054,60895,44940,30564,57978,15834,65267,44851,88264,38446,98310,7801,78134,30640,53310,53940,53785,3143,91201,8201,13676,41343,15677,50298,58436,74942,73960,73407,62577,62009,62540,61622,43907,43457,48946,64812,38997,86702,72065,4849,43734,85739,30773,21660,23450,90872,14218,70436,90413,97864,28404,55188,46785,93666,68272,94946,30367,74838,19348,44543,27176,18403,94709,50115,37468,28572,59659,43853,829,79550,10942,71806,20826,63000,1351,94329,65270,2395,77525,80816,46080,97822,33670,58447,54084,18049,25425,5357,43768,33366,22083,78896,12491,89497,5692,1870,22078,20366,10076,30178,69731,39029,96140,59633,2005,84294,50513,65062,33389,28910,69952,45124,75881,94181,89836,76950,92244,983,71898,19880,71148,97038,35002,61435,46001,17193,6824,23372,5829,15586,1223,3098,1889,8335,53780,60336,23794,67614,35362,21279,34158,27720,26172,37155,82114,7937,85470,43478,34642,56853,31512,33082,27712,88863,59113,82780,60257,14719,28596,55297,42326,53963,39768,58144,60746,93408,88281,49362,4862,20875,65853,19588,94119,63076,96846,81104,36348,19770,64603,43561,86706,41596,73235,68593,38466,30355,60801,60891,12843,19027,11811,97540,12947,55100,19145,95924,53188,63974,39911,50264,34126,67641,22727,99925,21713,1772,76721,47250,58199,15323,50494,94110,45056,57043,60578,7249,9924,57584,88979,53380,35411,95459,29116,58932,27588,79399,24268,83447,3429,37256,38081,20553,7336,23036,10073,63230,15966,62558,47083,32410,1823,55248,34044,13794,1189,86639,19058,38363,13553,71867,72845,98041,97436,92485,57201,49310,52891,58949,28941,27740,76532,417,12439,49872,17018,39947,54303,53228,91156,71096,71507,57957,48482,82357,45376,83329,9182,16007,29944,74140,85215,99676,98349,81363,3616,79220,63417,19165,59286,32374,9182,76760,13699,48517,92564,1438,22460,74813,69948,31275,93010,43732,95554,85789,87582,67591,66917,25918,72973,68462,7780,92997,46133,60170,68918,36375,54855,5116,99435,78399,40665,51050,69329,99006,91271,79672,97608,78424,27990,80230,35113,14837,68687,54184,33966,40365,93449,90546,72637,11338,92659,4105,80410,72772,85502,50680,45909,86472,24083,56371,54960,13907,75207,78482,64463,38572,34502,70584,46003,33041,42894,85989,42940,79569,49105,54878,32294,73205,84312,95455,5555,68740,21991,98533,21740,36985,36137,29075,17898,59656,60603,59422,87461,89986,20375,99360,91004,50416,51985,86038,8130,51217,51134,48924,55603,65389,4591,5513,90097,63901,93129,12950,28697,7117,836,1298,16882,58251,86885,63232,86930,43480,57794,66651,69829,39233,77182,3147,6301,62450,87511,21418,85680,360,27968,95934,18190,43097,38379,35090,92226,24758,30505,18385,31352,49027,48230,79657,27928,17664,57996,32796,45923,38880,70688,38312,28219,94261,46503,49339,84262,39560,51596,71572,48527,21546,34294,93222,45117,45015,6798,86811,88290,95870,87322,2012,4591,46154,29870,33726,2828,43787,35040,5888,54252,26831,26979,87603,25176,33281,64155,64776,26877,21077,5754,84495,49070,26197,19587,29459,22824,10144,18163,23351,36091,31891,40404,25520,83051,56829,7812,88699,56416,15328,90650,80456,1367,5839,37033,38517,34998,47344,76059,95723,68327,56044,42991,2956,74883,4109,92072,22604,4754,38884,91590,82391,77217,37264,26460,50478,25944,35931,78166,72430,74437,2363,20761,16215,93651,98742,47699,16731,54151,15162,91717,50497,78450,77716,85308,45426,34364,47868,4873,98108,86873,52,14198,84762,89029,60443,71267,28320,44510,18891,99390,29161,73011,17985,58355,39989,55509,52665,87232,77652,95477,29658,19788,84451,34489,44828,12560,63434,35476,17023,32678,12134,15812,53863,89760,8827,881,87022,39482,33696,78139,30498,86629,64757,88987,51623,15094,40026,29409,86465,81583,59835,76600,27,42477,91011,89600,10098,1254,74624,98620,86685,10936,88527,81295,42548,85334,74073,37624,86953,15471,94585,6020,19438,21113,33134,69139,71614,49253,39527,56063,95636,77807,20441,30975,70777,45716,78715,71223,99566,98714,35413,86961,23305,50642,77489,56074,36855,3483,83612,50033,51467,82360,5233,35954,46083,90410,6178,17283,74815,74952,71020,71622,99969,70970,18991,55659,55093,93827,83657,98070,5769,59484,90830,95179,73416,2158,85096,39936,44113,72666,85724,49260,91932,97909,27996,75864,60331,76250,24714,75462,27355,56771,55778,52300,59629,73417,25645,5462,24633,92056,82084,47187,42417,41069,85297,41219,17456,47940,55002,29440,86587,14048,19246,51497,18913,57279,98760,9368,62157,48925,79479,32931,3072,16648,310,95777,14316,20169,64269,92804,46286,42282,59057,15698,69967,64404,807,90506,26047,37111,93551,16114,33614,96875,26146,80063,77600,57741,26493,83093,76623,97918,70151,94173,64910,41721,88407,73375,77721,30185,61399,16882,68020,13670,59063,28559,60911,39934,11450,77985,38991,56177,75551,93661,34095,66722,34984,22756,79955,72609,94693,92912,60839,56586,34799,93325,70993,96619,61836,36594,90511,3693,70357,65814,40406,52515,38935,90231,37027,49003,11019,58578,20774,20304,17916,75112,97609,77284,44185,83407,21241,56082,64918,85482,78640,54254,95682,40131,37125,75157,15283,22873,57498,74905,55237,54357,15648,23044,90742,46024,39064,83706,42921,86421,88540,39137,85609,86857,2821,37526,83006,85079,66871,58367,69679,51944,31674,43291,15375,72501,39984,44486,1065,48393,9175,17949,63982,36727,33641,27886,30214,82245,75106,28557,54410,51588,91604,99520,18922,55942,4740,50985,22528,66998,54587,89887,2136,32394,58705,113,75478,78332,71604,9906,64805,33345,36045,58848,25555,71445,51287,6177,89742,30731,42386,13522,29936,2637,28916,50209,31970,25676,11269,25891,22849,31546,84760,78600,17653,90634,56077,44256,13288,85648,83261,8593,90286,2139,12584,43382,79715,79515,14829,34577,23244,12905,23028,71610,72014,18789,17907,36183,66597,56049,63277,86736,54748,81765,76211,238,44460,10582,54956,60893,27746,70698,96659,25443,660,36455,39077,65944,34148,83142,48538,63449,44971,13912,49604,91629,61801,66892,95059,40055,96318,42770,97921,46817,47094,319,59286,46298,85921,30391,60748,90108,24666,32300,77109,10863,46055,31702,79311,96177,88208,12326,94580,4698,7142,40396,63896,10750,90997,39237,42100,59674,50431,33486,44507,11804,89615,33357,74984,23986,68830,19957,85030,30468,81437,5331,77620,37263,45460,79766,78888,45896,69022,58874,69348,69692,23008,94971,65068,41129,25261,88462,73304,24925,50368,54823,55466,62898,15323,28727,29951,43558,39683,28712,46293,95938,97705,15681,25802,46685,93238,36683,39345,90342,95474,8605,68425,95770,95472,57466,11676,50796,76799,46410,12212,62098,42362,39768,92085,95855,90004,92713,28518,66786,40707,60356,2144,76204,36553,64492,99818,88394,73782,34635,94318,46210,67909,18998,87344,86997,23856,46819,92402,17462,11016,59367,87487,43084,52145,40988,86841,55242,88693,58545,16521,26649,32812,28593,74728,85538,25667,10241,53360,82251,33656,82528,69931,86908,19375,94643,91582,75095,14129,80210,54741,98798,2832,71350,72058,6457,80041,14913,59429,67951,12396,50639,99613,38229,55830,13028,20450,93318,25689,3575,10553,3256,3167,18172,92770,85675,13443,13793,99796,60523,88004,13489,10208,69944,91748,54491,37793,14780,15078,91802,89716,82957,3204,72011,3970,66867,37190,58146,47343,16194,21317,83613,71983,30650,64099,87300,61987,36782,32421,81875,4853,65294,71099,46175,90707,94009,16637,86346,49337,91913,86735,73558,91062,15949,62737,66176,7789,49679,31197,5072,8914,80971,10130,58578,55452,56525,57608,58217,75624,24789,62497,63977,67628,63488,65073,58992,99588,7513,57653,2275,31952,50531,63308,11855,58608,2265,64418,10065,44690,10069,84459,63443,97436,84354,32255,64094,88836,89617,26790,8333,92151,73469,92620,72774,44026,55584,97649,3132,40896,56463,54665,34680,95157,64509,35466,56981,39266,54812,54135,80770,68708,41171,63440,4972,86711,1750,26836,79328,72293,34123,17619,80000,80808,95671,44904,55763,29876,90366,37497,37468,11007,696,54068,92622,14294,2268,55307,82269,67320,77295,23827,46167,72952,14914,83421,35650,68116,28069,26189,6184,422,4114,88553,20674,7820,82883,31362,85415,14522,20900,36762,69388,11590,38785,6313,46858,4706,95887,28512,41052,39811,26550,47163,85810,98388,46563,35272,78699,90584,95069,93374,15809,11243,12972,59937,90298,43442,61090,7827,41021,45772,58850,16137,53441,79367,17832,25831,25807,85446,61217,98671,17781,86609,42662,37285,15971,20082,44473,33316,22084,29439,97932,57031,6261,89993,25873,10448,39030,99199,98682,30964,12625,52595,5776,61042,11657,68680,96503,23594,19536,88362,94121,53207,9494,73587,78884,7112,92727,23097,80024,87910,23050,53676,48083,60071,10443,52533,65566,23603,63597,94845,94174,15023,26061,72495,96836,31268,69271,84694,5792,91655,29263,34549,82139,78411,77315,85806,20039,91126,33567,86442,12805,72805,28940,40098,73077,69394,41100,14179,48519,38336,12213,70886,72032,9876,27984,55910,81157,6172,37458,79138,87995,81512,23476,60278,95774,64596,24482,43929,56845,22448,79846,80422,33868,57049,43596,48604,14769,9283,76625,67657,63517,963,40201,82817,69518,29011,13251,20160,78109,88334,51953,64604,25113,88985,57144,55850,48583,56229,69862,39538,30943,44021,13434,39982,93191,90687,59362,10191,61572,94704,30240,38756,59327,45057,2131,97305,23958,37588,85879,18388,23212,91983,65802,86222,10178,91335,14604,23447,15139,27244,1959,34651,95494,17036,83719,40043,5405,42668,39266,18033,15310,51511,79604,90916,15301,84589,85484,157,39230,21233,146,12957,21681,79625,86242,8172,40148,65047,7211,43118,97128,63575,5960,10772,38696,66064,86671,42155,20572,91442,16670,66022,95195,76935,35657,83768,69176,50505,89064,43538,52974,79427,2605,90251,99673,98886,80686,61841,9287,66198,53354,46958,62761,3559,31534,38615,92852,56648,15912,50938,75649,88033,42439,89798,69720,67247,39876,18410,71879,72264,46537,43309,9078,59391,5094,28619,41538,91892,13220,29493,56209,96757,93442,66258,31276,52220,70902,76171,90790,73334,44684,33504,84274,91948,76576,1970,4975,11746,46358,52100,15057,23458,61049,69500,97715,79270,97447,50713,16455,21563,2282,43408,51253,1196,4336,67553,24080,87310,19124,32383,25909,96988,76625,73057,77072,87846,38283,32872,76000,62629,65425,17444,37470,92183,85517,83842,38383,98990,15058,40444,97278,94167,11240,67273,90490,97825,19838,75643,7946,65529,21838,65550,87196,47859,11956,32116,54351,92747,72376,58260,21637,7955,15910,33654,82305,86025,84478,95758,12491,12187,30139,713,17009,5053,70554,84144,19579,23917,10098,47772,35594,35530,41534,63826,67632,14485,33174,27079,7013,26291,27122,82832,50444,52197,71113,95999,2688,61746,95378,64394,83431,73225,94607,45809,57258,79570,25660,7633,85519,31487,69204,58027,13513,94210,61493,42257,91208,93368,33203,99995,90763,52971,83135,19450,43415,64526,81778,39083,20496,79241,71628,65276,29926,48475,25940,35004,56346,16508,23621,78484,66000,57529,98525,99245,41716,31506,57518,18988,45946,75617,28335,85128,47291,68623,46395,35132,33806,27555,51346,87729,22640,17982,81363,54706]
pipe_list = [[2,1,47738],[3,1,69703],[4,2,69081],[5,1,56293],[6,3,2639],[7,6,93651],[8,1,40308],[9,1,97376],[10,2,48875],[11,10,694],[12,8,66572],[13,11,46321],[14,6,36345],[15,13,63471],[17,12,94942],[18,6,87698],[19,17,80719],[20,11,38134],[21,12,64972],[22,10,92522],[23,8,25713],[25,17,50590],[26,24,76743],[28,26,19033],[29,9,94495],[31,6,40808],[33,1,13648],[35,13,5519],[36,12,40618],[37,24,16644],[38,33,7549],[39,29,95247],[40,20,57468],[41,11,27141],[42,2,31665],[43,4,48455],[44,36,82641],[45,8,96368],[46,24,38535],[47,26,96961],[48,41,28292],[49,32,49107],[50,36,87950],[51,32,57160],[52,32,88076],[53,10,53621],[54,16,3282],[55,25,46281],[57,41,46504],[58,47,48746],[59,18,35133],[60,18,49749],[62,13,59420],[64,52,2698],[65,11,16455],[66,43,65636],[67,14,66030],[68,9,41250],[69,21,17143],[70,66,10939],[71,29,44964],[72,18,75213],[73,23,1235],[74,3,21010],[75,67,68921],[76,37,64772],[77,25,90872],[78,64,29819],[79,24,65964],[80,30,46318],[81,55,37496],[82,52,43108],[83,15,77389],[84,6,78172],[85,40,44932],[86,23,14545],[89,3,24538],[90,2,7520],[91,70,77155],[92,87,65058],[93,8,78772],[94,11,24940],[95,84,79176],[96,28,69715],[99,87,16516],[100,13,17057],[101,15,9984],[102,72,85281],[103,85,15511],[104,41,12824],[105,75,1326],[107,103,64679],[108,65,13235],[109,51,27413],[111,15,42979],[113,23,60159],[114,70,34095],[115,87,14405],[116,105,76429],[117,16,69239],[119,51,45322],[120,92,2757],[121,1,67728],[122,72,92648],[123,93,61697],[124,121,25364],[125,101,65526],[126,100,14288],[127,12,161],[128,39,71964],[129,104,20571],[130,40,57740],[131,123,10077],[132,58,66745],[133,130,21191],[134,11,27666],[135,57,92987],[136,17,9003],[137,103,94223],[138,106,72434],[139,25,82761],[140,40,1991],[141,17,38342],[142,57,69761],[143,75,6460],[144,14,12673],[145,33,88522],[146,36,53504],[148,113,59196],[149,8,4787],[150,74,58062],[151,136,31266],[153,51,46431],[154,79,40217],[155,81,50378],[156,150,4810],[157,30,80467],[158,110,13236],[159,135,16961],[160,106,94005],[161,142,90646],[162,102,64775],[163,74,74567],[164,8,84807],[165,95,99415],[166,26,22983],[167,65,1762],[168,113,25735],[169,96,33548],[170,25,53207],[171,5,67775],[172,96,49826],[174,98,66774],[175,118,98686],[176,44,77836],[179,71,45198],[180,162,87123],[181,12,42543],[182,171,51479],[183,150,83818],[185,112,9371],[186,72,45219],[187,137,99516],[188,61,5945],[189,103,43412],[190,167,59356],[191,26,17366],[192,38,57357],[193,25,74911],[194,29,52943],[195,152,31880],[196,143,60167],[197,137,14855],[198,12,24514],[199,154,25434],[200,119,9146],[201,181,91277],[202,93,6693],[203,173,86304],[204,17,57091],[205,182,31649],[206,179,55888],[207,170,54751],[208,122,19735],[209,6,73592],[211,87,95219],[214,40,22060],[215,140,3472],[216,31,74728],[217,68,3448],[218,28,13038],[220,93,35816],[221,202,91355],[222,72,72509],[223,181,98110],[225,61,30774],[227,178,67081],[228,31,73147],[229,156,88321],[230,70,86687],[231,160,33551],[232,72,54935],[233,221,50304],[234,127,23810],[235,152,18420],[236,143,1667],[237,55,46491],[238,80,51029],[239,34,64717],[240,58,5680],[241,219,34305],[242,181,37590],[243,227,40027],[245,107,31800],[246,120,34629],[247,166,35415],[248,204,52743],[249,115,96551],[250,49,45172],[251,12,81050],[252,11,4094],[254,140,72002],[255,235,9036],[256,144,32515],[257,129,11344],[258,246,56377],[259,234,90502],[260,207,47420],[261,50,43494],[262,25,10067],[263,235,83063],[264,164,93481],[265,201,72908],[266,108,93373],[267,213,68874],[268,106,77932],[270,40,54520],[271,85,31278],[274,237,81356],[275,272,78070],[276,230,27674],[277,269,31287],[279,34,86270],[280,51,10778],[281,68,66807],[282,224,57808],[284,48,40024],[285,126,3081],[286,102,75351],[287,116,64810],[288,27,58624],[289,155,51289],[290,35,67503],[291,90,60405],[292,259,40824],[293,192,49095],[295,43,44228],[296,72,22973],[297,252,47815],[298,111,6972],[299,282,87106],[300,2,21013],[301,284,22222],[302,269,81363],[303,285,52652],[304,58,2837],[305,137,77565],[306,225,35099],[307,196,59207],[308,242,52539],[309,38,53367],[310,148,47526],[311,24,9929],[313,102,86973],[314,167,33306],[315,237,31619],[316,72,85382],[317,1,81209],[318,112,92832],[319,165,34920],[321,105,28573],[322,47,83278],[323,13,7105],[324,181,79700],[325,141,96797],[326,146,3452],[327,44,9603],[328,227,53644],[329,193,64877],[330,72,19739],[331,248,43738],[332,46,37501],[333,144,5763],[334,322,27063],[335,131,65484],[336,189,19543],[337,107,38191],[338,149,16711],[339,109,85220],[341,145,38159],[342,61,64581],[343,102,64507],[344,77,98071],[345,292,57683],[347,65,32308],[348,163,73756],[349,247,54643],[350,7,40491],[351,92,24975],[352,267,1667],[353,110,13280],[354,40,53146],[356,304,14841],[357,192,2051],[358,90,98837],[359,158,16128],[360,83,13583],[361,12,78213],[362,185,42433],[363,7,70924],[364,321,52640],[365,157,88869],[366,200,15604],[367,85,86935],[368,188,96465],[369,107,82634],[370,103,44055],[371,274,4278],[372,183,36501],[373,76,85982],[374,212,95042],[376,277,85690],[377,295,93838],[379,303,7780],[380,76,75959],[381,48,60177],[382,145,75263],[383,185,85639],[384,54,40294],[385,123,3689],[386,112,87628],[387,340,26246],[388,56,57318],[389,309,21637],[390,55,39003],[391,303,91095],[392,328,99634],[393,76,51738],[394,373,41034],[395,229,4414],[399,374,31217],[400,240,96627],[401,233,54427],[402,170,67406],[403,57,4277],[404,139,72854],[405,181,90056],[407,271,80146],[408,67,90152],[409,109,84805],[410,303,44950],[411,109,84344],[412,97,64967],[413,254,25055],[414,189,28241],[415,13,32540],[416,302,33498],[417,159,33687],[418,43,13063],[419,113,46331],[420,277,34522],[421,23,56060],[422,1,88518],[423,64,45894],[424,105,56806],[425,406,45237],[426,177,53213],[427,336,40294],[428,151,39233],[429,273,9913],[430,83,2711],[431,244,47930],[432,82,56804],[433,339,70380],[434,151,47428],[435,397,58408],[437,178,63691],[438,315,25460],[440,163,36080],[441,384,87674],[442,159,4230],[443,402,72275],[444,24,15251],[445,247,61329],[446,14,85988],[447,241,45145],[449,288,57594],[450,5,5314],[451,226,98412],[452,198,12819],[453,374,33060],[454,300,63634],[455,281,61968],[456,266,25903],[457,62,99440],[458,182,80667],[459,187,71952],[460,104,76460],[462,175,24260],[463,445,93981],[464,457,4644],[465,138,54029],[466,235,8643],[467,68,82687],[468,461,24026],[469,374,56270],[470,62,76533],[471,434,57698],[472,277,63188],[473,360,80622],[474,271,65723],[475,151,3124],[476,316,49386],[477,461,42395],[479,193,20070],[480,252,41201],[481,179,91370],[482,230,43751],[483,90,19323],[484,128,92628],[485,235,68886],[486,429,82236],[487,193,6282],[488,431,86480],[489,247,86265],[490,331,13019],[491,51,58668],[492,273,82346],[493,421,91908],[494,259,52911],[495,308,31314],[496,340,19413],[498,95,18683],[499,289,41754],[502,151,78078],[503,128,1848],[504,384,28264],[505,129,78748],[506,90,76151],[507,502,3109],[508,192,6062],[509,420,91253],[511,235,66255],[512,54,15377],[513,275,10769],[514,458,7405],[515,145,98570],[517,329,17502],[518,208,85548],[519,80,40484],[520,135,24786],[521,423,10020],[522,8,54451],[523,478,23707],[524,53,78503],[525,27,78743],[526,389,68982],[527,450,78713],[528,303,16216],[531,338,62313],[532,222,31141],[533,172,1150],[535,232,64752],[536,285,50055],[537,439,97918],[538,287,51217],[539,45,94916],[540,43,88679],[542,8,79421],[543,253,10734],[544,344,78837],[545,32,11042],[546,167,40097],[547,152,21536],[548,33,99905],[550,449,54249],[551,380,23953],[552,109,30335],[553,165,9461],[554,523,53002],[555,14,25509],[556,254,79738],[557,231,37152],[558,211,71817],[559,87,4160],[560,75,79600],[562,100,8187],[563,255,57211],[564,325,69403],[565,550,78736],[566,498,41397],[567,220,33422],[568,167,87502],[569,132,92005],[570,58,68284],[571,383,55738],[572,290,25217],[574,329,82578],[575,255,37693],[576,406,26625],[577,334,98380],[578,226,68130],[580,430,31140],[581,65,69553],[582,163,31563],[583,314,26914],[585,439,78297],[586,376,80304],[587,404,46238],[588,235,41835],[589,372,58639],[590,28,84602],[591,475,59119],[592,452,13227],[593,384,75181],[594,166,93811],[595,152,19141],[596,495,1424],[597,466,98393],[598,123,1420],[599,115,32040],[600,395,74716],[602,408,82595],[603,471,42257],[604,294,97457],[605,147,57960],[606,355,75169],[607,298,98365],[608,438,22312],[609,372,24494],[610,26,15119],[611,249,59467],[612,303,31778],[613,353,33054],[614,428,62567],[615,380,62925],[616,494,18530],[617,268,99961],[618,537,92316],[619,261,3365],[620,148,32559],[621,475,63311],[622,603,56081],[623,342,16006],[624,29,4251],[625,182,84761],[626,513,34755],[627,342,74061],[628,97,39298],[629,620,63239],[631,201,42477],[632,434,40666],[634,488,16071],[635,72,86174],[636,252,18241],[637,183,98489],[638,309,27877],[639,300,21833],[640,25,81029],[641,61,3167],[642,241,91842],[643,632,5303],[645,165,47441],[646,558,72635],[647,196,46103],[648,412,52037],[649,39,32869],[650,399,73693],[651,215,93185],[653,204,81426],[654,115,24223],[655,260,52297],[656,289,20474],[657,79,10491],[658,280,57472],[659,261,14071],[660,93,87936],[661,613,96358],[662,463,38444],[663,393,81428],[664,145,91754],[665,384,24132],[666,510,21564],[668,563,64259],[669,459,39940],[670,495,38015],[671,531,56138],[672,39,38528],[673,240,44945],[674,220,63263],[675,535,92359],[676,305,63303],[677,76,77924],[678,44,14415],[679,22,52331],[680,515,94601],[681,277,61194],[682,175,4586],[683,521,5107],[684,535,77972],[685,80,68228],[686,603,63261],[687,62,62285],[688,90,90623],[689,19,2768],[690,334,42292],[691,106,33779],[692,169,85958],[693,274,20182],[694,430,3409],[696,5,81886],[697,534,24198],[698,306,60391],[699,298,14315],[700,228,71732],[701,97,29287],[702,20,45209],[703,358,51391],[704,538,56754],[705,675,3603],[706,396,93490],[707,589,87882],[708,389,42108],[709,124,55166],[710,45,62791],[711,163,48816],[712,218,70213],[713,500,52932],[714,563,22528],[715,709,71708],[716,449,70528],[717,119,35825],[719,665,27282],[720,660,78913],[721,669,3139],[722,176,60440],[723,279,19578],[724,242,25143],[725,256,17635],[727,108,74911],[728,304,56756],[729,559,62957],[730,39,45878],[731,499,3702],[732,493,29487],[733,321,86032],[734,652,53090],[735,383,85911],[736,233,69068],[737,585,30551],[738,635,30617],[739,500,24699],[740,231,41786],[741,733,65341],[742,210,37108],[743,688,94163],[744,249,94673],[745,367,24883],[746,502,34614],[747,21,62543],[748,402,11472],[749,168,39351],[750,729,92305],[751,602,38546],[752,516,76925],[753,165,34331],[754,505,35012],[755,515,33498],[756,428,23204],[757,180,28044],[758,727,75663],[759,554,87018],[760,55,55470],[761,542,16784],[762,36,43418],[763,315,2568],[764,601,23657],[765,701,69322],[767,553,20645],[768,126,87884],[769,271,46694],[770,159,61457],[771,100,52759],[773,112,84844],[775,763,27531],[776,141,9311],[777,166,42670],[778,307,49417],[779,484,65631],[780,175,42706],[781,60,84585],[782,92,41056],[783,350,38478],[784,556,65519],[786,150,51338],[787,648,84943],[788,202,94245],[789,238,36996],[791,193,46847],[792,79,59566],[793,722,19543],[794,787,58231],[795,713,8285],[797,265,80191],[798,231,68543],[799,481,60513],[800,78,17680],[801,778,85837],[803,334,33712],[804,185,79568],[805,423,7700],[806,227,49155],[807,188,78208],[808,465,41648],[809,747,30636],[810,400,15958],[811,123,9629],[812,180,18784],[813,251,94720],[814,44,48726],[815,350,48270],[816,195,51647],[817,174,49855],[819,439,6545],[820,11,15576],[821,607,78212],[822,301,86403],[823,23,56978],[824,74,44472],[825,199,81026],[826,518,15012],[827,66,5379],[828,643,34538],[829,68,42480],[830,390,8048],[834,467,17135],[835,728,14232],[837,258,98435],[838,577,95824],[839,97,41643],[840,57,62887],[842,341,11475],[843,664,12734],[844,613,60762],[845,699,32231],[846,686,61072],[847,730,79780],[848,732,44888],[849,529,90164],[850,472,46162],[851,782,94112],[852,119,9043],[853,613,4972],[854,750,96783],[855,365,40184],[857,502,49646],[858,39,17222],[859,798,23868],[860,784,61276],[861,9,40897],[862,760,41287],[863,571,73836],[864,53,65530],[865,424,16293],[866,479,99805],[867,293,60473],[868,862,65601],[869,674,79420],[870,354,28229],[871,642,63588],[872,408,98372],[873,631,22508],[874,679,41230],[875,393,58614],[877,577,12931],[878,306,84479],[879,700,56750],[880,79,2318],[882,405,58900],[883,246,63609],[884,128,92276],[886,615,88133],[887,664,32848],[888,772,12523],[889,354,51311],[890,72,29019],[891,452,67685],[892,765,82525],[893,424,44907],[894,18,82710],[895,691,43138],[897,396,75362],[898,630,96364],[899,44,73716],[900,136,30089],[901,404,15372],[902,566,73757],[903,772,11827],[904,537,55775],[905,577,16541],[906,66,23709],[907,512,38843],[908,215,58204],[909,872,66899],[910,256,50973],[911,757,98187],[912,539,77020],[913,594,93642],[914,838,40473],[915,36,83150],[916,538,82808],[917,268,35853],[918,163,46512],[919,608,52219],[920,239,3520],[922,879,95232],[923,917,13498],[925,287,80473],[926,497,66523],[927,572,31411],[928,488,54378],[929,94,54717],[930,306,83307],[931,269,1435],[932,838,29093],[933,392,75739],[934,845,58673],[935,269,55856],[936,861,7335],[937,503,22353],[938,614,72999],[940,527,74505],[941,880,35681],[942,307,74105],[944,587,72232],[945,758,61310],[946,656,75435],[947,918,84863],[948,760,68144],[950,146,36232],[951,790,54380],[952,195,46530],[953,596,36711],[954,237,58858],[955,480,32670],[956,670,20585],[957,880,36335],[959,204,76156],[960,556,20033],[961,919,77376],[962,432,6679],[963,673,81705],[964,798,29162],[965,63,22017],[966,606,95616],[967,597,33042],[968,59,48279],[969,645,96337],[970,904,36010],[971,852,32376],[972,711,22657],[973,461,87344],[974,265,56119],[975,809,2145],[976,431,3031],[978,172,7500],[979,466,66300],[980,719,89673],[982,868,74146],[985,488,29220],[986,314,67268],[987,648,74787],[988,780,68399],[989,876,89985],[991,319,61022],[992,970,94140],[994,912,8646],[995,237,88493],[996,606,44613],[997,162,68249],[998,802,76992],[999,948,142],[1000,209,75793],[1001,573,59174],[1002,43,13083],[1003,705,39415],[1004,204,23096],[1005,733,66632],[1006,443,34218],[1007,504,22789],[1008,433,4712],[1009,227,87182],[1010,585,75845],[1011,924,45012],[1013,32,49655],[1014,956,34045],[1015,585,99871],[1016,165,23422],[1017,222,70090],[1018,131,4799],[1019,298,10137],[1020,17,62654],[1021,674,59767],[1022,543,38471],[1023,81,90635],[1024,693,40530],[1025,179,76782],[1026,656,50950],[1027,1025,39924],[1028,775,82305],[1029,89,37380],[1030,466,3436],[1031,942,3108],[1032,503,60497],[1033,662,75790],[1034,698,25948],[1035,658,14508],[1036,49,29545],[1037,110,72979],[1038,635,76817],[1039,396,89270],[1040,726,92400],[1041,265,32648],[1042,58,38691],[1043,824,67013],[1044,937,86479],[1045,321,8925],[1046,1005,77773],[1047,1,3867],[1048,639,92435],[1049,511,55761],[1050,290,5780],[1052,314,55563],[1054,777,64571],[1055,169,33274],[1056,395,43129],[1057,380,7643],[1058,3,54680],[1059,369,35536],[1060,765,52963],[1061,620,71888],[1062,844,8313],[1063,77,55561],[1064,246,58025],[1065,573,71217],[1066,953,67196],[1067,850,38652],[1068,404,18157],[1069,688,79465],[1070,398,49372],[1071,811,17834],[1072,113,89264],[1073,950,18764],[1074,249,52315],[1075,508,96676],[1076,205,56823],[1077,69,36052],[1078,535,68233],[1079,209,70548],[1080,10,44244],[1081,322,84832],[1082,445,45100],[1083,211,9332],[1084,707,1318],[1085,1082,81349],[1086,585,32310],[1087,142,26914],[1088,290,1608],[1089,412,16027],[1090,230,17574],[1091,590,3667],[1092,131,94394],[1093,503,25813],[1094,817,6732],[1095,911,34510],[1096,1075,29322],[1098,122,16167],[1099,718,88523],[1100,341,33524],[1101,209,61470],[1102,737,5649],[1103,816,68072],[1104,383,79112],[1105,8,56329],[1106,296,70891],[1107,939,47258],[1108,316,82199],[1109,1057,95941],[1110,734,32426],[1111,205,40310],[1112,96,59012],[1113,637,30644],[1114,918,80458],[1116,934,61261],[1117,160,53114],[1118,920,23929],[1119,1083,42939],[1121,425,52848],[1122,736,87339],[1123,341,13196],[1124,896,43130],[1125,159,79416],[1126,652,49097],[1127,795,51540],[1128,200,25989],[1130,314,55977],[1131,396,62970],[1132,1120,91457],[1133,671,66368],[1134,588,47267],[1135,302,67616],[1137,1055,20153],[1138,604,79764],[1139,1120,9806],[1141,392,25134],[1142,107,67629],[1143,976,44635],[1144,1092,13396],[1145,649,86376],[1146,375,8645],[1147,748,42073],[1149,926,21895],[1150,508,5227],[1151,414,70775],[1152,483,87695],[1154,844,17674],[1155,261,4237],[1156,1037,63782],[1158,181,33619],[1160,930,65791],[1161,612,52644],[1162,179,51195],[1164,617,36372],[1165,1131,3021],[1166,175,96599],[1167,1110,45121],[1168,550,26907],[1169,936,16564],[1170,781,22595],[1171,697,41242],[1172,1042,2872],[1173,929,11288],[1174,1152,64176],[1175,836,64385],[1176,536,62023],[1177,28,65192],[1178,403,49169],[1179,1089,52684],[1180,1003,23285],[1181,245,10709],[1182,467,77569],[1183,869,90969],[1184,66,80805],[1186,52,93905],[1187,132,67779],[1188,1018,3776],[1189,430,42618],[1191,920,92628],[1192,633,71928],[1193,489,81572],[1194,654,86604],[1195,260,97465],[1196,438,46172],[1197,383,94700],[1198,205,54150],[1199,1058,88639],[1200,6,65317],[1201,460,64075],[1202,1147,52026],[1203,392,33745],[1204,171,58141],[1205,213,25994],[1206,1051,93346],[1207,655,64971],[1208,61,70082],[1209,323,53574],[1210,851,23274],[1212,99,79561],[1214,988,25202],[1215,437,52997],[1217,409,53208],[1218,566,75514],[1220,55,18345],[1221,583,39993],[1222,383,43736],[1223,58,89126],[1224,208,87296],[1225,636,26653],[1226,1056,45761],[1227,1169,85475],[1228,1191,1163],[1229,914,10299],[1230,733,38787],[1231,826,31161],[1232,1031,34397],[1233,1176,70306],[1234,614,28861],[1235,186,77236],[1236,445,87377],[1238,270,27954],[1239,919,8108],[1240,305,29752],[1241,1184,11046],[1242,167,62410],[1243,211,28122],[1244,405,9705],[1245,1179,35972],[1246,867,29933],[1247,687,35114],[1248,1211,32361],[1249,488,76533],[1250,980,35087],[1252,325,85518],[1253,233,52883],[1254,758,13974],[1255,252,45837],[1256,42,21241],[1258,237,96111],[1260,489,13272],[1261,622,37102],[1262,165,69520],[1263,105,32796],[1264,643,76341],[1265,230,71128],[1266,1007,5520],[1267,451,3947],[1268,418,25559],[1269,1011,81397],[1272,1140,45941],[1273,287,23685],[1274,437,35396],[1275,546,61773],[1276,691,95390],[1277,301,19217],[1278,850,43158],[1279,539,15041],[1280,928,39737],[1281,645,4852],[1282,24,660],[1283,288,19997],[1284,149,92813],[1285,645,45477],[1286,1196,52710],[1287,1213,33927],[1288,1019,76403],[1289,982,29179],[1290,1097,79488],[1291,407,10519],[1292,453,95212],[1293,689,58343],[1294,786,32397],[1295,954,87198],[1297,33,45087],[1298,130,33919],[1299,438,65874],[1301,891,66486],[1302,660,91062],[1303,324,97722],[1304,977,38274],[1305,332,9115],[1306,275,53315],[1307,475,15180],[1309,77,3618],[1310,15,817],[1311,866,98200],[1312,1275,41215],[1313,453,18003],[1314,774,33472],[1315,829,41664],[1316,727,35477],[1317,900,85457],[1318,836,97807],[1319,270,19983],[1320,57,52272],[1321,50,69480],[1322,640,66374],[1323,985,52237],[1327,602,76940],[1328,1195,60080],[1329,965,35528],[1331,1286,97551],[1332,659,15369],[1333,1236,54747],[1334,672,53675],[1335,1005,42342],[1337,255,22523],[1338,31,85005],[1340,546,66756],[1341,87,75143],[1342,267,97819],[1343,904,14100],[1344,415,64709],[1345,606,27299],[1346,76,83702],[1347,553,82536],[1348,777,9265],[1349,289,60834],[1350,628,34905],[1352,918,7354],[1353,1094,85293],[1354,1152,93585],[1355,1090,9968],[1356,1133,50747],[1357,403,68170],[1358,79,43585],[1359,284,42907],[1360,437,2781],[1361,139,44916],[1362,941,70289],[1363,816,21665],[1364,560,8039],[1365,599,96669],[1366,709,39232],[1367,1342,33114],[1368,520,87813],[1369,1099,52778],[1370,1360,27651],[1371,1333,18274],[1372,761,98168],[1373,78,56787],[1374,1137,46946],[1375,441,74025],[1376,1,82388],[1377,809,50555],[1378,578,71906],[1379,1109,46953],[1380,872,63656],[1381,265,36121],[1382,628,1836],[1384,1362,89530],[1385,466,60437],[1386,174,70709],[1387,212,35857],[1388,714,85894],[1390,79,41959],[1391,956,15072],[1392,1336,72179],[1393,943,49543],[1394,742,71990],[1395,33,80564],[1396,395,72409],[1397,272,33509],[1398,374,20935],[1399,1189,54118],[1400,156,48112],[1401,641,77927],[1402,708,35456],[1403,932,36480],[1404,363,6770],[1405,720,6354],[1406,955,52845],[1407,1184,24434],[1408,524,3214],[1411,252,9090],[1412,399,86243],[1414,1303,83643],[1415,972,8545],[1416,124,11194],[1417,802,93130],[1418,463,4767],[1419,815,54600],[1420,709,78688],[1421,903,44337],[1422,206,34953],[1423,1153,3977],[1424,524,36328],[1425,1380,5537],[1426,1323,35303],[1427,698,1750],[1429,405,95961],[1430,1370,50811],[1431,698,27110],[1432,1144,3364],[1433,634,93223],[1434,510,51984],[1435,1384,34399],[1436,955,13667],[1437,114,89895],[1438,172,87909],[1439,684,41386],[1440,578,73680],[1442,94,11570],[1443,1125,72975],[1444,693,90644],[1445,504,19217],[1446,438,6772],[1447,948,1716],[1449,471,38994],[1450,1348,44737],[1451,635,31484],[1452,343,66948],[1453,1229,54028],[1454,1155,53788],[1455,922,79470],[1456,915,10071],[1457,763,28453],[1458,242,30270],[1459,960,16786],[1460,504,88039],[1461,744,504],[1462,766,3757],[1463,399,46535],[1465,1369,61628],[1466,740,29576],[1467,970,69569],[1469,209,1311],[1470,1211,3423],[1471,203,48105],[1472,1423,4041],[1473,1187,10978],[1474,484,91630],[1475,1095,57861],[1476,993,60730],[1478,571,91846],[1479,978,98183],[1480,866,34749],[1482,881,25855],[1484,818,51154],[1485,336,24533],[1486,872,42933],[1487,183,53844],[1488,1033,20603],[1489,836,30269],[1490,445,50496],[1491,466,17030],[1492,935,11874],[1493,65,63073],[1494,659,46273],[1495,1269,11792],[1496,1283,76052],[1497,378,32385],[1498,1004,37938],[1499,1149,45061],[1500,243,89111],[1501,575,58758],[1502,351,50918],[1503,226,51596],[1504,638,40033],[1505,114,43023],[1506,382,29954],[1507,1467,18022],[1508,933,21304],[1509,64,49319],[1510,48,17495],[1512,878,26799],[1513,1051,59080],[1514,772,66307],[1515,650,150],[1516,1061,89686],[1517,1503,70284],[1518,428,21399],[1519,661,67953],[1520,360,98896],[1521,899,73839],[1523,607,57529],[1524,1110,49128],[1525,1499,13992],[1526,653,27492],[1527,1117,3176],[1528,1163,88320],[1529,1099,35055],[1530,446,77111],[1531,1297,87283],[1533,867,94366],[1534,336,58193],[1535,749,37723],[1536,881,95531],[1537,1263,31636],[1538,871,35956],[1539,973,83388],[1541,929,98098],[1542,1083,37605],[1543,703,71199],[1545,928,41847],[1546,52,36598],[1548,1507,68221],[1549,102,77115],[1550,360,18406],[1551,579,87917],[1552,816,8634],[1553,1257,27475],[1555,963,68216],[1556,518,31441],[1557,1208,18306],[1558,42,44602],[1559,1268,89670],[1560,344,76015],[1562,1099,60327],[1563,270,62330],[1564,264,1120],[1565,415,35580],[1566,907,18628],[1567,361,69036],[1568,220,94453],[1569,545,17955],[1570,570,66222],[1571,1067,15781],[1572,316,70691],[1573,599,70389],[1574,197,5458],[1576,329,83640],[1577,1233,60069],[1578,329,73243],[1579,1381,16601],[1580,504,36080],[1581,994,97818],[1582,778,71028],[1583,350,72599],[1585,489,31987],[1586,226,17809],[1588,509,49616],[1589,898,45742],[1590,164,27950],[1591,410,70390],[1592,462,70782],[1593,713,62525],[1594,577,49797],[1595,314,65903],[1596,1005,41782],[1597,1211,87593],[1598,514,37249],[1600,1400,47324],[1603,1217,68404],[1605,63,75344],[1606,197,54693],[1607,1254,12588],[1608,1599,77541],[1609,886,95297],[1610,1294,19267],[1611,219,77046],[1612,154,12553],[1613,1611,86957],[1614,488,81143],[1615,789,22585],[1616,1424,40659],[1617,1418,62971],[1618,1116,44672],[1619,1345,89108],[1620,479,35787],[1621,33,57663],[1622,1285,99990],[1624,511,19013],[1625,639,69512],[1626,107,86827],[1627,248,29520],[1628,285,27437],[1629,328,90067],[1630,541,37094],[1631,1345,41551],[1632,1285,1243],[1633,700,4791],[1634,538,59331],[1636,1032,36226],[1637,902,40634],[1638,627,29731],[1639,232,88111],[1640,449,75540],[1641,870,11970],[1642,1531,45912],[1643,960,61321],[1644,1010,71936],[1645,1262,56954],[1646,646,14393],[1647,1002,17946],[1648,1104,30771],[1649,1258,12721],[1650,783,37617],[1651,1116,86656],[1653,126,31037],[1654,1358,63417],[1655,44,34092],[1656,444,63915],[1657,243,23211],[1658,1133,90280],[1659,986,13253],[1661,446,74129],[1662,1419,93641],[1663,1501,42603],[1664,816,67618],[1665,1441,68390],[1666,1577,10125],[1668,827,93452],[1669,525,74127],[1670,1451,98405],[1671,343,67496],[1672,249,31842],[1673,911,87353],[1674,819,38218],[1675,1421,96335],[1676,1211,52690],[1677,429,71453],[1679,1513,91953],[1680,1452,96333],[1681,1217,21385],[1682,381,85010],[1683,357,76476],[1685,672,51761],[1686,799,27174],[1687,1272,75980],[1688,1316,95566],[1689,163,51804],[1690,268,27317],[1691,94,51345],[1692,1162,96221],[1693,1192,77713],[1694,495,6831],[1695,1624,82116],[1696,1274,3122],[1697,814,22576],[1698,1621,74345],[1699,1448,82232],[1700,447,3721],[1701,1407,90124],[1702,1324,45295],[1703,478,67666],[1704,1376,38472],[1705,1388,48044],[1706,490,92693],[1707,406,7171],[1708,1574,45952],[1709,847,70688],[1711,1438,43503],[1712,1387,7391],[1713,217,51541],[1714,108,94905],[1715,534,13561],[1716,1043,15883],[1717,49,4245],[1718,449,45025],[1719,1400,23033],[1720,385,23506],[1721,751,70620],[1723,1630,8951],[1724,565,68976],[1725,838,32234],[1726,436,84317],[1727,1161,47022],[1728,1461,28446],[1729,1477,1843],[1730,145,94685],[1732,543,33720],[1733,440,14736],[1734,1078,87857],[1735,1037,69408],[1737,630,22769],[1738,582,54720],[1739,1236,74201],[1740,231,96176],[1741,1197,73876],[1742,860,3916],[1743,537,29597],[1744,1139,92725],[1745,560,84200],[1746,687,2396],[1748,844,47005],[1749,107,39114],[1750,1120,83636],[1751,1180,22493],[1752,1169,54776],[1753,812,43142],[1754,118,92843],[1755,1551,88044],[1756,29,17750],[1757,1121,54955],[1758,74,83722],[1759,1626,85432],[1760,1654,75717],[1761,500,57458],[1762,34,14623],[1763,1396,43277],[1765,632,3048],[1766,1728,92011],[1767,263,74902],[1768,297,97870],[1769,459,69166],[1770,1692,47413],[1772,775,48526],[1776,1729,96141],[1777,1731,97984],[1778,818,78459],[1779,191,67375],[1780,1754,60017],[1781,153,73187],[1782,1511,41056],[1783,62,65831],[1784,462,63355],[1785,1213,31035],[1786,319,40700],[1787,1327,58985],[1788,167,13144],[1789,701,35706],[1790,1313,51676],[1791,1136,79675],[1792,1009,64676],[1793,1085,75851],[1794,981,77892],[1795,1186,99223],[1796,598,71713],[1797,41,39052],[1798,24,87352],[1799,1049,42673],[1800,566,8199],[1801,1511,1034],[1802,1710,73262],[1803,1371,57963],[1804,963,92321],[1805,1799,78675],[1806,844,53131],[1807,511,29487],[1808,700,6131],[1809,861,36393],[1810,933,65680],[1811,348,39805],[1813,1787,78922],[1814,1342,42262],[1815,1196,48537],[1816,1543,73324],[1817,599,14142],[1818,843,99035],[1819,950,6640],[1820,412,50055],[1822,94,89518],[1823,981,53656],[1824,436,18956],[1825,1132,91593],[1826,1144,58621],[1827,1501,87246],[1828,1773,6896],[1829,257,77524],[1830,1154,26458],[1831,1572,55352],[1832,68,55989],[1833,1283,17113],[1834,276,64939],[1835,460,99772],[1836,1657,31554],[1837,928,4852],[1838,110,86777],[1839,1550,45827],[1840,1694,76198],[1841,509,88927],[1842,20,93959],[1843,1040,75774],[1844,325,91123],[1845,1485,49970],[1846,695,64416],[1847,1032,99611],[1848,1498,57282],[1849,1814,51651],[1850,924,65723],[1851,719,919],[1852,1022,44442],[1853,993,22161],[1854,928,18480],[1855,242,68521],[1856,1563,34317],[1857,1353,58239],[1859,665,95723],[1860,1565,44461],[1861,1466,30587],[1862,1482,8575],[1863,723,50078],[1864,698,67924],[1865,1574,46831],[1866,129,87630],[1867,1473,73973],[1868,38,27159],[1869,1031,54410],[1871,582,87766],[1872,125,14295],[1873,554,16627],[1874,454,81563],[1875,252,29442],[1877,1237,62208],[1878,291,25696],[1879,1711,85100],[1880,1451,63006],[1881,405,96915],[1882,70,28853],[1884,1290,89445],[1885,1008,81169],[1886,1593,88575],[1887,1778,72240],[1888,1862,97522],[1889,1003,36783],[1890,1401,89958],[1891,939,49036],[1892,408,76540],[1893,1090,44045],[1895,1891,50184],[1896,1222,64981],[1897,962,60708],[1898,503,85364],[1899,472,70614],[1900,1890,88949],[1901,1849,58622],[1902,1133,51842],[1903,1291,15884],[1904,995,47988],[1905,864,94559],[1906,814,5637],[1907,297,52716],[1909,719,16146],[1910,4,61328],[1911,974,17770],[1912,525,38142],[1914,544,63974],[1915,790,34008],[1916,819,25758],[1917,190,81587],[1918,1295,47197],[1919,950,58540],[1921,316,74863],[1922,205,69432],[1923,1257,89666],[1924,1246,48622],[1925,284,91059],[1926,925,62645],[1927,1768,42437],[1928,142,49868],[1929,1565,77729],[1930,966,53905],[1931,1040,12986],[1932,450,42940],[1933,266,24792],[1934,961,15048],[1935,1418,36956],[1936,681,43429],[1939,26,12746],[1940,305,2675],[1943,1521,7465],[1944,1173,89692],[1945,843,27510],[1946,1106,533],[1947,270,77018],[1948,653,37992],[1949,1587,63235],[1951,43,66153],[1952,1559,63320],[1953,27,62204],[1954,1548,20215],[1955,742,52652],[1956,1151,75422],[1957,155,67678],[1958,1290,27941],[1959,176,1664],[1960,608,91306],[1961,901,44618],[1962,498,83362],[1963,1738,5425],[1965,264,18282],[1966,776,23375],[1967,635,70263],[1969,1388,51297],[1970,355,75499],[1971,888,78671],[1972,1502,75996],[1973,1102,77400],[1974,1073,13914],[1975,1072,18435],[1976,880,70359],[1977,998,98449],[1979,1523,92449],[1980,1428,98608],[1981,1866,92788],[1982,44,46856],[1984,1292,53419],[1985,1677,99922],[1986,1493,81544],[1987,70,48803],[1988,355,493],[1989,298,58494],[1990,437,76779],[1991,1926,52973],[1992,1008,43855],[1993,658,91051],[1994,926,69172],[1995,96,47836],[1996,399,99754],[1997,1610,71949],[1999,911,29650],[2002,1139,26260],[2003,1080,55828],[2004,337,38029],[2005,1591,83167],[2006,1285,36454],[2007,302,80646],[2008,1539,36943],[2009,528,66512],[2010,997,11298],[2011,1798,17912],[2012,1438,68428],[2013,559,16773],[2014,1853,10898],[2015,814,26628],[2016,12,14423],[2017,1770,95350],[2018,457,17296],[2019,816,34057],[2020,551,42332],[2021,1065,87948],[2022,1896,19232],[2024,725,48859],[2025,809,14971],[2026,1191,12510],[2027,1405,78781],[2028,1379,91749],[2029,2006,71425],[2030,644,32861],[2031,567,15118],[2032,513,22761],[2033,37,3972],[2034,531,40401],[2035,530,72439],[2036,860,17762],[2037,1221,4469],[2038,651,20414],[2039,1738,58964],[2040,720,10878],[2042,1924,3185],[2043,173,8734],[2044,27,46085],[2045,356,52285],[2046,106,90049],[2047,912,63547],[2048,548,24461],[2049,505,21897],[2050,713,95382],[2052,750,22527],[2053,447,46162],[2054,782,3186],[2055,1278,50925],[2056,956,68141],[2057,937,64222],[2058,1423,42756],[2060,1964,11746],[2061,804,21523],[2062,1251,31787],[2063,177,73889],[2064,608,89778],[2065,1144,36796],[2066,1767,16549],[2067,768,94389],[2068,459,92382],[2069,1142,48215],[2070,792,71644],[2072,168,7480],[2073,493,86608],[2074,230,29451],[2075,1712,4294],[2076,1619,66052],[2078,1512,34114],[2079,2014,34904],[2080,1865,11824],[2081,1403,11008],[2082,1797,24480],[2083,1699,31825],[2084,728,15569],[2085,1466,69270],[2086,1556,39695],[2087,1072,34681],[2088,1086,12274],[2089,1499,55968],[2090,1056,9137],[2091,396,64954],[2092,252,272],[2093,2089,10506],[2094,49,47608],[2095,179,5018],[2096,393,46944],[2097,740,32439],[2098,1584,2626],[2099,253,5671],[2100,1832,29061],[2102,830,66331],[2103,2077,94687],[2106,1080,81603],[2108,942,50145],[2109,181,84873],[2110,1992,90522],[2111,1434,48181],[2112,268,74161],[2113,235,13932],[2114,90,7279],[2115,84,66175],[2116,204,2935],[2117,816,37847],[2119,646,18610],[2120,1418,43803],[2121,473,40777],[2122,1014,34638],[2123,267,51479],[2124,1658,83842],[2125,731,47842],[2126,317,90276],[2127,1457,24267],[2128,1187,59737],[2129,482,2613],[2130,1652,62105],[2131,1558,86528],[2132,1479,94626],[2133,1306,44928],[2134,909,2115],[2135,686,1681],[2136,185,32278],[2138,2056,24445],[2139,791,91909],[2140,426,87245],[2141,1772,44133],[2142,1266,94485],[2143,1806,32702],[2144,1899,68142],[2145,701,17475],[2146,770,60592],[2147,682,41587],[2148,1996,40216],[2149,1464,95782],[2150,922,26684],[2152,450,43015],[2153,1619,28017],[2154,2061,64558],[2155,330,90257],[2156,991,15599],[2157,1632,35510],[2158,462,54597],[2160,345,12748]]



s = Solution()    
res = s.minCostToSupplyWater(n, wells_list, pipe_list)
print(res)



n= 2160
wells_list= [38081, 9906, 71867, 48475, 44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 95924, 42417, 90525, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 40675, 34549, 25622, 50209, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 66258, 66998, 86025, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 34386, 98070, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19

In [80]:
[len(x) for x in res if len(x)>100]

[140, 102]

In [5]:
def translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping):
    wells_dict_new = {}
    for i,x in enumerate(wells):
        i=i+1
        wells_dict_new[wells_mapping[i]]=x

    pipes_new = [[wells_mapping[x[0]],wells_mapping[x[1]],x[2]] for x in pipes]

    well_status_new = [wells_mapping[i] for i in well_status]
    pipe_status_new = [tuple(sorted([wells_mapping[x[0]],wells_mapping[x[1]]])) for x in pipe_status]
    
    return n,wells_dict_new,pipes_new,well_status_new,pipe_status_new

In [ ]:
# Base
n = 2160
wells_list = [16164,30030,19820,27644,84098,28288,66511,70326,16298,44697,23103,62955,11159,14602,29307,98398,37787,66430,30488,48570,73689,8153,34386,80699,33081,86182,91765,56117,99587,47926,64484,45055,70713,42689,46584,86633,88888,86696,89505,98661,53309,64292,15654,53562,5412,83789,76066,60577,98475,94666,60603,6301,70634,15731,91207,64378,90525,71587,42723,56647,65879,36201,20262,97595,39994,63029,98072,6978,53534,76282,50531,94408,25085,8957,23328,85237,19580,65626,41791,97149,85964,3052,82722,3114,10641,3542,9796,87078,31566,68901,9968,22758,59683,23465,69322,47732,71363,57455,85462,22508,9150,8204,74709,84196,2232,60748,86337,12960,35799,40488,63698,52285,67282,17252,28614,71747,75234,30524,81074,58572,77321,4152,49363,21735,70040,83031,45271,50979,50762,34059,52108,19311,16868,4664,47081,95570,1349,46009,20517,98740,86350,6993,47560,16713,61317,74923,86179,66612,36128,5027,67512,9403,21970,31534,31348,93175,39107,4026,89929,78887,20502,74687,56680,67724,83518,93561,19372,47740,83802,39243,68396,52863,64335,61468,63690,1264,77471,68742,69783,56203,23442,95657,74823,6649,30094,91865,87066,27170,49212,41526,82706,53923,32140,31037,80081,12341,98095,64620,47209,33917,98318,42496,49192,25761,88103,82947,36626,75537,88970,54675,54768,14210,81455,28588,73898,47871,15272,90124,46125,88501,5520,48391,92560,35980,26188,38325,97363,11507,77184,88176,26673,91150,1714,62043,45453,33413,255,77675,99874,21939,50557,79315,85479,5434,30923,63993,5392,4715,49549,45366,39422,40675,15632,25622,15429,3609,15280,97656,65743,3751,19773,65308,83107,9845,82593,77154,68741,5907,45311,75216,29384,13027,20779,82099,1100,1026,76317,87359,88375,10870,10678,31226,52699,18537,21275,67934,23362,85891,49656,35546,28982,17491,58730,9585,20839,48913,68340,94769,456,12630,55974,12601,19817,3082,89736,60727,60579,6078,95367,88804,73448,11061,72736,46787,21353,7000,22608,10771,56033,6581,44070,40441,24195,75038,56244,69479,60964,28948,54264,78624,3944,59794,13952,28649,12016,56472,31165,25008,68237,41472,97386,60336,69741,16651,49932,51650,95116,5096,92109,84940,21503,37006,51340,35958,62205,85963,84551,16101,26081,94333,10330,57023,64485,50586,90437,69623,5336,74743,71432,64569,20105,30774,29054,60895,44940,30564,57978,15834,65267,44851,88264,38446,98310,7801,78134,30640,53310,53940,53785,3143,91201,8201,13676,41343,15677,50298,58436,74942,73960,73407,62577,62009,62540,61622,43907,43457,48946,64812,38997,86702,72065,4849,43734,85739,30773,21660,23450,90872,14218,70436,90413,97864,28404,55188,46785,93666,68272,94946,30367,74838,19348,44543,27176,18403,94709,50115,37468,28572,59659,43853,829,79550,10942,71806,20826,63000,1351,94329,65270,2395,77525,80816,46080,97822,33670,58447,54084,18049,25425,5357,43768,33366,22083,78896,12491,89497,5692,1870,22078,20366,10076,30178,69731,39029,96140,59633,2005,84294,50513,65062,33389,28910,69952,45124,75881,94181,89836,76950,92244,983,71898,19880,71148,97038,35002,61435,46001,17193,6824,23372,5829,15586,1223,3098,1889,8335,53780,60336,23794,67614,35362,21279,34158,27720,26172,37155,82114,7937,85470,43478,34642,56853,31512,33082,27712,88863,59113,82780,60257,14719,28596,55297,42326,53963,39768,58144,60746,93408,88281,49362,4862,20875,65853,19588,94119,63076,96846,81104,36348,19770,64603,43561,86706,41596,73235,68593,38466,30355,60801,60891,12843,19027,11811,97540,12947,55100,19145,95924,53188,63974,39911,50264,34126,67641,22727,99925,21713,1772,76721,47250,58199,15323,50494,94110,45056,57043,60578,7249,9924,57584,88979,53380,35411,95459,29116,58932,27588,79399,24268,83447,3429,37256,38081,20553,7336,23036,10073,63230,15966,62558,47083,32410,1823,55248,34044,13794,1189,86639,19058,38363,13553,71867,72845,98041,97436,92485,57201,49310,52891,58949,28941,27740,76532,417,12439,49872,17018,39947,54303,53228,91156,71096,71507,57957,48482,82357,45376,83329,9182,16007,29944,74140,85215,99676,98349,81363,3616,79220,63417,19165,59286,32374,9182,76760,13699,48517,92564,1438,22460,74813,69948,31275,93010,43732,95554,85789,87582,67591,66917,25918,72973,68462,7780,92997,46133,60170,68918,36375,54855,5116,99435,78399,40665,51050,69329,99006,91271,79672,97608,78424,27990,80230,35113,14837,68687,54184,33966,40365,93449,90546,72637,11338,92659,4105,80410,72772,85502,50680,45909,86472,24083,56371,54960,13907,75207,78482,64463,38572,34502,70584,46003,33041,42894,85989,42940,79569,49105,54878,32294,73205,84312,95455,5555,68740,21991,98533,21740,36985,36137,29075,17898,59656,60603,59422,87461,89986,20375,99360,91004,50416,51985,86038,8130,51217,51134,48924,55603,65389,4591,5513,90097,63901,93129,12950,28697,7117,836,1298,16882,58251,86885,63232,86930,43480,57794,66651,69829,39233,77182,3147,6301,62450,87511,21418,85680,360,27968,95934,18190,43097,38379,35090,92226,24758,30505,18385,31352,49027,48230,79657,27928,17664,57996,32796,45923,38880,70688,38312,28219,94261,46503,49339,84262,39560,51596,71572,48527,21546,34294,93222,45117,45015,6798,86811,88290,95870,87322,2012,4591,46154,29870,33726,2828,43787,35040,5888,54252,26831,26979,87603,25176,33281,64155,64776,26877,21077,5754,84495,49070,26197,19587,29459,22824,10144,18163,23351,36091,31891,40404,25520,83051,56829,7812,88699,56416,15328,90650,80456,1367,5839,37033,38517,34998,47344,76059,95723,68327,56044,42991,2956,74883,4109,92072,22604,4754,38884,91590,82391,77217,37264,26460,50478,25944,35931,78166,72430,74437,2363,20761,16215,93651,98742,47699,16731,54151,15162,91717,50497,78450,77716,85308,45426,34364,47868,4873,98108,86873,52,14198,84762,89029,60443,71267,28320,44510,18891,99390,29161,73011,17985,58355,39989,55509,52665,87232,77652,95477,29658,19788,84451,34489,44828,12560,63434,35476,17023,32678,12134,15812,53863,89760,8827,881,87022,39482,33696,78139,30498,86629,64757,88987,51623,15094,40026,29409,86465,81583,59835,76600,27,42477,91011,89600,10098,1254,74624,98620,86685,10936,88527,81295,42548,85334,74073,37624,86953,15471,94585,6020,19438,21113,33134,69139,71614,49253,39527,56063,95636,77807,20441,30975,70777,45716,78715,71223,99566,98714,35413,86961,23305,50642,77489,56074,36855,3483,83612,50033,51467,82360,5233,35954,46083,90410,6178,17283,74815,74952,71020,71622,99969,70970,18991,55659,55093,93827,83657,98070,5769,59484,90830,95179,73416,2158,85096,39936,44113,72666,85724,49260,91932,97909,27996,75864,60331,76250,24714,75462,27355,56771,55778,52300,59629,73417,25645,5462,24633,92056,82084,47187,42417,41069,85297,41219,17456,47940,55002,29440,86587,14048,19246,51497,18913,57279,98760,9368,62157,48925,79479,32931,3072,16648,310,95777,14316,20169,64269,92804,46286,42282,59057,15698,69967,64404,807,90506,26047,37111,93551,16114,33614,96875,26146,80063,77600,57741,26493,83093,76623,97918,70151,94173,64910,41721,88407,73375,77721,30185,61399,16882,68020,13670,59063,28559,60911,39934,11450,77985,38991,56177,75551,93661,34095,66722,34984,22756,79955,72609,94693,92912,60839,56586,34799,93325,70993,96619,61836,36594,90511,3693,70357,65814,40406,52515,38935,90231,37027,49003,11019,58578,20774,20304,17916,75112,97609,77284,44185,83407,21241,56082,64918,85482,78640,54254,95682,40131,37125,75157,15283,22873,57498,74905,55237,54357,15648,23044,90742,46024,39064,83706,42921,86421,88540,39137,85609,86857,2821,37526,83006,85079,66871,58367,69679,51944,31674,43291,15375,72501,39984,44486,1065,48393,9175,17949,63982,36727,33641,27886,30214,82245,75106,28557,54410,51588,91604,99520,18922,55942,4740,50985,22528,66998,54587,89887,2136,32394,58705,113,75478,78332,71604,9906,64805,33345,36045,58848,25555,71445,51287,6177,89742,30731,42386,13522,29936,2637,28916,50209,31970,25676,11269,25891,22849,31546,84760,78600,17653,90634,56077,44256,13288,85648,83261,8593,90286,2139,12584,43382,79715,79515,14829,34577,23244,12905,23028,71610,72014,18789,17907,36183,66597,56049,63277,86736,54748,81765,76211,238,44460,10582,54956,60893,27746,70698,96659,25443,660,36455,39077,65944,34148,83142,48538,63449,44971,13912,49604,91629,61801,66892,95059,40055,96318,42770,97921,46817,47094,319,59286,46298,85921,30391,60748,90108,24666,32300,77109,10863,46055,31702,79311,96177,88208,12326,94580,4698,7142,40396,63896,10750,90997,39237,42100,59674,50431,33486,44507,11804,89615,33357,74984,23986,68830,19957,85030,30468,81437,5331,77620,37263,45460,79766,78888,45896,69022,58874,69348,69692,23008,94971,65068,41129,25261,88462,73304,24925,50368,54823,55466,62898,15323,28727,29951,43558,39683,28712,46293,95938,97705,15681,25802,46685,93238,36683,39345,90342,95474,8605,68425,95770,95472,57466,11676,50796,76799,46410,12212,62098,42362,39768,92085,95855,90004,92713,28518,66786,40707,60356,2144,76204,36553,64492,99818,88394,73782,34635,94318,46210,67909,18998,87344,86997,23856,46819,92402,17462,11016,59367,87487,43084,52145,40988,86841,55242,88693,58545,16521,26649,32812,28593,74728,85538,25667,10241,53360,82251,33656,82528,69931,86908,19375,94643,91582,75095,14129,80210,54741,98798,2832,71350,72058,6457,80041,14913,59429,67951,12396,50639,99613,38229,55830,13028,20450,93318,25689,3575,10553,3256,3167,18172,92770,85675,13443,13793,99796,60523,88004,13489,10208,69944,91748,54491,37793,14780,15078,91802,89716,82957,3204,72011,3970,66867,37190,58146,47343,16194,21317,83613,71983,30650,64099,87300,61987,36782,32421,81875,4853,65294,71099,46175,90707,94009,16637,86346,49337,91913,86735,73558,91062,15949,62737,66176,7789,49679,31197,5072,8914,80971,10130,58578,55452,56525,57608,58217,75624,24789,62497,63977,67628,63488,65073,58992,99588,7513,57653,2275,31952,50531,63308,11855,58608,2265,64418,10065,44690,10069,84459,63443,97436,84354,32255,64094,88836,89617,26790,8333,92151,73469,92620,72774,44026,55584,97649,3132,40896,56463,54665,34680,95157,64509,35466,56981,39266,54812,54135,80770,68708,41171,63440,4972,86711,1750,26836,79328,72293,34123,17619,80000,80808,95671,44904,55763,29876,90366,37497,37468,11007,696,54068,92622,14294,2268,55307,82269,67320,77295,23827,46167,72952,14914,83421,35650,68116,28069,26189,6184,422,4114,88553,20674,7820,82883,31362,85415,14522,20900,36762,69388,11590,38785,6313,46858,4706,95887,28512,41052,39811,26550,47163,85810,98388,46563,35272,78699,90584,95069,93374,15809,11243,12972,59937,90298,43442,61090,7827,41021,45772,58850,16137,53441,79367,17832,25831,25807,85446,61217,98671,17781,86609,42662,37285,15971,20082,44473,33316,22084,29439,97932,57031,6261,89993,25873,10448,39030,99199,98682,30964,12625,52595,5776,61042,11657,68680,96503,23594,19536,88362,94121,53207,9494,73587,78884,7112,92727,23097,80024,87910,23050,53676,48083,60071,10443,52533,65566,23603,63597,94845,94174,15023,26061,72495,96836,31268,69271,84694,5792,91655,29263,34549,82139,78411,77315,85806,20039,91126,33567,86442,12805,72805,28940,40098,73077,69394,41100,14179,48519,38336,12213,70886,72032,9876,27984,55910,81157,6172,37458,79138,87995,81512,23476,60278,95774,64596,24482,43929,56845,22448,79846,80422,33868,57049,43596,48604,14769,9283,76625,67657,63517,963,40201,82817,69518,29011,13251,20160,78109,88334,51953,64604,25113,88985,57144,55850,48583,56229,69862,39538,30943,44021,13434,39982,93191,90687,59362,10191,61572,94704,30240,38756,59327,45057,2131,97305,23958,37588,85879,18388,23212,91983,65802,86222,10178,91335,14604,23447,15139,27244,1959,34651,95494,17036,83719,40043,5405,42668,39266,18033,15310,51511,79604,90916,15301,84589,85484,157,39230,21233,146,12957,21681,79625,86242,8172,40148,65047,7211,43118,97128,63575,5960,10772,38696,66064,86671,42155,20572,91442,16670,66022,95195,76935,35657,83768,69176,50505,89064,43538,52974,79427,2605,90251,99673,98886,80686,61841,9287,66198,53354,46958,62761,3559,31534,38615,92852,56648,15912,50938,75649,88033,42439,89798,69720,67247,39876,18410,71879,72264,46537,43309,9078,59391,5094,28619,41538,91892,13220,29493,56209,96757,93442,66258,31276,52220,70902,76171,90790,73334,44684,33504,84274,91948,76576,1970,4975,11746,46358,52100,15057,23458,61049,69500,97715,79270,97447,50713,16455,21563,2282,43408,51253,1196,4336,67553,24080,87310,19124,32383,25909,96988,76625,73057,77072,87846,38283,32872,76000,62629,65425,17444,37470,92183,85517,83842,38383,98990,15058,40444,97278,94167,11240,67273,90490,97825,19838,75643,7946,65529,21838,65550,87196,47859,11956,32116,54351,92747,72376,58260,21637,7955,15910,33654,82305,86025,84478,95758,12491,12187,30139,713,17009,5053,70554,84144,19579,23917,10098,47772,35594,35530,41534,63826,67632,14485,33174,27079,7013,26291,27122,82832,50444,52197,71113,95999,2688,61746,95378,64394,83431,73225,94607,45809,57258,79570,25660,7633,85519,31487,69204,58027,13513,94210,61493,42257,91208,93368,33203,99995,90763,52971,83135,19450,43415,64526,81778,39083,20496,79241,71628,65276,29926,48475,25940,35004,56346,16508,23621,78484,66000,57529,98525,99245,41716,31506,57518,18988,45946,75617,28335,85128,47291,68623,46395,35132,33806,27555,51346,87729,22640,17982,81363,54706]
pipe_list = [[2,1,47738],[3,1,69703],[4,2,69081],[5,1,56293],[6,3,2639],[7,6,93651],[8,1,40308],[9,1,97376],[10,2,48875],[11,10,694],[12,8,66572],[13,11,46321],[14,6,36345],[15,13,63471],[17,12,94942],[18,6,87698],[19,17,80719],[20,11,38134],[21,12,64972],[22,10,92522],[23,8,25713],[25,17,50590],[26,24,76743],[28,26,19033],[29,9,94495],[31,6,40808],[33,1,13648],[35,13,5519],[36,12,40618],[37,24,16644],[38,33,7549],[39,29,95247],[40,20,57468],[41,11,27141],[42,2,31665],[43,4,48455],[44,36,82641],[45,8,96368],[46,24,38535],[47,26,96961],[48,41,28292],[49,32,49107],[50,36,87950],[51,32,57160],[52,32,88076],[53,10,53621],[54,16,3282],[55,25,46281],[57,41,46504],[58,47,48746],[59,18,35133],[60,18,49749],[62,13,59420],[64,52,2698],[65,11,16455],[66,43,65636],[67,14,66030],[68,9,41250],[69,21,17143],[70,66,10939],[71,29,44964],[72,18,75213],[73,23,1235],[74,3,21010],[75,67,68921],[76,37,64772],[77,25,90872],[78,64,29819],[79,24,65964],[80,30,46318],[81,55,37496],[82,52,43108],[83,15,77389],[84,6,78172],[85,40,44932],[86,23,14545],[89,3,24538],[90,2,7520],[91,70,77155],[92,87,65058],[93,8,78772],[94,11,24940],[95,84,79176],[96,28,69715],[99,87,16516],[100,13,17057],[101,15,9984],[102,72,85281],[103,85,15511],[104,41,12824],[105,75,1326],[107,103,64679],[108,65,13235],[109,51,27413],[111,15,42979],[113,23,60159],[114,70,34095],[115,87,14405],[116,105,76429],[117,16,69239],[119,51,45322],[120,92,2757],[121,1,67728],[122,72,92648],[123,93,61697],[124,121,25364],[125,101,65526],[126,100,14288],[127,12,161],[128,39,71964],[129,104,20571],[130,40,57740],[131,123,10077],[132,58,66745],[133,130,21191],[134,11,27666],[135,57,92987],[136,17,9003],[137,103,94223],[138,106,72434],[139,25,82761],[140,40,1991],[141,17,38342],[142,57,69761],[143,75,6460],[144,14,12673],[145,33,88522],[146,36,53504],[148,113,59196],[149,8,4787],[150,74,58062],[151,136,31266],[153,51,46431],[154,79,40217],[155,81,50378],[156,150,4810],[157,30,80467],[158,110,13236],[159,135,16961],[160,106,94005],[161,142,90646],[162,102,64775],[163,74,74567],[164,8,84807],[165,95,99415],[166,26,22983],[167,65,1762],[168,113,25735],[169,96,33548],[170,25,53207],[171,5,67775],[172,96,49826],[174,98,66774],[175,118,98686],[176,44,77836],[179,71,45198],[180,162,87123],[181,12,42543],[182,171,51479],[183,150,83818],[185,112,9371],[186,72,45219],[187,137,99516],[188,61,5945],[189,103,43412],[190,167,59356],[191,26,17366],[192,38,57357],[193,25,74911],[194,29,52943],[195,152,31880],[196,143,60167],[197,137,14855],[198,12,24514],[199,154,25434],[200,119,9146],[201,181,91277],[202,93,6693],[203,173,86304],[204,17,57091],[205,182,31649],[206,179,55888],[207,170,54751],[208,122,19735],[209,6,73592],[211,87,95219],[214,40,22060],[215,140,3472],[216,31,74728],[217,68,3448],[218,28,13038],[220,93,35816],[221,202,91355],[222,72,72509],[223,181,98110],[225,61,30774],[227,178,67081],[228,31,73147],[229,156,88321],[230,70,86687],[231,160,33551],[232,72,54935],[233,221,50304],[234,127,23810],[235,152,18420],[236,143,1667],[237,55,46491],[238,80,51029],[239,34,64717],[240,58,5680],[241,219,34305],[242,181,37590],[243,227,40027],[245,107,31800],[246,120,34629],[247,166,35415],[248,204,52743],[249,115,96551],[250,49,45172],[251,12,81050],[252,11,4094],[254,140,72002],[255,235,9036],[256,144,32515],[257,129,11344],[258,246,56377],[259,234,90502],[260,207,47420],[261,50,43494],[262,25,10067],[263,235,83063],[264,164,93481],[265,201,72908],[266,108,93373],[267,213,68874],[268,106,77932],[270,40,54520],[271,85,31278],[274,237,81356],[275,272,78070],[276,230,27674],[277,269,31287],[279,34,86270],[280,51,10778],[281,68,66807],[282,224,57808],[284,48,40024],[285,126,3081],[286,102,75351],[287,116,64810],[288,27,58624],[289,155,51289],[290,35,67503],[291,90,60405],[292,259,40824],[293,192,49095],[295,43,44228],[296,72,22973],[297,252,47815],[298,111,6972],[299,282,87106],[300,2,21013],[301,284,22222],[302,269,81363],[303,285,52652],[304,58,2837],[305,137,77565],[306,225,35099],[307,196,59207],[308,242,52539],[309,38,53367],[310,148,47526],[311,24,9929],[313,102,86973],[314,167,33306],[315,237,31619],[316,72,85382],[317,1,81209],[318,112,92832],[319,165,34920],[321,105,28573],[322,47,83278],[323,13,7105],[324,181,79700],[325,141,96797],[326,146,3452],[327,44,9603],[328,227,53644],[329,193,64877],[330,72,19739],[331,248,43738],[332,46,37501],[333,144,5763],[334,322,27063],[335,131,65484],[336,189,19543],[337,107,38191],[338,149,16711],[339,109,85220],[341,145,38159],[342,61,64581],[343,102,64507],[344,77,98071],[345,292,57683],[347,65,32308],[348,163,73756],[349,247,54643],[350,7,40491],[351,92,24975],[352,267,1667],[353,110,13280],[354,40,53146],[356,304,14841],[357,192,2051],[358,90,98837],[359,158,16128],[360,83,13583],[361,12,78213],[362,185,42433],[363,7,70924],[364,321,52640],[365,157,88869],[366,200,15604],[367,85,86935],[368,188,96465],[369,107,82634],[370,103,44055],[371,274,4278],[372,183,36501],[373,76,85982],[374,212,95042],[376,277,85690],[377,295,93838],[379,303,7780],[380,76,75959],[381,48,60177],[382,145,75263],[383,185,85639],[384,54,40294],[385,123,3689],[386,112,87628],[387,340,26246],[388,56,57318],[389,309,21637],[390,55,39003],[391,303,91095],[392,328,99634],[393,76,51738],[394,373,41034],[395,229,4414],[399,374,31217],[400,240,96627],[401,233,54427],[402,170,67406],[403,57,4277],[404,139,72854],[405,181,90056],[407,271,80146],[408,67,90152],[409,109,84805],[410,303,44950],[411,109,84344],[412,97,64967],[413,254,25055],[414,189,28241],[415,13,32540],[416,302,33498],[417,159,33687],[418,43,13063],[419,113,46331],[420,277,34522],[421,23,56060],[422,1,88518],[423,64,45894],[424,105,56806],[425,406,45237],[426,177,53213],[427,336,40294],[428,151,39233],[429,273,9913],[430,83,2711],[431,244,47930],[432,82,56804],[433,339,70380],[434,151,47428],[435,397,58408],[437,178,63691],[438,315,25460],[440,163,36080],[441,384,87674],[442,159,4230],[443,402,72275],[444,24,15251],[445,247,61329],[446,14,85988],[447,241,45145],[449,288,57594],[450,5,5314],[451,226,98412],[452,198,12819],[453,374,33060],[454,300,63634],[455,281,61968],[456,266,25903],[457,62,99440],[458,182,80667],[459,187,71952],[460,104,76460],[462,175,24260],[463,445,93981],[464,457,4644],[465,138,54029],[466,235,8643],[467,68,82687],[468,461,24026],[469,374,56270],[470,62,76533],[471,434,57698],[472,277,63188],[473,360,80622],[474,271,65723],[475,151,3124],[476,316,49386],[477,461,42395],[479,193,20070],[480,252,41201],[481,179,91370],[482,230,43751],[483,90,19323],[484,128,92628],[485,235,68886],[486,429,82236],[487,193,6282],[488,431,86480],[489,247,86265],[490,331,13019],[491,51,58668],[492,273,82346],[493,421,91908],[494,259,52911],[495,308,31314],[496,340,19413],[498,95,18683],[499,289,41754],[502,151,78078],[503,128,1848],[504,384,28264],[505,129,78748],[506,90,76151],[507,502,3109],[508,192,6062],[509,420,91253],[511,235,66255],[512,54,15377],[513,275,10769],[514,458,7405],[515,145,98570],[517,329,17502],[518,208,85548],[519,80,40484],[520,135,24786],[521,423,10020],[522,8,54451],[523,478,23707],[524,53,78503],[525,27,78743],[526,389,68982],[527,450,78713],[528,303,16216],[531,338,62313],[532,222,31141],[533,172,1150],[535,232,64752],[536,285,50055],[537,439,97918],[538,287,51217],[539,45,94916],[540,43,88679],[542,8,79421],[543,253,10734],[544,344,78837],[545,32,11042],[546,167,40097],[547,152,21536],[548,33,99905],[550,449,54249],[551,380,23953],[552,109,30335],[553,165,9461],[554,523,53002],[555,14,25509],[556,254,79738],[557,231,37152],[558,211,71817],[559,87,4160],[560,75,79600],[562,100,8187],[563,255,57211],[564,325,69403],[565,550,78736],[566,498,41397],[567,220,33422],[568,167,87502],[569,132,92005],[570,58,68284],[571,383,55738],[572,290,25217],[574,329,82578],[575,255,37693],[576,406,26625],[577,334,98380],[578,226,68130],[580,430,31140],[581,65,69553],[582,163,31563],[583,314,26914],[585,439,78297],[586,376,80304],[587,404,46238],[588,235,41835],[589,372,58639],[590,28,84602],[591,475,59119],[592,452,13227],[593,384,75181],[594,166,93811],[595,152,19141],[596,495,1424],[597,466,98393],[598,123,1420],[599,115,32040],[600,395,74716],[602,408,82595],[603,471,42257],[604,294,97457],[605,147,57960],[606,355,75169],[607,298,98365],[608,438,22312],[609,372,24494],[610,26,15119],[611,249,59467],[612,303,31778],[613,353,33054],[614,428,62567],[615,380,62925],[616,494,18530],[617,268,99961],[618,537,92316],[619,261,3365],[620,148,32559],[621,475,63311],[622,603,56081],[623,342,16006],[624,29,4251],[625,182,84761],[626,513,34755],[627,342,74061],[628,97,39298],[629,620,63239],[631,201,42477],[632,434,40666],[634,488,16071],[635,72,86174],[636,252,18241],[637,183,98489],[638,309,27877],[639,300,21833],[640,25,81029],[641,61,3167],[642,241,91842],[643,632,5303],[645,165,47441],[646,558,72635],[647,196,46103],[648,412,52037],[649,39,32869],[650,399,73693],[651,215,93185],[653,204,81426],[654,115,24223],[655,260,52297],[656,289,20474],[657,79,10491],[658,280,57472],[659,261,14071],[660,93,87936],[661,613,96358],[662,463,38444],[663,393,81428],[664,145,91754],[665,384,24132],[666,510,21564],[668,563,64259],[669,459,39940],[670,495,38015],[671,531,56138],[672,39,38528],[673,240,44945],[674,220,63263],[675,535,92359],[676,305,63303],[677,76,77924],[678,44,14415],[679,22,52331],[680,515,94601],[681,277,61194],[682,175,4586],[683,521,5107],[684,535,77972],[685,80,68228],[686,603,63261],[687,62,62285],[688,90,90623],[689,19,2768],[690,334,42292],[691,106,33779],[692,169,85958],[693,274,20182],[694,430,3409],[696,5,81886],[697,534,24198],[698,306,60391],[699,298,14315],[700,228,71732],[701,97,29287],[702,20,45209],[703,358,51391],[704,538,56754],[705,675,3603],[706,396,93490],[707,589,87882],[708,389,42108],[709,124,55166],[710,45,62791],[711,163,48816],[712,218,70213],[713,500,52932],[714,563,22528],[715,709,71708],[716,449,70528],[717,119,35825],[719,665,27282],[720,660,78913],[721,669,3139],[722,176,60440],[723,279,19578],[724,242,25143],[725,256,17635],[727,108,74911],[728,304,56756],[729,559,62957],[730,39,45878],[731,499,3702],[732,493,29487],[733,321,86032],[734,652,53090],[735,383,85911],[736,233,69068],[737,585,30551],[738,635,30617],[739,500,24699],[740,231,41786],[741,733,65341],[742,210,37108],[743,688,94163],[744,249,94673],[745,367,24883],[746,502,34614],[747,21,62543],[748,402,11472],[749,168,39351],[750,729,92305],[751,602,38546],[752,516,76925],[753,165,34331],[754,505,35012],[755,515,33498],[756,428,23204],[757,180,28044],[758,727,75663],[759,554,87018],[760,55,55470],[761,542,16784],[762,36,43418],[763,315,2568],[764,601,23657],[765,701,69322],[767,553,20645],[768,126,87884],[769,271,46694],[770,159,61457],[771,100,52759],[773,112,84844],[775,763,27531],[776,141,9311],[777,166,42670],[778,307,49417],[779,484,65631],[780,175,42706],[781,60,84585],[782,92,41056],[783,350,38478],[784,556,65519],[786,150,51338],[787,648,84943],[788,202,94245],[789,238,36996],[791,193,46847],[792,79,59566],[793,722,19543],[794,787,58231],[795,713,8285],[797,265,80191],[798,231,68543],[799,481,60513],[800,78,17680],[801,778,85837],[803,334,33712],[804,185,79568],[805,423,7700],[806,227,49155],[807,188,78208],[808,465,41648],[809,747,30636],[810,400,15958],[811,123,9629],[812,180,18784],[813,251,94720],[814,44,48726],[815,350,48270],[816,195,51647],[817,174,49855],[819,439,6545],[820,11,15576],[821,607,78212],[822,301,86403],[823,23,56978],[824,74,44472],[825,199,81026],[826,518,15012],[827,66,5379],[828,643,34538],[829,68,42480],[830,390,8048],[834,467,17135],[835,728,14232],[837,258,98435],[838,577,95824],[839,97,41643],[840,57,62887],[842,341,11475],[843,664,12734],[844,613,60762],[845,699,32231],[846,686,61072],[847,730,79780],[848,732,44888],[849,529,90164],[850,472,46162],[851,782,94112],[852,119,9043],[853,613,4972],[854,750,96783],[855,365,40184],[857,502,49646],[858,39,17222],[859,798,23868],[860,784,61276],[861,9,40897],[862,760,41287],[863,571,73836],[864,53,65530],[865,424,16293],[866,479,99805],[867,293,60473],[868,862,65601],[869,674,79420],[870,354,28229],[871,642,63588],[872,408,98372],[873,631,22508],[874,679,41230],[875,393,58614],[877,577,12931],[878,306,84479],[879,700,56750],[880,79,2318],[882,405,58900],[883,246,63609],[884,128,92276],[886,615,88133],[887,664,32848],[888,772,12523],[889,354,51311],[890,72,29019],[891,452,67685],[892,765,82525],[893,424,44907],[894,18,82710],[895,691,43138],[897,396,75362],[898,630,96364],[899,44,73716],[900,136,30089],[901,404,15372],[902,566,73757],[903,772,11827],[904,537,55775],[905,577,16541],[906,66,23709],[907,512,38843],[908,215,58204],[909,872,66899],[910,256,50973],[911,757,98187],[912,539,77020],[913,594,93642],[914,838,40473],[915,36,83150],[916,538,82808],[917,268,35853],[918,163,46512],[919,608,52219],[920,239,3520],[922,879,95232],[923,917,13498],[925,287,80473],[926,497,66523],[927,572,31411],[928,488,54378],[929,94,54717],[930,306,83307],[931,269,1435],[932,838,29093],[933,392,75739],[934,845,58673],[935,269,55856],[936,861,7335],[937,503,22353],[938,614,72999],[940,527,74505],[941,880,35681],[942,307,74105],[944,587,72232],[945,758,61310],[946,656,75435],[947,918,84863],[948,760,68144],[950,146,36232],[951,790,54380],[952,195,46530],[953,596,36711],[954,237,58858],[955,480,32670],[956,670,20585],[957,880,36335],[959,204,76156],[960,556,20033],[961,919,77376],[962,432,6679],[963,673,81705],[964,798,29162],[965,63,22017],[966,606,95616],[967,597,33042],[968,59,48279],[969,645,96337],[970,904,36010],[971,852,32376],[972,711,22657],[973,461,87344],[974,265,56119],[975,809,2145],[976,431,3031],[978,172,7500],[979,466,66300],[980,719,89673],[982,868,74146],[985,488,29220],[986,314,67268],[987,648,74787],[988,780,68399],[989,876,89985],[991,319,61022],[992,970,94140],[994,912,8646],[995,237,88493],[996,606,44613],[997,162,68249],[998,802,76992],[999,948,142],[1000,209,75793],[1001,573,59174],[1002,43,13083],[1003,705,39415],[1004,204,23096],[1005,733,66632],[1006,443,34218],[1007,504,22789],[1008,433,4712],[1009,227,87182],[1010,585,75845],[1011,924,45012],[1013,32,49655],[1014,956,34045],[1015,585,99871],[1016,165,23422],[1017,222,70090],[1018,131,4799],[1019,298,10137],[1020,17,62654],[1021,674,59767],[1022,543,38471],[1023,81,90635],[1024,693,40530],[1025,179,76782],[1026,656,50950],[1027,1025,39924],[1028,775,82305],[1029,89,37380],[1030,466,3436],[1031,942,3108],[1032,503,60497],[1033,662,75790],[1034,698,25948],[1035,658,14508],[1036,49,29545],[1037,110,72979],[1038,635,76817],[1039,396,89270],[1040,726,92400],[1041,265,32648],[1042,58,38691],[1043,824,67013],[1044,937,86479],[1045,321,8925],[1046,1005,77773],[1047,1,3867],[1048,639,92435],[1049,511,55761],[1050,290,5780],[1052,314,55563],[1054,777,64571],[1055,169,33274],[1056,395,43129],[1057,380,7643],[1058,3,54680],[1059,369,35536],[1060,765,52963],[1061,620,71888],[1062,844,8313],[1063,77,55561],[1064,246,58025],[1065,573,71217],[1066,953,67196],[1067,850,38652],[1068,404,18157],[1069,688,79465],[1070,398,49372],[1071,811,17834],[1072,113,89264],[1073,950,18764],[1074,249,52315],[1075,508,96676],[1076,205,56823],[1077,69,36052],[1078,535,68233],[1079,209,70548],[1080,10,44244],[1081,322,84832],[1082,445,45100],[1083,211,9332],[1084,707,1318],[1085,1082,81349],[1086,585,32310],[1087,142,26914],[1088,290,1608],[1089,412,16027],[1090,230,17574],[1091,590,3667],[1092,131,94394],[1093,503,25813],[1094,817,6732],[1095,911,34510],[1096,1075,29322],[1098,122,16167],[1099,718,88523],[1100,341,33524],[1101,209,61470],[1102,737,5649],[1103,816,68072],[1104,383,79112],[1105,8,56329],[1106,296,70891],[1107,939,47258],[1108,316,82199],[1109,1057,95941],[1110,734,32426],[1111,205,40310],[1112,96,59012],[1113,637,30644],[1114,918,80458],[1116,934,61261],[1117,160,53114],[1118,920,23929],[1119,1083,42939],[1121,425,52848],[1122,736,87339],[1123,341,13196],[1124,896,43130],[1125,159,79416],[1126,652,49097],[1127,795,51540],[1128,200,25989],[1130,314,55977],[1131,396,62970],[1132,1120,91457],[1133,671,66368],[1134,588,47267],[1135,302,67616],[1137,1055,20153],[1138,604,79764],[1139,1120,9806],[1141,392,25134],[1142,107,67629],[1143,976,44635],[1144,1092,13396],[1145,649,86376],[1146,375,8645],[1147,748,42073],[1149,926,21895],[1150,508,5227],[1151,414,70775],[1152,483,87695],[1154,844,17674],[1155,261,4237],[1156,1037,63782],[1158,181,33619],[1160,930,65791],[1161,612,52644],[1162,179,51195],[1164,617,36372],[1165,1131,3021],[1166,175,96599],[1167,1110,45121],[1168,550,26907],[1169,936,16564],[1170,781,22595],[1171,697,41242],[1172,1042,2872],[1173,929,11288],[1174,1152,64176],[1175,836,64385],[1176,536,62023],[1177,28,65192],[1178,403,49169],[1179,1089,52684],[1180,1003,23285],[1181,245,10709],[1182,467,77569],[1183,869,90969],[1184,66,80805],[1186,52,93905],[1187,132,67779],[1188,1018,3776],[1189,430,42618],[1191,920,92628],[1192,633,71928],[1193,489,81572],[1194,654,86604],[1195,260,97465],[1196,438,46172],[1197,383,94700],[1198,205,54150],[1199,1058,88639],[1200,6,65317],[1201,460,64075],[1202,1147,52026],[1203,392,33745],[1204,171,58141],[1205,213,25994],[1206,1051,93346],[1207,655,64971],[1208,61,70082],[1209,323,53574],[1210,851,23274],[1212,99,79561],[1214,988,25202],[1215,437,52997],[1217,409,53208],[1218,566,75514],[1220,55,18345],[1221,583,39993],[1222,383,43736],[1223,58,89126],[1224,208,87296],[1225,636,26653],[1226,1056,45761],[1227,1169,85475],[1228,1191,1163],[1229,914,10299],[1230,733,38787],[1231,826,31161],[1232,1031,34397],[1233,1176,70306],[1234,614,28861],[1235,186,77236],[1236,445,87377],[1238,270,27954],[1239,919,8108],[1240,305,29752],[1241,1184,11046],[1242,167,62410],[1243,211,28122],[1244,405,9705],[1245,1179,35972],[1246,867,29933],[1247,687,35114],[1248,1211,32361],[1249,488,76533],[1250,980,35087],[1252,325,85518],[1253,233,52883],[1254,758,13974],[1255,252,45837],[1256,42,21241],[1258,237,96111],[1260,489,13272],[1261,622,37102],[1262,165,69520],[1263,105,32796],[1264,643,76341],[1265,230,71128],[1266,1007,5520],[1267,451,3947],[1268,418,25559],[1269,1011,81397],[1272,1140,45941],[1273,287,23685],[1274,437,35396],[1275,546,61773],[1276,691,95390],[1277,301,19217],[1278,850,43158],[1279,539,15041],[1280,928,39737],[1281,645,4852],[1282,24,660],[1283,288,19997],[1284,149,92813],[1285,645,45477],[1286,1196,52710],[1287,1213,33927],[1288,1019,76403],[1289,982,29179],[1290,1097,79488],[1291,407,10519],[1292,453,95212],[1293,689,58343],[1294,786,32397],[1295,954,87198],[1297,33,45087],[1298,130,33919],[1299,438,65874],[1301,891,66486],[1302,660,91062],[1303,324,97722],[1304,977,38274],[1305,332,9115],[1306,275,53315],[1307,475,15180],[1309,77,3618],[1310,15,817],[1311,866,98200],[1312,1275,41215],[1313,453,18003],[1314,774,33472],[1315,829,41664],[1316,727,35477],[1317,900,85457],[1318,836,97807],[1319,270,19983],[1320,57,52272],[1321,50,69480],[1322,640,66374],[1323,985,52237],[1327,602,76940],[1328,1195,60080],[1329,965,35528],[1331,1286,97551],[1332,659,15369],[1333,1236,54747],[1334,672,53675],[1335,1005,42342],[1337,255,22523],[1338,31,85005],[1340,546,66756],[1341,87,75143],[1342,267,97819],[1343,904,14100],[1344,415,64709],[1345,606,27299],[1346,76,83702],[1347,553,82536],[1348,777,9265],[1349,289,60834],[1350,628,34905],[1352,918,7354],[1353,1094,85293],[1354,1152,93585],[1355,1090,9968],[1356,1133,50747],[1357,403,68170],[1358,79,43585],[1359,284,42907],[1360,437,2781],[1361,139,44916],[1362,941,70289],[1363,816,21665],[1364,560,8039],[1365,599,96669],[1366,709,39232],[1367,1342,33114],[1368,520,87813],[1369,1099,52778],[1370,1360,27651],[1371,1333,18274],[1372,761,98168],[1373,78,56787],[1374,1137,46946],[1375,441,74025],[1376,1,82388],[1377,809,50555],[1378,578,71906],[1379,1109,46953],[1380,872,63656],[1381,265,36121],[1382,628,1836],[1384,1362,89530],[1385,466,60437],[1386,174,70709],[1387,212,35857],[1388,714,85894],[1390,79,41959],[1391,956,15072],[1392,1336,72179],[1393,943,49543],[1394,742,71990],[1395,33,80564],[1396,395,72409],[1397,272,33509],[1398,374,20935],[1399,1189,54118],[1400,156,48112],[1401,641,77927],[1402,708,35456],[1403,932,36480],[1404,363,6770],[1405,720,6354],[1406,955,52845],[1407,1184,24434],[1408,524,3214],[1411,252,9090],[1412,399,86243],[1414,1303,83643],[1415,972,8545],[1416,124,11194],[1417,802,93130],[1418,463,4767],[1419,815,54600],[1420,709,78688],[1421,903,44337],[1422,206,34953],[1423,1153,3977],[1424,524,36328],[1425,1380,5537],[1426,1323,35303],[1427,698,1750],[1429,405,95961],[1430,1370,50811],[1431,698,27110],[1432,1144,3364],[1433,634,93223],[1434,510,51984],[1435,1384,34399],[1436,955,13667],[1437,114,89895],[1438,172,87909],[1439,684,41386],[1440,578,73680],[1442,94,11570],[1443,1125,72975],[1444,693,90644],[1445,504,19217],[1446,438,6772],[1447,948,1716],[1449,471,38994],[1450,1348,44737],[1451,635,31484],[1452,343,66948],[1453,1229,54028],[1454,1155,53788],[1455,922,79470],[1456,915,10071],[1457,763,28453],[1458,242,30270],[1459,960,16786],[1460,504,88039],[1461,744,504],[1462,766,3757],[1463,399,46535],[1465,1369,61628],[1466,740,29576],[1467,970,69569],[1469,209,1311],[1470,1211,3423],[1471,203,48105],[1472,1423,4041],[1473,1187,10978],[1474,484,91630],[1475,1095,57861],[1476,993,60730],[1478,571,91846],[1479,978,98183],[1480,866,34749],[1482,881,25855],[1484,818,51154],[1485,336,24533],[1486,872,42933],[1487,183,53844],[1488,1033,20603],[1489,836,30269],[1490,445,50496],[1491,466,17030],[1492,935,11874],[1493,65,63073],[1494,659,46273],[1495,1269,11792],[1496,1283,76052],[1497,378,32385],[1498,1004,37938],[1499,1149,45061],[1500,243,89111],[1501,575,58758],[1502,351,50918],[1503,226,51596],[1504,638,40033],[1505,114,43023],[1506,382,29954],[1507,1467,18022],[1508,933,21304],[1509,64,49319],[1510,48,17495],[1512,878,26799],[1513,1051,59080],[1514,772,66307],[1515,650,150],[1516,1061,89686],[1517,1503,70284],[1518,428,21399],[1519,661,67953],[1520,360,98896],[1521,899,73839],[1523,607,57529],[1524,1110,49128],[1525,1499,13992],[1526,653,27492],[1527,1117,3176],[1528,1163,88320],[1529,1099,35055],[1530,446,77111],[1531,1297,87283],[1533,867,94366],[1534,336,58193],[1535,749,37723],[1536,881,95531],[1537,1263,31636],[1538,871,35956],[1539,973,83388],[1541,929,98098],[1542,1083,37605],[1543,703,71199],[1545,928,41847],[1546,52,36598],[1548,1507,68221],[1549,102,77115],[1550,360,18406],[1551,579,87917],[1552,816,8634],[1553,1257,27475],[1555,963,68216],[1556,518,31441],[1557,1208,18306],[1558,42,44602],[1559,1268,89670],[1560,344,76015],[1562,1099,60327],[1563,270,62330],[1564,264,1120],[1565,415,35580],[1566,907,18628],[1567,361,69036],[1568,220,94453],[1569,545,17955],[1570,570,66222],[1571,1067,15781],[1572,316,70691],[1573,599,70389],[1574,197,5458],[1576,329,83640],[1577,1233,60069],[1578,329,73243],[1579,1381,16601],[1580,504,36080],[1581,994,97818],[1582,778,71028],[1583,350,72599],[1585,489,31987],[1586,226,17809],[1588,509,49616],[1589,898,45742],[1590,164,27950],[1591,410,70390],[1592,462,70782],[1593,713,62525],[1594,577,49797],[1595,314,65903],[1596,1005,41782],[1597,1211,87593],[1598,514,37249],[1600,1400,47324],[1603,1217,68404],[1605,63,75344],[1606,197,54693],[1607,1254,12588],[1608,1599,77541],[1609,886,95297],[1610,1294,19267],[1611,219,77046],[1612,154,12553],[1613,1611,86957],[1614,488,81143],[1615,789,22585],[1616,1424,40659],[1617,1418,62971],[1618,1116,44672],[1619,1345,89108],[1620,479,35787],[1621,33,57663],[1622,1285,99990],[1624,511,19013],[1625,639,69512],[1626,107,86827],[1627,248,29520],[1628,285,27437],[1629,328,90067],[1630,541,37094],[1631,1345,41551],[1632,1285,1243],[1633,700,4791],[1634,538,59331],[1636,1032,36226],[1637,902,40634],[1638,627,29731],[1639,232,88111],[1640,449,75540],[1641,870,11970],[1642,1531,45912],[1643,960,61321],[1644,1010,71936],[1645,1262,56954],[1646,646,14393],[1647,1002,17946],[1648,1104,30771],[1649,1258,12721],[1650,783,37617],[1651,1116,86656],[1653,126,31037],[1654,1358,63417],[1655,44,34092],[1656,444,63915],[1657,243,23211],[1658,1133,90280],[1659,986,13253],[1661,446,74129],[1662,1419,93641],[1663,1501,42603],[1664,816,67618],[1665,1441,68390],[1666,1577,10125],[1668,827,93452],[1669,525,74127],[1670,1451,98405],[1671,343,67496],[1672,249,31842],[1673,911,87353],[1674,819,38218],[1675,1421,96335],[1676,1211,52690],[1677,429,71453],[1679,1513,91953],[1680,1452,96333],[1681,1217,21385],[1682,381,85010],[1683,357,76476],[1685,672,51761],[1686,799,27174],[1687,1272,75980],[1688,1316,95566],[1689,163,51804],[1690,268,27317],[1691,94,51345],[1692,1162,96221],[1693,1192,77713],[1694,495,6831],[1695,1624,82116],[1696,1274,3122],[1697,814,22576],[1698,1621,74345],[1699,1448,82232],[1700,447,3721],[1701,1407,90124],[1702,1324,45295],[1703,478,67666],[1704,1376,38472],[1705,1388,48044],[1706,490,92693],[1707,406,7171],[1708,1574,45952],[1709,847,70688],[1711,1438,43503],[1712,1387,7391],[1713,217,51541],[1714,108,94905],[1715,534,13561],[1716,1043,15883],[1717,49,4245],[1718,449,45025],[1719,1400,23033],[1720,385,23506],[1721,751,70620],[1723,1630,8951],[1724,565,68976],[1725,838,32234],[1726,436,84317],[1727,1161,47022],[1728,1461,28446],[1729,1477,1843],[1730,145,94685],[1732,543,33720],[1733,440,14736],[1734,1078,87857],[1735,1037,69408],[1737,630,22769],[1738,582,54720],[1739,1236,74201],[1740,231,96176],[1741,1197,73876],[1742,860,3916],[1743,537,29597],[1744,1139,92725],[1745,560,84200],[1746,687,2396],[1748,844,47005],[1749,107,39114],[1750,1120,83636],[1751,1180,22493],[1752,1169,54776],[1753,812,43142],[1754,118,92843],[1755,1551,88044],[1756,29,17750],[1757,1121,54955],[1758,74,83722],[1759,1626,85432],[1760,1654,75717],[1761,500,57458],[1762,34,14623],[1763,1396,43277],[1765,632,3048],[1766,1728,92011],[1767,263,74902],[1768,297,97870],[1769,459,69166],[1770,1692,47413],[1772,775,48526],[1776,1729,96141],[1777,1731,97984],[1778,818,78459],[1779,191,67375],[1780,1754,60017],[1781,153,73187],[1782,1511,41056],[1783,62,65831],[1784,462,63355],[1785,1213,31035],[1786,319,40700],[1787,1327,58985],[1788,167,13144],[1789,701,35706],[1790,1313,51676],[1791,1136,79675],[1792,1009,64676],[1793,1085,75851],[1794,981,77892],[1795,1186,99223],[1796,598,71713],[1797,41,39052],[1798,24,87352],[1799,1049,42673],[1800,566,8199],[1801,1511,1034],[1802,1710,73262],[1803,1371,57963],[1804,963,92321],[1805,1799,78675],[1806,844,53131],[1807,511,29487],[1808,700,6131],[1809,861,36393],[1810,933,65680],[1811,348,39805],[1813,1787,78922],[1814,1342,42262],[1815,1196,48537],[1816,1543,73324],[1817,599,14142],[1818,843,99035],[1819,950,6640],[1820,412,50055],[1822,94,89518],[1823,981,53656],[1824,436,18956],[1825,1132,91593],[1826,1144,58621],[1827,1501,87246],[1828,1773,6896],[1829,257,77524],[1830,1154,26458],[1831,1572,55352],[1832,68,55989],[1833,1283,17113],[1834,276,64939],[1835,460,99772],[1836,1657,31554],[1837,928,4852],[1838,110,86777],[1839,1550,45827],[1840,1694,76198],[1841,509,88927],[1842,20,93959],[1843,1040,75774],[1844,325,91123],[1845,1485,49970],[1846,695,64416],[1847,1032,99611],[1848,1498,57282],[1849,1814,51651],[1850,924,65723],[1851,719,919],[1852,1022,44442],[1853,993,22161],[1854,928,18480],[1855,242,68521],[1856,1563,34317],[1857,1353,58239],[1859,665,95723],[1860,1565,44461],[1861,1466,30587],[1862,1482,8575],[1863,723,50078],[1864,698,67924],[1865,1574,46831],[1866,129,87630],[1867,1473,73973],[1868,38,27159],[1869,1031,54410],[1871,582,87766],[1872,125,14295],[1873,554,16627],[1874,454,81563],[1875,252,29442],[1877,1237,62208],[1878,291,25696],[1879,1711,85100],[1880,1451,63006],[1881,405,96915],[1882,70,28853],[1884,1290,89445],[1885,1008,81169],[1886,1593,88575],[1887,1778,72240],[1888,1862,97522],[1889,1003,36783],[1890,1401,89958],[1891,939,49036],[1892,408,76540],[1893,1090,44045],[1895,1891,50184],[1896,1222,64981],[1897,962,60708],[1898,503,85364],[1899,472,70614],[1900,1890,88949],[1901,1849,58622],[1902,1133,51842],[1903,1291,15884],[1904,995,47988],[1905,864,94559],[1906,814,5637],[1907,297,52716],[1909,719,16146],[1910,4,61328],[1911,974,17770],[1912,525,38142],[1914,544,63974],[1915,790,34008],[1916,819,25758],[1917,190,81587],[1918,1295,47197],[1919,950,58540],[1921,316,74863],[1922,205,69432],[1923,1257,89666],[1924,1246,48622],[1925,284,91059],[1926,925,62645],[1927,1768,42437],[1928,142,49868],[1929,1565,77729],[1930,966,53905],[1931,1040,12986],[1932,450,42940],[1933,266,24792],[1934,961,15048],[1935,1418,36956],[1936,681,43429],[1939,26,12746],[1940,305,2675],[1943,1521,7465],[1944,1173,89692],[1945,843,27510],[1946,1106,533],[1947,270,77018],[1948,653,37992],[1949,1587,63235],[1951,43,66153],[1952,1559,63320],[1953,27,62204],[1954,1548,20215],[1955,742,52652],[1956,1151,75422],[1957,155,67678],[1958,1290,27941],[1959,176,1664],[1960,608,91306],[1961,901,44618],[1962,498,83362],[1963,1738,5425],[1965,264,18282],[1966,776,23375],[1967,635,70263],[1969,1388,51297],[1970,355,75499],[1971,888,78671],[1972,1502,75996],[1973,1102,77400],[1974,1073,13914],[1975,1072,18435],[1976,880,70359],[1977,998,98449],[1979,1523,92449],[1980,1428,98608],[1981,1866,92788],[1982,44,46856],[1984,1292,53419],[1985,1677,99922],[1986,1493,81544],[1987,70,48803],[1988,355,493],[1989,298,58494],[1990,437,76779],[1991,1926,52973],[1992,1008,43855],[1993,658,91051],[1994,926,69172],[1995,96,47836],[1996,399,99754],[1997,1610,71949],[1999,911,29650],[2002,1139,26260],[2003,1080,55828],[2004,337,38029],[2005,1591,83167],[2006,1285,36454],[2007,302,80646],[2008,1539,36943],[2009,528,66512],[2010,997,11298],[2011,1798,17912],[2012,1438,68428],[2013,559,16773],[2014,1853,10898],[2015,814,26628],[2016,12,14423],[2017,1770,95350],[2018,457,17296],[2019,816,34057],[2020,551,42332],[2021,1065,87948],[2022,1896,19232],[2024,725,48859],[2025,809,14971],[2026,1191,12510],[2027,1405,78781],[2028,1379,91749],[2029,2006,71425],[2030,644,32861],[2031,567,15118],[2032,513,22761],[2033,37,3972],[2034,531,40401],[2035,530,72439],[2036,860,17762],[2037,1221,4469],[2038,651,20414],[2039,1738,58964],[2040,720,10878],[2042,1924,3185],[2043,173,8734],[2044,27,46085],[2045,356,52285],[2046,106,90049],[2047,912,63547],[2048,548,24461],[2049,505,21897],[2050,713,95382],[2052,750,22527],[2053,447,46162],[2054,782,3186],[2055,1278,50925],[2056,956,68141],[2057,937,64222],[2058,1423,42756],[2060,1964,11746],[2061,804,21523],[2062,1251,31787],[2063,177,73889],[2064,608,89778],[2065,1144,36796],[2066,1767,16549],[2067,768,94389],[2068,459,92382],[2069,1142,48215],[2070,792,71644],[2072,168,7480],[2073,493,86608],[2074,230,29451],[2075,1712,4294],[2076,1619,66052],[2078,1512,34114],[2079,2014,34904],[2080,1865,11824],[2081,1403,11008],[2082,1797,24480],[2083,1699,31825],[2084,728,15569],[2085,1466,69270],[2086,1556,39695],[2087,1072,34681],[2088,1086,12274],[2089,1499,55968],[2090,1056,9137],[2091,396,64954],[2092,252,272],[2093,2089,10506],[2094,49,47608],[2095,179,5018],[2096,393,46944],[2097,740,32439],[2098,1584,2626],[2099,253,5671],[2100,1832,29061],[2102,830,66331],[2103,2077,94687],[2106,1080,81603],[2108,942,50145],[2109,181,84873],[2110,1992,90522],[2111,1434,48181],[2112,268,74161],[2113,235,13932],[2114,90,7279],[2115,84,66175],[2116,204,2935],[2117,816,37847],[2119,646,18610],[2120,1418,43803],[2121,473,40777],[2122,1014,34638],[2123,267,51479],[2124,1658,83842],[2125,731,47842],[2126,317,90276],[2127,1457,24267],[2128,1187,59737],[2129,482,2613],[2130,1652,62105],[2131,1558,86528],[2132,1479,94626],[2133,1306,44928],[2134,909,2115],[2135,686,1681],[2136,185,32278],[2138,2056,24445],[2139,791,91909],[2140,426,87245],[2141,1772,44133],[2142,1266,94485],[2143,1806,32702],[2144,1899,68142],[2145,701,17475],[2146,770,60592],[2147,682,41587],[2148,1996,40216],[2149,1464,95782],[2150,922,26684],[2152,450,43015],[2153,1619,28017],[2154,2061,64558],[2155,330,90257],[2156,991,15599],[2157,1632,35510],[2158,462,54597],[2160,345,12748]]


In [30]:
# # Res0
# n = 2160
# wells = [90790, 86908, 95870, 90231, 10772, 20761, 16508, 93129, 69679, 98310, 18033, 87846, 57023, 75157, 25873, 14316, 52285, 80808, 78484, 30094, 39911, 42940, 85334, 16521, 48583, 87310, 41534, 72058, 39345, 92151, 33082, 55466, 33806, 72774, 47772, 40441, 92226, 28649, 32410, 91201, 56586, 19817, 82780, 58992, 51253, 86702, 65267, 17781, 52, 50985, 76950, 70698, 63517, 95474, 1254, 417, 56229, 10208, 93325, 42668, 97921, 39982, 95195, 91913, 30773, 39266, 25891, 319, 83518, 56033, 48482, 78600, 86706, 87729, 34123, 64805, 22640, 60603, 55603, 21563, 99566, 91655, 81295, 39811, 85806, 97363, 91948, 18385, 28948, 29936, 67657, 79311, 77807, 68742, 85479, 37468, 95472, 696, 90997, 86857, 22849, 836, 13676, 47699, 3616, 2605, 80699, 62450, 86182, 31534, 56117, 71610, 85096, 72666, 88888, 93561, 87582, 64603, 54254, 36553, 58355, 83789, 64394, 24789, 73448, 71806, 82706, 47094, 59286, 47209, 11338, 85237, 59794, 46055, 41791, 88836, 95459, 88208, 90124, 92109, 62558, 73057, 92244, 71604, 45460, 5839, 38383, 26061, 5392, 11007, 44851, 80422, 65566, 8201, 54357, 31512, 84294, 78699, 44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 42417, 90525, 95924, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 40675, 34549, 25622, 50209, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 45426, 68623, 63230, 89760, 40707, 28697, 52533, 50505, 71628, 95758, 46210, 79515, 28572, 78450, 82245, 21353, 41716, 8130, 74073, 25676, 98742, 88281, 21838, 64378, 53940, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 34386, 98070, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19058, 38363, 67282, 97436, 37468, 28941, 77321, 21735, 39947, 45271, 75551, 22756, 94693, 74923, 63417, 66612, 36128, 85415, 74813, 28512, 26550, 67591, 47740, 68396, 15283, 15648, 23442, 95657, 99006, 53923, 40365, 93449, 64620, 98318, 80410, 88103, 13907, 62043, 21740, 36137, 79315, 86038, 19773, 82593, 43382, 34577, 23476, 27968, 58730, 48230, 60893, 32796, 82817, 6078, 95367, 88804, 84262, 48538, 69862, 69479, 60964, 38756, 25008, 7142, 23986, 84551, 7812, 79766, 21233, 86242, 25261, 74883, 5960, 66064, 77217, 53785, 46293, 83768, 25802, 54151, 80686, 14218, 90413, 97864, 73782, 94318, 67909, 84451, 91892, 12560, 17023, 52220, 2395, 80816, 18049, 40026, 29409, 14129, 19124, 32383, 59429, 76000, 10553, 35002, 85675, 78715, 60336, 91802, 5233, 83107, 61987, 12187, 9403, 96846, 38880, 44828, 91629, 91208, 80081, 90763, 7249, 95777, 89497, 29116, 76600, 94643, 12326, 76625, 75881, 23050, 30468, 45453, 88407, 15429, 9287, 4664, 85462, 95682, 99676, 60801, 28614, 9796, 88540, 24080, 27588, 78139, 34984, 57049, 27996, 43787, 52595, 90004, 23465, 97595, 43097, 30505, 38229, 28404, 34642, 7780, 65626, 6301, 59674, 26460, 69720, 60578, 27355, 61622, 86841, 66722, 36683, 36855, 66871, 75649, 3143, 65308, 255, 85964, 82099, 29459, 20105, 40665, 79570, 91207, 50416, 33081, 63434, 31348, 93222, 90872, 33134, 15654, 80000, 51287, 4975, 10443, 6313, 51340, 79955, 1823, 26081, 22084, 74952, 40488, 64155, 4026, 12972, 30355, 17619, 82357, 83135, 93442, 13793, 16455, 50444, 11746, 2688, 46001, 60523, 71350, 56177, 98108, 76171, 83261, 17283, 74815, 79220, 10870, 21970, 45055, 94119, 15949, 43561, 61746, 98475, 60603, 58850, 33917, 88987, 85502, 33281, 76623, 71898, 35799, 69623, 96836, 81074, 45366, 39064, 65294, 80063, 33726, 3072, 61317, 97386, 78887, 73205, 87022, 93191, 26673, 38466, 83431, 13489, 11016, 49070, 37111, 95934, 45057, 19579, 3609, 32872, 75207, 78399, 26493, 74140, 36594, 99796, 59057, 85989, 97540, 56244, 37006, 81778, 68741, 81455, 23044, 75038, 64485, 73304, 30943, 98760, 90108, 81437, 61468, 70688, 63826, 85789, 98095, 31197, 84459, 1349, 61572, 89736, 90251, 75106, 42894, 84312, 1065, 3483, 50033, 10178, 40404, 97656, 92747, 60331, 63232, 34998, 63993, 58572, 22758, 4109, 79569, 1714, 62577, 89887, 35132, 64418, 32394, 49105, 82832, 45056, 310, 55002, 94174, 89716, 25831, 15272, 61090, 6978, 19580, 36183, 75864, 97918, 49362, 86465, 28559, 19348, 79138, 58146, 35650, 86885, 7820, 70357, 66430, 55297, 39768, 20774, 39768, 74687, 95554, 48913, 64492, 99818, 92997, 82084, 91865, 42723, 7000, 56647, 68687, 94408, 78624, 79479, 75537, 36727, 69741, 86222, 1196, 59633, 48391, 95494, 54878, 15471, 8204, 4152, 91150, 4972, 69139, 98714, 2956, 12439, 50979, 16298, 34799, 9182, 21637, 93238, 55093, 99587, 31275, 89505, 99390, 95378, 69783, 31037, 57279, 50531, 82947, 84495, 70584, 88362, 98041, 1223, 56077, 64099, 29307, 66597, 56082, 94769, 74838, 40201, 51050, 97608, 47250, 84274, 43415, 35040, 97305, 82722, 69731, 9150, 94333, 63698, 94971, 86961, 70040, 40131, 66511, 45923, 86930, 58217, 44904, 84940, 72430, 1026, 23917, 97609, 48527, 63029, 44684, 19246, 76282, 32300, 29926, 28557, 91335, 67951, 28910, 88176, 58848, 17252, 54823, 14837, 54135, 15162, 71747, 93408, 23362, 14294, 78134, 52108, 86442, 21241, 27928, 52300, 79399, 79367, 35413, 49363, 51650, 37264, 55248, 95570, 7117, 94946, 13553, 3167, 88979, 18789, 37787, 18403, 86350, 89986, 63982, 72264, 67512, 96140, 31352, 19880, 85297, 47940, 20826, 71363, 45946, 60278, 85921, 4591, 83329, 68830, 82139, 49310, 98620, 4849, 27990, 21713, 78166, 57653, 90650, 17036, 90916, 1298, 60579, 12341, 36762, 18190, 71572, 7937, 48393, 46286, 89742, 1970, 95636, 27176, 15139, 94210, 95938, 35090, 94585, 58608, 44113, 77184, 32255, 15677, 83447, 52197, 90286, 66651, 5027, 53441, 65068, 93175, 76760, 2828, 68327, 76250, 34126, 78896, 54084, 96988, 43442, 5331, 10098, 39077, 65550, 81104, 97649, 39237, 33641, 63449, 77109, 41721, 84589, 98990, 28596, 69948, 55763, 72501, 25113, 19450, 97705, 89064, 22083, 73469, 31506, 36091, 18922, 7955, 55659, 45772, 44070, 82957, 98072, 2232, 23328, 33413, 73235, 14602, 33345, 47560, 16713, 13952, 93827, 64812, 15681, 32140, 50513, 44026, 89836, 38336, 57518, 99925, 96875, 46785, 15809, 43457, 94121, 96659, 35272, 83006, 40098, 17193, 55100, 69322, 43558, 47083, 13699, 57144, 21681, 45909, 24083, 22460, 56680, 44473, 15323, 72637, 60748, 28712, 57031, 39876, 51623, 95477, 50497, 43853, 38785, 52974, 33041, 71148, 91590, 71113, 94173, 50298, 96619, 20082, 94607, 55850, 15632, 77489, 65853, 31268, 42921, 25176, 85891, 65270, 92852, 31546, 19770, 86711, 14769, 72772, 27712, 90511, 16137, 91765, 75643, 95774, 35931, 64757, 39030, 60746, 42770, 22448, 18913, 11676, 12491, 60911, 85079, 38312, 53360, 86587, 35546, 90830, 50264, 86873, 44486, 16731, 88394, 67628, 23794, 42282, 1264, 42439, 55778, 74942, 42991, 47732, 83802, 52863, 30185, 59835, 42326, 69967, 50431, 39936, 22604, 53676, 16670, 51588, 41219, 37190, 54768, 3256, 47343, 56416, 9845, 49337, 71879, 91156, 26188, 84762, 10191, 60839, 6178, 72845, 60727, 8605, 60336, 57466, 41129, 79672, 27170, 65879, 23028, 81583, 4336, 72805, 28940, 12213, 40896, 59484, 35362, 33614, 79270, 16882, 99199, 18988, 66176, 70436, 89029, 72376, 72973, 75462, 79604, 45311, 43538, 87603, 76317, 30178, 31970, 32116, 29075, 46817, 86953, 23036, 56981, 10936, 80971, 23827, 17949, 38884, 77600, 69388, 20441, 72293, 23305, 6457, 82269, 17444, 20039, 56209, 47871, 56648, 59367, 71983, 50639, 46503, 45896, 55509, 19588, 65047, 81512, 60071, 4706, 44460, 58199, 46858, 48083, 1889, 59422, 87196, 52891, 66786, 55452, 56063, 25555, 3098, 26189, 90584, 44507, 93318, 95179, 90742, 20674, 1959, 98388, 54351, 50557, 46125, 1351, 62205, 33504, 10771, 99995, 71614, 25761, 85215, 88004, 64604, 2832, 46958, 51953, 21418, 69829, 3559, 91582, 44510, 98525, 6301, 5072, 2275, 85470, 46133, 77675, 8333, 97149, 77182, 47926, 26831, 93368, 28982, 30030, 69952, 97822, 58705, 64292, 12630, 37588, 91983, 71099, 68901, 54303, 65802, 42362, 90298, 44021, 20366, 42548, 91604, 43907, 7013, 24714, 47344, 86736, 46080, 16007, 3575, 73960, 22078, 60895, 46819, 23008, 78411, 33567, 33670, 51497, 57584, 41069, 70554, 42689, 7112, 59362, 99874, 54960, 88375, 77716, 93551, 2268, 45124, 27555, 24758, 82305, 28219, 43118, 59659, 37624, 35954, 59327, 39989, 9175, 58027, 20779, 30367, 94181, 31534, 23450, 5513, 75617, 92713, 57741, 47081, 43478, 79550, 89929, 25807, 35466, 65944, 39482, 20262, 92727, 69348, 80770, 35594, 36782, 113, 17653, 13794, 65743, 17491, 97038, 12601, 73375, 17916, 64094, 89617, 10942, 39527, 12905, 40055, 89993, 16114, 53863, 79328, 68593, 43480, 18998, 86179, 7336, 47291, 23447, 3114, 15094, 83719, 92804, 73011, 39266, 55242, 93010, 83407, 19311, 85963, 881, 76066, 65276, 21939, 3082, 18991, 63974, 40406, 81875, 71587, 7946, 85724, 19820, 82360, 28288, 19438, 16648, 88970, 47187, 31566, 43084, 64484, 15023, 76211, 2136, 63443, 87232, 99360, 64463, 23958, 41596, 91004, 96177, 20517, 97932, 58949, 21279, 74437, 98398, 75234, 15731, 86735, 90097, 22508, 46584, 24195, 11159, 19027, 90366, 58144, 83706, 95157, 21275, 83031, 64918, 5754, 14913, 44690, 69022, 54675, 15912, 95455, 67632, 60443, 2012, 88462, 61217, 41171, 50478, 63977, 62761, 29951, 52971, 19536, 38081, 9906, 71867, 9585, 9876, 99613, 71267, 37033, 25667, 90687, 4853, 54410, 54812, 11507, 1367, 70886, 78424, 49003, 44543, 8827, 58260, 57529, 81765, 34294, 68237, 31226, 360, 99969, 13251, 35657, 78482, 68425, 38517, 63575, 2139, 18891, 3693, 72032, 68680, 19587, 28320, 9494, 23594, 72495, 15966, 82528, 30774, 74823, 12134, 1438, 14914, 88699, 8153, 66917, 22873, 87322, 25918, 96503, 56203, 20375, 17664, 28518, 78640, 76532, 95887, 95671, 75478, 37256, 58447, 77154, 69176, 77471, 93666, 99245, 84478, 92770, 46009, 68116, 49549, 35004, 19838, 64335, 3147, 36045, 29161, 37497, 37526, 82883, 4740, 86421, 422, 52145, 5769, 82391, 63488, 57258, 15323, 91271, 65814, 41021, 53963, 34158, 83842, 92485, 1100, 56346, 72014, 17462, 12396, 38446, 94329, 41052, 45376, 69944, 32931, 28619, 80041, 21503, 38991, 49656, 83612, 81363, 77652, 6824, 46003, 94009, 56049, 91932, 51134, 13522, 64404, 17018, 26979, 67934, 44256, 31891, 91011, 35980, 67247, 27644, 79625, 66258, 5405, 49339, 45117, 2821, 4591, 96318, 6172, 40396, 43408, 72011, 56044, 63901, 46685, 25944, 87300, 11590, 30524, 29944, 94167, 15971, 15058, 28335, 68462, 58251, 33366, 63690, 84694, 99588, 86685, 85609, 25689, 5888, 61435, 53310, 41472, 95855, 77295, 55237, 62009, 77985, 36985, 24666, 20496, 60891, 98682, 94580, 98533, 53780, 15586, 93374, 983, 66892, 660, 78884, 32294, 83093, 92659, 23372, 7513, 85680, 50680, 86472, 67273, 19165, 24925, 38572, 88290, 27079, 6261, 3429, 51217, 92560, 74743, 21077, 49027, 12491, 56845, 33868, 14719, 30391, 13288, 88033, 59937, 91748, 63000, 38379, 4114, 76935, 33696, 30640, 88527, 53354, 92402, 26197, 10448, 40444, 69204, 15310, 48519, 13434, 90437, 39107, 26877, 64596, 1189, 70993, 14829, 27746, 90506, 67724, 65073, 14604, 70777, 40043, 66867, 9924, 17982, 41343, 29054, 71445, 77525, 54741, 46563, 15812, 44185, 94709, 58436, 55188, 18163, 92072, 10750, 72952, 15375, 15301, 11269, 45015, 51985, 20875, 88985, 30975, 47868, 42662, 26649, 54706, 49932, 33966, 17456, 87910, 10069, 18388, 92085, 89615, 1772, 26836, 54491, 77284, 58545, 9368, 28069, 61049, 93651, 43309, 32421, 61042, 10678, 2144, 33174, 85810, 33316, 3052, 52665, 5116, 37793, 37458, 14048, 58578, 99520, 65529, 18172, 2158, 85030, 46410, 34148, 67641, 62098, 11811, 7633, 99673, 62629, 39984, 85482, 26291, 20450, 80456, 35530, 29263, 81157, 66998, 86025, 31952, 49253, 83657, 13912, 51511, 68740, 86997, 86609, 20839, 73334, 43929, 50115, 98671, 16194, 85446, 12947, 73587, 2131, 73077, 13028, 7801, 4862, 59286, 91126, 42477, 75624, 73225, 64910, 39029, 16651, 90707, 54264, 82114, 16101, 95069, 9078, 77620, 60257, 43732, 42496, 85517, 19145, 88501, 59683, 37155, 86811, 5692, 12016, 45716, 38325, 62497, 48604, 45809, 27740, 71507, 21546, 80024, 16637, 17907, 22608, 37027, 23244, 60356, 29658, 6020, 3132, 51596, 87487, 66000, 13027, 69692, 97436, 11855, 46395, 18410, 33654, 21991, 96757, 14198, 52515, 39243, 30564, 11243, 31674, 57498, 46298, 85519, 39230, 5792, 39137, 90634, 5336, 50642, 76204, 10076, 92912, 46024, 3751, 98349, 46167, 65389, 29439, 33389, 11804, 42257, 61399, 21113, 42386, 68020, 54855, 30488, 37470, 59391, 54068, 829, 79846, 44971, 27244, 55307, 33203, 85648, 85484, 48946, 56074, 87078, 35411, 30650, 5555, 38615, 4873, 91442, 76576, 31702, 9182, 5412, 90546, 5096, 9283, 5094, 41100, 2005, 1870, 43768, 40148, 88693, 62898, 2265, 54748, 12950, 35113, 68918, 30964, 69271, 36201, 55942, 12843, 71622, 56829, 90342, 74984, 33656, 29870, 49604, 78888, 47859, 7789, 46083, 68708, 72736, 64509, 73407, 79657, 37263, 15910, 87066, 25425, 43596, 74728, 23351, 24482, 34635, 98798, 15834, 30731, 23212, 23856, 48924, 76721, 83613, 22727, 10098, 67553, 69518, 95770, 30498, 76799, 48517, 67614, 61493, 85128, 52100, 54587, 8593, 76059, 10241, 963, 14780, 11450, 44940, 72065, 90490, 36348, 15078, 20900, 31487, 26790, 67320, 13513, 57996, 63896, 17985, 57201, 63440, 20553, 238, 20572, 64269, 66198, 19957, 26146, 807, 6649, 60748, 99435, 28916, 4754, 61801, 50368, 87511, 56371, 75112, 9968, 97128, 41526, 8957, 39683, 64776, 34489, 92564, 65425, 46358, 84354, 95723, 34044, 157, 60170, 23621, 71020, 6177, 80210, 22824, 76625, 6184, 92620, 26047, 30240, 34651, 28593, 34502, 79427, 50586, 49192, 40988, 86639, 11956, 61836, 8335, 6581, 87995, 24268, 41538, 5434, 17898, 50796, 3204, 71096, 83051, 59629, 27122, 13220, 4105, 34364, 5907, 91717, 47163, 26172, 87461, 19375, 13670, 456, 50531, 8172, 38696, 8914, 89600, 56771, 32812, 48475, 32678, 33357, 69394, 97825, 42155, 56525, 5053, 77721, 20160, 27984, 91062, 16882, 25940, 51467, 69500, 12625, 72609, 25660, 22528, 54252, 46537, 7827, 5520, 49679, 57608, 10073, 4715, 34680, 38935, 48925, 59113, 14210, 2282, 83142, 17832, 15328, 46175, 23603, 39560, 62157, 5462, 85538, 36626, 36375, 27886, 57957, 92183, 77072, 79241, 15057, 39422, 58874, 69931, 90410, 32374, 38283, 7211, 27, 29384, 83421, 10863, 51944, 11657, 5829, 25909, 34095, 89798, 39233, 13443, 713, 86671, 98886, 29011, 81363, 12584, 56463, 23458, 20502, 30139, 21660, 78332, 84760, 12957, 88334, 69329, 25443, 21317, 25645, 77315, 11240, 62737, 88553, 63597, 37125, 39934, 87359, 33486, 39083, 42100, 11061, 52699, 71223, 54184, 55910, 10330, 73558, 3944, 29493, 85308, 36455, 53207, 53228, 59656, 38997, 87344, 29876, 57794, 10065, 57455, 54665, 23097, 61841, 70902, 14522, 16215, 53188, 6798, 10130, 4698, 37285, 51346, 19788, 14485, 86629, 97715, 94845, 14179, 3970, 57043, 94110, 57978]
# pipes = [[3, 2, 30269], [3, 4, 64385], [6, 7, 2115], [17, 8, 84844], [10, 9, 43736], [9, 11, 64981], [20, 10, 85639], [10, 21, 55738], [10, 22, 85911], [10, 16, 79112], [10, 14, 94700], [11, 12, 19232], [20, 13, 42433], [14, 15, 73876], [16, 18, 30771], [17, 20, 9371], [20, 19, 32278], [23, 24, 60730], [23, 25, 22161], [25, 26, 10898], [26, 27, 34904], [31, 29, 36328], [29, 30, 40659], [31, 32, 3214], [34, 33, 28017], [34, 35, 66052], [36, 38, 27063], [38, 37, 33712], [42, 39, 31778], [39, 41, 52644], [42, 40, 91095], [41, 48, 47022], [42, 47, 7780], [42, 46, 44950], [42, 43, 16216], [43, 45, 66512], [46, 44, 70390], [51, 49, 54378], [49, 53, 4852], [49, 57, 18480], [51, 50, 76533], [51, 56, 16071], [51, 55, 29220], [55, 52, 52237], [52, 54, 35303], [59, 58, 88320], [65, 61, 64709], [64, 62, 44461], [64, 63, 77729], [65, 64, 35580], [72, 66, 1243], [66, 77, 35510], [71, 67, 4852], [73, 68, 82536], [69, 70, 34920], [69, 73, 9461], [69, 71, 47441], [69, 78, 34331], [69, 81, 23422], [69, 76, 69520], [70, 83, 61022], [70, 82, 40700], [71, 72, 45477], [72, 80, 36454], [73, 79, 20645], [83, 74, 15599], [76, 75, 56954], [90, 84, 3122], [93, 85, 64676], [94, 86, 67081], [86, 95, 40027], [86, 89, 53644], [86, 88, 49155], [86, 93, 87182], [96, 87, 76779], [96, 90, 35396], [98, 91, 31554], [96, 92, 2781], [92, 99, 27651], [94, 96, 63691], [95, 98, 23211], [96, 100, 52997], [99, 97, 50811], [107, 101, 660], [116, 102, 42670], [102, 113, 64571], [102, 129, 9265], [132, 103, 51738], [103, 123, 46944], [137, 104, 93642], [135, 105, 10491], [109, 106, 12746], [107, 109, 76743], [107, 115, 16644], [107, 122, 38535], [107, 135, 65964], [107, 125, 9929], [107, 126, 15251], [135, 108, 59566], [109, 111, 19033], [109, 116, 22983], [109, 127, 17366], [109, 141, 15119], [135, 110, 40217], [110, 130, 25434], [110, 136, 12553], [111, 139, 13038], [133, 112, 9115], [151, 114, 7643], [115, 132, 64772], [115, 147, 3972], [116, 149, 35415], [116, 137, 93811], [132, 117, 77924], [151, 118, 23953], [118, 142, 42332], [143, 119, 81572], [129, 120, 44737], [146, 121, 35681], [121, 138, 70289], [122, 133, 37501], [143, 124, 31987], [126, 150, 63915], [127, 148, 67375], [132, 128, 83702], [139, 131, 70213], [132, 151, 75959], [135, 134, 43585], [135, 146, 2318], [135, 145, 41959], [149, 140, 54643], [149, 143, 86265], [143, 144, 13272], [153, 152, 6896], [154, 155, 33745], [157, 156, 23707], [157, 158, 67666], [159, 160, 694], [159, 171, 53621], [159, 172, 44244], [160, 161, 38134], [160, 168, 27141], [160, 176, 16455], [160, 209, 4094], [160, 223, 15576], [161, 167, 57468], [161, 200, 45209], [185, 162, 48215], [215, 163, 62330], [163, 227, 34317], [225, 164, 55563], [197, 165, 40097], [165, 208, 61773], [165, 226, 66756], [237, 166, 35536], [167, 178, 44932], [167, 190, 57740], [167, 192, 1991], [167, 204, 22060], [167, 215, 54520], [167, 235, 53146], [168, 170, 28292], [168, 173, 46504], [168, 180, 12824], [168, 214, 39052], [209, 169, 272], [170, 217, 40024], [170, 240, 60177], [170, 259, 17495], [171, 234, 65530], [172, 254, 55828], [173, 193, 69761], [173, 244, 4277], [173, 228, 62887], [173, 220, 52272], [197, 174, 87502], [193, 175, 26914], [176, 184, 13235], [176, 197, 1762], [176, 233, 32308], [176, 256, 63073], [225, 177, 26914], [177, 201, 39993], [178, 179, 15511], [179, 183, 64679], [179, 199, 43412], [179, 238, 44055], [180, 189, 20571], [180, 252, 76460], [236, 181, 11970], [225, 182, 55977], [183, 207, 31800], [183, 231, 38191], [183, 237, 82634], [183, 185, 67629], [183, 203, 39114], [257, 186, 13253], [209, 187, 18241], [187, 202, 26653], [247, 188, 70775], [188, 248, 75422], [189, 213, 11344], [189, 229, 87630], [190, 191, 21191], [190, 216, 33919], [192, 205, 3472], [192, 211, 72002], [193, 242, 49868], [240, 194, 85010], [244, 195, 49169], [207, 196, 10709], [197, 225, 33306], [197, 210, 13144], [252, 198, 64075], [199, 230, 19543], [199, 247, 28241], [201, 260, 4469], [205, 243, 58204], [215, 206, 27954], [208, 218, 41215], [209, 221, 47815], [209, 258, 41201], [209, 241, 9090], [209, 232, 29442], [211, 246, 25055], [222, 212, 19217], [215, 219, 19983], [217, 222, 22222], [221, 239, 52716], [253, 224, 49970], [225, 257, 67268], [229, 251, 92788], [230, 249, 40294], [230, 253, 24533], [231, 255, 38029], [235, 236, 28229], [250, 245, 13667], [258, 250, 32670], [261, 262, 26684], [273, 263, 22312], [263, 274, 52219], [263, 270, 89778], [274, 264, 77376], [264, 268, 15048], [273, 265, 6772], [278, 266, 27531], [266, 267, 48526], [267, 277, 44133], [271, 269, 24267], [278, 271, 28453], [273, 272, 65874], [276, 273, 25460], [274, 275, 8108], [276, 278, 2568], [281, 279, 8646], [282, 280, 15041], [282, 281, 77020], [281, 283, 63547], [284, 285, 57318], [286, 288, 56293], [286, 289, 40308], [286, 298, 13648], [286, 324, 67728], [286, 401, 88518], [286, 296, 3867], [412, 287, 7405], [287, 308, 37249], [288, 341, 67775], [288, 410, 5314], [288, 346, 81886], [289, 291, 66572], [289, 295, 25713], [289, 334, 4787], [289, 290, 54451], [289, 313, 56329], [291, 294, 64972], [291, 299, 40618], [291, 327, 161], [291, 344, 42543], [291, 350, 24514], [291, 417, 14423], [393, 292, 68982], [361, 293, 32648], [294, 309, 17143], [294, 356, 62543], [295, 310, 1235], [295, 316, 14545], [295, 320, 60159], [295, 400, 56060], [295, 374, 56978], [340, 297, 7480], [298, 301, 7549], [298, 362, 45087], [298, 315, 57663], [299, 303, 82641], [299, 305, 87950], [299, 331, 53504], [299, 359, 43418], [299, 397, 83150], [319, 300, 71888], [300, 420, 89686], [301, 347, 57357], [301, 372, 53367], [301, 379, 27159], [382, 302, 16601], [303, 378, 9603], [303, 339, 14415], [303, 369, 48726], [303, 392, 73716], [303, 322, 34092], [303, 409, 46856], [405, 304, 18764], [304, 406, 13914], [305, 360, 43494], [305, 368, 69480], [353, 306, 56823], [309, 307, 36052], [423, 311, 34638], [411, 312, 13227], [421, 314, 1424], [314, 407, 36711], [353, 317, 40310], [360, 318, 3365], [333, 319, 32559], [320, 333, 59196], [320, 340, 25735], [320, 399, 46331], [345, 321, 84761], [351, 323, 42477], [323, 384, 22508], [324, 325, 25364], [325, 349, 55166], [325, 394, 11194], [372, 326, 27877], [326, 418, 40033], [327, 355, 23810], [424, 328, 5227], [360, 329, 4237], [329, 402, 53788], [344, 330, 33619], [331, 377, 3452], [331, 405, 36232], [360, 332, 14071], [332, 375, 15369], [332, 415, 46273], [333, 373, 47526], [334, 380, 16711], [383, 335, 76476], [421, 336, 38015], [336, 408, 20585], [421, 337, 6831], [337, 370, 76198], [369, 338, 22576], [340, 357, 39351], [341, 345, 51479], [341, 343, 58141], [353, 342, 54150], [344, 351, 91277], [344, 358, 37590], [345, 353, 31649], [345, 412, 80667], [347, 366, 49095], [347, 383, 2051], [347, 424, 6062], [393, 348, 42108], [348, 388, 35456], [349, 352, 71708], [349, 381, 39232], [349, 396, 78688], [350, 411, 12819], [351, 361, 72908], [353, 391, 69432], [358, 354, 25143], [356, 367, 30636], [357, 425, 37723], [358, 371, 52539], [358, 404, 30270], [358, 376, 68521], [361, 365, 80191], [361, 413, 56119], [361, 382, 36121], [389, 363, 66486], [405, 364, 6640], [367, 414, 2145], [367, 419, 14971], [369, 386, 5637], [369, 416, 26628], [371, 421, 31314], [372, 393, 21637], [408, 385, 15072], [413, 387, 17770], [411, 389, 67685], [405, 390, 58540], [392, 422, 73839], [410, 395, 42940], [397, 403, 10071], [422, 398, 7465], [408, 423, 34045], [440, 426, 3436], [449, 427, 83063], [427, 447, 74902], [432, 428, 8634], [447, 429, 16549], [430, 436, 31880], [430, 449, 18420], [430, 431, 21536], [430, 441, 19141], [436, 432, 51647], [432, 439, 68072], [432, 444, 21665], [432, 445, 34057], [432, 437, 37847], [436, 433, 46530], [451, 434, 22523], [449, 435, 13932], [449, 438, 41835], [438, 450, 47267], [449, 440, 8643], [440, 442, 66300], [440, 448, 60437], [440, 443, 17030], [449, 446, 68886], [449, 451, 9036], [459, 454, 16516], [454, 460, 79561], [456, 455, 86604], [458, 456, 24223], [459, 457, 4160], [457, 461, 16773], [459, 458, 14405], [462, 463, 33042], [467, 464, 17674], [464, 465, 26458], [467, 466, 8313], [467, 468, 47005], [470, 469, 11570], [479, 471, 2698], [471, 478, 29819], [471, 475, 45894], [471, 474, 49319], [478, 472, 17680], [475, 473, 7700], [475, 476, 10020], [476, 477, 5107], [478, 480, 56787], [485, 481, 15372], [481, 482, 44618], [485, 483, 46238], [485, 484, 18157], [488, 486, 4041], [487, 488, 3977], [499, 489, 40530], [501, 490, 18345], [505, 491, 67678], [501, 492, 39003], [492, 506, 8048], [503, 493, 10067], [501, 494, 46491], [494, 496, 81356], [494, 504, 58858], [501, 495, 37496], [495, 505, 50378], [496, 498, 4278], [496, 499, 20182], [502, 497, 41287], [506, 500, 66331], [503, 501, 46281], [501, 502, 55470], [509, 507, 13063], [507, 511, 25559], [509, 508, 13083], [508, 510, 17946], [521, 515, 13280], [515, 517, 33054], [520, 516, 63782], [517, 522, 4972], [523, 518, 16128], [520, 519, 69408], [521, 520, 72979], [521, 523, 13236], [525, 527, 72635], [527, 526, 14393], [527, 528, 18610], [530, 529, 92449], [537, 532, 55968], [532, 534, 10506], [539, 533, 69172], [535, 539, 66523], [537, 536, 13992], [538, 537, 45061], [539, 538, 21895], [541, 540, 53419], [544, 542, 14508], [552, 543, 29545], [545, 544, 57472], [553, 545, 10778], [553, 546, 46431], [546, 563, 73187], [547, 552, 49107], [547, 553, 57160], [547, 548, 11042], [548, 549, 17955], [561, 550, 30335], [552, 551, 47608], [552, 565, 45172], [552, 554, 4245], [553, 561, 27413], [553, 564, 45322], [553, 560, 58668], [564, 555, 9146], [555, 562, 15604], [555, 559, 25989], [558, 556, 32376], [564, 557, 35825], [564, 558, 9043], [566, 567, 18306], [572, 568, 13196], [572, 569, 11475], [572, 570, 33524], [571, 572, 38159], [573, 577, 33551], [573, 583, 53114], [577, 574, 41786], [574, 581, 29576], [574, 579, 32439], [584, 575, 29162], [581, 576, 30587], [577, 578, 37152], [577, 584, 68543], [583, 580, 3176], [584, 582, 23868], [587, 589, 17635], [589, 588, 48859], [592, 591, 49097], [592, 596, 53090], [595, 593, 45121], [595, 594, 49128], [596, 595, 32426], [598, 597, 69403], [601, 599, 1667], [601, 600, 51479], [602, 601, 68874], [602, 603, 25994], [605, 606, 6770], [609, 607, 58239], [612, 608, 6732], [608, 609, 85293], [611, 610, 70709], [611, 612, 49855], [619, 613, 11824], [620, 614, 63303], [618, 615, 14855], [615, 616, 5458], [615, 617, 54693], [616, 619, 46831], [618, 620, 77565], [620, 622, 29752], [620, 621, 2675], [623, 624, 65341], [623, 625, 38787], [626, 627, 39924], [635, 630, 56377], [633, 631, 3186], [635, 632, 58025], [637, 633, 41056], [635, 634, 63609], [636, 635, 34629], [637, 636, 2757], [640, 639, 69068], [640, 641, 54427], [640, 642, 52883], [648, 646, 30551], [646, 649, 5649], [650, 647, 12274], [648, 650, 32310], [656, 654, 3448], [654, 655, 51541], [657, 659, 55561], [657, 658, 3618], [664, 662, 3031], [662, 663, 44635], [666, 665, 73324], [693, 667, 67496], [686, 668, 84585], [668, 670, 22595], [680, 669, 96333], [671, 684, 35133], [671, 686, 49749], [671, 688, 75213], [697, 672, 31141], [703, 673, 64752], [673, 677, 92359], [673, 681, 77972], [673, 682, 68233], [705, 674, 23285], [681, 675, 41386], [701, 676, 64775], [676, 700, 68249], [677, 687, 3603], [688, 678, 22973], [708, 679, 31484], [679, 694, 63006], [693, 680, 66948], [688, 683, 45219], [683, 692, 77236], [688, 685, 85382], [685, 696, 49386], [687, 705, 39415], [688, 701, 85281], [688, 702, 92648], [688, 697, 72509], [688, 703, 54935], [688, 689, 19739], [688, 708, 86174], [688, 707, 29019], [702, 690, 16167], [702, 691, 19735], [701, 693, 64507], [700, 695, 11298], [697, 706, 70090], [705, 698, 36783], [708, 699, 30617], [703, 704, 88111], [718, 709, 71964], [709, 730, 1848], [710, 716, 94495], [721, 711, 51195], [718, 712, 32869], [719, 713, 64222], [725, 714, 34953], [719, 715, 86479], [716, 718, 95247], [716, 724, 44964], [716, 722, 52943], [716, 729, 4251], [716, 728, 17750], [718, 717, 38528], [718, 727, 45878], [718, 726, 17222], [730, 719, 22353], [721, 720, 5018], [724, 721, 45198], [721, 725, 55888], [730, 723, 25813], [752, 731, 76403], [749, 732, 18406], [732, 738, 45827], [733, 746, 77389], [733, 748, 9984], [733, 750, 42979], [733, 734, 817], [737, 735, 42618], [735, 751, 54118], [750, 736, 6972], [736, 740, 14315], [736, 752, 10137], [736, 742, 58494], [746, 737, 2711], [737, 741, 31140], [737, 739, 3409], [740, 744, 32231], [747, 743, 40777], [753, 745, 14295], [746, 749, 13583], [749, 747, 80622], [748, 753, 65526], [755, 760, 40491], [760, 756, 48270], [760, 757, 38478], [757, 759, 37617], [760, 758, 72599], [766, 761, 23709], [776, 762, 27674], [776, 763, 29451], [766, 764, 80805], [764, 772, 11046], [764, 779, 24434], [766, 765, 5379], [766, 769, 10939], [769, 767, 48803], [776, 768, 17574], [768, 770, 9968], [769, 778, 34095], [769, 776, 86687], [769, 773, 28853], [775, 771, 2613], [778, 774, 43023], [776, 775, 43751], [776, 777, 71128], [784, 780, 56754], [784, 781, 59331], [784, 782, 82808], [783, 785, 64810], [785, 784, 51217], [796, 787, 3689], [787, 794, 23506], [796, 788, 10077], [788, 795, 4799], [793, 789, 71713], [795, 790, 3776], [796, 791, 9629], [791, 792, 17834], [796, 793, 1420], [800, 798, 30089], [802, 799, 62567], [799, 811, 28861], [807, 800, 9003], [800, 813, 31266], [809, 801, 9311], [801, 812, 23375], [813, 802, 39233], [802, 810, 23204], [802, 804, 21399], [814, 803, 63311], [814, 805, 59119], [814, 806, 15180], [807, 809, 38342], [813, 808, 47428], [813, 814, 3124], [819, 817, 45100], [817, 818, 81349], [820, 830, 64967], [820, 828, 39298], [820, 831, 29287], [820, 824, 41643], [831, 821, 17475], [830, 822, 50055], [828, 823, 34905], [830, 825, 52037], [825, 829, 74787], [828, 826, 1836], [831, 827, 35706], [832, 835, 12931], [832, 833, 16541], [832, 834, 49797], [836, 837, 88949], [839, 838, 49417], [840, 839, 59207], [842, 841, 27174], [844, 843, 15012], [843, 845, 31161], [849, 848, 43855], [848, 852, 90522], [850, 849, 4712], [849, 851, 81169], [854, 853, 93130], [854, 855, 76992], [867, 856, 47324], [860, 857, 43129], [857, 862, 9137], [868, 858, 88321], [858, 860, 4414], [863, 859, 19267], [860, 861, 74716], [864, 863, 32397], [865, 864, 51338], [865, 868, 4810], [867, 866, 23033], [868, 867, 48112], [869, 870, 12734], [869, 871, 32848], [873, 872, 71217], [875, 874, 4644], [875, 876, 17296], [878, 877, 7391], [877, 879, 4294], [882, 881, 24461], [886, 884, 18274], [885, 886, 54747], [888, 887, 50747], [892, 888, 66368], [888, 889, 51842], [891, 890, 40401], [891, 892, 56138], [897, 896, 43803], [899, 897, 4767], [897, 900, 62971], [897, 898, 36956], [902, 903, 29933], [905, 906, 15883], [910, 907, 28573], [907, 919, 8925], [915, 908, 31636], [914, 909, 66030], [909, 911, 68921], [909, 920, 90152], [911, 910, 1326], [910, 915, 32796], [911, 916, 6460], [916, 912, 1667], [914, 913, 25509], [914, 917, 12673], [917, 918, 5763], [922, 923, 20070], [922, 925, 6282], [923, 924, 35787], [926, 927, 32702], [932, 928, 26625], [930, 929, 52848], [929, 933, 54955], [932, 930, 45237], [932, 931, 7171], [934, 935, 45295], [939, 936, 75514], [939, 937, 8199], [940, 938, 18683], [938, 939, 41397], [943, 946, 27282], [946, 944, 919], [946, 945, 16146], [948, 947, 3139], [949, 960, 36080], [949, 951, 31563], [949, 952, 48816], [949, 959, 46512], [949, 961, 51804], [960, 950, 14736], [951, 955, 54720], [952, 957, 22657], [959, 953, 7354], [957, 954, 8545], [955, 956, 5425], [959, 958, 84863], [964, 963, 29487], [968, 965, 75362], [968, 966, 64954], [968, 967, 62970], [967, 969, 3021], [975, 970, 33720], [973, 971, 5671], [974, 972, 44442], [973, 975, 10734], [975, 974, 38471], [978, 977, 23274], [990, 979, 58624], [979, 980, 57594], [979, 982, 19997], [980, 983, 54249], [980, 986, 70528], [980, 984, 75540], [980, 989, 45025], [990, 981, 62204], [982, 985, 17113], [983, 988, 26907], [990, 987, 78743], [990, 991, 46085], [996, 993, 55775], [993, 994, 36010], [993, 997, 14100], [996, 995, 29597], [1002, 998, 58621], [999, 1002, 13396], [1002, 1000, 3364], [1002, 1001, 36796], [1004, 1005, 51154], [1007, 1006, 1608], [1007, 1009, 25217], [1007, 1008, 5780], [1009, 1010, 31411], [1012, 1011, 10299], [1011, 1013, 54028], [1015, 1014, 49616], [1017, 1018, 1664], [1020, 1019, 49372], [1022, 1023, 33548], [1022, 1024, 49826], [1022, 1028, 59012], [1023, 1030, 33274], [1024, 1027, 1150], [1024, 1026, 7500], [1030, 1025, 20153], [1025, 1029, 46946], [1032, 1033, 42437], [1037, 1034, 28122], [1037, 1035, 9332], [1035, 1036, 37605], [1041, 1042, 1120], [1041, 1043, 18282], [1058, 1044, 3167], [1044, 1055, 77927], [1058, 1045, 30774], [1045, 1051, 35099], [1051, 1046, 83307], [1046, 1048, 65791], [1056, 1047, 67924], [1056, 1049, 25948], [1053, 1050, 16006], [1051, 1056, 60391], [1056, 1052, 1750], [1058, 1053, 64581], [1056, 1054, 27110], [1058, 1057, 5945], [1060, 1059, 38274], [1062, 1061, 17912], [1066, 1063, 42673], [1067, 1064, 29487], [1067, 1065, 19013], [1067, 1066, 55761], [1068, 1070, 9806], [1070, 1069, 26260], [1070, 1071, 92725], [1079, 1072, 68142], [1078, 1073, 15781], [1083, 1074, 34522], [1080, 1075, 1435], [1085, 1076, 50925], [1083, 1077, 61194], [1077, 1081, 43429], [1082, 1078, 38652], [1084, 1079, 70614], [1080, 1083, 31287], [1084, 1082, 46162], [1082, 1085, 43158], [1083, 1084, 63188], [1087, 1086, 22527], [1090, 1088, 27299], [1088, 1091, 41551], [1090, 1089, 44613], [1095, 1093, 60069], [1093, 1094, 10125], [1096, 1097, 43130], [1099, 1100, 71936], [1102, 1103, 42603], [1110, 1108, 20215], [1109, 1111, 18022], [1111, 1110, 68221], [1114, 1113, 49543], [1115, 1116, 63974], [1119, 1118, 47413], [1124, 1125, 35012], [1124, 1126, 21897], [1132, 1128, 19217], [1132, 1129, 36080], [1132, 1130, 22789], [1130, 1131, 5520], [1135, 1134, 38472], [1137, 1136, 59080], [1136, 1139, 91953], [1137, 1138, 93346], [1145, 1141, 3721], [1145, 1142, 46162], [1144, 1143, 34305], [1143, 1145, 45145], [1146, 1147, 493], [1151, 1149, 2935], [1151, 1150, 23096], [1150, 1155, 37938], [1151, 1152, 81426], [1152, 1153, 27492], [1152, 1156, 37992], [1155, 1154, 57282], [1159, 1158, 58231], [1162, 1161, 11874], [1164, 1163, 91909], [1171, 1167, 40484], [1171, 1168, 68228], [1171, 1169, 51029], [1169, 1172, 36996], [1172, 1170, 22585], [1173, 1171, 46318], [1186, 1175, 7279], [1186, 1176, 60405], [1176, 1184, 25696], [1177, 1181, 31665], [1177, 1186, 7520], [1177, 1182, 21013], [1186, 1178, 19323], [1182, 1179, 63634], [1179, 1183, 81563], [1181, 1180, 21241], [1181, 1185, 44602], [1182, 1187, 21833], [1190, 1188, 85100], [1189, 1190, 43503], [1195, 1194, 9705], [1200, 1199, 18003], [1199, 1208, 51676], [1205, 1200, 33060], [1203, 1201, 73693], [1201, 1202, 150], [1205, 1203, 31217], [1203, 1206, 46535], [1205, 1204, 56270], [1205, 1207, 20935], [1212, 1211, 3667], [1215, 1218, 64717], [1215, 1220, 86270], [1215, 1216, 14623], [1219, 1217, 50078], [1218, 1221, 3520], [1220, 1219, 19578], [1221, 1222, 23929], [1227, 1225, 64558], [1226, 1227, 21523], [1230, 1228, 6545], [1228, 1229, 25758], [1234, 1232, 3108], [1232, 1235, 34397], [1232, 1233, 54410], [1234, 1236, 50145], [1237, 1238, 9913], [1238, 1239, 82236], [1249, 1241, 33687], [1249, 1242, 61457], [1242, 1243, 60592], [1245, 1244, 72975], [1249, 1245, 79416], [1246, 1249, 16961], [1246, 1247, 24786], [1249, 1248, 4230], [1251, 1250, 8951], [1253, 1252, 35528], [1254, 1253, 22017], [1256, 1255, 43277], [1260, 1259, 27475], [1265, 1262, 18530], [1263, 1264, 40824], [1263, 1265, 52911], [1266, 1267, 67616], [1269, 1270, 86957], [1271, 1272, 34218], [1279, 1277, 20033], [1277, 1281, 16786], [1277, 1278, 61321], [1279, 1280, 65519], [1282, 1283, 57960], [1290, 1288, 49036], [1288, 1291, 50184], [1290, 1289, 47258], [1294, 1292, 10978], [1300, 1293, 44945], [1293, 1297, 81705], [1295, 1294, 67779], [1294, 1299, 59737], [1306, 1295, 66745], [1301, 1296, 14841], [1296, 1307, 52285], [1297, 1305, 68216], [1298, 1306, 48746], [1306, 1300, 5680], [1306, 1301, 2837], [1306, 1302, 38691], [1302, 1304, 2872], [1306, 1303, 68284], [1309, 1308, 54680], [1309, 1311, 2639], [1309, 1316, 24538], [1316, 1310, 37380], [1311, 1318, 40808], [1311, 1314, 73592], [1314, 1312, 75793], [1314, 1313, 61470], [1314, 1315, 70548], [1314, 1317, 1311], [1325, 1320, 35477], [1324, 1321, 13974], [1321, 1322, 12588], [1324, 1323, 61310], [1325, 1324, 75663], [1327, 1326, 16627], [1327, 1328, 87018], [1330, 1329, 44916], [1332, 1331, 22769], [1337, 1333, 15377], [1333, 1334, 38843], [1334, 1338, 18628], [1335, 1337, 3282], [1335, 1336, 69239], [1340, 1339, 52759], [1343, 1340, 17057], [1340, 1350, 14288], [1340, 1344, 8187], [1343, 1341, 5519], [1343, 1342, 7105], [1342, 1347, 53574], [1350, 1345, 31037], [1349, 1346, 50055], [1349, 1348, 27437], [1350, 1349, 3081], [1358, 1355, 71990], [1356, 1358, 37108], [1358, 1357, 52652], [1362, 1359, 11008], [1361, 1360, 29093], [1360, 1362, 36480], [1361, 1363, 32234], [1365, 1364, 40634], [1366, 1367, 63235], [1371, 1373, 56081], [1373, 1372, 37102], [1377, 1375, 65680], [1377, 1376, 21304], [1378, 1379, 25855], [1379, 1380, 8575], [1387, 1383, 4791], [1384, 1387, 71732], [1387, 1385, 56750], [1387, 1386, 6131], [1389, 1390, 6679], [1391, 1392, 24445], [1394, 1393, 41664], [1401, 1398, 92400], [1398, 1399, 75774], [1398, 1400, 12986], [1405, 1404, 47197], [1410, 1406, 7335], [1406, 1407, 16564], [1407, 1409, 54776], [1410, 1408, 36393], [1413, 1414, 60017], [1417, 1415, 24494], [1418, 1416, 53844], [1418, 1417, 36501], [1424, 1422, 41230], [1423, 1424, 52331], [1431, 1428, 43142], [1429, 1430, 28044], [1429, 1431, 18784], [1434, 1433, 71928], [1433, 1435, 77713], [1437, 1436, 12721], [1440, 1439, 25903], [1440, 1441, 24792], [1442, 1443, 53213], [1442, 1445, 73889], [1443, 1444, 87245], [1449, 1448, 31842], [1452, 1451, 8734], [1457, 1458, 21385], [1460, 1459, 32361], [1460, 1462, 3423], [1460, 1461, 52690], [1464, 1465, 45742], [1471, 1468, 24198], [1468, 1469, 41242], [1471, 1470, 13561], [1475, 1472, 10769], [1472, 1474, 34755], [1472, 1473, 22761], [1475, 1477, 53315], [1477, 1476, 44928], [1480, 1479, 29954], [1485, 1484, 35055], [1487, 1486, 75996], [1488, 1487, 50918], [1490, 1494, 41754], [1490, 1492, 20474], [1492, 1491, 50950], [1492, 1493, 75435], [1494, 1495, 3702], [1499, 1498, 52963], [1502, 1501, 30644], [1507, 1505, 29179], [1506, 1507, 74146], [1510, 1511, 61328], [1518, 1520, 33114], [1518, 1519, 42262], [1522, 1521, 36943], [1524, 1523, 12523], [1524, 1526, 11827], [1526, 1525, 44337], [1530, 1531, 20414], [1535, 1534, 41587], [1538, 1535, 4586], [1538, 1536, 42706], [1536, 1541, 68399], [1538, 1537, 24260], [1537, 1540, 70782], [1537, 1539, 63355], [1541, 1542, 25202], [1547, 1545, 19413], [1547, 1546, 26246], [1548, 1549, 68390], [1552, 1550, 52026], [1551, 1553, 11472], [1553, 1552, 42073], [1556, 1558, 8039], [1556, 1557, 84200], [1561, 1559, 34614], [1561, 1560, 3109], [1563, 1562, 92693], [1570, 1566, 57458], [1570, 1567, 24699], [1572, 1568, 51540], [1570, 1569, 52932], [1569, 1572, 8285], [1569, 1571, 62525], [1576, 1574, 78913], [1574, 1577, 6354], [1574, 1575, 10878], [1578, 1579, 14232], [1578, 1580, 15569], [1582, 1583, 23657], [1588, 1587, 41648], [1593, 1594, 27941], [1597, 1596, 77111], [1601, 1600, 53905], [1606, 1607, 3916], [1606, 1608, 17762], [1614, 1613, 88869], [1613, 1615, 40184], [1617, 1618, 36372], [1622, 1623, 27950], [1631, 1630, 41034], [1633, 1632, 3947], [1639, 1638, 58408], [1640, 1641, 16293], [1640, 1642, 44907], [1650, 1649, 16784], [1653, 1651, 65723], [1653, 1652, 45012], [1655, 1654, 1843], [1657, 1656, 12748], [1658, 1659, 1318], [1665, 1663, 73680], [1665, 1664, 71906], [1667, 1666, 45912], [1670, 1669, 57861], [1673, 1670, 34510], [1673, 1671, 87353], [1673, 1672, 29650], [1678, 1680, 82232], [1680, 1679, 31825], [1687, 1688, 52684], [1688, 1689, 35972], [1693, 1694, 34399], [1703, 1701, 12510], [1703, 1702, 1163], [1706, 1705, 26799], [1705, 1707, 34114], [1708, 1709, 78922], [1710, 1711, 31787], [1713, 1712, 41782], [1713, 1714, 77773], [1713, 1715, 42342], [1717, 1718, 504], [1718, 1719, 28446], [1723, 1722, 18956], [1723, 1724, 84317], [1727, 1726, 68976], [1731, 1730, 1034], [1736, 1735, 77892], [1737, 1738, 2626], [1741, 1742, 76015], [1743, 1744, 17502], [1748, 1746, 48044], [1748, 1747, 51297], [1754, 1750, 63263], [1755, 1751, 6693], [1753, 1752, 15118], [1754, 1753, 33422], [1755, 1754, 35816], [1758, 1757, 17135], [1761, 1762, 17809], [1763, 1764, 29061], [1765, 1766, 5303], [1765, 1768, 3048], [1766, 1767, 34538], [1775, 1774, 1716], [1775, 1776, 142], [1781, 1782, 33509], [1784, 1783, 77541], [1786, 1787, 11746], [1790, 1791, 11288], [1801, 1800, 31035], [1801, 1802, 33927], [1806, 1805, 38994], [1810, 1808, 64971], [1809, 1810, 52297], [1820, 1819, 45941], [1822, 1821, 2768], [1834, 1832, 10519], [1832, 1833, 15884], [1841, 1842, 62645], [1842, 1843, 52973], [1846, 1847, 62791], [1854, 1852, 42395], [1854, 1853, 24026], [1860, 1859, 33472], [1865, 1862, 62285], [1862, 1866, 35114], [1862, 1863, 2396], [1865, 1864, 65831], [1869, 1871, 63656], [1869, 1872, 42933], [1871, 1870, 5537], [1874, 1875, 72179], [1878, 1879, 36226], [1882, 1883, 15958], [1886, 1887, 71952], [1890, 1889, 34749], [1894, 1893, 32385], [1898, 1897, 3757], [1909, 1907, 51984], [1907, 1910, 48181], [1909, 1908, 21564], [1912, 1911, 40216], [1920, 1919, 8645], [1933, 1934, 29731], [1938, 1939, 533], [1944, 1945, 33779], [1945, 1947, 43138], [1945, 1946, 95390], [1951, 1950, 19543], [1955, 1954, 81587], [1964, 1963, 95297], [1965, 1964, 88133], [1967, 1968, 1681], [1970, 1971, 11792], [1976, 1975, 44672], [1983, 1984, 48105], [1991, 1990, 14142], [1998, 1996, 35956], [1997, 1998, 63588], [1999, 2001, 18435], [1999, 2000, 34681], [2005, 2003, 13498], [2004, 2005, 35853], [2007, 2008, 33498], [2024, 2023, 3185], [2046, 2047, 29520], [2060, 2059, 29322], [2065, 2066, 32861], [2073, 2072, 20603], [2074, 2073, 75790], [2104, 2103, 64416], [2124, 2125, 69036]]

# well_status=[1, 21, 24, 28, 30, 31, 32, 33, 34, 37, 40, 41, 52, 57, 58, 59, 62, 69, 71, 72, 77, 78, 79, 80, 83, 86, 95, 96, 97, 99, 101, 103, 105, 106, 111, 112, 114, 122, 123, 125, 127, 128, 129, 130, 131, 132, 135, 142, 143, 146, 147, 150, 155, 161, 165, 171, 172, 174, 177, 179, 180, 181, 184, 185, 187, 188, 189, 197, 198, 201, 202, 203, 204, 205, 209, 210, 217, 219, 220, 223, 226, 228, 232, 235, 242, 243, 244, 247, 250, 251, 253, 254, 256, 258, 259, 260, 261, 267, 269, 272, 274, 275, 277, 278, 284, 290, 291, 293, 295, 299, 303, 309, 315, 318, 321, 328, 330, 331, 332, 333, 340, 341, 345, 348, 350, 351, 354, 356, 364, 365, 368, 371, 376, 380, 382, 385, 388, 389, 401, 405, 406, 410, 411, 412, 413, 415, 416, 418, 423, 426, 427, 429, 431, 434, 436, 439, 440, 445, 447, 449, 451, 453, 460, 461, 466, 469, 470, 471, 479, 483, 486, 492, 498, 499, 500, 506, 510, 513, 516, 518, 522, 524, 526, 528, 531, 532, 535, 539, 546, 551, 552, 556, 562, 564, 567, 571, 572, 573, 574, 575, 576, 577, 579, 582, 585, 600, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 615, 624, 625, 629, 633, 634, 638, 646, 648, 649, 651, 652, 657, 658, 659, 664, 665, 667, 668, 671, 672, 673, 682, 683, 690, 693, 695, 699, 700, 702, 705, 706, 707, 712, 716, 722, 724, 725, 726, 733, 734, 737, 741, 743, 744, 745, 748, 749, 750, 751, 757, 760, 764, 767, 769, 770, 773, 774, 778, 779, 780, 781, 784, 785, 786, 788, 789, 790, 791, 795, 797, 800, 803, 805, 808, 810, 811, 813, 814, 815, 817, 818, 820, 821, 822, 823, 824, 825, 826, 828, 830, 832, 838, 840, 846, 848, 849, 853, 854, 857, 860, 862, 863, 865, 867, 869, 872, 873, 874, 875, 879, 881, 883, 886, 889, 890, 891, 892, 893, 894, 895, 898, 903, 904, 911, 913, 914, 915, 916, 917, 918, 920, 922, 927, 929, 930, 932, 934, 942, 943, 945, 947, 948, 955, 957, 960, 967, 969, 970, 972, 973, 976, 977, 980, 989, 996, 997, 999, 1001, 1002, 1004, 1005, 1008, 1012, 1014, 1026, 1027, 1032, 1035, 1037, 1038, 1039, 1040, 1043, 1044, 1047, 1049, 1051, 1054, 1055, 1056, 1057, 1058, 1061, 1063, 1066, 1071, 1073, 1074, 1077, 1078, 1079, 1080, 1081, 1082, 1085, 1086, 1088, 1090, 1096, 1105, 1106, 1107, 1110, 1114, 1115, 1118, 1119, 1124, 1126, 1127, 1129, 1132, 1134, 1136, 1137, 1138, 1143, 1144, 1147, 1148, 1149, 1150, 1152, 1154, 1159, 1161, 1167, 1168, 1170, 1172, 1173, 1176, 1180, 1183, 1185, 1188, 1189, 1190, 1191, 1192, 1193, 1195, 1197, 1201, 1204, 1206, 1208, 1211, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1224, 1225, 1227, 1228, 1229, 1236, 1238, 1240, 1243, 1244, 1245, 1246, 1250, 1252, 1253, 1255, 1256, 1257, 1259, 1261, 1262, 1263, 1264, 1265, 1268, 1271, 1273, 1274, 1281, 1285, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1298, 1299, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1311, 1312, 1314, 1318, 1321, 1322, 1323, 1325, 1327, 1330, 1333, 1334, 1336, 1340, 1348, 1349, 1350, 1351, 1354, 1356, 1360, 1361, 1362, 1366, 1368, 1373, 1375, 1377, 1379, 1382, 1383, 1384, 1386, 1387, 1389, 1392, 1395, 1397, 1400, 1405, 1408, 1411, 1412, 1413, 1414, 1415, 1418, 1419, 1421, 1423, 1426, 1427, 1428, 1430, 1434, 1435, 1438, 1444, 1445, 1446, 1448, 1450, 1453, 1454, 1455, 1457, 1458, 1459, 1460, 1461, 1462, 1464, 1465, 1467, 1468, 1471, 1472, 1473, 1474, 1477, 1478, 1480, 1481, 1482, 1485, 1488, 1489, 1490, 1491, 1492, 1494, 1496, 1497, 1498, 1499, 1501, 1502, 1504, 1506, 1508, 1509, 1510, 1511, 1512, 1514, 1516, 1518, 1521, 1522, 1523, 1524, 1526, 1528, 1530, 1531, 1532, 1534, 1536, 1537, 1540, 1541, 1542, 1544, 1546, 1547, 1548, 1550, 1552, 1553, 1555, 1557, 1560, 1561, 1562, 1564, 1565, 1566, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 1575, 1577, 1578, 1579, 1580, 1583, 1584, 1587, 1588, 1589, 1591, 1592, 1594, 1595, 1596, 1597, 1598, 1599, 1601, 1602, 1603, 1605, 1607, 1608, 1610, 1611, 1612, 1613, 1615, 1617, 1618, 1619, 1621, 1622, 1624, 1626, 1630, 1631, 1632, 1633, 1635, 1637, 1639, 1640, 1642, 1644, 1645, 1647, 1648, 1653, 1655, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1667, 1668, 1669, 1671, 1672, 1677, 1679, 1680, 1681, 1682, 1685, 1686, 1687, 1688, 1689, 1691, 1692, 1693, 1694, 1695, 1696, 1698, 1700, 1701, 1703, 1704, 1707, 1708, 1709, 1711, 1712, 1713, 1717, 1718, 1719, 1720, 1721, 1723, 1724, 1725, 1726, 1728, 1729, 1731, 1732, 1733, 1735, 1736, 1737, 1738, 1739, 1741, 1743, 1745, 1747, 1749, 1750, 1751, 1752, 1753, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1766, 1767, 1768, 1771, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1782, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1798, 1800, 1801, 1803, 1806, 1809, 1810, 1811, 1812, 1814, 1815, 1817, 1818, 1820, 1821, 1823, 1824, 1825, 1827, 1828, 1830, 1832, 1834, 1835, 1836, 1837, 1838, 1840, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1860, 1861, 1862, 1863, 1864, 1866, 1867, 1870, 1871, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1882, 1883, 1884, 1886, 1888, 1889, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1909, 1910, 1911, 1913, 1914, 1915, 1917, 1918, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1935, 1936, 1937, 1939, 1940, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1952, 1953, 1955, 1956, 1957, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1974, 1976, 1977, 1978, 1979, 1981, 1983, 1984, 1986, 1987, 1988, 1989, 1990, 1991, 1993, 1994, 1995, 1996, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160]
# pipe_status=[(1, 11), (1, 13), (1, 116), (2, 23), (2, 127), (3, 61), (3, 125), (4, 5), (4, 49), (6, 14), (6, 42), (6, 65), (6, 132), (7, 108), (8, 76), (9, 24), (10, 25), (10, 31), (10, 89), (10, 115), (12, 55), (13, 16), (14, 74), (15, 34), (16, 94), (17, 97), (18, 37), (18, 54), (18, 93), (18, 107), (18, 124), (19, 120), (19, 121), (20, 75), (22, 24), (26, 138), (27, 126), (29, 136), (32, 68), (33, 75), (34, 48), (35, 55), (36, 60), (38, 66), (38, 99), (39, 40), (40, 109), (41, 87), (41, 133), (42, 70), (43, 139), (44, 75), (44, 117), (45, 59), (46, 92), (46, 120), (47, 75), (47, 90), (48, 88), (49, 95), (50, 98), (51, 123), (52, 85), (52, 136), (53, 84), (55, 72), (56, 60), (60, 68), (62, 81), (62, 98), (62, 139), (63, 103), (64, 67), (64, 96), (65, 126), (69, 73), (72, 140), (73, 91), (73, 119), (76, 97), (78, 104), (79, 120), (82, 129), (82, 134), (84, 101), (87, 108), (100, 123), (102, 128), (107, 137), (110, 125), (112, 118), (113, 137), (123, 138), (141, 142), (143, 157), (144, 164), (145, 149), (146, 164), (147, 148), (147, 153), (147, 158), (149, 154), (149, 156), (149, 161), (149, 162), (151, 168), (152, 166), (155, 167), (157, 159), (157, 160), (157, 166), (163, 166), (166, 168), (169, 171), (170, 171), (173, 184), (173, 186), (174, 175), (175, 176), (177, 183), (178, 180), (182, 185), (189, 196), (190, 195), (191, 200), (192, 197), (193, 202), (194, 198), (195, 199), (197, 200), (199, 202), (205, 206), (207, 211), (208, 209), (208, 211), (209, 212), (209, 213), (214, 217), (215, 217), (216, 223), (217, 218), (217, 221), (217, 222), (224, 230), (225, 229), (226, 231), (227, 232), (228, 229), (230, 232), (233, 237), (233, 241), (233, 244), (233, 246), (233, 248), (234, 236), (235, 245), (235, 249), (236, 246), (238, 244), (239, 247), (240, 249), (251, 255), (252, 253), (257, 258), (261, 262), (261, 263), (264, 265), (264, 273), (265, 270), (266, 267), (268, 270), (268, 271), (270, 272), (274, 276), (278, 279), (280, 288), (280, 291), (281, 291), (282, 290), (283, 288), (285, 289), (285, 290), (286, 288), (287, 291), (292, 293), (293, 294), (296, 297), (296, 308), (297, 298), (297, 313), (297, 346), (297, 360), (298, 337), (299, 322), (300, 364), (301, 362), (302, 334), (302, 363), (303, 374), (304, 329), (304, 341), (305, 307), (305, 310), (305, 317), (306, 346), (307, 377), (307, 396), (309, 391), (310, 357), (310, 381), (311, 334), (312, 330), (313, 321), (313, 334), (313, 370), (313, 393), (314, 338), (314, 362), (315, 316), (316, 336), (316, 375), (317, 326), (318, 373), (319, 362), (320, 344), (323, 394), (324, 339), (324, 346), (325, 384), (326, 350), (326, 366), (327, 328), (327, 353), (329, 342), (330, 379), (333, 344), (334, 347), (334, 362), (335, 389), (336, 367), (336, 384), (338, 397), (343, 352), (345, 355), (346, 358), (346, 369), (346, 378), (348, 383), (349, 359), (352, 356), (354, 359), (361, 390), (362, 394), (367, 386), (367, 390), (368, 392), (372, 373), (382, 387), (387, 395), (398, 407), (399, 400), (400, 411), (401, 402), (403, 410), (404, 405), (406, 408), (407, 410), (409, 410), (413, 414), (416, 417), (419, 424), (420, 422), (421, 424), (422, 426), (424, 425), (425, 426), (427, 430), (428, 429), (432, 443), (433, 447), (434, 443), (434, 448), (435, 445), (436, 446), (437, 443), (438, 440), (438, 441), (439, 444), (442, 448), (450, 451), (452, 464), (453, 456), (454, 455), (455, 456), (455, 457), (458, 461), (458, 463), (459, 461), (460, 464), (461, 462), (465, 470), (466, 467), (466, 474), (468, 475), (469, 472), (470, 475), (471, 473), (473, 476), (477, 481), (478, 481), (479, 481), (480, 481), (482, 483), (482, 484), (483, 485), (487, 492), (488, 497), (489, 491), (489, 493), (489, 494), (490, 498), (493, 497), (493, 498), (495, 498), (496, 497), (500, 502), (501, 502), (503, 507), (504, 505), (504, 508), (505, 507), (506, 507), (509, 511), (509, 512), (511, 513), (511, 521), (514, 519), (515, 521), (516, 519), (517, 518), (520, 521), (523, 524), (525, 526), (525, 529), (526, 527), (530, 531), (531, 534), (533, 534), (535, 536), (535, 537), (535, 538), (540, 546), (540, 557), (541, 542), (541, 556), (543, 545), (543, 559), (544, 548), (544, 550), (544, 558), (544, 560), (545, 547), (545, 549), (545, 554), (548, 552), (551, 555), (553, 561), (554, 557), (556, 561), (563, 564), (565, 568), (566, 567), (566, 568), (567, 569), (570, 571), (576, 578), (579, 580), (579, 581), (583, 607), (584, 615), (586, 608), (586, 622), (587, 617), (587, 627), (588, 589), (589, 599), (589, 600), (589, 613), (590, 615), (591, 609), (592, 626), (593, 616), (594, 603), (595, 614), (596, 623), (597, 598), (597, 629), (600, 626), (601, 620), (605, 619), (605, 623), (606, 612), (607, 618), (608, 628), (611, 614), (612, 620), (612, 627), (612, 630), (612, 631), (614, 616), (614, 621), (614, 632), (635, 638), (635, 647), (636, 643), (637, 642), (638, 639), (638, 643), (639, 644), (640, 644), (641, 642), (642, 645), (645, 646), (649, 650), (653, 657), (654, 657), (655, 656), (655, 657), (660, 662), (661, 664), (662, 663), (663, 664), (666, 668), (669, 682), (669, 684), (670, 695), (671, 680), (672, 703), (673, 679), (674, 699), (675, 685), (676, 686), (677, 692), (677, 706), (678, 691), (681, 686), (683, 694), (685, 703), (686, 687), (686, 701), (686, 705), (688, 700), (689, 700), (691, 699), (693, 698), (695, 704), (696, 703), (697, 706), (707, 708), (707, 711), (709, 711), (710, 711), (712, 713), (712, 714), (715, 716), (716, 717), (716, 718), (718, 719), (720, 721), (720, 722), (723, 725), (723, 728), (727, 728), (728, 729), (729, 730), (731, 732), (732, 733), (732, 738), (734, 735), (736, 737), (738, 739), (740, 742), (741, 742), (745, 746), (747, 748), (752, 757), (752, 758), (753, 754), (754, 756), (755, 757), (755, 759), (756, 757), (760, 762), (760, 768), (761, 765), (761, 770), (761, 777), (763, 767), (764, 775), (766, 770), (771, 775), (772, 777), (776, 777), (781, 782), (783, 784), (787, 788), (792, 793), (792, 794), (792, 795), (794, 796), (797, 798), (797, 799), (801, 803), (802, 803), (803, 804), (805, 806), (806, 807), (809, 810), (811, 812), (816, 817), (818, 819), (826, 827), (829, 831), (829, 832), (830, 834), (830, 835), (833, 836), (835, 837), (836, 837), (839, 841), (839, 844), (840, 841), (840, 842), (840, 843), (841, 847), (844, 845), (850, 851), (851, 853), (852, 853), (855, 857), (856, 858), (857, 859), (858, 859), (860, 861), (864, 865), (866, 870), (868, 871), (869, 871), (870, 872), (875, 876), (875, 877), (875, 878), (880, 881), (880, 882), (884, 886), (884, 888), (885, 886), (885, 887), (895, 896), (895, 897), (899, 903), (900, 903), (901, 902), (901, 903), (904, 905), (905, 906), (907, 909), (907, 911), (908, 909), (909, 910), (911, 912), (918, 919), (920, 921), (922, 924), (923, 924), (924, 925), (926, 933), (926, 939), (926, 941), (927, 935), (927, 938), (928, 930), (928, 936), (931, 940), (932, 940), (937, 942), (939, 940), (944, 945), (946, 952), (946, 957), (947, 951), (949, 950), (949, 951), (949, 953), (949, 958), (949, 961), (950, 962), (950, 963), (953, 959), (954, 963), (955, 956), (964, 966), (964, 967), (965, 966), (968, 969), (970, 971), (973, 975), (974, 983), (976, 984), (977, 994), (978, 979), (979, 980), (981, 992), (982, 983), (983, 985), (983, 996), (984, 991), (984, 992), (986, 990), (986, 993), (986, 995), (987, 989), (988, 995), (989, 995), (998, 1003), (1000, 1002), (1000, 1003), (1004, 1006), (1007, 1009), (1007, 1010), (1008, 1009), (1011, 1012), (1013, 1014), (1015, 1023), (1016, 1021), (1017, 1020), (1018, 1025), (1019, 1020), (1020, 1021), (1021, 1023), (1022, 1023), (1023, 1026), (1024, 1026), (1025, 1026), (1028, 1036), (1029, 1036), (1029, 1037), (1030, 1036), (1031, 1033), (1033, 1034), (1034, 1035), (1041, 1043), (1042, 1043), (1044, 1045), (1046, 1047), (1046, 1048), (1050, 1052), (1051, 1052), (1053, 1054), (1058, 1059), (1059, 1060), (1061, 1062), (1063, 1064), (1065, 1066), (1066, 1067), (1068, 1071), (1069, 1071), (1070, 1071), (1070, 1072), (1074, 1075), (1076, 1077), (1082, 1083), (1084, 1085), (1086, 1087), (1088, 1089), (1091, 1098), (1091, 1101), (1091, 1102), (1092, 1096), (1093, 1101), (1094, 1106), (1095, 1105), (1097, 1099), (1099, 1106), (1100, 1101), (1101, 1104), (1102, 1103), (1104, 1106), (1107, 1108), (1108, 1109), (1111, 1112), (1111, 1115), (1113, 1114), (1116, 1118), (1117, 1118), (1119, 1120), (1121, 1125), (1122, 1126), (1123, 1124), (1124, 1125), (1128, 1130), (1129, 1130), (1131, 1132), (1131, 1135), (1133, 1134), (1137, 1141), (1137, 1145), (1139, 1144), (1140, 1142), (1140, 1143), (1146, 1167), (1149, 1155), (1151, 1162), (1153, 1161), (1153, 1165), (1153, 1166), (1155, 1163), (1155, 1164), (1156, 1167), (1157, 1158), (1158, 1161), (1158, 1162), (1160, 1167), (1169, 1170), (1171, 1174), (1172, 1174), (1173, 1175), (1176, 1177), (1177, 1178), (1179, 1180), (1181, 1184), (1182, 1183), (1182, 1184), (1185, 1186), (1187, 1188), (1194, 1195), (1195, 1196), (1198, 1199), (1198, 1200), (1198, 1201), (1201, 1203), (1202, 1203), (1204, 1205), (1206, 1207), (1209, 1213), (1210, 1211), (1211, 1212), (1212, 1213), (1214, 1216), (1221, 1222), (1223, 1226), (1225, 1226), (1229, 1230), (1229, 1232), (1230, 1231), (1233, 1235), (1233, 1236), (1234, 1237), (1235, 1237), (1238, 1239), (1238, 1242), (1240, 1241), (1246, 1247), (1247, 1248), (1247, 1249), (1250, 1251), (1254, 1255), (1257, 1258), (1260, 1261), (1263, 1267), (1266, 1267), (1269, 1271), (1269, 1272), (1270, 1271), (1274, 1275), (1276, 1282), (1277, 1283), (1278, 1279), (1278, 1280), (1278, 1281), (1279, 1282), (1283, 1284), (1283, 1285), (1286, 1287), (1293, 1294), (1293, 1296), (1294, 1295), (1297, 1298), (1299, 1300), (1301, 1302), (1301, 1304), (1310, 1313), (1311, 1313), (1314, 1316), (1315, 1316), (1317, 1318), (1317, 1319), (1320, 1321), (1324, 1327), (1326, 1330), (1326, 1331), (1328, 1332), (1329, 1330), (1330, 1332), (1335, 1336), (1337, 1344), (1337, 1346), (1338, 1345), (1338, 1346), (1339, 1343), (1340, 1345), (1341, 1342), (1341, 1346), (1343, 1346), (1347, 1349), (1351, 1353), (1352, 1359), (1353, 1358), (1355, 1360), (1356, 1364), (1357, 1365), (1359, 1365), (1360, 1365), (1361, 1363), (1367, 1368), (1369, 1372), (1369, 1380), (1370, 1374), (1370, 1376), (1370, 1379), (1371, 1380), (1372, 1375), (1373, 1378), (1380, 1381), (1384, 1385), (1388, 1389), (1388, 1390), (1391, 1392), (1392, 1393), (1394, 1395), (1395, 1396), (1398, 1399), (1398, 1400), (1398, 1401), (1402, 1404), (1403, 1404), (1403, 1405), (1406, 1407), (1406, 1408), (1409, 1410), (1409, 1411), (1415, 1416), (1417, 1418), (1420, 1421), (1422, 1424), (1424, 1425), (1425, 1426), (1429, 1434), (1430, 1431), (1430, 1433), (1432, 1433), (1435, 1442), (1436, 1438), (1437, 1440), (1437, 1442), (1438, 1439), (1440, 1441), (1443, 1444), (1446, 1447), (1449, 1451), (1450, 1452), (1451, 1453), (1456, 1457), (1462, 1463), (1465, 1466), (1468, 1469), (1469, 1470), (1474, 1475), (1475, 1476), (1479, 1480), (1482, 1483), (1483, 1484), (1486, 1489), (1487, 1489), (1493, 1494), (1494, 1495), (1500, 1501), (1502, 1503), (1503, 1505), (1507, 1508), (1513, 1514), (1515, 1516), (1515, 1517), (1518, 1519), (1518, 1520), (1524, 1525), (1527, 1529), (1528, 1529), (1532, 1533), (1534, 1535), (1537, 1538), (1537, 1539), (1543, 1544), (1545, 1546), (1547, 1549), (1550, 1551), (1553, 1554), (1555, 1556), (1557, 1558), (1559, 1560), (1563, 1565), (1566, 1567), (1576, 1577), (1580, 1581), (1582, 1583), (1585, 1586), (1586, 1588), (1589, 1590), (1592, 1593), (1599, 1600), (1604, 1606), (1605, 1606), (1609, 1610), (1613, 1614), (1616, 1617), (1619, 1620), (1622, 1623), (1625, 1626), (1627, 1630), (1628, 1629), (1628, 1630), (1633, 1634), (1635, 1636), (1637, 1638), (1641, 1642), (1642, 1643), (1645, 1646), (1649, 1650), (1649, 1653), (1650, 1654), (1651, 1652), (1651, 1653), (1655, 1657), (1656, 1658), (1665, 1666), (1669, 1670), (1671, 1674), (1672, 1675), (1673, 1675), (1676, 1678), (1677, 1678), (1683, 1684), (1683, 1685), (1689, 1690), (1695, 1697), (1698, 1699), (1701, 1702), (1705, 1706), (1706, 1707), (1708, 1710), (1713, 1714), (1714, 1716), (1715, 1716), (1722, 1723), (1727, 1728), (1729, 1730), (1733, 1734), (1739, 1740), (1741, 1742), (1743, 1744), (1745, 1746), (1747, 1748), (1753, 1754), (1755, 1756), (1763, 1764), (1765, 1766), (1769, 1770), (1769, 1772), (1771, 1772), (1772, 1773), (1781, 1782), (1781, 1783), (1784, 1786), (1785, 1786), (1795, 1796), (1796, 1797), (1798, 1799), (1802, 1803), (1804, 1805), (1804, 1807), (1806, 1807), (1808, 1809), (1812, 1813), (1816, 1817), (1818, 1819), (1821, 1822), (1826, 1827), (1829, 1830), (1831, 1832), (1833, 1834), (1839, 1840), (1840, 1841), (1859, 1860), (1865, 1866), (1868, 1869), (1869, 1870), (1872, 1873), (1879, 1880), (1881, 1882), (1885, 1886), (1887, 1888), (1889, 1890), (1905, 1907), (1906, 1907), (1908, 1909), (1912, 1913), (1916, 1917), (1918, 1919), (1934, 1935), (1937, 1938), (1940, 1941), (1951, 1952), (1954, 1955), (1958, 1959), (1959, 1960), (1973, 1975), (1974, 1975), (1979, 1980), (1982, 1983), (1984, 1985), (1991, 1992), (1997, 1998), (1997, 1999), (2007, 2008), (2056, 2057), (2066, 2067), (2109, 2110), (2124, 2125), (2125, 2126)]
# wells_mapping={1: 286, 2: 287, 3: 288, 4: 289, 5: 290, 6: 291, 7: 292, 8: 293, 9: 294, 10: 295, 11: 296, 12: 297, 13: 298, 14: 299, 15: 300, 16: 301, 17: 302, 18: 303, 19: 304, 20: 305, 21: 306, 22: 307, 23: 308, 24: 309, 25: 310, 26: 311, 27: 312, 28: 313, 29: 314, 30: 315, 31: 316, 32: 317, 33: 318, 34: 319, 35: 320, 36: 321, 37: 322, 38: 323, 39: 324, 40: 325, 41: 326, 42: 327, 43: 328, 44: 329, 45: 330, 46: 331, 47: 332, 48: 333, 49: 334, 50: 335, 51: 336, 52: 337, 53: 338, 54: 339, 55: 340, 56: 341, 57: 342, 58: 343, 59: 344, 60: 345, 61: 346, 62: 347, 63: 348, 64: 349, 65: 350, 66: 351, 67: 352, 68: 353, 69: 354, 70: 355, 71: 356, 72: 357, 73: 358, 74: 359, 75: 360, 76: 361, 77: 362, 78: 363, 79: 364, 80: 365, 81: 366, 82: 367, 83: 368, 84: 369, 85: 370, 86: 371, 87: 372, 88: 373, 89: 374, 90: 375, 91: 376, 92: 377, 93: 378, 94: 379, 95: 380, 96: 381, 97: 382, 98: 383, 99: 384, 100: 385, 101: 386, 102: 387, 103: 388, 104: 389, 105: 390, 106: 391, 107: 392, 108: 393, 109: 394, 110: 395, 111: 396, 112: 397, 113: 398, 114: 399, 115: 400, 116: 401, 117: 402, 118: 403, 119: 404, 120: 405, 121: 406, 122: 407, 123: 408, 124: 409, 125: 410, 126: 411, 127: 412, 128: 413, 129: 414, 130: 415, 131: 416, 132: 417, 133: 418, 134: 419, 135: 420, 136: 421, 137: 422, 138: 423, 139: 424, 140: 425, 141: 934, 142: 935, 143: 426, 144: 427, 145: 428, 146: 429, 147: 430, 148: 431, 149: 432, 150: 433, 151: 434, 152: 435, 153: 436, 154: 437, 155: 438, 156: 439, 157: 440, 158: 441, 159: 442, 160: 443, 161: 444, 162: 445, 163: 446, 164: 447, 165: 448, 166: 449, 167: 450, 168: 451, 169: 36, 170: 37, 171: 38, 172: 1668, 173: 1056, 174: 1057, 175: 1058, 176: 1044, 177: 1045, 178: 1046, 179: 1047, 180: 1048, 181: 1049, 182: 1050, 183: 1051, 184: 1052, 185: 1053, 186: 1054, 187: 1055, 188: 2040, 189: 1072, 190: 1073, 191: 1074, 192: 1075, 193: 1076, 194: 1077, 195: 1078, 196: 1079, 197: 1080, 198: 1081, 199: 1082, 200: 1083, 201: 1084, 202: 1085, 203: 2137, 204: 2000, 205: 2001, 206: 1999, 207: 1568, 208: 1569, 209: 1570, 210: 1571, 211: 1572, 212: 1566, 213: 1567, 214: 39, 215: 40, 216: 41, 217: 42, 218: 43, 219: 44, 220: 45, 221: 46, 222: 47, 223: 48, 224: 515, 225: 516, 226: 517, 227: 518, 228: 519, 229: 520, 230: 521, 231: 522, 232: 523, 233: 96, 234: 97, 235: 98, 236: 99, 237: 100, 238: 84, 239: 85, 240: 86, 241: 87, 242: 88, 243: 89, 244: 90, 245: 91, 246: 92, 247: 93, 248: 94, 249: 95, 250: 2153, 251: 848, 252: 849, 253: 850, 254: 851, 255: 852, 256: 1881, 257: 1480, 258: 1479, 259: 2057, 260: 1794, 261: 1556, 262: 1557, 263: 1558, 264: 1199, 265: 1200, 266: 1201, 267: 1202, 268: 1203, 269: 1204, 270: 1205, 271: 1206, 272: 1207, 273: 1208, 274: 1400, 275: 1401, 276: 1398, 277: 1399, 278: 1874, 279: 1875, 280: 820, 281: 821, 282: 822, 283: 823, 284: 824, 285: 825, 286: 826, 287: 827, 288: 828, 289: 829, 290: 830, 291: 831, 292: 853, 293: 854, 294: 855, 295: 1884, 296: 159, 297: 160, 298: 161, 299: 162, 300: 163, 301: 164, 302: 165, 303: 166, 304: 167, 305: 168, 306: 169, 307: 170, 308: 171, 309: 172, 310: 173, 311: 174, 312: 175, 313: 176, 314: 177, 315: 178, 316: 179, 317: 180, 318: 181, 319: 182, 320: 183, 321: 184, 322: 185, 323: 186, 324: 187, 325: 188, 326: 189, 327: 190, 328: 191, 329: 192, 330: 193, 331: 194, 332: 195, 333: 196, 334: 197, 335: 198, 336: 199, 337: 200, 338: 201, 339: 202, 340: 203, 341: 204, 342: 205, 343: 206, 344: 207, 345: 208, 346: 209, 347: 210, 348: 211, 349: 212, 350: 213, 351: 214, 352: 215, 353: 216, 354: 217, 355: 218, 356: 219, 357: 220, 358: 221, 359: 222, 360: 223, 361: 224, 362: 225, 363: 226, 364: 227, 365: 228, 366: 229, 367: 230, 368: 231, 369: 232, 370: 233, 371: 234, 372: 235, 373: 236, 374: 237, 375: 238, 376: 239, 377: 240, 378: 241, 379: 242, 380: 243, 381: 244, 382: 245, 383: 246, 384: 247, 385: 248, 386: 249, 387: 250, 388: 251, 389: 252, 390: 253, 391: 254, 392: 255, 393: 256, 394: 257, 395: 258, 396: 259, 397: 260, 398: 8, 399: 9, 400: 10, 401: 11, 402: 12, 403: 13, 404: 14, 405: 15, 406: 16, 407: 17, 408: 18, 409: 19, 410: 20, 411: 21, 412: 22, 413: 1019, 414: 1020, 415: 1371, 416: 1372, 417: 1373, 418: 1803, 419: 630, 420: 631, 421: 632, 422: 633, 423: 634, 424: 635, 425: 636, 426: 637, 427: 1264, 428: 1265, 429: 1262, 430: 1263, 431: 489, 432: 490, 433: 491, 434: 492, 435: 493, 436: 494, 437: 495, 438: 496, 439: 497, 440: 498, 441: 499, 442: 500, 443: 501, 444: 502, 445: 503, 446: 504, 447: 505, 448: 506, 449: 1602, 450: 1656, 451: 1657, 452: 864, 453: 865, 454: 866, 455: 867, 456: 868, 457: 856, 458: 857, 459: 858, 460: 859, 461: 860, 462: 861, 463: 862, 464: 863, 465: 576, 466: 577, 467: 578, 468: 579, 469: 580, 470: 581, 471: 582, 472: 583, 473: 584, 474: 573, 475: 574, 476: 575, 477: 568, 478: 569, 479: 570, 480: 571, 481: 572, 482: 1552, 483: 1553, 484: 1550, 485: 1551, 486: 1092, 487: 1184, 488: 1185, 489: 1186, 490: 1187, 491: 1175, 492: 1176, 493: 1177, 494: 1178, 495: 1179, 496: 1180, 497: 1181, 498: 1182, 499: 1183, 500: 1996, 501: 1997, 502: 1998, 503: 928, 504: 929, 505: 930, 506: 931, 507: 932, 508: 933, 509: 960, 510: 961, 511: 949, 512: 950, 513: 951, 514: 952, 515: 953, 516: 954, 517: 955, 518: 956, 519: 957, 520: 958, 521: 959, 522: 1966, 523: 1744, 524: 1743, 525: 64, 526: 65, 527: 61, 528: 62, 529: 63, 530: 1669, 531: 1670, 532: 1671, 533: 1672, 534: 1673, 535: 832, 536: 833, 537: 834, 538: 835, 539: 731, 540: 732, 541: 733, 542: 734, 543: 735, 544: 736, 545: 737, 546: 738, 547: 739, 548: 740, 549: 741, 550: 742, 551: 743, 552: 744, 553: 745, 554: 746, 555: 747, 556: 748, 557: 749, 558: 750, 559: 751, 560: 752, 561: 753, 562: 1906, 563: 1548, 564: 1549, 565: 1359, 566: 1360, 567: 1361, 568: 1362, 569: 1363, 570: 841, 571: 842, 572: 1573, 573: 604, 574: 586, 575: 2130, 576: 1488, 577: 1486, 578: 1487, 579: 1124, 580: 1125, 581: 1126, 582: 128, 583: 129, 584: 130, 585: 131, 586: 132, 587: 133, 588: 134, 589: 135, 590: 136, 591: 137, 592: 138, 593: 139, 594: 140, 595: 141, 596: 142, 597: 143, 598: 144, 599: 145, 600: 146, 601: 147, 602: 148, 603: 149, 604: 150, 605: 151, 606: 101, 607: 102, 608: 103, 609: 104, 610: 105, 611: 106, 612: 107, 613: 108, 614: 109, 615: 110, 616: 111, 617: 112, 618: 113, 619: 114, 620: 115, 621: 116, 622: 117, 623: 118, 624: 119, 625: 120, 626: 121, 627: 122, 628: 123, 629: 124, 630: 125, 631: 126, 632: 127, 633: 1772, 634: 1804, 635: 907, 636: 908, 637: 909, 638: 910, 639: 911, 640: 912, 641: 913, 642: 914, 643: 915, 644: 916, 645: 917, 646: 918, 647: 919, 648: 920, 649: 1897, 650: 1898, 651: 1636, 652: 1273, 653: 1141, 654: 1142, 655: 1143, 656: 1144, 657: 1145, 658: 1483, 659: 1381, 660: 1408, 661: 1409, 662: 1410, 663: 1406, 664: 1407, 665: 667, 666: 668, 667: 669, 668: 670, 669: 671, 670: 672, 671: 673, 672: 674, 673: 675, 674: 676, 675: 677, 676: 678, 677: 679, 678: 680, 679: 681, 680: 682, 681: 683, 682: 684, 683: 685, 684: 686, 685: 687, 686: 688, 687: 689, 688: 690, 689: 691, 690: 692, 691: 693, 692: 694, 693: 695, 694: 696, 695: 697, 696: 698, 697: 699, 698: 700, 699: 701, 700: 702, 701: 703, 702: 704, 703: 705, 704: 706, 705: 707, 706: 708, 707: 481, 708: 482, 709: 483, 710: 484, 711: 485, 712: 488, 713: 486, 714: 487, 715: 1006, 716: 1007, 717: 1008, 718: 1009, 719: 1010, 720: 1578, 721: 1579, 722: 1580, 723: 532, 724: 533, 725: 534, 726: 535, 727: 536, 728: 537, 729: 538, 730: 539, 731: 1248, 732: 1249, 733: 1241, 734: 1242, 735: 1243, 736: 1244, 737: 1245, 738: 1246, 739: 1247, 740: 944, 741: 945, 742: 946, 743: 943, 744: 2012, 745: 1392, 746: 1391, 747: 1893, 748: 1894, 749: 2094, 750: 1807, 751: 1953, 752: 454, 753: 455, 754: 456, 755: 457, 756: 458, 757: 459, 758: 460, 759: 461, 760: 768, 761: 769, 762: 770, 763: 771, 764: 772, 765: 773, 766: 774, 767: 775, 768: 776, 769: 777, 770: 778, 771: 779, 772: 761, 773: 762, 774: 763, 775: 764, 776: 765, 777: 766, 778: 767, 779: 1844, 780: 1716, 781: 881, 782: 882, 783: 2008, 784: 2007, 785: 754, 786: 1637, 787: 1693, 788: 1694, 789: 1904, 790: 2025, 791: 1936, 792: 1232, 793: 1233, 794: 1234, 795: 1235, 796: 1236, 797: 1705, 798: 1706, 799: 1707, 800: 1690, 801: 1068, 802: 1069, 803: 1070, 804: 1071, 805: 1800, 806: 1801, 807: 1802, 808: 1388, 809: 1448, 810: 1449, 811: 1632, 812: 1633, 813: 1209, 814: 1635, 815: 1925, 816: 1404, 817: 1405, 818: 33, 819: 34, 820: 35, 821: 1691, 822: 1648, 823: 1979, 824: 2029, 825: 28, 826: 1115, 827: 1116, 828: 1992, 829: 1024, 830: 1025, 831: 1026, 832: 1027, 833: 1028, 834: 1029, 835: 1030, 836: 1022, 837: 1023, 838: 1619, 839: 1536, 840: 1537, 841: 1538, 842: 1539, 843: 1540, 844: 1541, 845: 1542, 846: 1534, 847: 1535, 848: 1684, 849: 1626, 850: 1523, 851: 1524, 852: 1525, 853: 1526, 854: 970, 855: 971, 856: 972, 857: 973, 858: 974, 859: 975, 860: 1784, 861: 1783, 862: 1512, 863: 2077, 864: 1501, 865: 1502, 866: 1167, 867: 1168, 868: 1169, 869: 1170, 870: 1171, 871: 1172, 872: 1173, 873: 2002, 874: 1402, 875: 640, 876: 641, 877: 642, 878: 639, 879: 1106, 880: 1011, 881: 1012, 882: 1013, 883: 2019, 884: 507, 885: 508, 886: 509, 887: 510, 888: 511, 889: 2090, 890: 628, 891: 2139, 892: 1845, 893: 2014, 894: 2006, 895: 1041, 896: 1042, 897: 1043, 898: 1223, 899: 1128, 900: 1129, 901: 1130, 902: 1131, 903: 1132, 904: 1722, 905: 1723, 906: 1724, 907: 1490, 908: 1491, 909: 1492, 910: 1493, 911: 1494, 912: 1495, 913: 904, 914: 1773, 915: 816, 916: 1500, 917: 1850, 918: 605, 919: 606, 920: 591, 921: 592, 922: 593, 923: 594, 924: 595, 925: 596, 926: 800, 927: 801, 928: 802, 929: 803, 930: 804, 931: 805, 932: 806, 933: 807, 934: 808, 935: 809, 936: 810, 937: 811, 938: 812, 939: 813, 940: 814, 941: 798, 942: 799, 943: 1, 944: 152, 945: 153, 946: 66, 947: 67, 948: 68, 949: 69, 950: 70, 951: 71, 952: 72, 953: 73, 954: 74, 955: 75, 956: 76, 957: 77, 958: 78, 959: 79, 960: 80, 961: 81, 962: 82, 963: 83, 964: 1288, 965: 1289, 966: 1290, 967: 1291, 968: 1014, 969: 1015, 970: 1786, 971: 1787, 972: 2143, 973: 542, 974: 543, 975: 544, 976: 545, 977: 546, 978: 547, 979: 548, 980: 549, 981: 550, 982: 551, 983: 552, 984: 553, 985: 554, 986: 555, 987: 556, 988: 557, 989: 558, 990: 559, 991: 560, 992: 561, 993: 562, 994: 563, 995: 564, 996: 565, 997: 1799, 998: 755, 999: 756, 1000: 757, 1001: 758, 1002: 759, 1003: 760, 1004: 884, 1005: 885, 1006: 886, 1007: 1907, 1008: 1908, 1009: 1909, 1010: 1910, 1011: 1821, 1012: 1822, 1013: 1282, 1014: 1283, 1015: 1344, 1016: 1345, 1017: 1346, 1018: 1347, 1019: 1348, 1020: 1349, 1021: 1350, 1022: 1339, 1023: 1340, 1024: 1341, 1025: 1342, 1026: 1343, 1027: 1828, 1028: 480, 1029: 471, 1030: 472, 1031: 473, 1032: 474, 1033: 475, 1034: 476, 1035: 477, 1036: 478, 1037: 479, 1038: 780, 1039: 781, 1040: 782, 1041: 783, 1042: 784, 1043: 785, 1044: 1666, 1045: 1667, 1046: 843, 1047: 844, 1048: 845, 1049: 2123, 1050: 624, 1051: 625, 1052: 623, 1053: 1331, 1054: 1332, 1055: 1395, 1056: 1610, 1057: 2140, 1058: 2, 1059: 3, 1060: 4, 1061: 1790, 1062: 1791, 1063: 1211, 1064: 1212, 1065: 874, 1066: 875, 1067: 876, 1068: 965, 1069: 966, 1070: 967, 1071: 968, 1072: 969, 1073: 1188, 1074: 1189, 1075: 1190, 1076: 1920, 1077: 1919, 1078: 1592, 1079: 1599, 1080: 2079, 1081: 1877, 1082: 1984, 1083: 1983, 1084: 977, 1085: 978, 1086: 1451, 1087: 1452, 1088: 1250, 1089: 1251, 1090: 1532, 1091: 263, 1092: 264, 1093: 265, 1094: 266, 1095: 267, 1096: 268, 1097: 269, 1098: 270, 1099: 271, 1100: 272, 1101: 273, 1102: 274, 1103: 275, 1104: 276, 1105: 277, 1106: 278, 1107: 156, 1108: 157, 1109: 158, 1110: 1098, 1111: 1152, 1112: 1153, 1113: 1154, 1114: 1155, 1115: 1156, 1116: 1149, 1117: 1150, 1118: 1151, 1119: 566, 1120: 567, 1121: 608, 1122: 609, 1123: 610, 1124: 611, 1125: 612, 1126: 607, 1127: 1355, 1128: 1356, 1129: 1357, 1130: 1358, 1131: 1862, 1132: 1863, 1133: 1864, 1134: 1865, 1135: 1866, 1136: 1268, 1137: 49, 1138: 50, 1139: 51, 1140: 52, 1141: 53, 1142: 54, 1143: 55, 1144: 56, 1145: 57, 1146: 709, 1147: 710, 1148: 711, 1149: 712, 1150: 713, 1151: 714, 1152: 715, 1153: 716, 1154: 717, 1155: 718, 1156: 719, 1157: 720, 1158: 721, 1159: 722, 1160: 723, 1161: 724, 1162: 725, 1163: 726, 1164: 727, 1165: 728, 1166: 729, 1167: 730, 1168: 2112, 1169: 905, 1170: 906, 1171: 936, 1172: 937, 1173: 938, 1174: 939, 1175: 940, 1176: 1237, 1177: 1238, 1178: 1239, 1179: 1886, 1180: 1887, 1181: 1034, 1182: 1035, 1183: 1036, 1184: 1037, 1185: 1736, 1186: 1735, 1187: 1259, 1188: 1260, 1189: 1989, 1190: 2076, 1191: 512, 1192: 2086, 1193: 2071, 1194: 1560, 1195: 1561, 1196: 1559, 1197: 1733, 1198: 1472, 1199: 1473, 1200: 1474, 1201: 1475, 1202: 1476, 1203: 1477, 1204: 963, 1205: 964, 1206: 1976, 1207: 1975, 1208: 1750, 1209: 1751, 1210: 1752, 1211: 1753, 1212: 1754, 1213: 1755, 1214: 1680, 1215: 1678, 1216: 1679, 1217: 1503, 1218: 1988, 1219: 1166, 1220: 2069, 1221: 1096, 1222: 1097, 1223: 528, 1224: 525, 1225: 526, 1226: 527, 1227: 1133, 1228: 1686, 1229: 922, 1230: 923, 1231: 924, 1232: 925, 1233: 646, 1234: 647, 1235: 648, 1236: 649, 1237: 650, 1238: 993, 1239: 994, 1240: 995, 1241: 996, 1242: 997, 1243: 1533, 1244: 1287, 1245: 1280, 1246: 1281, 1247: 1277, 1248: 1278, 1249: 1279, 1250: 1510, 1251: 1511, 1252: 1928, 1253: 1692, 1254: 665, 1255: 666, 1256: 840, 1257: 838, 1258: 839, 1259: 1817, 1260: 1366, 1261: 1367, 1262: 1370, 1263: 1383, 1264: 1384, 1265: 1385, 1266: 1386, 1267: 1387, 1268: 1396, 1269: 1088, 1270: 1089, 1271: 1090, 1272: 1091, 1273: 1838, 1274: 1158, 1275: 1159, 1276: 613, 1277: 614, 1278: 615, 1279: 616, 1280: 617, 1281: 618, 1282: 619, 1283: 620, 1284: 621, 1285: 622, 1286: 1600, 1287: 1601, 1288: 1823, 1289: 2136, 1290: 1683, 1291: 1257, 1292: 1105, 1293: 1442, 1294: 1443, 1295: 1444, 1296: 1445, 1297: 1859, 1298: 1860, 1299: 1617, 1300: 1618, 1301: 1136, 1302: 1137, 1303: 1138, 1304: 1139, 1305: 2042, 1306: 2084, 1307: 1851, 1308: 921, 1309: 998, 1310: 999, 1311: 1000, 1312: 1001, 1313: 1002, 1314: 1776, 1315: 1774, 1316: 1775, 1317: 869, 1318: 870, 1319: 871, 1320: 1562, 1321: 1563, 1322: 1312, 1323: 1313, 1324: 1314, 1325: 1315, 1326: 1316, 1327: 1317, 1328: 1318, 1329: 1308, 1330: 1309, 1331: 1310, 1332: 1311, 1333: 1914, 1334: 1779, 1335: 1596, 1336: 1597, 1337: 787, 1338: 788, 1339: 789, 1340: 790, 1341: 791, 1342: 792, 1343: 793, 1344: 794, 1345: 795, 1346: 796, 1347: 817, 1348: 818, 1349: 819, 1350: 1798, 1351: 1292, 1352: 1293, 1353: 1294, 1354: 1295, 1355: 1296, 1356: 1297, 1357: 1298, 1358: 1299, 1359: 1300, 1360: 1301, 1361: 1302, 1362: 1303, 1363: 1304, 1364: 1305, 1365: 1306, 1366: 1307, 1367: 284, 1368: 285, 1369: 979, 1370: 980, 1371: 981, 1372: 982, 1373: 983, 1374: 984, 1375: 985, 1376: 986, 1377: 987, 1378: 988, 1379: 989, 1380: 990, 1381: 991, 1382: 1792, 1383: 1644, 1384: 1882, 1385: 1883, 1386: 2085, 1387: 1120, 1388: 1640, 1389: 1641, 1390: 1642, 1391: 887, 1392: 888, 1393: 889, 1394: 890, 1395: 891, 1396: 892, 1397: 896, 1398: 897, 1399: 898, 1400: 899, 1401: 900, 1402: 1416, 1403: 1417, 1404: 1418, 1405: 1415, 1406: 1440, 1407: 1441, 1408: 1439, 1409: 1832, 1410: 1833, 1411: 1834, 1412: 1564, 1413: 1351, 1414: 2142, 1415: 1530, 1416: 1531, 1417: 1134, 1418: 1135, 1419: 1963, 1420: 1964, 1421: 1965, 1422: 23, 1423: 24, 1424: 25, 1425: 26, 1426: 27, 1427: 651, 1428: 2151, 1429: 1320, 1430: 1321, 1431: 1322, 1432: 1323, 1433: 1324, 1434: 1325, 1435: 1216, 1436: 1217, 1437: 1218, 1438: 1219, 1439: 1220, 1440: 1221, 1441: 1222, 1442: 1215, 1443: 1032, 1444: 1033, 1445: 1224, 1446: 2072, 1447: 2073, 1448: 2074, 1449: 279, 1450: 280, 1451: 281, 1452: 282, 1453: 283, 1454: 1980, 1455: 2034, 1456: 1587, 1457: 1588, 1458: 1696, 1459: 60, 1460: 2105, 1461: 2117, 1462: 2046, 1463: 2047, 1464: 1140, 1465: 626, 1466: 627, 1467: 2051, 1468: 1712, 1469: 1713, 1470: 1714, 1471: 1715, 1472: 976, 1473: 1576, 1474: 1577, 1475: 1574, 1476: 1575, 1477: 2061, 1478: 1397, 1479: 1649, 1480: 1650, 1481: 1839, 1482: 656, 1483: 654, 1484: 655, 1485: 2064, 1486: 1428, 1487: 1429, 1488: 1430, 1489: 1431, 1490: 1867, 1491: 1489, 1492: 894, 1493: 1768, 1494: 1765, 1495: 1766, 1496: 1767, 1497: 962, 1498: 1148, 1499: 1438, 1500: 1389, 1501: 1390, 1502: 1459, 1503: 1460, 1504: 1461, 1505: 1462, 1506: 2010, 1507: 902, 1508: 903, 1509: 1704, 1510: 2099, 1511: 1509, 1512: 2158, 1513: 1889, 1514: 1890, 1515: 877, 1516: 878, 1517: 879, 1518: 1228, 1519: 1229, 1520: 1230, 1521: 1777, 1522: 2070, 1523: 1456, 1524: 1521, 1525: 1522, 1526: 1412, 1527: 1746, 1528: 1747, 1529: 1748, 1530: 2080, 1531: 1513, 1532: 2024, 1533: 2023, 1534: 1004, 1535: 1005, 1536: 1697, 1537: 657, 1538: 658, 1539: 659, 1540: 2081, 1541: 2028, 1542: 1941, 1543: 1950, 1544: 1951, 1545: 1763, 1546: 1764, 1547: 587, 1548: 588, 1549: 589, 1550: 1741, 1551: 1742, 1552: 1808, 1553: 1809, 1554: 1810, 1555: 1413, 1556: 1414, 1557: 1846, 1558: 1847, 1559: 1658, 1560: 1659, 1561: 1813, 1562: 2128, 1563: 1613, 1564: 1614, 1565: 1615, 1566: 836, 1567: 837, 1568: 1604, 1569: 1554, 1570: 1122, 1571: 2039, 1572: 1645, 1573: 2111, 1574: 1353, 1575: 1725, 1576: 1622, 1577: 1623, 1578: 1544, 1579: 2107, 1580: 1102, 1581: 1103, 1582: 1805, 1583: 1806, 1584: 1589, 1585: 1944, 1586: 1945, 1587: 1946, 1588: 1947, 1589: 1256, 1590: 1255, 1591: 1628, 1592: 2065, 1593: 2066, 1594: 1937, 1595: 1586, 1596: 846, 1597: 531, 1598: 1978, 1599: 1457, 1600: 1458, 1601: 1892, 1602: 1896, 1603: 893, 1604: 1701, 1605: 1702, 1606: 1703, 1607: 1677, 1608: 1880, 1609: 1163, 1610: 1164, 1611: 1527, 1612: 1852, 1613: 1853, 1614: 1854, 1615: 2108, 1616: 1436, 1617: 1437, 1618: 1258, 1619: 261, 1620: 262, 1621: 644, 1622: 926, 1623: 927, 1624: 1856, 1625: 1638, 1626: 1639, 1627: 32, 1628: 29, 1629: 30, 1630: 31, 1631: 1932, 1632: 2017, 1633: 1582, 1634: 1583, 1635: 1819, 1636: 1820, 1637: 1520, 1638: 1518, 1639: 1519, 1640: 1948, 1641: 1664, 1642: 1665, 1643: 1663, 1644: 1972, 1645: 664, 1646: 662, 1647: 663, 1648: 2119, 1649: 1333, 1650: 1334, 1651: 1335, 1652: 1336, 1653: 1337, 1654: 1338, 1655: 1108, 1656: 1109, 1657: 1110, 1658: 1111, 1659: 1876, 1660: 2044, 1661: 1021, 1662: 1467, 1663: 1620, 1664: 1496, 1665: 1099, 1666: 1100, 1667: 2101, 1668: 1986, 1669: 540, 1670: 541, 1671: 1063, 1672: 1064, 1673: 1065, 1674: 1066, 1675: 1067, 1676: 1651, 1677: 1652, 1678: 1653, 1679: 1603, 1680: 1284, 1681: 1927, 1682: 1165, 1683: 1424, 1684: 1422, 1685: 1423, 1686: 941, 1687: 2141, 1688: 2109, 1689: 1933, 1690: 1934, 1691: 1982, 1692: 1285, 1693: 1698, 1694: 2135, 1695: 2003, 1696: 2004, 1697: 2005, 1698: 1938, 1699: 1939, 1700: 645, 1701: 2104, 1702: 2103, 1703: 2159, 1704: 2155, 1705: 1252, 1706: 1253, 1707: 1254, 1708: 1505, 1709: 1506, 1710: 1507, 1711: 2063, 1712: 2038, 1713: 1872, 1714: 1869, 1715: 1870, 1716: 1871, 1717: 1411, 1718: 2116, 1719: 1921, 1720: 1815, 1721: 992, 1722: 872, 1723: 873, 1724: 643, 1725: 2052, 1726: 1780, 1727: 947, 1728: 948, 1729: 1113, 1730: 1114, 1731: 1700, 1732: 1497, 1733: 1761, 1734: 1762, 1735: 2114, 1736: 1421, 1737: 1855, 1738: 1225, 1739: 1226, 1740: 1227, 1741: 1364, 1742: 1365, 1743: 1878, 1744: 1879, 1745: 154, 1746: 155, 1747: 58, 1748: 59, 1749: 1729, 1750: 1987, 1751: 1276, 1752: 1107, 1753: 1968, 1754: 1967, 1755: 1630, 1756: 1631, 1757: 452, 1758: 1609, 1759: 883, 1760: 1214, 1761: 1675, 1762: 1354, 1763: 1059, 1764: 1060, 1765: 6, 1766: 7, 1767: 2082, 1768: 1695, 1769: 464, 1770: 465, 1771: 466, 1772: 467, 1773: 468, 1774: 2154, 1775: 1899, 1776: 1121, 1777: 1662, 1778: 1660, 1779: 1517, 1780: 2035, 1781: 1433, 1782: 1434, 1783: 1435, 1784: 1545, 1785: 1546, 1786: 1547, 1787: 1797, 1788: 1962, 1789: 1403, 1790: 1447, 1791: 1756, 1792: 1621, 1793: 1482, 1794: 2016, 1795: 1717, 1796: 1718, 1797: 1719, 1798: 1061, 1799: 1062, 1800: 1427, 1801: 1213, 1802: 1757, 1803: 1758, 1804: 1468, 1805: 1469, 1806: 1470, 1807: 1471, 1808: 1484, 1809: 1485, 1810: 1977, 1811: 1039, 1812: 1269, 1813: 1270, 1814: 1454, 1815: 1685, 1816: 469, 1817: 470, 1818: 1912, 1819: 1911, 1820: 1481, 1821: 462, 1822: 463, 1823: 1785, 1824: 1994, 1825: 1818, 1826: 1194, 1827: 1195, 1828: 1827, 1829: 1730, 1830: 1731, 1831: 1393, 1832: 1394, 1833: 529, 1834: 530, 1835: 1728, 1836: 1590, 1837: 2097, 1838: 1591, 1839: 1378, 1840: 1379, 1841: 1380, 1842: 1952, 1843: 2087, 1844: 1463, 1845: 2120, 1846: 1931, 1847: 2033, 1848: 2067, 1849: 2156, 1850: 1740, 1851: 1174, 1852: 895, 1853: 1788, 1854: 1040, 1855: 1769, 1856: 1961, 1857: 1917, 1858: 2009, 1859: 1118, 1860: 1119, 1861: 1374, 1862: 1885, 1863: 660, 1864: 1624, 1865: 1161, 1866: 1162, 1867: 1584, 1868: 1608, 1869: 1606, 1870: 1607, 1871: 1778, 1872: 1498, 1873: 1499, 1874: 653, 1875: 815, 1876: 2095, 1877: 1873, 1878: 1923, 1879: 1266, 1880: 1267, 1881: 1464, 1882: 1465, 1883: 1555, 1884: 1825, 1885: 1737, 1886: 1738, 1887: 1726, 1888: 1727, 1889: 1708, 1890: 1709, 1891: 2089, 1892: 1616, 1893: 1826, 1894: 2133, 1895: 1901, 1896: 1676, 1897: 1759, 1898: 590, 1899: 1453, 1900: 1629, 1901: 2115, 1902: 1674, 1903: 1581, 1904: 513, 1905: 599, 1906: 600, 1907: 601, 1908: 602, 1909: 603, 1910: 2075, 1911: 1840, 1912: 1093, 1913: 1094, 1914: 1095, 1915: 652, 1916: 2059, 1917: 2060, 1918: 1781, 1919: 1782, 1920: 1959, 1921: 1960, 1922: 2149, 1923: 2078, 1924: 1814, 1925: 1543, 1926: 1771, 1927: 1319, 1928: 2113, 1929: 1739, 1930: 2110, 1931: 901, 1932: 2131, 1933: 2020, 1934: 1146, 1935: 1147, 1936: 786, 1937: 1710, 1938: 1711, 1939: 1681, 1940: 1970, 1941: 1971, 1942: 2037, 1943: 1929, 1944: 1770, 1945: 2102, 1946: 585, 1947: 1627, 1948: 1895, 1949: 1516, 1950: 1643, 1951: 1329, 1952: 1330, 1953: 1913, 1954: 1954, 1955: 1955, 1956: 1985, 1957: 1942, 1958: 1376, 1959: 1377, 1960: 1375, 1961: 1849, 1962: 1646, 1963: 1352, 1964: 1918, 1965: 2050, 1966: 1915, 1967: 1720, 1968: 2041, 1969: 1198, 1970: 1478, 1971: 1514, 1972: 629, 1973: 1328, 1974: 1326, 1975: 1327, 1976: 2032, 1977: 2160, 1978: 1760, 1979: 1593, 1980: 1594, 1981: 1382, 1982: 1086, 1983: 1087, 1984: 1017, 1985: 1018, 1986: 1993, 1987: 1647, 1988: 2146, 1989: 1508, 1990: 1745, 1991: 2124, 1992: 2125, 1993: 1160, 1994: 1888, 1995: 1101, 1996: 453, 1997: 1688, 1998: 1689, 1999: 1687, 2000: 1891, 2001: 1016, 2002: 1261, 2003: 2053, 2004: 2049, 2005: 2145, 2006: 1930, 2007: 1272, 2008: 1271, 2009: 1275, 2010: 2129, 2011: 1425, 2012: 1003, 2013: 2088, 2014: 1565, 2015: 1905, 2016: 2036, 2017: 1995, 2018: 1192, 2019: 1796, 2020: 2152, 2021: 514, 2022: 1197, 2023: 1196, 2024: 1837, 2025: 1104, 2026: 1528, 2027: 1922, 2028: 1835, 2029: 1789, 2030: 2043, 2031: 942, 2032: 1902, 2033: 1793, 2034: 2132, 2035: 1981, 2036: 1974, 2037: 1795, 2038: 1455, 2039: 2134, 2040: 2068, 2041: 1943, 2042: 1210, 2043: 1924, 2044: 2026, 2045: 1420, 2046: 1958, 2047: 1038, 2048: 1612, 2049: 2013, 2050: 2056, 2051: 1274, 2052: 1903, 2053: 1231, 2054: 2121, 2055: 2055, 2056: 1990, 2057: 1991, 2058: 524, 2059: 1031, 2060: 1811, 2061: 1824, 2062: 1193, 2063: 2148, 2064: 1157, 2065: 2011, 2066: 597, 2067: 598, 2068: 2091, 2069: 2093, 2070: 1949, 2071: 1935, 2072: 1940, 2073: 1857, 2074: 1432, 2075: 1369, 2076: 2030, 2077: 1529, 2078: 2048, 2079: 2027, 2080: 1829, 2081: 1816, 2082: 1112, 2083: 1515, 2084: 2118, 2085: 2100, 2086: 1368, 2087: 1973, 2088: 1830, 2089: 1916, 2090: 2031, 2091: 638, 2092: 2062, 2093: 1699, 2094: 2092, 2095: 1605, 2096: 2147, 2097: 1749, 2098: 5, 2099: 880, 2100: 1123, 2101: 2098, 2102: 1611, 2103: 797, 2104: 2126, 2105: 1956, 2106: 2127, 2107: 2021, 2108: 2058, 2109: 1654, 2110: 1655, 2111: 1117, 2112: 1419, 2113: 2150, 2114: 2144, 2115: 1734, 2116: 1836, 2117: 1240, 2118: 1426, 2119: 1858, 2120: 2083, 2121: 2122, 2122: 1286, 2123: 1848, 2124: 1841, 2125: 1842, 2126: 1843, 2127: 1831, 2128: 1625, 2129: 1868, 2130: 661, 2131: 1595, 2132: 1900, 2133: 1969, 2134: 1466, 2135: 1682, 2136: 1861, 2137: 1504, 2138: 2054, 2139: 1732, 2140: 1127, 2141: 2106, 2142: 1598, 2143: 1661, 2144: 1634, 2145: 1812, 2146: 1191, 2147: 2138, 2148: 1926, 2149: 2045, 2150: 2157, 2151: 1446, 2152: 1585, 2153: 1721, 2154: 847, 2155: 2018, 2156: 2022, 2157: 2096, 2158: 1957, 2159: 2015, 2160: 1450}
# res = 70033241
# n,wells_dict_new,pipes_new,well_status_new,pipe_status_new = translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping)

# pack0 = [wells_dict_new,pipes_new,well_status_new,pipe_status_new]


In [85]:
n= 242
wells_list= [44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 42417, 95924, 90525, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 40675, 34549, 25622, 50209, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 98070, 34386, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19058, 38363, 97436, 67282, 37468, 28941, 77321, 21735, 39947, 45271, 75551, 22756, 94693, 74923, 85415, 66612, 36128, 63417, 74813, 28512, 26550, 67591, 47740, 68396, 15283, 15648, 23442, 95657, 99006, 53923, 40365, 93449, 64620, 98318, 80410, 88103, 13907, 62043, 21740, 36137, 79315, 86038, 19773, 82593, 43382, 34577, 23476, 27968, 58730, 48230, 60893, 32796, 82817, 6078, 95367, 88804, 84262, 48538, 69862, 69479, 60964, 38756, 25008, 7142, 84551, 23986, 7812, 79766, 21233, 86242, 25261, 74883, 5960, 66064, 77217, 53785, 46293, 83768, 25802, 54151, 80686, 14218, 90413, 97864, 73782, 94318, 67909, 84451, 91892, 12560, 17023, 52220, 2395, 80816, 18049, 40026, 29409, 14129, 19124, 32383, 59429, 76000, 10553, 35002, 85675, 78715, 60336, 91802]
pipe_list= [[1, 2, 694], [1, 13, 53621], [1, 14, 44244], [2, 3, 38134], [2, 10, 27141], [2, 18, 16455], [2, 51, 4094], [2, 65, 15576], [3, 9, 57468], [3, 42, 45209], [27, 4, 48215], [57, 5, 62330], [5, 69, 34317], [67, 6, 55563], [39, 7, 40097], [7, 50, 61773], [7, 68, 66756], [79, 8, 35536], [9, 20, 44932], [9, 32, 57740], [9, 34, 1991], [9, 46, 22060], [9, 57, 54520], [9, 77, 53146], [10, 12, 28292], [10, 16, 46504], [10, 22, 12824], [10, 56, 39052], [51, 11, 272], [12, 59, 40024], [12, 82, 60177], [12, 101, 17495], [13, 76, 65530], [14, 96, 55828], [39, 15, 87502], [16, 35, 69761], [16, 86, 4277], [16, 70, 62887], [16, 62, 52272], [35, 17, 26914], [18, 26, 13235], [18, 39, 1762], [18, 75, 32308], [18, 98, 63073], [67, 19, 26914], [19, 43, 39993], [20, 21, 15511], [21, 25, 64679], [21, 41, 43412], [21, 80, 44055], [22, 31, 20571], [22, 94, 76460], [78, 23, 11970], [67, 24, 55977], [25, 49, 31800], [25, 73, 38191], [25, 79, 82634], [25, 27, 67629], [25, 45, 39114], [99, 28, 13253], [51, 29, 18241], [29, 44, 26653], [89, 30, 70775], [30, 90, 75422], [31, 55, 11344], [31, 71, 87630], [32, 33, 21191], [32, 58, 33919], [34, 47, 3472], [34, 53, 72002], [35, 84, 49868], [82, 36, 85010], [86, 37, 49169], [49, 38, 10709], [39, 67, 33306], [39, 52, 13144], [94, 40, 64075], [41, 72, 19543], [41, 89, 28241], [43, 102, 4469], [47, 85, 58204], [57, 48, 27954], [50, 60, 41215], [51, 63, 47815], [51, 100, 41201], [51, 83, 9090], [51, 74, 29442], [53, 88, 25055], [64, 54, 19217], [57, 61, 19983], [59, 64, 22222], [63, 81, 52716], [95, 66, 49970], [67, 99, 67268], [71, 93, 92788], [72, 91, 40294], [72, 95, 24533], [73, 97, 38029], [77, 78, 28229], [92, 87, 13667], [100, 92, 32670], [103, 105, 56293], [103, 106, 40308], [103, 115, 13648], [103, 141, 67728], [103, 218, 88518], [103, 112, 3867], [229, 104, 7405], [104, 125, 37249], [105, 158, 67775], [105, 227, 5314], [105, 163, 81886], [106, 108, 66572], [106, 113, 25713], [106, 151, 4787], [106, 107, 54451], [106, 130, 56329], [108, 111, 64972], [108, 116, 40618], [108, 144, 161], [108, 161, 42543], [108, 167, 24514], [108, 234, 14423], [210, 109, 68982], [178, 110, 32648], [111, 126, 17143], [111, 173, 62543], [113, 127, 1235], [113, 133, 14545], [113, 138, 60159], [113, 217, 56060], [113, 191, 56978], [157, 114, 7480], [115, 118, 7549], [115, 179, 45087], [115, 132, 57663], [116, 120, 82641], [116, 122, 87950], [116, 148, 53504], [116, 176, 43418], [116, 214, 83150], [136, 117, 71888], [117, 237, 89686], [118, 164, 57357], [118, 189, 53367], [118, 196, 27159], [200, 119, 16601], [120, 195, 9603], [120, 156, 14415], [120, 186, 48726], [120, 209, 73716], [120, 139, 34092], [120, 226, 46856], [222, 121, 18764], [121, 223, 13914], [122, 177, 43494], [122, 185, 69480], [170, 123, 56823], [126, 124, 36052], [240, 128, 34638], [228, 129, 13227], [238, 131, 1424], [131, 224, 36711], [170, 134, 40310], [177, 135, 3365], [150, 136, 32559], [162, 137, 84761], [138, 150, 59196], [138, 157, 25735], [138, 216, 46331], [168, 140, 42477], [140, 201, 22508], [141, 142, 25364], [142, 166, 55166], [142, 211, 11194], [189, 143, 27877], [143, 235, 40033], [144, 172, 23810], [241, 145, 5227], [177, 146, 4237], [146, 219, 53788], [161, 147, 33619], [148, 194, 3452], [148, 222, 36232], [199, 149, 76476], [150, 190, 47526], [151, 197, 16711], [177, 152, 14071], [152, 192, 15369], [152, 232, 46273], [238, 153, 38015], [153, 225, 20585], [238, 154, 6831], [154, 187, 76198], [186, 155, 22576], [157, 174, 39351], [158, 162, 51479], [158, 160, 58141], [170, 159, 54150], [161, 168, 91277], [161, 175, 37590], [162, 170, 31649], [162, 229, 80667], [164, 183, 49095], [164, 199, 2051], [164, 241, 6062], [210, 165, 42108], [165, 205, 35456], [166, 169, 71708], [166, 198, 39232], [166, 213, 78688], [167, 228, 12819], [168, 178, 72908], [170, 208, 69432], [175, 171, 25143], [173, 184, 30636], [174, 242, 37723], [175, 188, 52539], [175, 221, 30270], [175, 193, 68521], [178, 182, 80191], [178, 230, 56119], [178, 200, 36121], [206, 180, 66486], [222, 181, 6640], [184, 231, 2145], [184, 236, 14971], [186, 203, 5637], [186, 233, 26628], [188, 238, 31314], [189, 210, 21637], [225, 202, 15072], [230, 204, 17770], [228, 206, 67685], [222, 207, 58540], [209, 239, 73839], [227, 212, 42940], [214, 220, 10071], [239, 215, 7465], [225, 240, 34045]]
well_status= [4, 8, 14, 20, 23, 26, 33, 35, 36, 37, 38, 45, 46, 50, 53, 55, 56, 59, 61, 69, 70, 73, 76, 81, 85, 87, 90, 93, 94, 103, 123, 126, 130, 132, 133, 134, 135, 136, 139, 142, 143, 154, 159, 160, 161, 164, 171, 173, 174, 179, 180, 181, 182, 185, 188, 197, 198, 200, 201, 203, 205, 207, 208, 213, 214, 216, 224, 225, 227, 229, 230, 231, 232, 233, 234, 237]
pipe_status= [(1, 2), (1, 13), (2, 3), (2, 18), (2, 51), (2, 65), (3, 42), (4, 27), (5, 69), (6, 67), (7, 39), (7, 68), (8, 79), (9, 34), (9, 46), (10, 12), (10, 16), (10, 22), (11, 51), (12, 82), (12, 101), (14, 96), (15, 39), (16, 62), (16, 86), (17, 35), (18, 26), (18, 39), (18, 75), (18, 98), (19, 43), (19, 67), (20, 21), (21, 41), (21, 80), (22, 31), (23, 78), (24, 67), (25, 49), (28, 99), (29, 44), (29, 51), (30, 89), (31, 55), (31, 71), (32, 33), (32, 58), (34, 47), (35, 84), (38, 49), (39, 52), (39, 67), (40, 94), (41, 72), (41, 89), (43, 102), (48, 57), (50, 60), (51, 63), (51, 74), (51, 83), (53, 88), (54, 64), (57, 61), (59, 64), (66, 95), (67, 99), (72, 91), (72, 95), (73, 97), (77, 78), (87, 92), (92, 100), (103, 112), (103, 115), (103, 218), (104, 125), (104, 229), (105, 163), (105, 227), (106, 107), (106, 151), (108, 116), (108, 144), (108, 167), (108, 234), (109, 210), (110, 178), (111, 126), (113, 127), (113, 133), (113, 191), (113, 217), (114, 157), (115, 118), (116, 176), (117, 136), (118, 196), (119, 200), (120, 139), (120, 156), (120, 195), (120, 209), (120, 226), (121, 222), (121, 223), (122, 177), (124, 126), (128, 240), (129, 228), (131, 238), (134, 170), (135, 177), (136, 150), (137, 162), (138, 157), (140, 168), (140, 201), (141, 142), (142, 211), (143, 189), (143, 235), (144, 172), (145, 241), (146, 177), (146, 219), (147, 161), (148, 194), (148, 222), (149, 199), (150, 190), (151, 197), (152, 177), (152, 192), (153, 225), (154, 187), (154, 238), (155, 186), (157, 174), (158, 162), (162, 170), (164, 183), (164, 199), (164, 241), (165, 205), (166, 169), (166, 198), (167, 228), (171, 175), (174, 242), (175, 193), (175, 221), (178, 200), (180, 206), (181, 222), (184, 231), (184, 236), (186, 203), (189, 210), (202, 225), (204, 230), (209, 239), (212, 227), (214, 220), (215, 239), (225, 240)]
wells_mapping= {1: 10, 2: 11, 3: 20, 4: 2069, 5: 1563, 6: 1052, 7: 546, 8: 1059, 9: 40, 10: 41, 11: 2092, 12: 48, 13: 53, 14: 1080, 15: 568, 16: 57, 17: 1087, 18: 65, 19: 583, 20: 85, 21: 103, 22: 104, 23: 1641, 24: 1130, 25: 107, 26: 108, 27: 1142, 28: 1659, 29: 636, 30: 1151, 31: 129, 32: 130, 33: 133, 34: 140, 35: 142, 36: 1682, 37: 1178, 38: 1181, 39: 167, 40: 1201, 41: 189, 42: 702, 43: 1221, 44: 1225, 45: 1749, 46: 214, 47: 215, 48: 1238, 49: 245, 50: 1275, 51: 252, 52: 1788, 53: 254, 54: 1277, 55: 257, 56: 1797, 57: 270, 58: 1298, 59: 284, 60: 1312, 61: 1319, 62: 1320, 63: 297, 64: 301, 65: 820, 66: 1845, 67: 314, 68: 1340, 69: 1856, 70: 840, 71: 1866, 72: 336, 73: 337, 74: 1875, 75: 347, 76: 864, 77: 354, 78: 870, 79: 369, 80: 370, 81: 1907, 82: 381, 83: 1411, 84: 1928, 85: 908, 86: 403, 87: 1436, 88: 413, 89: 414, 90: 1956, 91: 427, 92: 955, 93: 1981, 94: 460, 95: 1485, 96: 2003, 97: 2004, 98: 1493, 99: 986, 100: 480, 101: 1510, 102: 2037, 103: 1, 104: 514, 105: 5, 106: 8, 107: 522, 108: 12, 109: 526, 110: 1041, 111: 21, 112: 1047, 113: 23, 114: 2072, 115: 33, 116: 36, 117: 1061, 118: 38, 119: 1579, 120: 44, 121: 1073, 122: 50, 123: 1076, 124: 1077, 125: 1598, 126: 69, 127: 73, 128: 2122, 129: 592, 130: 1105, 131: 596, 132: 1621, 133: 86, 134: 1111, 135: 619, 136: 620, 137: 625, 138: 113, 139: 1655, 140: 631, 141: 121, 142: 124, 143: 638, 144: 127, 145: 1150, 146: 1155, 147: 1158, 148: 146, 149: 1683, 150: 148, 151: 149, 152: 659, 153: 670, 154: 1694, 155: 1697, 156: 678, 157: 168, 158: 171, 159: 1198, 160: 1204, 161: 181, 162: 182, 163: 696, 164: 192, 165: 708, 166: 709, 167: 198, 168: 201, 169: 715, 170: 205, 171: 724, 172: 234, 173: 747, 174: 749, 175: 242, 176: 762, 177: 261, 178: 265, 179: 1297, 180: 1301, 181: 1819, 182: 797, 183: 293, 184: 809, 185: 1321, 186: 814, 187: 1840, 188: 308, 189: 309, 190: 310, 191: 823, 192: 1332, 193: 1855, 194: 326, 195: 327, 196: 1868, 197: 338, 198: 1366, 199: 357, 200: 1381, 201: 873, 202: 1391, 203: 1906, 204: 1911, 205: 1402, 206: 891, 207: 1919, 208: 1922, 209: 899, 210: 389, 211: 1416, 212: 1932, 213: 1420, 214: 915, 215: 1943, 216: 419, 217: 421, 218: 422, 219: 1454, 220: 1456, 221: 1458, 222: 950, 223: 1974, 224: 953, 225: 956, 226: 1982, 227: 450, 228: 452, 229: 458, 230: 974, 231: 975, 232: 1494, 233: 2015, 234: 2016, 235: 1504, 236: 2025, 237: 1516, 238: 495, 239: 1521, 240: 1014, 241: 508, 242: 1535}
res = 6917605

n,wells_dict_new,pipes_new,well_status_new,pipe_status_new = translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping)
pack1 = [wells_dict_new,pipes_new,well_status_new,pipe_status_new]


KeyError: 243

In [96]:
# Res1
n= 2160
wells_list= [38081, 9906, 71867, 48475, 44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 95924, 42417, 90525, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 40675, 34549, 25622, 50209, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 66258, 66998, 86025, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 34386, 98070, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19058, 38363, 67282, 97436, 28941, 37468, 77321, 21735, 39947, 45271, 75551, 22756, 94693, 74923, 63417, 66612, 36128, 85415, 74813, 28512, 26550, 67591, 47740, 68396, 15283, 15648, 23442, 95657, 99006, 53923, 40365, 93449, 64620, 98318, 80410, 88103, 13907, 62043, 21740, 36137, 79315, 86038, 19773, 82593, 43382, 34577, 23476, 27968, 58730, 60893, 48230, 32796, 82817, 6078, 48538, 95367, 84262, 88804, 69862, 69479, 60964, 38756, 25008, 7142, 23986, 84551, 7812, 79766, 21233, 86242, 25261, 74883, 5960, 66064, 77217, 53785, 46293, 83768, 25802, 54151, 80686, 14218, 90413, 97864, 73782, 94318, 67909, 84451, 91892, 12560, 17023, 52220, 2395, 80816, 18049, 40026, 29409, 14129, 19124, 32383, 59429, 76000, 10553, 35002, 85675, 78715, 60336, 91802, 33316, 5233, 83107, 61987, 12187, 9403, 96846, 38880, 44828, 91629, 91208, 80081, 90763, 7249, 95777, 89497, 76600, 94643, 29116, 12326, 76625, 75881, 23050, 30468, 45453, 88407, 15429, 78134, 52108, 86442, 21241, 27928, 52300, 79399, 79367, 35413, 49363, 85891, 65270, 92852, 31546, 19770, 86711, 14769, 72772, 27712, 90511, 16137, 91765, 75643, 11450, 44940, 46175, 58545, 9368, 28069, 61049, 93651, 32410, 91201, 56586, 19817, 82780, 58992, 51253, 86702, 65267, 17781, 35650, 86885, 7820, 66430, 70357, 55297, 39768, 20774, 39768, 74687, 95554, 48913, 64492, 99818, 92997, 82084, 91865, 42723, 7000, 56647, 68687, 94408, 78624, 79479, 75537, 36727, 69741, 86222, 1196, 59633, 48391, 95494, 54878, 15471, 8204, 4972, 91150, 69139, 98714, 4152, 12439, 2956, 71363, 45946, 60278, 85921, 4591, 83329, 68830, 82139, 49310, 98620, 4849, 27990, 88699, 8153, 66917, 37264, 55248, 95570, 7117, 94946, 13553, 3167, 88979, 18789, 37787, 18403, 63982, 89986, 86350, 72264, 67512, 96140, 44070, 82957, 98072, 2232, 23328, 73235, 13952, 14602, 47560, 16713, 33345, 33413, 93827, 64812, 14913, 67553, 97656, 92747, 60331, 63232, 34998, 63993, 58572, 22758, 18988, 66176, 70436, 89029, 72376, 72973, 75462, 79604, 45311, 43538, 87603, 76317, 30178, 31970, 99969, 13251, 35657, 78482, 39811, 85806, 97363, 91948, 18385, 28948, 67657, 90997, 79311, 77807, 68742, 85479, 37468, 95472, 696, 29936, 86857, 37285, 38935, 85470, 46133, 77675, 8333, 97149, 77182, 47926, 11811, 23958, 41596, 91004, 13699, 57144, 21681, 45909, 75112, 26172, 87461, 93010, 55242, 881, 19311, 83407, 85963, 76066, 21939, 3082, 18991, 81875, 63974, 40406, 65276, 71587, 7946, 5094, 85334, 16521, 48583, 87310, 41534, 45426, 68623, 36855, 66871, 75649, 3143, 65308, 255, 85964, 82099, 29459, 20105, 40665, 79570, 91207, 50416, 33081, 63434, 31348, 93222, 2275, 39934, 34984, 27996, 57049, 43787, 52595, 14829, 78884, 32294, 83093, 92659, 23372, 7513, 85680, 50444, 11746, 2688, 46001, 60523, 71350, 56177, 98108, 54252, 13794, 65743, 17491, 97038, 33614, 79270, 16882, 99199, 963, 53676, 16670, 69944, 32931, 64378, 53940, 54184, 86465, 19348, 28559, 46817, 86953, 23036, 56981, 38572, 88290, 27079, 85297, 47940, 20826, 24083, 22460, 39422, 75038, 56077, 64099, 29307, 66597, 56082, 94769, 74838, 40201, 51050, 97608, 47250, 84274, 43415, 35040, 97305, 82722, 69731, 9150, 94333, 63698, 94971, 86961, 70040, 7013, 69518, 52, 50985, 76950, 70698, 63517, 95474, 1254, 417, 56229, 93129, 69679, 98310, 18033, 87846, 57023, 75157, 25873, 14316, 52285, 80808, 78484, 30094, 39911, 42940, 20441, 72293, 34158, 83842, 92485, 1100, 56346, 72014, 14837, 54135, 15162, 71747, 93408, 23362, 55188, 18163, 92072, 19957, 26460, 69720, 60578, 27355, 61622, 97595, 43097, 38229, 30505, 28404, 34642, 7780, 65626, 6301, 59674, 50680, 76250, 34126, 5520, 25645, 5412, 90546, 30355, 82357, 17619, 83135, 33041, 71148, 27588, 78139, 37125, 33806, 72774, 47772, 73407, 79657, 22849, 836, 13676, 47699, 3616, 2605, 80699, 62450, 86182, 31534, 56117, 71610, 85096, 72666, 88888, 93561, 87582, 64603, 54254, 36553, 58355, 83789, 64394, 24789, 73448, 71806, 82706, 47094, 59286, 47209, 11338, 85237, 59794, 46055, 41791, 88836, 95459, 88208, 90124, 92109, 62558, 73057, 92244, 71604, 45460, 5839, 38383, 26061, 5392, 11007, 44851, 71096, 3204, 83051, 36091, 18922, 13793, 93442, 48604, 45809, 35980, 87511, 56371, 37033, 25667, 90687, 38283, 63896, 60891, 98682, 94580, 85079, 10069, 69022, 54675, 15912, 95455, 50979, 16298, 34799, 9182, 21637, 93238, 55093, 99587, 31275, 89505, 99390, 95378, 69783, 31037, 57279, 50531, 82947, 84495, 70584, 88362, 98041, 1223, 53354, 23856, 48924, 26188, 91156, 84762, 10191, 60839, 6178, 72845, 60727, 8605, 60336, 57466, 41129, 79672, 27170, 65879, 57957, 92183, 96659, 35272, 15375, 31674, 86587, 35546, 90830, 50264, 86873, 23450, 5513, 92713, 75617, 57741, 47081, 43478, 79550, 89929, 30943, 98760, 90108, 81437, 61468, 70688, 46298, 94709, 58436, 46286, 94009, 29876, 61801, 28593, 85724, 19820, 82360, 28288, 19438, 16648, 88970, 47187, 31566, 43084, 64484, 39266, 25891, 81295, 319, 83518, 48482, 78600, 86706, 87729, 34123, 64805, 22640, 60603, 55603, 21563, 99566, 91655, 56033, 59391, 85648, 85484, 48946, 3429, 51217, 12396, 38446, 97715, 72430, 1026, 23917, 97609, 48527, 19246, 63029, 44684, 76282, 32300, 29926, 28557, 91335, 67951, 28910, 88176, 58848, 17252, 54823, 87322, 45896, 55509, 50796, 7633, 1264, 42439, 21713, 78166, 57653, 90650, 47732, 83802, 52863, 30185, 59835, 42326, 69967, 50431, 39936, 93374, 983, 39560, 78887, 73205, 93191, 87022, 26673, 38466, 83431, 13489, 11016, 49070, 37111, 95934, 37470, 51497, 57584, 360, 36762, 18190, 4698, 38312, 53360, 47291, 81765, 34294, 21279, 74437, 98398, 75234, 15731, 86735, 51340, 79955, 1823, 26081, 22084, 74952, 40488, 64155, 4026, 19838, 64335, 32116, 29075, 27244, 21991, 42689, 7112, 59362, 99874, 54960, 88375, 77716, 93551, 4740, 86421, 422, 52145, 63826, 85789, 98095, 31197, 84459, 1349, 61572, 89736, 90251, 75106, 78896, 54084, 96988, 54748, 12950, 27555, 24758, 82305, 32394, 86609, 68740, 86997, 4105, 49656, 83612, 81363, 77652, 6824, 46003, 90097, 22508, 46584, 24195, 11159, 19027, 90366, 58144, 83706, 95157, 21275, 83031, 14719, 3483, 50033, 97825, 42155, 99995, 25761, 71614, 85215, 88004, 64604, 2832, 46958, 85462, 95682, 99676, 60801, 28614, 9796, 88540, 24080, 65550, 81104, 15812, 17036, 90916, 54812, 11507, 1367, 70886, 78424, 6261, 74823, 30774, 15966, 82528, 25807, 35466, 39029, 83006, 40098, 17193, 55100, 69322, 95770, 31506, 16731, 44486, 88394, 33203, 30030, 28982, 69952, 93368, 97822, 58705, 64292, 12630, 37588, 91983, 71099, 68901, 54303, 44256, 31891, 91011, 12584, 14048, 58578, 99520, 95938, 35090, 94585, 52891, 97128, 41526, 86629, 20779, 30367, 94181, 91748, 63000, 74743, 37006, 81778, 68741, 81455, 23044, 28518, 42921, 25176, 54706, 49932, 32140, 89836, 44026, 50513, 51596, 56829, 90342, 74984, 33656, 56044, 63901, 46685, 25944, 63230, 89760, 40707, 28697, 52533, 50505, 71628, 95758, 46210, 79515, 28572, 78450, 82245, 21353, 41716, 8130, 92564, 76211, 2136, 63443, 87232, 99360, 64463, 37458, 44543, 8827, 1889, 59422, 87196, 60748, 99435, 28916, 4754, 86736, 46080, 16007, 3575, 73960, 22078, 60895, 46819, 23008, 78411, 25918, 29944, 94167, 20366, 80971, 23827, 17949, 9283, 49105, 82832, 45056, 310, 55002, 90490, 19450, 97705, 89064, 22083, 73469, 58608, 44113, 77184, 32255, 15677, 83447, 52197, 90286, 66651, 5027, 53441, 65068, 93175, 84312, 42894, 1065, 17283, 74815, 79220, 10870, 21970, 45055, 94119, 15949, 43561, 61746, 98475, 60603, 58850, 33917, 88987, 85502, 33281, 76623, 71898, 35799, 69623, 96836, 81074, 45366, 17832, 12957, 47871, 45117, 3147, 55778, 74942, 74073, 21838, 98742, 88281, 25676, 3256, 99925, 96875, 46785, 15809, 43457, 94121, 53188, 47083, 61435, 53310, 41472, 13513, 67632, 60443, 2012, 88462, 61217, 23447, 2268, 2821, 59656, 77471, 93666, 99245, 84478, 27644, 79625, 72736, 43732, 42496, 85517, 19145, 88501, 59683, 77525, 71445, 47344, 20082, 94607, 55850, 15632, 77489, 65853, 48083, 2005, 1870, 43768, 18891, 3693, 72032, 68680, 19587, 59629, 13220, 27122, 26493, 74140, 36594, 99796, 59057, 85989, 76935, 33696, 79569, 1714, 62577, 89887, 20039, 97540, 56244, 87359, 31226, 70554, 9876, 99613, 71267, 17907, 15272, 61090, 6978, 29493, 40444, 10448, 26197, 56209, 51985, 20875, 90437, 39107, 26877, 31952, 49253, 83657, 13912, 65944, 39482, 20262, 22448, 18913, 11676, 12491, 60911, 88693, 71223, 39237, 33641, 63449, 14604, 56680, 44473, 15323, 72637, 60748, 28712, 57031, 39876, 51623, 95477, 50497, 43853, 38785, 86811, 5692, 17444, 72501, 64776, 35954, 59327, 39989, 9175, 58027, 94845, 14914, 54068, 62205, 33504, 157, 88334, 69329, 9182, 20839, 10942, 39527, 40404, 38336, 57518, 66786, 55452, 56063, 25555, 3098, 78399, 68116, 49549, 46858, 29263, 81157, 90298, 42362, 65802, 23212, 35132, 66198, 64269, 34095, 14522, 87995, 24268, 86908, 95870, 90231, 28219, 43118, 59659, 17898, 19580, 36183, 75864, 85446, 12947, 77315, 23028, 81583, 55237, 62009, 77985, 36985, 9968, 76059, 77072, 6457, 82269, 64404, 17018, 71572, 7937, 48393, 29011, 51588, 41219, 54768, 37190, 94174, 4591, 51467, 80422, 65566, 75624, 73225, 83719, 92804, 73011, 39266, 98388, 54351, 50557, 46125, 1351, 27746, 39233, 42991, 32812, 64094, 89617, 18388, 6020, 29658, 60356, 66000, 45376, 91590, 71113, 94173, 50298, 96619, 29951, 41171, 50478, 77295, 95855, 10772, 62157, 5462, 98533, 53780, 15586, 51944, 25443, 28619, 80041, 21503, 34148, 13434, 42662, 26649, 91126, 42477, 74728, 23351, 49192, 40988, 238, 90872, 33134, 15654, 80000, 51287, 55307, 71622, 1298, 60579, 12341, 69500, 4336, 72805, 85030, 46410, 11061, 44460, 46563, 829, 90584, 44507, 35594, 43442, 10098, 5331, 65529, 31512, 84294, 78699, 39077, 33486, 89742, 30731, 13288, 88033, 86671, 9845, 49337, 71879, 1438, 98886, 68237, 37497, 26189, 32678, 76721, 23594, 72495, 28335, 68462, 58251, 33366, 63690, 84694, 99588, 86685, 85609, 10443, 60071, 4706, 28940, 12213, 40896, 59484, 35362, 17653, 19588, 65047, 95671, 75478, 15094, 20761, 16508, 36348, 14198, 52515, 25660, 86179, 7336, 23244, 12905, 70902, 93318, 95179, 90742, 20674, 97921, 39982, 95195, 91913, 30773, 97932, 58949, 9924, 40055, 35411, 68918, 30964, 69271, 36201, 55942, 80063, 33726, 3072, 61317, 97386, 2158, 57258, 38615, 63597, 70777, 20502, 34635, 38991, 35931, 64757, 39030, 60746, 42770, 46167, 4862, 58260, 57529, 88985, 30975, 47868, 45716, 96503, 56203, 20375, 17664, 3609, 32872, 75207, 87487, 660, 7801, 1970, 95636, 27176, 15139, 94210, 90004, 23465, 73334, 37155, 30564, 38884, 77600, 42548, 16215, 38997, 59286, 24666, 40043, 15078, 53228, 25909, 36626, 6177, 80210, 77109, 41721, 84589, 98990, 28596, 69948, 78640, 76532, 95887, 46395, 92727, 69348, 66892, 39683, 20450, 80456, 35530, 42257, 11956, 69388, 5116, 91271, 41021, 65814, 53963, 3970, 17916, 34502, 88527, 8335, 91604, 43907, 45057, 2265, 90790, 66511, 58217, 86930, 45923, 44904, 84940, 34364, 5907, 91717, 57455, 31702, 59113, 27740, 71507, 21546, 80024, 54264, 82114, 67273, 19165, 24925, 86472, 15058, 67934, 22604, 12601, 73375, 76799, 48517, 67614, 61493, 4853, 713, 26836, 54491, 39064, 65294, 98349, 3751, 46024, 43309, 6798, 41538, 5754, 96177, 20517, 37256, 46083, 68708, 10130, 14780, 63575, 2139, 16101, 15681, 5336, 9078, 95069, 77620, 44690, 5053, 44021, 51953, 10073, 4715, 34680, 52971, 97436, 11855, 41343, 29054, 58199, 92085, 89615, 1772, 10678, 49679, 50586, 44185, 13027, 69692, 31352, 76171, 83261, 45124, 56074, 94110, 11243, 54410, 29161, 61841, 86841, 66722, 36683, 34651, 13522, 64485, 73304, 84354, 95723, 34044, 1959, 31534, 20900, 5792, 39137, 90634, 16114, 38325, 62497, 21077, 92151, 55466, 33082, 39345, 67320, 49027, 12491, 54587, 91582, 44510, 98525, 6301, 17985, 2282, 54665, 33357, 58874, 64509, 11657, 6172, 96318, 40396, 57201, 63440, 51511, 62629, 39984, 85482, 29870, 83142, 83613, 85538, 49604, 78888, 58447, 77154, 69176, 63977, 62761, 11804, 62098, 71020, 57978, 21113, 42282, 60170, 23621, 46537, 3052, 6581, 49339, 40441, 92226, 28649, 85128, 52100, 27984, 16651, 90707, 34489, 92620, 26047, 41052, 6184, 87910, 79138, 58146, 79241, 5072, 20496, 73077, 13028, 85519, 13670, 2131, 16882, 69394, 28320, 64596, 54741, 89993, 67641, 12843, 2144, 33174, 85810, 47859, 4664, 23305, 12016, 14210, 4975, 37027, 72058, 44971, 42668, 76760, 2828, 68327, 40131, 53863, 79328, 68593, 43480, 18998, 42386, 68020, 7827, 1189, 70993, 55910, 36375, 92770, 5434, 56463, 33670, 52665, 20553, 10330, 73558, 76625, 88553, 4114, 61836, 5555, 68425, 17982, 30498, 18410, 33654, 43408, 72011, 76204, 10076, 65389, 82391, 63488, 43929, 50115, 98671, 39243, 47343, 36782, 113, 14485, 26979, 56648, 50639, 71983, 59367, 72065, 72952, 33567, 27886, 3944, 15910, 25425, 87066, 41069, 56416, 38696, 16194, 37624, 57996, 33868, 32421, 10936, 50368, 10065, 57043, 30240, 16637, 61042, 89600, 8957, 5405, 47163, 67628, 23794, 59937, 12972, 26831, 19536, 72609, 54855, 30488, 49003, 10208, 93325, 36045, 12625, 15328, 57608, 10863, 49362, 56049, 66867, 51650, 56525, 29439, 8914, 36455, 85308, 9494, 37793, 87078, 8201, 54357, 79427, 35113, 77721, 46358, 21418, 69829, 81363, 95774, 21317, 25113, 11269, 807, 91932, 51134, 62737, 40148, 67247, 5096, 3114, 33966, 17456, 23097, 39230, 80770, 11240, 77284, 15023, 30391, 24714, 14294, 56771, 67724, 65073, 52974, 24482, 15301, 48519, 30650, 29384, 55763, 3132, 53207, 26291, 31487, 15971, 7955, 7211, 33389, 456, 96757, 42100, 30640, 15057, 65425, 25940, 55659, 45772, 20572, 22873, 3559, 89716, 12134, 98798, 15834, 22608, 30524, 4873, 91442, 76576, 91062, 52699, 69931, 90410, 32374, 87300, 31268, 57498, 82883, 37526, 46009, 7789, 57794, 43596, 39083, 5888, 26790, 64910, 10178, 4109, 83421, 25689, 14179, 16455, 26146, 6313, 15323, 37263, 19579, 90506, 92912, 87344, 81512, 43558, 56845, 9585, 5829, 6649, 8593, 50531, 97918, 21660, 60257, 11590, 50642, 89798, 22528, 48925, 79846, 19880, 61399, 94329, 8172, 23458, 10771, 45015, 18172, 97649, 22727, 73587, 92560, 22824, 69204, 25831, 38517, 9287, 92402, 5769, 19375, 35004, 99673, 23603, 10241, 51346, 46503, 38379, 78332, 17462, 10750, 84760, 19788, 64918, 27, 64418, 15310, 13443, 41100, 86639, 20160, 62898, 30139, 10098]
pipe_list= [[1, 3, 56081], [3, 2, 37102], [5, 6, 694], [5, 17, 53621], [5, 19, 44244], [6, 7, 38134], [6, 14, 27141], [6, 22, 16455], [6, 55, 4094], [6, 69, 15576], [7, 13, 57468], [7, 46, 45209], [31, 8, 48215], [61, 9, 62330], [9, 73, 34317], [71, 10, 55563], [43, 11, 40097], [11, 54, 61773], [11, 72, 66756], [83, 12, 35536], [13, 24, 44932], [13, 36, 57740], [13, 38, 1991], [13, 50, 22060], [13, 61, 54520], [13, 81, 53146], [14, 16, 28292], [14, 20, 46504], [14, 26, 12824], [14, 60, 39052], [55, 15, 272], [16, 63, 40024], [16, 86, 60177], [16, 105, 17495], [17, 80, 65530], [43, 18, 87502], [19, 100, 55828], [20, 39, 69761], [20, 90, 4277], [20, 74, 62887], [20, 66, 52272], [39, 21, 26914], [22, 30, 13235], [22, 43, 1762], [22, 79, 32308], [22, 102, 63073], [71, 23, 26914], [23, 47, 39993], [24, 25, 15511], [25, 29, 64679], [25, 45, 43412], [25, 84, 44055], [26, 35, 20571], [26, 98, 76460], [82, 27, 11970], [71, 28, 55977], [29, 53, 31800], [29, 77, 38191], [29, 83, 82634], [29, 31, 67629], [29, 49, 39114], [103, 32, 13253], [55, 33, 18241], [33, 48, 26653], [93, 34, 70775], [34, 94, 75422], [35, 59, 11344], [35, 75, 87630], [36, 37, 21191], [36, 62, 33919], [38, 51, 3472], [38, 57, 72002], [39, 88, 49868], [86, 40, 85010], [90, 41, 49169], [53, 42, 10709], [43, 71, 33306], [43, 56, 13144], [98, 44, 64075], [45, 76, 19543], [45, 93, 28241], [47, 106, 4469], [51, 89, 58204], [61, 52, 27954], [54, 64, 41215], [55, 67, 47815], [55, 104, 41201], [55, 87, 9090], [55, 78, 29442], [57, 92, 25055], [68, 58, 19217], [61, 65, 19983], [63, 68, 22222], [67, 85, 52716], [99, 70, 49970], [71, 103, 67268], [75, 97, 92788], [76, 95, 40294], [76, 99, 24533], [77, 101, 38029], [81, 82, 28229], [96, 91, 13667], [104, 96, 32670], [108, 109, 31787], [110, 112, 56293], [110, 113, 40308], [110, 122, 13648], [110, 148, 67728], [110, 225, 88518], [110, 120, 3867], [236, 111, 7405], [111, 132, 37249], [112, 165, 67775], [112, 234, 5314], [112, 170, 81886], [113, 115, 66572], [113, 119, 25713], [113, 158, 4787], [113, 114, 54451], [113, 137, 56329], [115, 118, 64972], [115, 123, 40618], [115, 151, 161], [115, 168, 42543], [115, 174, 24514], [115, 241, 14423], [217, 116, 68982], [185, 117, 32648], [118, 133, 17143], [118, 180, 62543], [119, 134, 1235], [119, 140, 14545], [119, 144, 60159], [119, 224, 56060], [119, 198, 56978], [164, 121, 7480], [122, 125, 7549], [122, 186, 45087], [122, 139, 57663], [123, 127, 82641], [123, 129, 87950], [123, 155, 53504], [123, 183, 43418], [123, 221, 83150], [143, 124, 71888], [124, 244, 89686], [125, 171, 57357], [125, 197, 53367], [125, 203, 27159], [206, 126, 16601], [127, 202, 9603], [127, 163, 14415], [127, 193, 48726], [127, 216, 73716], [127, 147, 34092], [127, 233, 46856], [229, 128, 18764], [128, 230, 13914], [129, 184, 43494], [129, 191, 69480], [177, 130, 56823], [133, 131, 36052], [247, 135, 34638], [235, 136, 13227], [245, 138, 1424], [138, 231, 36711], [177, 141, 40310], [184, 142, 3365], [157, 143, 32559], [144, 157, 59196], [144, 164, 25735], [144, 223, 46331], [169, 145, 84761], [175, 146, 42477], [146, 208, 22508], [148, 149, 25364], [149, 173, 55166], [149, 218, 11194], [197, 150, 27877], [150, 242, 40033], [151, 179, 23810], [248, 152, 5227], [184, 153, 4237], [153, 226, 53788], [168, 154, 33619], [155, 201, 3452], [155, 229, 36232], [184, 156, 14071], [156, 196, 15369], [156, 239, 46273], [157, 199, 47526], [158, 204, 16711], [207, 159, 76476], [245, 160, 38015], [160, 232, 20585], [245, 161, 6831], [161, 194, 76198], [193, 162, 22576], [164, 181, 39351], [165, 169, 51479], [165, 167, 58141], [177, 166, 54150], [168, 175, 91277], [168, 182, 37590], [169, 177, 31649], [169, 236, 80667], [171, 190, 49095], [171, 207, 2051], [171, 248, 6062], [217, 172, 42108], [172, 212, 35456], [173, 176, 71708], [173, 205, 39232], [173, 220, 78688], [174, 235, 12819], [175, 185, 72908], [177, 215, 69432], [182, 178, 25143], [180, 192, 30636], [181, 249, 37723], [182, 195, 52539], [182, 228, 30270], [182, 200, 68521], [185, 189, 80191], [185, 237, 56119], [185, 206, 36121], [213, 187, 66486], [229, 188, 6640], [192, 238, 2145], [192, 243, 14971], [193, 210, 5637], [193, 240, 26628], [195, 245, 31314], [197, 217, 21637], [232, 209, 15072], [237, 211, 17770], [235, 213, 67685], [229, 214, 58540], [216, 246, 73839], [234, 219, 42940], [221, 227, 10071], [246, 222, 7465], [232, 247, 34045], [265, 251, 3436], [274, 252, 83063], [252, 272, 74902], [257, 253, 8634], [272, 254, 16549], [255, 261, 31880], [255, 274, 18420], [255, 256, 21536], [255, 268, 19141], [261, 257, 51647], [257, 264, 68072], [257, 269, 21665], [257, 270, 34057], [257, 262, 37847], [261, 258, 46530], [276, 259, 22523], [274, 260, 13932], [274, 263, 41835], [263, 275, 47267], [274, 265, 8643], [265, 266, 66300], [265, 273, 60437], [265, 267, 17030], [274, 271, 68886], [274, 276, 9036], [286, 277, 3689], [277, 284, 23506], [286, 278, 10077], [278, 285, 4799], [283, 279, 71713], [285, 280, 3776], [286, 281, 9629], [281, 282, 17834], [286, 283, 1420], [298, 287, 58624], [287, 288, 57594], [287, 290, 19997], [288, 291, 54249], [288, 294, 70528], [288, 292, 75540], [288, 297, 45025], [298, 289, 62204], [290, 293, 17113], [291, 296, 26907], [298, 295, 78743], [298, 299, 46085], [301, 300, 8645], [304, 303, 57861], [307, 304, 34510], [307, 305, 87353], [307, 306, 29650], [311, 308, 31778], [308, 310, 52644], [311, 309, 91095], [310, 317, 47022], [311, 316, 7780], [311, 315, 44950], [311, 312, 16216], [312, 314, 66512], [315, 313, 70390], [344, 318, 67496], [337, 319, 84585], [319, 322, 22595], [331, 320, 96333], [321, 335, 35133], [321, 337, 49749], [321, 339, 75213], [348, 323, 31141], [354, 324, 64752], [324, 328, 92359], [324, 332, 77972], [324, 333, 68233], [355, 325, 23285], [332, 326, 41386], [352, 327, 64775], [327, 351, 68249], [328, 338, 3603], [339, 329, 22973], [358, 330, 31484], [330, 345, 63006], [344, 331, 66948], [339, 334, 45219], [334, 343, 77236], [339, 336, 85382], [336, 347, 49386], [338, 355, 39415], [339, 352, 85281], [339, 357, 92648], [339, 348, 72509], [339, 354, 54935], [339, 340, 19739], [339, 358, 86174], [339, 359, 29019], [357, 341, 16167], [357, 342, 19735], [352, 344, 64507], [351, 346, 11298], [348, 356, 70090], [355, 349, 36783], [358, 350, 30617], [354, 353, 88111], [360, 370, 64967], [360, 368, 39298], [360, 371, 29287], [360, 364, 41643], [371, 361, 17475], [370, 362, 50055], [368, 363, 34905], [370, 365, 52037], [365, 369, 74787], [368, 366, 1836], [371, 367, 35706], [374, 372, 41230], [373, 374, 52331], [377, 375, 30089], [379, 376, 62567], [376, 386, 28861], [384, 377, 9003], [377, 390, 31266], [388, 378, 9311], [378, 389, 23375], [390, 379, 39233], [379, 387, 23204], [379, 381, 21399], [391, 380, 63311], [391, 382, 59119], [391, 383, 15180], [384, 388, 38342], [390, 385, 47428], [390, 391, 3124], [395, 392, 28573], [392, 404, 8925], [402, 393, 31636], [399, 394, 66030], [394, 396, 68921], [394, 405, 90152], [396, 395, 1326], [395, 402, 32796], [396, 400, 6460], [399, 397, 25509], [401, 398, 5763], [399, 401, 12673], [400, 403, 1667], [413, 408, 56377], [411, 409, 3186], [413, 410, 58025], [415, 411, 41056], [413, 412, 63609], [414, 413, 34629], [415, 414, 2757], [423, 416, 68142], [422, 417, 15781], [427, 418, 34522], [424, 419, 1435], [429, 420, 50925], [427, 421, 61194], [421, 425, 43429], [426, 422, 38652], [428, 423, 70614], [424, 427, 31287], [428, 426, 46162], [426, 429, 43158], [427, 428, 63188], [433, 430, 92400], [430, 431, 75774], [430, 432, 12986], [449, 434, 3122], [443, 435, 64676], [444, 436, 67081], [436, 445, 40027], [436, 439, 53644], [436, 438, 49155], [436, 443, 87182], [446, 437, 76779], [448, 440, 31554], [442, 441, 27651], [441, 447, 50811], [446, 442, 2781], [444, 446, 63691], [445, 448, 23211], [446, 450, 52997], [446, 449, 35396], [457, 453, 40484], [457, 454, 68228], [457, 455, 51029], [455, 458, 36996], [458, 456, 22585], [459, 457, 46318], [462, 461, 16627], [462, 463, 87018], [464, 467, 27282], [467, 465, 919], [467, 466, 16146], [469, 470, 33498], [478, 471, 44945], [471, 473, 81705], [475, 472, 10978], [473, 481, 68216], [485, 474, 66745], [474, 475, 67779], [475, 484, 59737], [479, 476, 14841], [476, 486, 52285], [477, 485, 48746], [485, 478, 5680], [485, 479, 2837], [485, 480, 38691], [480, 483, 2872], [485, 482, 68284], [488, 489, 60730], [488, 490, 22161], [490, 491, 10898], [491, 492, 34904], [493, 494, 26684], [505, 495, 40530], [507, 496, 18345], [511, 497, 67678], [507, 498, 39003], [498, 512, 8048], [509, 499, 10067], [507, 500, 46491], [500, 502, 81356], [500, 510, 58858], [507, 501, 37496], [501, 511, 50378], [502, 504, 4278], [502, 505, 20182], [508, 503, 41287], [512, 506, 66331], [509, 507, 46281], [507, 508, 55470], [518, 515, 17674], [515, 517, 26458], [518, 516, 8313], [518, 519, 47005], [525, 521, 57458], [525, 522, 24699], [527, 523, 51540], [525, 524, 52932], [524, 527, 8285], [524, 526, 62525], [533, 528, 55968], [528, 530, 10506], [535, 529, 69172], [531, 535, 66523], [533, 532, 13992], [534, 533, 45061], [535, 534, 21895], [540, 537, 18530], [538, 539, 40824], [538, 540, 52911], [541, 543, 9806], [543, 542, 26260], [543, 544, 92725], [546, 547, 42437], [549, 548, 35055], [550, 551, 57318], [554, 553, 3031], [553, 555, 44635], [558, 556, 27299], [556, 559, 41551], [558, 557, 44613], [560, 561, 14232], [560, 562, 15569], [565, 563, 45100], [563, 564, 81349], [567, 566, 3139], [591, 570, 76403], [588, 571, 18406], [571, 577, 45827], [572, 585, 77389], [572, 587, 9984], [572, 589, 42979], [572, 573, 817], [576, 574, 42618], [574, 590, 54118], [589, 575, 6972], [575, 579, 14315], [575, 591, 10137], [575, 581, 58494], [585, 576, 2711], [576, 580, 31140], [576, 578, 3409], [579, 583, 32231], [586, 582, 40777], [592, 584, 14295], [585, 588, 13583], [588, 586, 80622], [587, 592, 65526], [597, 595, 54378], [595, 599, 4852], [595, 603, 18480], [597, 596, 76533], [597, 602, 16071], [597, 601, 29220], [601, 598, 52237], [598, 600, 35303], [613, 604, 84844], [606, 605, 43736], [605, 607, 64981], [616, 606, 85639], [606, 617, 55738], [606, 618, 85911], [606, 612, 79112], [606, 610, 94700], [607, 608, 19232], [616, 609, 42433], [610, 611, 73876], [612, 614, 30771], [613, 616, 9371], [616, 615, 32278], [619, 620, 71936], [624, 621, 10769], [621, 623, 34755], [621, 622, 22761], [624, 626, 53315], [626, 625, 44928], [631, 627, 56754], [631, 628, 59331], [631, 629, 82808], [630, 632, 64810], [632, 631, 51217], [633, 634, 16293], [633, 635, 44907], [641, 637, 15372], [637, 638, 44618], [641, 639, 46238], [641, 640, 18157], [650, 642, 2698], [642, 649, 29819], [642, 646, 45894], [642, 644, 49319], [649, 643, 17680], [646, 645, 7700], [646, 647, 10020], [647, 648, 5107], [649, 651, 56787], [654, 653, 71217], [657, 658, 62791], [659, 660, 72635], [660, 661, 14393], [660, 662, 18610], [664, 663, 29487], [665, 666, 33042], [669, 668, 28017], [669, 670, 66052], [671, 672, 15958], [679, 673, 660], [688, 674, 42670], [674, 685, 64571], [674, 701, 9265], [704, 675, 51738], [675, 695, 46944], [709, 676, 93642], [707, 677, 10491], [681, 678, 12746], [679, 681, 76743], [679, 687, 16644], [679, 694, 38535], [679, 707, 65964], [679, 697, 9929], [679, 698, 15251], [707, 680, 59566], [681, 683, 19033], [681, 688, 22983], [681, 699, 17366], [681, 713, 15119], [707, 682, 40217], [682, 702, 25434], [682, 708, 12553], [683, 711, 13038], [705, 684, 9115], [723, 686, 7643], [687, 704, 64772], [687, 719, 3972], [688, 721, 35415], [688, 709, 93811], [704, 689, 77924], [723, 690, 23953], [690, 714, 42332], [715, 691, 81572], [701, 692, 44737], [718, 693, 35681], [693, 710, 70289], [694, 705, 37501], [715, 696, 31987], [698, 722, 63915], [699, 720, 67375], [704, 700, 83702], [711, 703, 70213], [704, 723, 75959], [707, 706, 43585], [707, 718, 2318], [707, 717, 41959], [721, 712, 54643], [721, 715, 86265], [715, 716, 13272], [724, 726, 63588], [726, 725, 35956], [727, 728, 29933], [729, 730, 92449], [731, 732, 29061], [735, 734, 19543], [736, 737, 25855], [737, 738, 8575], [741, 743, 8039], [741, 742, 84200], [749, 746, 71990], [747, 749, 37108], [749, 748, 52652], [759, 750, 71964], [750, 771, 1848], [751, 757, 94495], [762, 752, 51195], [759, 753, 32869], [760, 754, 64222], [766, 755, 34953], [760, 756, 86479], [757, 759, 95247], [757, 765, 44964], [757, 763, 52943], [757, 770, 4251], [757, 769, 17750], [759, 758, 38528], [759, 768, 45878], [759, 767, 17222], [771, 760, 22353], [762, 761, 5018], [765, 762, 45198], [762, 766, 55888], [771, 764, 25813], [774, 773, 3757], [789, 775, 30774], [775, 782, 35099], [789, 776, 3167], [776, 786, 77927], [782, 777, 83307], [777, 779, 65791], [787, 778, 67924], [787, 780, 25948], [784, 781, 16006], [782, 787, 60391], [787, 783, 1750], [789, 784, 64581], [787, 785, 27110], [789, 788, 5945], [790, 791, 32861], [792, 793, 45295], [797, 796, 1608], [797, 799, 25217], [797, 798, 5780], [799, 800, 31411], [809, 801, 33687], [809, 802, 61457], [802, 804, 60592], [805, 803, 72975], [809, 805, 79416], [806, 809, 16961], [806, 807, 24786], [809, 808, 4230], [812, 810, 58239], [815, 811, 6732], [811, 812, 85293], [814, 813, 70709], [814, 815, 49855], [818, 817, 58408], [825, 824, 54680], [825, 827, 2639], [825, 832, 24538], [832, 826, 37380], [827, 834, 40808], [827, 830, 73592], [830, 828, 75793], [830, 829, 61470], [830, 831, 70548], [830, 833, 1311], [841, 835, 1243], [835, 846, 35510], [840, 836, 4852], [852, 837, 61022], [837, 843, 15599], [842, 838, 82536], [839, 852, 34920], [839, 842, 9461], [839, 840, 47441], [839, 847, 34331], [839, 850, 23422], [839, 845, 69520], [840, 841, 45477], [841, 849, 36454], [842, 848, 20645], [845, 844, 56954], [852, 851, 40700], [856, 854, 10519], [854, 855, 15884], [857, 858, 23657], [860, 859, 29954], [868, 862, 23709], [877, 863, 27674], [877, 864, 29451], [868, 865, 80805], [865, 873, 11046], [865, 880, 24434], [868, 866, 5379], [877, 867, 17574], [867, 871, 9968], [868, 870, 10939], [870, 869, 48803], [870, 879, 34095], [870, 877, 86687], [870, 874, 28853], [876, 872, 2613], [879, 875, 43023], [877, 876, 43751], [877, 878, 71128], [883, 882, 49543], [886, 887, 1664], [888, 891, 12931], [888, 889, 16541], [888, 890, 49797], [892, 893, 33548], [892, 894, 49826], [892, 898, 59012], [893, 900, 33274], [894, 897, 1150], [894, 896, 7500], [900, 895, 20153], [895, 899, 46946], [902, 901, 92693], [904, 908, 33551], [904, 914, 53114], [908, 905, 41786], [905, 912, 29576], [905, 910, 32439], [912, 906, 30587], [915, 907, 29162], [908, 909, 37152], [908, 915, 68543], [914, 911, 3176], [915, 913, 23868], [918, 917, 3667], [921, 920, 27174], [923, 924, 51154], [927, 926, 41664], [932, 928, 15377], [928, 929, 38843], [929, 933, 18628], [930, 932, 3282], [930, 931, 69239], [940, 934, 13280], [934, 936, 33054], [939, 935, 63782], [936, 941, 4972], [942, 937, 16128], [939, 938, 69408], [940, 939, 72979], [940, 942, 13236], [944, 943, 8734], [946, 945, 22527], [949, 952, 64717], [949, 954, 86270], [949, 950, 14623], [953, 951, 50078], [952, 955, 3520], [954, 953, 19578], [955, 956, 23929], [958, 957, 32361], [958, 960, 3423], [958, 959, 52690], [967, 961, 11824], [968, 962, 63303], [966, 963, 14855], [963, 964, 5458], [963, 965, 54693], [964, 967, 46831], [966, 968, 77565], [968, 970, 29752], [968, 969, 2675], [972, 971, 4644], [972, 973, 17296], [975, 974, 33472], [978, 976, 64558], [977, 978, 21523], [982, 980, 28446], [981, 982, 504], [984, 988, 41754], [984, 986, 20474], [986, 985, 50950], [986, 987, 75435], [988, 989, 3702], [991, 990, 52759], [994, 991, 17057], [991, 1001, 14288], [991, 995, 8187], [994, 992, 5519], [994, 993, 7105], [993, 998, 53574], [1001, 996, 31037], [1000, 997, 50055], [1000, 999, 27437], [1001, 1000, 3081], [1003, 1004, 39924], [1006, 1005, 3185], [1008, 1007, 2935], [1008, 1010, 81426], [1008, 1009, 23096], [1009, 1013, 37938], [1010, 1011, 27492], [1010, 1014, 37992], [1013, 1012, 57282], [1020, 1015, 16516], [1015, 1021, 79561], [1017, 1016, 86604], [1019, 1017, 24223], [1020, 1018, 4160], [1018, 1022, 16773], [1020, 1019, 14405], [1024, 1023, 24461], [1026, 1027, 88949], [1032, 1028, 4791], [1029, 1032, 71732], [1032, 1030, 56750], [1032, 1031, 6131], [1034, 1035, 36501], [1034, 1037, 53844], [1035, 1036, 24494], [1039, 1038, 8951], [1044, 1041, 75514], [1044, 1042, 8199], [1045, 1043, 18683], [1043, 1044, 41397], [1048, 1049, 10299], [1049, 1050, 54028], [1052, 1058, 31665], [1052, 1063, 7520], [1052, 1059, 21013], [1063, 1053, 60405], [1053, 1061, 25696], [1063, 1054, 19323], [1063, 1055, 7279], [1059, 1056, 63634], [1056, 1060, 81563], [1058, 1057, 21241], [1058, 1062, 44602], [1059, 1064, 21833], [1067, 1065, 29179], [1066, 1067, 74146], [1069, 1070, 52684], [1070, 1071, 35972], [1073, 1072, 93130], [1073, 1074, 76992], [1077, 1076, 81587], [1079, 1080, 9913], [1080, 1081, 82236], [1083, 1082, 77111], [1087, 1085, 1667], [1087, 1086, 51479], [1088, 1087, 68874], [1088, 1089, 25994], [1092, 1091, 23274], [1094, 1093, 12748], [1095, 1098, 20070], [1095, 1096, 6282], [1098, 1097, 35787], [1100, 1102, 63656], [1100, 1103, 42933], [1102, 1101, 5537], [1105, 1104, 12523], [1105, 1107, 11827], [1107, 1106, 44337], [1118, 1108, 22312], [1108, 1119, 52219], [1108, 1115, 89778], [1119, 1109, 77376], [1109, 1113, 15048], [1118, 1110, 6772], [1123, 1111, 27531], [1111, 1112, 48526], [1112, 1122, 44133], [1116, 1114, 24267], [1123, 1116, 28453], [1118, 1117, 65874], [1121, 1118, 25460], [1119, 1120, 8108], [1121, 1123, 2568], [1130, 1125, 35477], [1129, 1126, 13974], [1126, 1127, 12588], [1129, 1128, 61310], [1130, 1129, 75663], [1132, 1133, 6679], [1134, 1135, 35012], [1134, 1136, 21897], [1137, 1138, 33779], [1138, 1140, 43138], [1138, 1139, 95390], [1142, 1141, 18003], [1141, 1150, 51676], [1147, 1142, 33060], [1145, 1143, 73693], [1143, 1144, 150], [1147, 1145, 31217], [1145, 1148, 46535], [1147, 1146, 56270], [1147, 1149, 20935], [1152, 1153, 20414], [1157, 1155, 60069], [1155, 1156, 10125], [1161, 1159, 30551], [1159, 1162, 5649], [1163, 1160, 12274], [1161, 1163, 32310], [1166, 1165, 43803], [1168, 1166, 4767], [1166, 1169, 62971], [1166, 1167, 36956], [1181, 1170, 47324], [1174, 1171, 43129], [1171, 1176, 9137], [1182, 1172, 88321], [1172, 1174, 4414], [1177, 1173, 19267], [1174, 1175, 74716], [1178, 1177, 32397], [1179, 1178, 51338], [1179, 1182, 4810], [1181, 1180, 23033], [1182, 1181, 48112], [1184, 1183, 65341], [1184, 1185, 38787], [1188, 1186, 14508], [1196, 1187, 29545], [1189, 1188, 57472], [1197, 1189, 10778], [1197, 1190, 46431], [1190, 1207, 73187], [1191, 1196, 49107], [1191, 1197, 57160], [1191, 1192, 11042], [1192, 1193, 17955], [1205, 1194, 30335], [1196, 1195, 47608], [1196, 1209, 45172], [1196, 1198, 4245], [1197, 1205, 27413], [1197, 1208, 45322], [1197, 1204, 58668], [1208, 1199, 9146], [1199, 1206, 15604], [1199, 1203, 25989], [1202, 1200, 32376], [1208, 1201, 35825], [1208, 1202, 9043], [1216, 1215, 49372], [1219, 1217, 8646], [1219, 1218, 63547], [1220, 1219, 77020], [1220, 1221, 15041], [1227, 1223, 26625], [1225, 1224, 52848], [1224, 1228, 54955], [1227, 1225, 45237], [1227, 1226, 7171], [1233, 1231, 19413], [1233, 1232, 26246], [1238, 1235, 11008], [1237, 1236, 29093], [1236, 1238, 36480], [1237, 1239, 32234], [1244, 1245, 53213], [1244, 1247, 73889], [1245, 1246, 87245], [1248, 1249, 61328], [1255, 1251, 63263], [1256, 1252, 6693], [1254, 1253, 15118], [1255, 1254, 33422], [1256, 1255, 35816], [1257, 1258, 3947], [1265, 1260, 33720], [1263, 1261, 5671], [1264, 1262, 44442], [1263, 1265, 10734], [1265, 1264, 38471], [1269, 1267, 42395], [1269, 1268, 24026], [1274, 1270, 7335], [1270, 1271, 16564], [1271, 1273, 54776], [1274, 1272, 36393], [1275, 1276, 18435], [1275, 1277, 34681], [1279, 1278, 49097], [1279, 1283, 53090], [1282, 1280, 45121], [1282, 1281, 49128], [1283, 1282, 32426], [1285, 1284, 53905], [1287, 1286, 69068], [1287, 1288, 54427], [1287, 1289, 52883], [1292, 1291, 69403], [1298, 1296, 65680], [1298, 1297, 21304], [1302, 1300, 3448], [1300, 1301, 51541], [1306, 1304, 17762], [1306, 1305, 3916], [1309, 1308, 16784], [1311, 1310, 88869], [1310, 1312, 40184], [1314, 1313, 41782], [1314, 1315, 77773], [1314, 1316, 42342], [1318, 1317, 35528], [1319, 1318, 22017], [1324, 1320, 58621], [1321, 1324, 13396], [1324, 1322, 3364], [1324, 1323, 36796], [1329, 1327, 18274], [1328, 1329, 54747], [1331, 1342, 36080], [1331, 1333, 31563], [1331, 1334, 48816], [1331, 1341, 46512], [1331, 1343, 51804], [1342, 1332, 14736], [1333, 1337, 54720], [1334, 1339, 22657], [1341, 1335, 7354], [1339, 1336, 8545], [1337, 1338, 5425], [1341, 1340, 84863], [1345, 1344, 17135], [1351, 1349, 3108], [1349, 1352, 34397], [1349, 1350, 54410], [1351, 1353, 50145], [1357, 1358, 493], [1361, 1360, 64416], [1364, 1365, 34218], [1367, 1368, 32702], [1373, 1369, 19217], [1373, 1370, 36080], [1373, 1371, 22789], [1371, 1372, 5520], [1376, 1375, 31842], [1378, 1379, 78922], [1381, 1380, 43503], [1380, 1382, 85100], [1386, 1385, 533], [1390, 1389, 14142], [1392, 1391, 30269], [1392, 1393, 64385], [1396, 1394, 6545], [1394, 1395, 25758], [1398, 1400, 55561], [1398, 1399, 3618], [1402, 1401, 68976], [1405, 1404, 38274], [1408, 1406, 52026], [1407, 1409, 11472], [1409, 1408, 42073], [1413, 1414, 42603], [1416, 1415, 30644], [1418, 1417, 15012], [1417, 1419, 31161], [1423, 1421, 28122], [1423, 1422, 9332], [1422, 1424, 37605], [1429, 1428, 6896], [1430, 1431, 2626], [1434, 1432, 49036], [1432, 1435, 50184], [1434, 1433, 47258], [1440, 1436, 3721], [1440, 1437, 46162], [1439, 1438, 34305], [1438, 1440, 45145], [1445, 1446, 86957], [1449, 1448, 142], [1449, 1450, 1716], [1456, 1453, 75362], [1456, 1454, 64954], [1456, 1455, 62970], [1455, 1457, 3021], [1460, 1459, 40634], [1462, 1461, 68390], [1465, 1464, 29322], [1468, 1466, 34614], [1468, 1467, 3109], [1472, 1471, 75996], [1473, 1472, 50918], [1477, 1476, 1843], [1479, 1478, 77892], [1481, 1480, 34749], [1482, 1483, 48105], [1487, 1485, 13063], [1485, 1489, 25559], [1487, 1486, 13083], [1486, 1488, 17946], [1493, 1492, 49417], [1494, 1493, 59207], [1497, 1496, 17912], [1498, 1499, 34399], [1505, 1504, 38472], [1509, 1507, 7391], [1507, 1508, 4294], [1512, 1511, 23707], [1512, 1513, 67666], [1518, 1519, 27941], [1521, 1522, 1120], [1521, 1523, 18282], [1531, 1532, 60017], [1534, 1533, 41587], [1537, 1534, 4586], [1537, 1535, 42706], [1535, 1540, 68399], [1537, 1536, 24260], [1536, 1539, 70782], [1536, 1538, 63355], [1540, 1541, 25202], [1544, 1543, 47413], [1548, 1545, 42673], [1549, 1546, 29487], [1549, 1547, 19013], [1549, 1548, 55761], [1551, 1552, 63974], [1554, 1553, 12721], [1556, 1557, 2115], [1559, 1560, 11288], [1562, 1563, 57960], [1568, 1567, 59080], [1567, 1570, 91953], [1568, 1569, 93346], [1575, 1571, 64709], [1574, 1572, 44461], [1574, 1573, 77729], [1575, 1574, 35580], [1577, 1576, 22769], [1584, 1581, 62285], [1581, 1585, 35114], [1581, 1582, 2396], [1584, 1583, 65831], [1590, 1586, 13196], [1590, 1587, 11475], [1590, 1588, 33524], [1589, 1590, 38159], [1602, 1599, 55775], [1599, 1600, 36010], [1599, 1603, 14100], [1602, 1601, 29597], [1606, 1607, 24445], [1610, 1608, 65723], [1610, 1609, 45012], [1615, 1612, 43142], [1613, 1614, 28044], [1613, 1615, 18784], [1616, 1618, 17635], [1618, 1617, 48859], [1623, 1622, 43855], [1622, 1626, 90522], [1624, 1623, 4712], [1623, 1625, 81169], [1628, 1627, 11570], [1632, 1633, 43130], [1644, 1645, 11792], [1647, 1646, 50747], [1651, 1647, 66368], [1647, 1648, 51842], [1650, 1649, 40401], [1650, 1651, 56138], [1653, 1652, 71928], [1652, 1654, 77713], [1657, 1656, 43277], [1661, 1660, 26799], [1660, 1662, 34114], [1670, 1667, 24198], [1667, 1669, 41242], [1670, 1668, 13561], [1677, 1676, 9705], [1681, 1686, 40491], [1686, 1682, 72599], [1686, 1683, 38478], [1683, 1685, 37617], [1686, 1684, 48270], [1689, 1687, 13498], [1688, 1689, 35853], [1693, 1694, 5303], [1693, 1696, 3048], [1694, 1695, 34538], [1697, 1698, 17502], [1702, 1699, 10878], [1700, 1702, 78913], [1702, 1701, 6354], [1706, 1707, 67616], [1710, 1708, 51984], [1708, 1711, 48181], [1710, 1709, 21564], [1715, 1714, 45912], [1716, 1717, 18306], [1719, 1718, 52297], [1718, 1720, 64971], [1726, 1725, 44916], [1728, 1729, 36226], [1733, 1732, 47197], [1739, 1737, 51297], [1739, 1738, 48044], [1745, 1746, 29520], [1749, 1748, 77541], [1751, 1750, 41034], [1755, 1753, 73680], [1755, 1754, 71906], [1760, 1761, 33509], [1764, 1763, 53419], [1774, 1772, 4041], [1773, 1774, 3977], [1777, 1778, 6770], [1780, 1779, 95297], [1781, 1780, 88133], [1786, 1785, 31035], [1786, 1787, 33927], [1789, 1790, 17809], [1795, 1792, 40659], [1794, 1793, 3214], [1794, 1795, 36328], [1798, 1797, 41648], [1801, 1800, 11874], [1803, 1802, 91909], [1812, 1811, 42262], [1812, 1813, 33114], [1814, 1815, 29731], [1819, 1817, 12510], [1819, 1818, 1163], [1824, 1825, 72179], [1827, 1826, 25903], [1827, 1828, 24792], [1829, 1830, 63235], [1837, 1838, 1681], [1843, 1845, 27063], [1845, 1844, 33712], [1847, 1846, 40216], [1849, 1850, 76015], [1853, 1852, 44672], [1858, 1857, 73324], [1863, 1862, 1034], [1875, 1877, 82232], [1877, 1876, 31825], [1888, 1889, 12734], [1888, 1890, 32848], [1894, 1892, 20033], [1892, 1896, 16786], [1892, 1893, 61321], [1894, 1895, 65519], [1898, 1897, 45941], [1900, 1901, 36372], [1910, 1911, 69036], [1920, 1921, 11746], [1923, 1922, 36943], [1925, 1924, 38994], [1927, 1928, 45742], [1930, 1929, 18956], [1930, 1931, 84317], [1935, 1934, 27475], [1940, 1938, 20215], [1941, 1939, 18022], [1939, 1940, 68221], [1949, 1948, 71952], [1970, 1969, 49616], [1977, 1976, 2768], [1980, 1979, 88320], [1998, 1999, 33745], [2005, 2004, 58231], [2013, 2012, 52963], [2019, 2020, 1318], [2031, 2032, 27950], [2055, 2056, 15883], [2063, 2062, 32385], [2066, 2067, 62645], [2067, 2068, 52973], [2072, 2071, 20603], [2073, 2072, 75790], [2078, 2077, 21385]]
well_status= [1, 2, 4, 8, 12, 19, 24, 27, 30, 37, 39, 40, 41, 42, 49, 50, 54, 57, 59, 60, 63, 65, 73, 74, 77, 80, 85, 89, 91, 94, 97, 98, 107, 108, 110, 130, 133, 137, 139, 140, 141, 142, 143, 147, 149, 150, 161, 166, 167, 168, 171, 178, 180, 181, 186, 187, 188, 189, 191, 195, 204, 205, 206, 208, 210, 212, 214, 215, 220, 221, 223, 231, 232, 234, 236, 237, 238, 239, 240, 241, 244, 250, 251, 254, 255, 258, 263, 269, 273, 280, 291, 293, 295, 297, 300, 302, 304, 305, 311, 313, 314, 317, 318, 320, 322, 324, 325, 326, 335, 336, 343, 346, 348, 352, 353, 357, 358, 359, 364, 370, 371, 373, 376, 378, 380, 381, 383, 385, 395, 398, 405, 406, 407, 412, 415, 416, 424, 425, 428, 429, 431, 432, 433, 438, 439, 443, 448, 449, 451, 452, 454, 456, 459, 460, 461, 464, 466, 468, 469, 472, 473, 474, 479, 480, 482, 486, 487, 489, 492, 493, 495, 498, 500, 503, 504, 509, 511, 513, 514, 516, 520, 525, 526, 529, 530, 531, 536, 537, 539, 543, 545, 547, 549, 551, 552, 554, 555, 558, 562, 564, 565, 567, 568, 569, 570, 577, 582, 583, 587, 593, 594, 595, 596, 601, 602, 607, 611, 612, 616, 617, 618, 619, 624, 627, 628, 629, 632, 634, 636, 637, 644, 648, 650, 652, 654, 655, 656, 657, 659, 661, 663, 665, 667, 668, 670, 671, 673, 674, 675, 676, 677, 678, 682, 691, 692, 696, 700, 703, 718, 720, 721, 722, 723, 725, 728, 729, 732, 733, 735, 737, 739, 740, 741, 744, 745, 746, 748, 751, 752, 753, 754, 756, 758, 763, 765, 771, 772, 773, 775, 778, 779, 780, 783, 784, 786, 788, 790, 793, 794, 795, 797, 801, 802, 805, 810, 814, 816, 818, 819, 820, 821, 822, 823, 825, 828, 829, 831, 833, 836, 838, 844, 846, 849, 853, 856, 857, 859, 861, 863, 864, 867, 869, 873, 876, 878, 879, 881, 882, 884, 885, 886, 888, 895, 897, 902, 903, 908, 911, 912, 913, 916, 917, 919, 921, 922, 923, 925, 927, 932, 936, 938, 942, 943, 946, 947, 948, 950, 953, 957, 959, 966, 970, 972, 975, 976, 977, 979, 981, 983, 988, 994, 1002, 1003, 1006, 1008, 1013, 1014, 1020, 1023, 1025, 1026, 1028, 1029, 1030, 1033, 1036, 1038, 1040, 1042, 1043, 1046, 1047, 1048, 1051, 1053, 1059, 1060, 1065, 1066, 1068, 1069, 1073, 1075, 1077, 1078, 1079, 1083, 1084, 1085, 1089, 1090, 1092, 1094, 1095, 1099, 1103, 1107, 1113, 1122, 1123, 1124, 1126, 1130, 1131, 1133, 1134, 1139, 1140, 1144, 1146, 1149, 1151, 1152, 1154, 1156, 1157, 1158, 1162, 1164, 1165, 1168, 1173, 1174, 1179, 1185, 1186, 1189, 1190, 1193, 1202, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1218, 1221, 1222, 1226, 1229, 1230, 1233, 1234, 1237, 1240, 1241, 1242, 1243, 1244, 1248, 1250, 1251, 1254, 1258, 1259, 1260, 1263, 1266, 1267, 1268, 1271, 1276, 1277, 1278, 1280, 1285, 1287, 1290, 1292, 1293, 1294, 1295, 1296, 1299, 1302, 1303, 1305, 1307, 1309, 1311, 1312, 1313, 1316, 1319, 1320, 1322, 1323, 1325, 1326, 1327, 1328, 1330, 1333, 1336, 1338, 1343, 1345, 1346, 1347, 1348, 1352, 1354, 1355, 1356, 1358, 1359, 1361, 1362, 1363, 1364, 1366, 1367, 1373, 1374, 1376, 1377, 1378, 1381, 1382, 1383, 1384, 1386, 1387, 1388, 1390, 1391, 1394, 1397, 1398, 1402, 1403, 1404, 1409, 1410, 1411, 1412, 1413, 1416, 1418, 1420, 1424, 1425, 1426, 1427, 1429, 1431, 1435, 1440, 1441, 1442, 1443, 1444, 1445, 1447, 1448, 1451, 1452, 1456, 1458, 1459, 1461, 1463, 1465, 1468, 1469, 1470, 1471, 1473, 1474, 1475, 1477, 1479, 1481, 1483, 1484, 1487, 1490, 1491, 1492, 1494, 1495, 1496, 1499, 1500, 1501, 1502, 1503, 1505, 1506, 1509, 1510, 1511, 1514, 1515, 1516, 1517, 1518, 1520, 1521, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1533, 1536, 1542, 1544, 1545, 1546, 1550, 1551, 1554, 1555, 1557, 1558, 1559, 1561, 1563, 1564, 1565, 1566, 1569, 1570, 1572, 1575, 1577, 1578, 1579, 1580, 1582, 1584, 1588, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1601, 1604, 1605, 1607, 1609, 1611, 1614, 1615, 1616, 1617, 1619, 1620, 1621, 1622, 1624, 1625, 1628, 1629, 1630, 1631, 1632, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1647, 1650, 1653, 1655, 1657, 1658, 1659, 1660, 1663, 1664, 1665, 1666, 1668, 1671, 1672, 1673, 1674, 1675, 1677, 1678, 1679, 1680, 1682, 1684, 1685, 1687, 1688, 1690, 1691, 1692, 1693, 1695, 1697, 1700, 1701, 1703, 1704, 1705, 1706, 1709, 1712, 1713, 1714, 1716, 1719, 1720, 1721, 1722, 1723, 1724, 1726, 1727, 1728, 1730, 1731, 1733, 1734, 1735, 1736, 1737, 1740, 1741, 1742, 1743, 1744, 1745, 1747, 1749, 1751, 1752, 1755, 1756, 1757, 1758, 1759, 1760, 1762, 1763, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1774, 1775, 1776, 1777, 1779, 1781, 1782, 1783, 1784, 1785, 1788, 1789, 1791, 1794, 1796, 1798, 1799, 1801, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1813, 1814, 1816, 1818, 1820, 1821, 1822, 1823, 1824, 1826, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1838, 1839, 1840, 1841, 1842, 1845, 1847, 1848, 1849, 1851, 1853, 1854, 1855, 1856, 1858, 1859, 1860, 1861, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1889, 1891, 1895, 1896, 1897, 1899, 1900, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1922, 1925, 1926, 1928, 1929, 1932, 1933, 1935, 1936, 1937, 1938, 1939, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1970, 1971, 1972, 1973, 1974, 1975, 1977, 1978, 1979, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 2000, 2001, 2002, 2003, 2004, 2006, 2007, 2008, 2009, 2010, 2011, 2013, 2014, 2015, 2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2056, 2057, 2058, 2059, 2060, 2061, 2063, 2064, 2065, 2066, 2069, 2070, 2071, 2073, 2074, 2075, 2076, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160]
pipe_status= [(2, 3), (5, 6), (5, 17), (6, 7), (6, 22), (6, 55), (6, 69), (7, 46), (8, 31), (9, 73), (10, 71), (11, 43), (11, 72), (12, 83), (13, 38), (13, 50), (14, 16), (14, 20), (14, 26), (15, 55), (16, 86), (16, 105), (18, 43), (19, 100), (20, 66), (20, 90), (21, 39), (22, 30), (22, 43), (22, 79), (22, 102), (23, 47), (23, 71), (24, 25), (25, 45), (25, 84), (26, 35), (27, 82), (28, 71), (29, 53), (32, 103), (33, 48), (33, 55), (34, 93), (35, 59), (35, 75), (36, 37), (36, 62), (38, 51), (39, 88), (42, 53), (43, 56), (43, 71), (44, 98), (45, 76), (45, 93), (47, 106), (52, 61), (54, 64), (55, 67), (55, 78), (55, 87), (57, 92), (58, 68), (61, 65), (63, 68), (70, 99), (71, 103), (76, 95), (76, 99), (77, 101), (81, 82), (91, 96), (96, 104), (108, 109), (110, 120), (110, 122), (110, 225), (111, 132), (111, 236), (112, 170), (112, 234), (113, 114), (113, 158), (115, 123), (115, 151), (115, 174), (115, 241), (116, 217), (117, 185), (118, 133), (119, 134), (119, 140), (119, 198), (119, 224), (121, 164), (122, 125), (123, 183), (124, 143), (125, 203), (126, 206), (127, 147), (127, 163), (127, 202), (127, 216), (127, 233), (128, 229), (128, 230), (129, 184), (131, 133), (135, 247), (136, 235), (138, 245), (141, 177), (142, 184), (143, 157), (144, 164), (145, 169), (146, 175), (146, 208), (148, 149), (149, 218), (150, 197), (150, 242), (151, 179), (152, 248), (153, 184), (153, 226), (154, 168), (155, 201), (155, 229), (156, 184), (156, 196), (157, 199), (158, 204), (159, 207), (160, 232), (161, 194), (161, 245), (162, 193), (164, 181), (165, 169), (169, 177), (171, 190), (171, 207), (171, 248), (172, 212), (173, 176), (173, 205), (174, 235), (178, 182), (181, 249), (182, 200), (182, 228), (185, 206), (187, 213), (188, 229), (192, 238), (192, 243), (193, 210), (197, 217), (209, 232), (211, 237), (216, 246), (219, 234), (221, 227), (222, 246), (232, 247), (251, 265), (252, 272), (253, 257), (254, 272), (255, 256), (255, 261), (255, 268), (257, 262), (257, 264), (257, 269), (257, 270), (259, 276), (260, 274), (263, 275), (265, 266), (265, 267), (265, 274), (271, 274), (274, 276), (277, 284), (277, 286), (278, 285), (278, 286), (279, 283), (280, 285), (281, 282), (281, 286), (283, 286), (287, 290), (287, 298), (288, 292), (288, 294), (288, 297), (289, 298), (290, 293), (291, 296), (298, 299), (300, 301), (303, 304), (304, 307), (306, 307), (308, 311), (309, 311), (310, 317), (311, 312), (311, 315), (311, 316), (319, 322), (321, 335), (321, 337), (323, 348), (324, 333), (325, 355), (326, 332), (327, 352), (328, 338), (329, 339), (330, 345), (330, 358), (331, 344), (334, 339), (336, 347), (338, 355), (339, 340), (339, 354), (339, 359), (341, 357), (342, 357), (344, 352), (346, 351), (348, 356), (349, 355), (350, 358), (360, 368), (360, 371), (361, 371), (362, 370), (363, 368), (365, 369), (365, 370), (366, 368), (367, 371), (372, 374), (373, 374), (375, 377), (376, 386), (377, 384), (377, 390), (378, 388), (378, 389), (379, 381), (379, 387), (382, 391), (383, 391), (390, 391), (392, 395), (392, 404), (393, 402), (394, 399), (395, 396), (395, 402), (396, 400), (397, 399), (398, 401), (399, 401), (400, 403), (408, 413), (409, 411), (410, 413), (411, 415), (413, 414), (414, 415), (416, 423), (417, 422), (418, 427), (419, 424), (420, 429), (421, 425), (422, 426), (424, 427), (426, 429), (430, 432), (434, 449), (435, 443), (436, 445), (437, 446), (440, 448), (441, 442), (441, 447), (442, 446), (444, 446), (445, 448), (446, 449), (446, 450), (453, 457), (455, 458), (456, 458), (457, 459), (461, 462), (462, 463), (465, 467), (466, 467), (469, 470), (471, 478), (472, 475), (473, 481), (475, 484), (476, 479), (477, 485), (478, 485), (479, 485), (480, 483), (488, 490), (490, 491), (491, 492), (493, 494), (496, 507), (497, 511), (498, 507), (498, 512), (499, 509), (500, 510), (501, 507), (502, 504), (502, 505), (503, 508), (506, 512), (515, 517), (515, 518), (516, 518), (518, 519), (521, 525), (522, 525), (523, 527), (524, 525), (524, 527), (528, 530), (528, 533), (532, 533), (533, 534), (534, 535), (537, 540), (538, 539), (541, 543), (542, 543), (543, 544), (546, 547), (548, 549), (550, 551), (553, 554), (556, 558), (556, 559), (557, 558), (560, 561), (560, 562), (563, 565), (566, 567), (571, 577), (571, 588), (572, 573), (572, 587), (574, 576), (574, 590), (575, 579), (575, 581), (575, 589), (575, 591), (576, 578), (576, 580), (576, 585), (579, 583), (582, 586), (584, 592), (585, 588), (587, 592), (595, 599), (595, 603), (597, 602), (598, 600), (598, 601), (604, 613), (605, 606), (606, 617), (607, 608), (609, 616), (610, 611), (612, 614), (613, 616), (615, 616), (619, 620), (621, 622), (621, 623), (621, 624), (624, 626), (625, 626), (630, 632), (631, 632), (633, 634), (633, 635), (637, 638), (637, 641), (639, 641), (640, 641), (642, 649), (642, 650), (643, 649), (645, 646), (646, 647), (647, 648), (649, 651), (653, 654), (657, 658), (660, 661), (660, 662), (663, 664), (665, 666), (668, 669), (671, 672), (673, 679), (674, 685), (674, 701), (675, 695), (675, 704), (676, 709), (678, 681), (679, 687), (679, 694), (679, 697), (679, 698), (680, 707), (681, 683), (681, 688), (681, 699), (681, 713), (682, 702), (682, 708), (683, 711), (684, 705), (686, 723), (687, 719), (689, 704), (690, 714), (690, 723), (693, 710), (693, 718), (694, 705), (696, 715), (706, 707), (707, 717), (707, 718), (712, 721), (715, 716), (724, 726), (725, 726), (727, 728), (729, 730), (731, 732), (734, 735), (736, 737), (737, 738), (741, 742), (741, 743), (747, 749), (748, 749), (750, 771), (753, 759), (755, 766), (757, 765), (757, 769), (757, 770), (759, 767), (759, 768), (760, 771), (761, 762), (762, 765), (762, 766), (764, 771), (773, 774), (775, 782), (776, 789), (777, 779), (781, 784), (783, 787), (785, 787), (788, 789), (790, 791), (792, 793), (796, 797), (797, 798), (797, 799), (799, 800), (801, 809), (802, 804), (803, 805), (806, 807), (806, 809), (808, 809), (810, 812), (811, 815), (813, 814), (814, 815), (817, 818), (824, 825), (825, 827), (825, 832), (826, 832), (827, 834), (830, 833), (835, 841), (835, 846), (836, 840), (837, 843), (837, 852), (839, 840), (839, 842), (839, 847), (839, 850), (839, 852), (842, 848), (844, 845), (851, 852), (854, 855), (854, 856), (857, 858), (859, 860), (862, 868), (865, 873), (865, 880), (866, 868), (867, 871), (867, 877), (868, 870), (870, 874), (870, 879), (872, 876), (875, 879), (882, 883), (886, 887), (888, 889), (888, 890), (888, 891), (892, 893), (892, 898), (893, 900), (894, 896), (894, 897), (895, 899), (895, 900), (901, 902), (904, 908), (905, 908), (905, 910), (906, 912), (907, 915), (908, 909), (911, 914), (913, 915), (917, 918), (920, 921), (923, 924), (926, 927), (928, 929), (928, 932), (929, 933), (930, 931), (930, 932), (934, 940), (935, 939), (936, 941), (937, 942), (938, 939), (940, 942), (943, 944), (945, 946), (949, 950), (949, 952), (951, 953), (952, 955), (953, 954), (955, 956), (957, 958), (958, 960), (961, 967), (962, 968), (963, 964), (963, 965), (963, 966), (964, 967), (968, 969), (968, 970), (971, 972), (972, 973), (974, 975), (977, 978), (980, 982), (981, 982), (984, 986), (984, 988), (985, 986), (986, 987), (988, 989), (990, 991), (991, 994), (991, 995), (991, 1001), (992, 994), (993, 994), (993, 998), (996, 1001), (997, 1000), (999, 1000), (1000, 1001), (1003, 1004), (1005, 1006), (1007, 1008), (1008, 1009), (1010, 1011), (1010, 1014), (1012, 1013), (1015, 1020), (1015, 1021), (1016, 1017), (1017, 1019), (1018, 1020), (1018, 1022), (1019, 1020), (1023, 1024), (1026, 1027), (1028, 1032), (1031, 1032), (1034, 1035), (1034, 1037), (1035, 1036), (1038, 1039), (1041, 1044), (1042, 1044), (1043, 1045), (1048, 1049), (1049, 1050), (1052, 1058), (1052, 1059), (1052, 1063), (1053, 1061), (1054, 1063), (1055, 1063), (1056, 1059), (1057, 1058), (1058, 1062), (1059, 1064), (1065, 1067), (1069, 1070), (1070, 1071), (1072, 1073), (1073, 1074), (1076, 1077), (1079, 1080), (1080, 1081), (1082, 1083), (1085, 1087), (1086, 1087), (1088, 1089), (1091, 1092), (1093, 1094), (1095, 1096), (1095, 1098), (1097, 1098), (1100, 1102), (1100, 1103), (1101, 1102), (1104, 1105), (1105, 1107), (1106, 1107), (1108, 1115), (1108, 1118), (1108, 1119), (1109, 1113), (1110, 1118), (1111, 1123), (1112, 1122), (1114, 1116), (1116, 1123), (1117, 1118), (1118, 1121), (1119, 1120), (1121, 1123), (1125, 1130), (1126, 1127), (1126, 1129), (1128, 1129), (1132, 1133), (1134, 1135), (1134, 1136), (1137, 1138), (1138, 1140), (1141, 1142), (1141, 1150), (1142, 1147), (1143, 1144), (1145, 1147), (1145, 1148), (1147, 1149), (1152, 1153), (1155, 1156), (1159, 1161), (1159, 1162), (1160, 1163), (1161, 1163), (1166, 1167), (1166, 1168), (1166, 1169), (1170, 1181), (1171, 1174), (1171, 1176), (1172, 1174), (1173, 1177), (1174, 1175), (1177, 1178), (1179, 1182), (1180, 1181), (1181, 1182), (1183, 1184), (1184, 1185), (1186, 1188), (1187, 1196), (1189, 1197), (1190, 1207), (1191, 1192), (1192, 1193), (1194, 1205), (1195, 1196), (1196, 1198), (1196, 1209), (1197, 1204), (1197, 1205), (1199, 1203), (1199, 1206), (1199, 1208), (1200, 1202), (1201, 1208), (1202, 1208), (1215, 1216), (1217, 1219), (1218, 1219), (1220, 1221), (1223, 1227), (1224, 1225), (1224, 1228), (1225, 1227), (1226, 1227), (1231, 1233), (1232, 1233), (1235, 1238), (1236, 1237), (1236, 1238), (1237, 1239), (1244, 1245), (1244, 1247), (1245, 1246), (1248, 1249), (1252, 1256), (1253, 1254), (1254, 1255), (1255, 1256), (1257, 1258), (1261, 1263), (1262, 1264), (1263, 1265), (1264, 1265), (1268, 1269), (1270, 1271), (1270, 1274), (1271, 1273), (1272, 1274), (1275, 1276), (1278, 1279), (1280, 1282), (1281, 1282), (1282, 1283), (1284, 1285), (1286, 1287), (1287, 1288), (1287, 1289), (1291, 1292), (1296, 1298), (1297, 1298), (1300, 1301), (1300, 1302), (1304, 1306), (1305, 1306), (1308, 1309), (1310, 1312), (1313, 1314), (1314, 1315), (1317, 1318), (1318, 1319), (1321, 1324), (1322, 1324), (1327, 1329), (1331, 1333), (1331, 1341), (1331, 1342), (1332, 1342), (1334, 1339), (1335, 1341), (1336, 1339), (1337, 1338), (1340, 1341), (1344, 1345), (1349, 1350), (1349, 1351), (1349, 1352), (1351, 1353), (1357, 1358), (1360, 1361), (1364, 1365), (1367, 1368), (1369, 1373), (1370, 1373), (1371, 1372), (1371, 1373), (1375, 1376), (1378, 1379), (1380, 1381), (1385, 1386), (1389, 1390), (1391, 1392), (1392, 1393), (1394, 1395), (1394, 1396), (1398, 1399), (1398, 1400), (1401, 1402), (1404, 1405), (1406, 1408), (1407, 1409), (1408, 1409), (1413, 1414), (1415, 1416), (1417, 1418), (1417, 1419), (1421, 1423), (1422, 1423), (1422, 1424), (1428, 1429), (1430, 1431), (1432, 1434), (1432, 1435), (1433, 1434), (1436, 1440), (1437, 1440), (1438, 1439), (1438, 1440), (1445, 1446), (1448, 1449), (1449, 1450), (1453, 1456), (1454, 1456), (1455, 1456), (1455, 1457), (1459, 1460), (1461, 1462), (1464, 1465), (1466, 1468), (1467, 1468), (1472, 1473), (1476, 1477), (1478, 1479), (1480, 1481), (1482, 1483), (1485, 1487), (1485, 1489), (1486, 1487), (1486, 1488), (1492, 1493), (1496, 1497), (1498, 1499), (1504, 1505), (1507, 1508), (1507, 1509), (1511, 1512), (1512, 1513), (1518, 1519), (1521, 1522), (1521, 1523), (1531, 1532), (1534, 1537), (1535, 1537), (1535, 1540), (1536, 1537), (1536, 1538), (1536, 1539), (1540, 1541), (1543, 1544), (1545, 1548), (1546, 1549), (1547, 1549), (1551, 1552), (1553, 1554), (1556, 1557), (1559, 1560), (1562, 1563), (1567, 1568), (1567, 1570), (1571, 1575), (1573, 1574), (1574, 1575), (1576, 1577), (1581, 1582), (1581, 1585), (1583, 1584), (1586, 1590), (1587, 1590), (1588, 1590), (1589, 1590), (1599, 1600), (1599, 1603), (1601, 1602), (1606, 1607), (1608, 1610), (1609, 1610), (1612, 1615), (1613, 1615), (1616, 1618), (1622, 1626), (1623, 1624), (1627, 1628), (1632, 1633), (1644, 1645), (1646, 1647), (1647, 1648), (1649, 1650), (1650, 1651), (1652, 1653), (1652, 1654), (1656, 1657), (1660, 1661), (1660, 1662), (1667, 1669), (1667, 1670), (1668, 1670), (1676, 1677), (1681, 1686), (1683, 1685), (1683, 1686), (1687, 1689), (1693, 1694), (1693, 1696), (1697, 1698), (1699, 1702), (1701, 1702), (1706, 1707), (1708, 1710), (1708, 1711), (1709, 1710), (1714, 1715), (1716, 1717), (1718, 1719), (1725, 1726), (1728, 1729), (1732, 1733), (1737, 1739), (1738, 1739), (1745, 1746), (1748, 1749), (1750, 1751), (1753, 1755), (1754, 1755), (1760, 1761), (1763, 1764), (1772, 1774), (1773, 1774), (1777, 1778), (1780, 1781), (1785, 1786), (1786, 1787), (1789, 1790), (1792, 1795), (1793, 1794), (1794, 1795), (1797, 1798), (1800, 1801), (1802, 1803), (1812, 1813), (1814, 1815), (1817, 1819), (1818, 1819), (1824, 1825), (1826, 1827), (1827, 1828), (1829, 1830), (1837, 1838), (1843, 1845), (1844, 1845), (1846, 1847), (1849, 1850), (1852, 1853), (1857, 1858), (1862, 1863), (1876, 1877), (1888, 1889), (1888, 1890), (1892, 1893), (1892, 1894), (1892, 1896), (1897, 1898), (1900, 1901), (1910, 1911), (1920, 1921), (1922, 1923), (1924, 1925), (1927, 1928), (1929, 1930), (1930, 1931), (1934, 1935), (1938, 1940), (1939, 1941), (1948, 1949), (1969, 1970), (1976, 1977), (1979, 1980), (1998, 1999), (2004, 2005), (2012, 2013), (2019, 2020), (2031, 2032), (2055, 2056), (2062, 2063), (2066, 2067), (2067, 2068), (2071, 2072), (2077, 2078)]
wells_mapping= {1: 603, 2: 1261, 3: 622, 4: 2130, 5: 10, 6: 11, 7: 20, 8: 2069, 9: 1563, 10: 1052, 11: 546, 12: 1059, 13: 40, 14: 41, 15: 2092, 16: 48, 17: 53, 18: 568, 19: 1080, 20: 57, 21: 1087, 22: 65, 23: 583, 24: 85, 25: 103, 26: 104, 27: 1641, 28: 1130, 29: 107, 30: 108, 31: 1142, 32: 1659, 33: 636, 34: 1151, 35: 129, 36: 130, 37: 133, 38: 140, 39: 142, 40: 1682, 41: 1178, 42: 1181, 43: 167, 44: 1201, 45: 189, 46: 702, 47: 1221, 48: 1225, 49: 1749, 50: 214, 51: 215, 52: 1238, 53: 245, 54: 1275, 55: 252, 56: 1788, 57: 254, 58: 1277, 59: 257, 60: 1797, 61: 270, 62: 1298, 63: 284, 64: 1312, 65: 1319, 66: 1320, 67: 297, 68: 301, 69: 820, 70: 1845, 71: 314, 72: 1340, 73: 1856, 74: 840, 75: 1866, 76: 336, 77: 337, 78: 1875, 79: 347, 80: 864, 81: 354, 82: 870, 83: 369, 84: 370, 85: 1907, 86: 381, 87: 1411, 88: 1928, 89: 908, 90: 403, 91: 1436, 92: 413, 93: 414, 94: 1956, 95: 427, 96: 955, 97: 1981, 98: 460, 99: 1485, 100: 2003, 101: 2004, 102: 1493, 103: 986, 104: 480, 105: 1510, 106: 2037, 107: 1980, 108: 1251, 109: 2062, 110: 1, 111: 514, 112: 5, 113: 8, 114: 522, 115: 12, 116: 526, 117: 1041, 118: 21, 119: 23, 120: 1047, 121: 2072, 122: 33, 123: 36, 124: 1061, 125: 38, 126: 1579, 127: 44, 128: 1073, 129: 50, 130: 1076, 131: 1077, 132: 1598, 133: 69, 134: 73, 135: 2122, 136: 592, 137: 1105, 138: 596, 139: 1621, 140: 86, 141: 1111, 142: 619, 143: 620, 144: 113, 145: 625, 146: 631, 147: 1655, 148: 121, 149: 124, 150: 638, 151: 127, 152: 1150, 153: 1155, 154: 1158, 155: 146, 156: 659, 157: 148, 158: 149, 159: 1683, 160: 670, 161: 1694, 162: 1697, 163: 678, 164: 168, 165: 171, 166: 1198, 167: 1204, 168: 181, 169: 182, 170: 696, 171: 192, 172: 708, 173: 709, 174: 198, 175: 201, 176: 715, 177: 205, 178: 724, 179: 234, 180: 747, 181: 749, 182: 242, 183: 762, 184: 261, 185: 265, 186: 1297, 187: 1301, 188: 1819, 189: 797, 190: 293, 191: 1321, 192: 809, 193: 814, 194: 1840, 195: 308, 196: 1332, 197: 309, 198: 823, 199: 310, 200: 1855, 201: 326, 202: 327, 203: 1868, 204: 338, 205: 1366, 206: 1381, 207: 357, 208: 873, 209: 1391, 210: 1906, 211: 1911, 212: 1402, 213: 891, 214: 1919, 215: 1922, 216: 899, 217: 389, 218: 1416, 219: 1932, 220: 1420, 221: 915, 222: 1943, 223: 419, 224: 421, 225: 422, 226: 1454, 227: 1456, 228: 1458, 229: 950, 230: 1974, 231: 953, 232: 956, 233: 1982, 234: 450, 235: 452, 236: 458, 237: 974, 238: 975, 239: 1494, 240: 2015, 241: 2016, 242: 1504, 243: 2025, 244: 1516, 245: 495, 246: 1521, 247: 1014, 248: 508, 249: 1535, 250: 1734, 251: 1030, 252: 263, 253: 1552, 254: 2066, 255: 152, 256: 547, 257: 816, 258: 952, 259: 1337, 260: 2113, 261: 195, 262: 2117, 263: 588, 264: 1103, 265: 466, 266: 979, 267: 1491, 268: 595, 269: 1363, 270: 2019, 271: 485, 272: 1767, 273: 1385, 274: 235, 275: 1134, 276: 255, 277: 385, 278: 131, 279: 1796, 280: 1188, 281: 811, 282: 1071, 283: 598, 284: 1720, 285: 1018, 286: 123, 287: 288, 288: 449, 289: 1953, 290: 1283, 291: 550, 292: 1640, 293: 1833, 294: 716, 295: 525, 296: 1168, 297: 1718, 298: 27, 299: 2044, 300: 1146, 301: 375, 302: 1559, 303: 1475, 304: 1095, 305: 1673, 306: 1999, 307: 911, 308: 612, 309: 391, 310: 1161, 311: 303, 312: 528, 313: 1591, 314: 2009, 315: 410, 316: 379, 317: 1727, 318: 1671, 319: 781, 320: 1680, 321: 18, 322: 1170, 323: 532, 324: 535, 325: 1180, 326: 1439, 327: 162, 328: 675, 329: 296, 330: 1451, 331: 1452, 332: 684, 333: 1078, 334: 186, 335: 59, 336: 316, 337: 60, 338: 705, 339: 72, 340: 330, 341: 1098, 342: 208, 343: 1235, 344: 343, 345: 1880, 346: 2010, 347: 476, 348: 222, 349: 1889, 350: 738, 351: 997, 352: 102, 353: 1639, 354: 232, 355: 1003, 356: 1017, 357: 122, 358: 635, 359: 890, 360: 97, 361: 2145, 362: 1820, 363: 1350, 364: 839, 365: 648, 366: 1382, 367: 1789, 368: 628, 369: 987, 370: 412, 371: 701, 372: 874, 373: 22, 374: 679, 375: 900, 376: 614, 377: 136, 378: 776, 379: 428, 380: 621, 381: 1518, 382: 591, 383: 1307, 384: 17, 385: 434, 386: 1234, 387: 756, 388: 141, 389: 1966, 390: 151, 391: 475, 392: 321, 393: 1537, 394: 67, 395: 105, 396: 75, 397: 555, 398: 333, 399: 14, 400: 143, 401: 144, 402: 1263, 403: 236, 404: 1045, 405: 408, 406: 1503, 407: 2012, 408: 258, 409: 2054, 410: 1064, 411: 782, 412: 883, 413: 246, 414: 120, 415: 92, 416: 2144, 417: 1571, 418: 420, 419: 931, 420: 2055, 421: 681, 422: 1067, 423: 1899, 424: 269, 425: 1936, 426: 850, 427: 277, 428: 472, 429: 1278, 430: 1040, 431: 1843, 432: 1931, 433: 726, 434: 1696, 435: 1792, 436: 227, 437: 1990, 438: 806, 439: 328, 440: 1836, 441: 1370, 442: 1360, 443: 1009, 444: 178, 445: 243, 446: 437, 447: 1430, 448: 1657, 449: 1274, 450: 1215, 451: 1730, 452: 1174, 453: 519, 454: 685, 455: 238, 456: 1615, 457: 80, 458: 789, 459: 30, 460: 563, 461: 1873, 462: 554, 463: 759, 464: 665, 465: 1851, 466: 1909, 467: 719, 468: 1183, 469: 515, 470: 755, 471: 673, 472: 1473, 473: 963, 474: 132, 475: 1187, 476: 356, 477: 47, 478: 240, 479: 304, 480: 1042, 481: 1555, 482: 570, 483: 1172, 484: 2128, 485: 58, 486: 2045, 487: 1971, 488: 993, 489: 1476, 490: 1853, 491: 2014, 492: 2079, 493: 922, 494: 2150, 495: 1024, 496: 1220, 497: 1957, 498: 390, 499: 262, 500: 237, 501: 81, 502: 274, 503: 862, 504: 371, 505: 693, 506: 2102, 507: 55, 508: 760, 509: 25, 510: 954, 511: 155, 512: 830, 513: 1595, 514: 1145, 515: 1154, 516: 1062, 517: 1830, 518: 844, 519: 1748, 520: 1300, 521: 1761, 522: 739, 523: 1127, 524: 713, 525: 500, 526: 1593, 527: 795, 528: 2089, 529: 1994, 530: 2093, 531: 497, 532: 1525, 533: 1499, 534: 1149, 535: 926, 536: 847, 537: 616, 538: 259, 539: 292, 540: 494, 541: 1120, 542: 2002, 543: 1139, 544: 1744, 545: 1838, 546: 1768, 547: 1927, 548: 1529, 549: 1099, 550: 56, 551: 388, 552: 706, 553: 976, 554: 431, 555: 1143, 556: 1345, 557: 996, 558: 606, 559: 1631, 560: 728, 561: 835, 562: 2084, 563: 1082, 564: 1085, 565: 445, 566: 721, 567: 669, 568: 251, 569: 324, 570: 1288, 571: 1550, 572: 15, 573: 1310, 574: 1189, 575: 298, 576: 430, 577: 1839, 578: 694, 579: 699, 580: 580, 581: 1989, 582: 2121, 583: 845, 584: 1872, 585: 83, 586: 473, 587: 101, 588: 360, 589: 111, 590: 1399, 591: 1019, 592: 125, 593: 2085, 594: 1841, 595: 928, 596: 1249, 597: 488, 598: 1323, 599: 1837, 600: 1426, 601: 985, 602: 634, 603: 1854, 604: 773, 605: 1222, 606: 383, 607: 1896, 608: 2022, 609: 362, 610: 1197, 611: 1741, 612: 1104, 613: 112, 614: 1648, 615: 2136, 616: 185, 617: 571, 618: 735, 619: 1010, 620: 1644, 621: 513, 622: 2032, 623: 626, 624: 275, 625: 2133, 626: 1306, 627: 704, 628: 1634, 629: 916, 630: 116, 631: 538, 632: 287, 633: 424, 634: 865, 635: 893, 636: 1383, 637: 901, 638: 1961, 639: 587, 640: 1068, 641: 404, 642: 64, 643: 800, 644: 1509, 645: 805, 646: 423, 647: 521, 648: 683, 649: 78, 650: 52, 651: 1373, 652: 718, 653: 1065, 654: 573, 655: 221, 656: 1074, 657: 45, 658: 710, 659: 558, 660: 646, 661: 1646, 662: 2119, 663: 732, 664: 493, 665: 597, 666: 967, 667: 1196, 668: 2153, 669: 1619, 670: 2076, 671: 400, 672: 810, 673: 1282, 674: 777, 675: 393, 676: 913, 677: 657, 678: 1939, 679: 24, 680: 792, 681: 26, 682: 154, 683: 28, 684: 1305, 685: 1054, 686: 1057, 687: 37, 688: 166, 689: 677, 690: 551, 691: 1193, 692: 1450, 693: 941, 694: 46, 695: 2096, 696: 1585, 697: 311, 698: 444, 699: 191, 700: 1346, 701: 1348, 702: 199, 703: 712, 704: 76, 705: 332, 706: 1358, 707: 79, 708: 1612, 709: 594, 710: 1362, 711: 218, 712: 349, 713: 610, 714: 2020, 715: 489, 716: 1260, 717: 1390, 718: 880, 719: 2033, 720: 1779, 721: 247, 722: 1656, 723: 380, 724: 642, 725: 1538, 726: 871, 727: 867, 728: 1246, 729: 1523, 730: 1979, 731: 1832, 732: 2100, 733: 224, 734: 793, 735: 722, 736: 881, 737: 1482, 738: 1862, 739: 2023, 740: 1368, 741: 560, 742: 1745, 743: 1364, 744: 1219, 745: 1605, 746: 1394, 747: 210, 748: 1955, 749: 742, 750: 128, 751: 9, 752: 1162, 753: 649, 754: 2057, 755: 1422, 756: 1044, 757: 29, 758: 672, 759: 39, 760: 937, 761: 2095, 762: 179, 763: 194, 764: 1093, 765: 71, 766: 206, 767: 858, 768: 730, 769: 1756, 770: 624, 771: 503, 772: 1947, 773: 1462, 774: 766, 775: 225, 776: 641, 777: 930, 778: 1864, 779: 1160, 780: 1034, 781: 623, 782: 306, 783: 1427, 784: 342, 785: 1431, 786: 1401, 787: 698, 788: 188, 789: 61, 790: 644, 791: 2030, 792: 1324, 793: 1702, 794: 1226, 795: 1224, 796: 1088, 797: 290, 798: 1050, 799: 572, 800: 927, 801: 417, 802: 770, 803: 1443, 804: 2146, 805: 1125, 806: 135, 807: 520, 808: 442, 809: 159, 810: 1857, 811: 1094, 812: 1353, 813: 1386, 814: 174, 815: 817, 816: 1349, 817: 435, 818: 397, 819: 1108, 820: 1561, 821: 1652, 822: 1338, 823: 1479, 824: 1058, 825: 3, 826: 1029, 827: 6, 828: 1000, 829: 1101, 830: 209, 831: 1079, 832: 89, 833: 1469, 834: 31, 835: 1632, 836: 1281, 837: 991, 838: 1347, 839: 165, 840: 645, 841: 1285, 842: 553, 843: 2156, 844: 1645, 845: 1262, 846: 2157, 847: 753, 848: 767, 849: 2006, 850: 1016, 851: 1786, 852: 319, 853: 1970, 854: 1291, 855: 1903, 856: 407, 857: 601, 858: 764, 859: 1506, 860: 382, 861: 2001, 862: 906, 863: 276, 864: 2074, 865: 1184, 866: 827, 867: 1090, 868: 66, 869: 1987, 870: 70, 871: 1355, 872: 2129, 873: 1241, 874: 1882, 875: 1505, 876: 482, 877: 230, 878: 1265, 879: 114, 880: 1407, 881: 837, 882: 1393, 883: 943, 884: 1433, 885: 2104, 886: 176, 887: 1959, 888: 577, 889: 905, 890: 1594, 891: 877, 892: 96, 893: 169, 894: 172, 895: 1137, 896: 978, 897: 533, 898: 1112, 899: 1374, 900: 1055, 901: 1706, 902: 490, 903: 824, 904: 160, 905: 740, 906: 1861, 907: 964, 908: 231, 909: 557, 910: 2097, 911: 1527, 912: 1466, 913: 859, 914: 1117, 915: 798, 916: 2029, 917: 1091, 918: 590, 919: 796, 920: 1686, 921: 799, 922: 1365, 923: 818, 924: 1484, 925: 2149, 926: 1315, 927: 829, 928: 512, 929: 907, 930: 16, 931: 117, 932: 54, 933: 1566, 934: 353, 935: 1156, 936: 613, 937: 359, 938: 1735, 939: 1037, 940: 110, 941: 853, 942: 158, 943: 2043, 944: 173, 945: 2052, 946: 750, 947: 1886, 948: 745, 949: 34, 950: 1762, 951: 1863, 952: 239, 953: 723, 954: 279, 955: 920, 956: 1118, 957: 1248, 958: 1211, 959: 1676, 960: 1470, 961: 2080, 962: 676, 963: 197, 964: 1574, 965: 1606, 966: 137, 967: 1865, 968: 305, 969: 1940, 970: 1240, 971: 464, 972: 457, 973: 2018, 974: 1314, 975: 774, 976: 2154, 977: 804, 978: 2061, 979: 1255, 980: 1728, 981: 744, 982: 1461, 983: 714, 984: 289, 985: 1026, 986: 656, 987: 946, 988: 499, 989: 731, 990: 771, 991: 100, 992: 35, 993: 323, 994: 13, 995: 562, 996: 1653, 997: 536, 998: 1209, 999: 1628, 1000: 285, 1001: 126, 1002: 530, 1003: 1025, 1004: 1027, 1005: 2042, 1006: 1924, 1007: 2116, 1008: 204, 1009: 1004, 1010: 653, 1011: 1526, 1012: 1848, 1013: 1498, 1014: 1948, 1015: 99, 1016: 1194, 1017: 654, 1018: 559, 1019: 115, 1020: 87, 1021: 1212, 1022: 2013, 1023: 2048, 1024: 548, 1025: 959, 1026: 1890, 1027: 1900, 1028: 1633, 1029: 228, 1030: 879, 1031: 1808, 1032: 700, 1033: 1739, 1034: 183, 1035: 372, 1036: 609, 1037: 1487, 1038: 1723, 1039: 1630, 1040: 474, 1041: 1218, 1042: 1800, 1043: 498, 1044: 566, 1045: 95, 1046: 1429, 1047: 2142, 1048: 914, 1049: 1229, 1050: 1453, 1051: 2115, 1052: 2, 1053: 291, 1054: 483, 1055: 2114, 1056: 454, 1057: 1256, 1058: 42, 1059: 300, 1060: 1874, 1061: 1878, 1062: 1558, 1063: 90, 1064: 639, 1065: 1289, 1066: 868, 1067: 982, 1068: 1296, 1069: 1089, 1070: 1179, 1071: 1245, 1072: 1417, 1073: 802, 1074: 998, 1075: 629, 1076: 1917, 1077: 190, 1078: 969, 1079: 273, 1080: 429, 1081: 486, 1082: 1530, 1083: 446, 1084: 368, 1085: 352, 1086: 2123, 1087: 267, 1088: 213, 1089: 1205, 1090: 1444, 1091: 1210, 1092: 851, 1093: 2160, 1094: 345, 1095: 193, 1096: 487, 1097: 1620, 1098: 479, 1099: 825, 1100: 872, 1101: 1425, 1102: 1380, 1103: 1486, 1104: 888, 1105: 772, 1106: 1421, 1107: 903, 1108: 608, 1109: 961, 1110: 1446, 1111: 775, 1112: 1772, 1113: 1934, 1114: 2127, 1115: 2064, 1116: 1457, 1117: 1299, 1118: 438, 1119: 919, 1120: 1239, 1121: 315, 1122: 2141, 1123: 763, 1124: 667, 1125: 1316, 1126: 1254, 1127: 1607, 1128: 945, 1129: 758, 1130: 727, 1131: 1815, 1132: 432, 1133: 962, 1134: 505, 1135: 754, 1136: 2049, 1137: 106, 1138: 691, 1139: 1276, 1140: 895, 1141: 1313, 1142: 453, 1143: 650, 1144: 1515, 1145: 399, 1146: 469, 1147: 374, 1148: 1463, 1149: 1398, 1150: 1790, 1151: 680, 1152: 651, 1153: 2038, 1154: 470, 1155: 1577, 1156: 1666, 1157: 1233, 1158: 1834, 1159: 737, 1160: 2088, 1161: 585, 1162: 1102, 1163: 1086, 1164: 2041, 1165: 2120, 1166: 1418, 1167: 1935, 1168: 463, 1169: 1617, 1170: 1600, 1171: 1056, 1172: 229, 1173: 1610, 1174: 395, 1175: 600, 1176: 2090, 1177: 1294, 1178: 786, 1179: 150, 1180: 1719, 1181: 1400, 1182: 156, 1183: 741, 1184: 733, 1185: 1230, 1186: 1035, 1187: 1036, 1188: 658, 1189: 280, 1190: 153, 1191: 32, 1192: 545, 1193: 1569, 1194: 552, 1195: 2094, 1196: 49, 1197: 51, 1198: 1717, 1199: 200, 1200: 971, 1201: 717, 1202: 852, 1203: 1128, 1204: 491, 1205: 109, 1206: 366, 1207: 1781, 1208: 119, 1209: 250, 1210: 1721, 1211: 1908, 1212: 216, 1213: 831, 1214: 790, 1215: 1070, 1216: 398, 1217: 994, 1218: 2047, 1219: 912, 1220: 539, 1221: 1279, 1222: 1517, 1223: 576, 1224: 1121, 1225: 425, 1226: 1707, 1227: 406, 1228: 1757, 1229: 569, 1230: 611, 1231: 496, 1232: 387, 1233: 340, 1234: 2109, 1235: 2081, 1236: 932, 1237: 838, 1238: 1403, 1239: 1725, 1240: 1884, 1241: 1661, 1242: 1216, 1243: 752, 1244: 177, 1245: 426, 1246: 2140, 1247: 2063, 1248: 4, 1249: 1910, 1250: 313, 1251: 674, 1252: 202, 1253: 2031, 1254: 567, 1255: 220, 1256: 93, 1257: 451, 1258: 1267, 1259: 884, 1260: 1732, 1261: 2099, 1262: 1852, 1263: 253, 1264: 1022, 1265: 543, 1266: 1769, 1267: 477, 1268: 468, 1269: 461, 1270: 936, 1271: 1169, 1272: 1809, 1273: 1752, 1274: 861, 1275: 1072, 1276: 1975, 1277: 2087, 1278: 1126, 1279: 652, 1280: 1167, 1281: 1524, 1282: 1110, 1283: 734, 1284: 1930, 1285: 966, 1286: 736, 1287: 233, 1288: 401, 1289: 1253, 1290: 1793, 1291: 564, 1292: 325, 1293: 278, 1294: 282, 1295: 2071, 1296: 1810, 1297: 1508, 1298: 933, 1299: 1308, 1300: 217, 1301: 1713, 1302: 68, 1303: 1976, 1304: 2036, 1305: 1742, 1306: 860, 1307: 1977, 1308: 761, 1309: 542, 1310: 365, 1311: 157, 1312: 855, 1313: 1596, 1314: 1005, 1315: 1046, 1316: 1335, 1317: 1329, 1318: 965, 1319: 63, 1320: 1826, 1321: 1092, 1322: 1432, 1323: 2065, 1324: 1144, 1325: 1474, 1326: 1015, 1327: 1371, 1328: 1236, 1329: 1333, 1330: 1883, 1331: 163, 1332: 1733, 1333: 582, 1334: 711, 1335: 1352, 1336: 1415, 1337: 1738, 1338: 1963, 1339: 972, 1340: 947, 1341: 918, 1342: 440, 1343: 1689, 1344: 834, 1345: 467, 1346: 2028, 1347: 1227, 1348: 854, 1349: 1031, 1350: 1869, 1351: 942, 1352: 1232, 1353: 2108, 1354: 1776, 1355: 1669, 1356: 1658, 1357: 355, 1358: 1988, 1359: 1904, 1360: 1846, 1361: 695, 1362: 663, 1363: 295, 1364: 443, 1365: 1006, 1366: 869, 1367: 1806, 1368: 2143, 1369: 1445, 1370: 1580, 1371: 1007, 1372: 1266, 1373: 504, 1374: 692, 1375: 1672, 1376: 249, 1377: 1691, 1378: 1787, 1379: 1813, 1380: 1711, 1381: 1438, 1382: 1879, 1383: 1877, 1384: 2152, 1385: 1946, 1386: 1106, 1387: 1152, 1388: 1684, 1389: 1817, 1390: 599, 1391: 1489, 1392: 836, 1393: 1175, 1394: 819, 1395: 1916, 1396: 439, 1397: 751, 1398: 77, 1399: 1309, 1400: 1063, 1401: 1724, 1402: 565, 1403: 1791, 1404: 1304, 1405: 977, 1406: 1202, 1407: 402, 1408: 1147, 1409: 748, 1410: 91, 1411: 885, 1412: 2021, 1413: 1501, 1414: 1663, 1415: 1113, 1416: 637, 1417: 826, 1418: 518, 1419: 1231, 1420: 1842, 1421: 1243, 1422: 1083, 1423: 211, 1424: 1542, 1425: 1777, 1426: 769, 1427: 1028, 1428: 1828, 1429: 1773, 1430: 1584, 1431: 2098, 1432: 1891, 1433: 1107, 1434: 939, 1435: 1895, 1436: 1700, 1437: 2053, 1438: 241, 1439: 219, 1440: 447, 1441: 1322, 1442: 788, 1443: 889, 1444: 1478, 1445: 1611, 1446: 1613, 1447: 1876, 1448: 999, 1449: 948, 1450: 1447, 1451: 2137, 1452: 647, 1453: 897, 1454: 2091, 1455: 1131, 1456: 396, 1457: 1165, 1458: 1412, 1459: 1637, 1460: 902, 1461: 1665, 1462: 1441, 1463: 1920, 1464: 1096, 1465: 1075, 1466: 746, 1467: 507, 1468: 502, 1469: 1223, 1470: 1325, 1471: 1972, 1472: 1502, 1473: 351, 1474: 1330, 1475: 1859, 1476: 1729, 1477: 1477, 1478: 1794, 1479: 981, 1480: 1480, 1481: 866, 1482: 203, 1483: 1471, 1484: 1317, 1485: 418, 1486: 1002, 1487: 43, 1488: 1647, 1489: 1268, 1490: 1662, 1491: 1039, 1492: 778, 1493: 307, 1494: 196, 1495: 2000, 1496: 2011, 1497: 1798, 1498: 1384, 1499: 1435, 1500: 312, 1501: 1318, 1502: 1701, 1503: 441, 1504: 1704, 1505: 1376, 1506: 2077, 1507: 1712, 1508: 2075, 1509: 1387, 1510: 2046, 1511: 523, 1512: 478, 1513: 1703, 1514: 1328, 1515: 1375, 1516: 1270, 1517: 1271, 1518: 1290, 1519: 1958, 1520: 1923, 1521: 264, 1522: 1564, 1523: 1965, 1524: 668, 1525: 1942, 1526: 339, 1527: 1654, 1528: 1674, 1529: 957, 1530: 579, 1531: 1754, 1532: 1780, 1533: 2147, 1534: 682, 1535: 780, 1536: 462, 1537: 175, 1538: 1784, 1539: 1592, 1540: 988, 1541: 1214, 1542: 1771, 1543: 1770, 1544: 1692, 1545: 1799, 1546: 1807, 1547: 1624, 1548: 1049, 1549: 511, 1550: 1286, 1551: 544, 1552: 1914, 1553: 1649, 1554: 1258, 1555: 973, 1556: 909, 1557: 2134, 1558: 549, 1559: 929, 1560: 1173, 1561: 2103, 1562: 147, 1563: 605, 1564: 1302, 1565: 1303, 1566: 1983, 1567: 1513, 1568: 1051, 1569: 1206, 1570: 1679, 1571: 1344, 1572: 1860, 1573: 1929, 1574: 1565, 1575: 415, 1576: 1737, 1577: 630, 1578: 589, 1579: 1341, 1580: 593, 1581: 687, 1582: 1746, 1583: 1783, 1584: 62, 1585: 1247, 1586: 1123, 1587: 842, 1588: 1100, 1589: 145, 1590: 341, 1591: 1053, 1592: 2101, 1593: 1952, 1594: 1775, 1595: 1012, 1596: 161, 1597: 1455, 1598: 1148, 1599: 904, 1600: 970, 1601: 1743, 1602: 537, 1603: 1343, 1604: 1667, 1605: 541, 1606: 2056, 1607: 2138, 1608: 1850, 1609: 1011, 1610: 924, 1611: 1013, 1612: 1753, 1613: 180, 1614: 757, 1615: 812, 1616: 256, 1617: 2024, 1618: 725, 1619: 1468, 1620: 1326, 1621: 384, 1622: 1992, 1623: 1008, 1624: 433, 1625: 1885, 1626: 2110, 1627: 1442, 1628: 94, 1629: 1986, 1630: 516, 1631: 376, 1632: 896, 1633: 1124, 1634: 992, 1635: 910, 1636: 409, 1637: 661, 1638: 1354, 1639: 1892, 1640: 1534, 1641: 640, 1642: 2017, 1643: 207, 1644: 1269, 1645: 1495, 1646: 1356, 1647: 1133, 1648: 1902, 1649: 2034, 1650: 531, 1651: 671, 1652: 1192, 1653: 633, 1654: 1693, 1655: 2151, 1656: 1763, 1657: 1396, 1658: 1339, 1659: 1414, 1660: 1512, 1661: 878, 1662: 2078, 1663: 2112, 1664: 2051, 1665: 1687, 1666: 690, 1667: 697, 1668: 1715, 1669: 1171, 1670: 534, 1671: 1540, 1672: 1182, 1673: 729, 1674: 990, 1675: 506, 1676: 1244, 1677: 405, 1678: 1870, 1679: 1601, 1680: 1985, 1681: 7, 1682: 1583, 1683: 783, 1684: 815, 1685: 1650, 1686: 350, 1687: 923, 1688: 268, 1689: 917, 1690: 98, 1691: 1359, 1692: 527, 1693: 632, 1694: 643, 1695: 828, 1696: 1765, 1697: 329, 1698: 517, 1699: 2040, 1700: 660, 1701: 1405, 1702: 720, 1703: 2035, 1704: 286, 1705: 894, 1706: 302, 1707: 1135, 1708: 1434, 1709: 666, 1710: 510, 1711: 2111, 1712: 1556, 1713: 2068, 1714: 1642, 1715: 1531, 1716: 1208, 1717: 1557, 1718: 655, 1719: 260, 1720: 1207, 1721: 1968, 1722: 833, 1723: 1973, 1724: 857, 1725: 1361, 1726: 139, 1727: 602, 1728: 1032, 1729: 1636, 1730: 1578, 1731: 1533, 1732: 1918, 1733: 1295, 1734: 358, 1735: 1419, 1736: 367, 1737: 1969, 1738: 1705, 1739: 1388, 1740: 1604, 1741: 2070, 1742: 1858, 1743: 1847, 1744: 607, 1745: 248, 1746: 1627, 1747: 2118, 1748: 1608, 1749: 1599, 1750: 394, 1751: 373, 1752: 581, 1753: 1440, 1754: 1378, 1755: 578, 1756: 281, 1757: 1573, 1758: 364, 1759: 1186, 1760: 272, 1761: 1397, 1762: 807, 1763: 1984, 1764: 1292, 1765: 484, 1766: 1023, 1767: 584, 1768: 1708, 1769: 1242, 1770: 938, 1771: 1944, 1772: 1472, 1773: 1153, 1774: 1423, 1775: 1888, 1776: 1273, 1777: 363, 1778: 1404, 1779: 1609, 1780: 886, 1781: 615, 1782: 1887, 1783: 1951, 1784: 1685, 1785: 1785, 1786: 1213, 1787: 1287, 1788: 1119, 1789: 226, 1790: 1586, 1791: 856, 1792: 1616, 1793: 1408, 1794: 524, 1795: 1424, 1796: 1664, 1797: 808, 1798: 465, 1799: 1252, 1800: 1492, 1801: 935, 1802: 2139, 1803: 791, 1804: 940, 1805: 2007, 1806: 1626, 1807: 1379, 1808: 1395, 1809: 1629, 1810: 1751, 1811: 1814, 1812: 1342, 1813: 1367, 1814: 627, 1815: 1638, 1816: 1898, 1817: 2026, 1818: 1228, 1819: 1191, 1820: 841, 1821: 1331, 1822: 1547, 1823: 1481, 1824: 1336, 1825: 1392, 1826: 456, 1827: 266, 1828: 1933, 1829: 1587, 1830: 1949, 1831: 1377, 1832: 1437, 1833: 1038, 1834: 377, 1835: 1001, 1836: 1109, 1837: 686, 1838: 2135, 1839: 1967, 1840: 82, 1841: 320, 1842: 822, 1843: 322, 1844: 803, 1845: 334, 1846: 2148, 1847: 1996, 1848: 1811, 1849: 344, 1850: 1560, 1851: 951, 1852: 1618, 1853: 1116, 1854: 1695, 1855: 1675, 1856: 1766, 1857: 1816, 1858: 1543, 1859: 2126, 1860: 1575, 1861: 2125, 1862: 1801, 1863: 1511, 1864: 2105, 1865: 1141, 1866: 1871, 1867: 779, 1868: 1802, 1869: 934, 1870: 1822, 1871: 1496, 1872: 1740, 1873: 574, 1874: 561, 1875: 1448, 1876: 2083, 1877: 1699, 1878: 2050, 1879: 134, 1880: 1020, 1881: 335, 1882: 212, 1883: 1993, 1884: 1176, 1885: 1500, 1886: 1334, 1887: 1894, 1888: 664, 1889: 843, 1890: 887, 1891: 1195, 1892: 960, 1893: 1643, 1894: 556, 1895: 784, 1896: 1459, 1897: 1272, 1898: 1140, 1899: 1714, 1900: 617, 1901: 1164, 1902: 1812, 1903: 688, 1904: 1520, 1905: 244, 1906: 1625, 1907: 455, 1908: 944, 1909: 604, 1910: 361, 1911: 1567, 1912: 1835, 1913: 1678, 1914: 1677, 1915: 1166, 1916: 743, 1917: 1428, 1918: 2158, 1919: 968, 1920: 1964, 1921: 2060, 1922: 2008, 1923: 1539, 1924: 1449, 1925: 471, 1926: 768, 1927: 898, 1928: 1589, 1929: 1824, 1930: 436, 1931: 1726, 1932: 170, 1933: 1544, 1934: 1553, 1935: 1257, 1936: 2082, 1937: 849, 1938: 1954, 1939: 1507, 1940: 1548, 1941: 1467, 1942: 411, 1943: 1668, 1944: 1795, 1945: 1237, 1946: 331, 1947: 2059, 1948: 459, 1949: 187, 1950: 1081, 1951: 875, 1952: 1921, 1953: 1545, 1954: 995, 1955: 813, 1956: 1829, 1957: 1554, 1958: 989, 1959: 1406, 1960: 1603, 1961: 586, 1962: 1867, 1963: 1562, 1964: 1750, 1965: 983, 1966: 74, 1967: 1893, 1968: 1698, 1969: 1588, 1970: 509, 1971: 1710, 1972: 1709, 1973: 848, 1974: 1755, 1975: 1157, 1976: 689, 1977: 19, 1978: 1177, 1979: 1528, 1980: 1163, 1981: 1264, 1982: 1747, 1983: 876, 1984: 1582, 1985: 1357, 1986: 540, 1987: 1311, 1988: 1541, 1989: 346, 1990: 1581, 1991: 1736, 1992: 1576, 1993: 1327, 1994: 921, 1995: 1759, 1996: 1532, 1997: 88, 1998: 392, 1999: 1203, 2000: 1938, 2001: 703, 2002: 1136, 2003: 1995, 2004: 794, 2005: 787, 2006: 2159, 2007: 1821, 2008: 1546, 2009: 1849, 2010: 1280, 2011: 1114, 2012: 1060, 2013: 765, 2014: 1570, 2015: 1913, 2016: 1962, 2017: 348, 2018: 84, 2019: 707, 2020: 1084, 2021: 1764, 2022: 1905, 2023: 1635, 2024: 2039, 2025: 1185, 2026: 1778, 2027: 1351, 2028: 1066, 2029: 1660, 2030: 1069, 2031: 164, 2032: 1590, 2033: 1937, 2034: 1823, 2035: 1901, 2036: 1805, 2037: 1549, 2038: 271, 2039: 1651, 2040: 1623, 2041: 1758, 2042: 2086, 2043: 2106, 2044: 1731, 2045: 2058, 2046: 1915, 2047: 481, 2048: 299, 2049: 1978, 2050: 1372, 2051: 386, 2052: 1997, 2053: 2027, 2054: 2131, 2055: 1043, 2056: 1716, 2057: 1925, 2058: 1199, 2059: 1950, 2060: 1536, 2061: 958, 2062: 1497, 2063: 378, 2064: 317, 2065: 118, 2066: 925, 2067: 1926, 2068: 1991, 2069: 1568, 2070: 283, 2071: 1488, 2072: 1033, 2073: 662, 2074: 1551, 2075: 1782, 2076: 1200, 2077: 1681, 2078: 1217, 2079: 138, 2080: 1572, 2081: 785, 2082: 1831, 2083: 2124, 2084: 846, 2085: 1614, 2086: 1132, 2087: 1881, 2088: 892, 2089: 1670, 2090: 1514, 2091: 1804, 2092: 2005, 2093: 1122, 2094: 1690, 2095: 1410, 2096: 1389, 2097: 2073, 2098: 1115, 2099: 1159, 2100: 1460, 2101: 1818, 2102: 1413, 2103: 1825, 2104: 294, 2105: 501, 2106: 184, 2107: 1293, 2108: 1597, 2109: 1129, 2110: 416, 2111: 529, 2112: 1688, 2113: 1021, 2114: 1960, 2115: 1250, 2116: 1097, 2117: 1827, 2118: 492, 2119: 1138, 2120: 448, 2121: 1912, 2122: 1998, 2123: 318, 2124: 832, 2125: 1519, 2126: 1622, 2127: 575, 2128: 1760, 2129: 223, 2130: 863, 2131: 2107, 2132: 1722, 2133: 882, 2134: 1945, 2135: 1464, 2136: 1048, 2137: 1490, 2138: 2132, 2139: 1941, 2140: 1774, 2141: 1483, 2142: 2155, 2143: 821, 2144: 801, 2145: 1259, 2146: 1465, 2147: 1369, 2148: 1284, 2149: 949, 2150: 1190, 2151: 980, 2152: 1602, 2153: 1897, 2154: 1522, 2155: 1803, 2156: 618, 2157: 1844, 2158: 1409, 2159: 2067, 2160: 984}
res = 70045451

n,wells_dict_new,pipes_new,well_status_new,pipe_status_new = translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping)

pack1 = [wells_dict_new,pipes_new,well_status_new,pipe_status_new]


In [87]:
# res2
n= 2160
wells_list= [85479, 37468, 95472, 696, 29936, 86857, 85806, 39811, 97363, 18385, 91948, 28948, 67657, 90997, 79311, 77807, 68742, 41721, 84589, 98990, 28596, 69948, 77109, 12601, 73375, 11450, 44940, 87995, 24268, 91655, 78600, 48482, 83518, 86706, 87729, 22640, 64805, 34123, 60603, 55603, 21563, 99566, 81295, 56033, 39266, 25891, 319, 76211, 2136, 63443, 87232, 99360, 64463, 74813, 28512, 26550, 67591, 47740, 68396, 15283, 15648, 23442, 95657, 99006, 53923, 40365, 93449, 64620, 98318, 80410, 88103, 13907, 62043, 21740, 36137, 79315, 86038, 19773, 82593, 43382, 34577, 23476, 27968, 58730, 48230, 60893, 32796, 82817, 6078, 95367, 88804, 84262, 48538, 69862, 69479, 60964, 38756, 25008, 7142, 23986, 84551, 7812, 79766, 21233, 86242, 25261, 74883, 5960, 66064, 77217, 53785, 46293, 83768, 25802, 54151, 80686, 14218, 90413, 97864, 73782, 94318, 67909, 84451, 91892, 12560, 17023, 52220, 2395, 80816, 18049, 40026, 29409, 14129, 19124, 32383, 59429, 76000, 10553, 35002, 85675, 78715, 60336, 91802, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 34386, 98070, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19058, 38363, 67282, 97436, 37468, 28941, 77321, 21735, 39947, 45271, 75551, 22756, 94693, 74923, 63417, 66612, 36128, 85415, 69394, 35650, 86885, 7820, 66430, 70357, 55297, 39768, 20774, 39768, 74687, 95554, 48913, 64492, 99818, 92997, 82084, 91865, 42723, 56647, 7000, 68687, 94408, 78624, 79479, 75537, 36727, 69741, 86222, 1196, 59633, 48391, 95494, 54878, 15471, 8204, 4152, 91150, 4972, 69139, 98714, 2956, 12439, 78482, 99969, 13251, 35657, 52300, 79399, 79367, 35413, 49363, 78134, 52108, 21241, 86442, 27928, 72430, 1026, 23917, 97609, 48527, 63029, 44684, 19246, 76282, 32300, 29926, 28557, 91335, 67951, 28910, 88176, 58848, 17252, 54823, 80422, 65566, 9906, 71867, 38081, 95179, 90742, 20674, 93318, 14837, 54135, 15162, 71747, 93408, 23362, 44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 42417, 90525, 95924, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 34549, 40675, 25622, 50209, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 26189, 48604, 45809, 87066, 25425, 33357, 69022, 54675, 15912, 95455, 4873, 91442, 76576, 64485, 73304, 88553, 80063, 33726, 3072, 61317, 97386, 42548, 33868, 51650, 90687, 37033, 25667, 42689, 7112, 59362, 99874, 54960, 88375, 77716, 93551, 43480, 18998, 53863, 79328, 68593, 92727, 69348, 97825, 42155, 31674, 22849, 836, 13676, 3616, 47699, 2605, 62450, 80699, 86182, 31534, 56117, 71610, 85096, 72666, 88888, 93561, 87582, 64603, 54254, 36553, 58355, 83789, 64394, 24789, 73448, 71806, 82706, 47094, 59286, 47209, 11338, 88836, 85237, 46055, 41791, 59794, 88208, 95459, 90124, 92109, 62558, 73057, 92244, 71604, 45460, 5839, 38383, 26061, 5392, 11007, 44851, 49932, 54706, 9494, 62629, 39984, 85482, 157, 78884, 32294, 83093, 92659, 23372, 7513, 85680, 20496, 13794, 65743, 17491, 97038, 37125, 40665, 79570, 91207, 50416, 33081, 63434, 31348, 93222, 36855, 66871, 75649, 3143, 65308, 255, 85964, 82099, 29459, 20105, 42662, 26649, 50979, 16298, 34799, 9182, 21637, 93238, 55093, 99587, 31275, 89505, 99390, 95378, 69783, 31037, 57279, 50531, 82947, 84495, 70584, 88362, 98041, 1223, 34148, 44486, 88394, 16731, 25918, 85446, 12947, 94607, 55850, 15632, 77489, 65853, 20082, 59629, 27122, 13220, 26493, 74140, 36594, 99796, 59057, 85989, 30030, 69952, 97822, 58705, 64292, 12630, 37588, 91983, 71099, 68901, 54303, 93368, 28982, 85470, 46133, 77675, 8333, 97149, 77182, 47926, 68116, 49549, 46958, 99995, 71614, 25761, 85215, 88004, 64604, 2832, 96503, 56203, 17664, 20375, 11269, 57996, 56680, 44473, 15323, 72637, 60748, 28712, 57031, 39876, 51623, 95477, 50497, 43853, 38785, 80210, 6177, 86179, 7336, 34489, 69931, 90410, 32374, 67632, 60443, 2012, 88462, 61217, 37111, 95934, 78887, 73205, 87022, 93191, 26673, 38466, 83431, 13489, 11016, 49070, 56044, 63901, 46685, 25944, 17283, 74815, 79220, 10870, 21970, 45055, 94119, 15949, 43561, 61746, 98475, 60603, 58850, 33917, 88987, 85502, 33281, 76623, 71898, 35799, 69623, 96836, 81074, 45366, 53207, 87196, 59422, 1889, 45376, 46009, 63230, 89760, 40707, 28697, 52533, 50505, 71628, 78450, 95758, 46210, 79515, 28572, 82245, 21353, 41716, 8130, 35466, 25807, 47344, 12396, 38446, 94174, 3429, 51217, 20779, 30367, 94181, 58545, 9368, 28069, 93651, 61049, 14719, 90251, 75106, 63826, 85789, 98095, 31197, 84459, 61572, 1349, 89736, 18891, 72032, 3693, 68680, 19587, 17832, 68237, 97656, 92747, 60331, 63232, 34998, 63993, 58572, 22758, 58447, 77154, 69176, 86465, 28559, 19348, 63977, 62761, 56077, 64099, 29307, 66597, 56082, 94769, 74838, 40201, 51050, 97608, 47250, 84274, 43415, 35040, 97305, 82722, 69731, 9150, 94333, 63698, 94971, 86961, 70040, 49192, 40988, 1970, 95636, 27176, 15139, 94210, 46819, 86736, 46080, 16007, 3575, 73960, 22078, 23008, 60895, 78411, 17907, 79672, 27170, 65879, 26188, 84762, 91156, 60839, 10191, 6178, 72845, 60727, 8605, 60336, 57466, 41129, 9287, 963, 3609, 75207, 32872, 26172, 87461, 37006, 81778, 68741, 81455, 23044, 23212, 76204, 10076, 24083, 22460, 40441, 92226, 28649, 50444, 11746, 2688, 46001, 60523, 71350, 56177, 98108, 14210, 1959, 54812, 11507, 1367, 70886, 78424, 43309, 27746, 11061, 2282, 30943, 98760, 90108, 81437, 61468, 70688, 98798, 15834, 88334, 69329, 51340, 79955, 1823, 22084, 26081, 74952, 40488, 64155, 4026, 5792, 39137, 90634, 44256, 31891, 91011, 75624, 73225, 39029, 10750, 69204, 68740, 86997, 86609, 44070, 82957, 98072, 2232, 23328, 33413, 73235, 14602, 33345, 47560, 16713, 13952, 93827, 64812, 45015, 50796, 33203, 15812, 14198, 52515, 33316, 91748, 63000, 2005, 1870, 43768, 48519, 65529, 90097, 22508, 46584, 24195, 11159, 19027, 90366, 58144, 83706, 95157, 21275, 83031, 74942, 55778, 90342, 74984, 33656, 56829, 39083, 90437, 39107, 26877, 19450, 97705, 89064, 22083, 73469, 92770, 40404, 20450, 80456, 35530, 713, 20572, 10330, 73558, 86841, 66722, 36683, 10178, 7013, 91590, 71113, 50298, 96619, 94173, 39560, 66511, 45923, 86930, 58217, 44904, 84940, 72973, 75462, 79604, 45311, 43538, 87603, 76317, 30178, 31970, 18988, 66176, 70436, 89029, 72376, 71020, 31952, 49253, 83657, 13912, 66867, 59674, 97595, 43097, 30505, 38229, 28404, 34642, 7780, 65626, 6301, 24482, 10208, 93325, 7789, 72495, 23594, 30240, 76760, 2828, 68327, 87344, 11657, 5434, 75112, 45117, 63690, 84694, 99588, 86685, 85609, 28335, 68462, 58251, 33366, 32410, 91201, 56586, 19817, 82780, 58992, 51253, 86702, 65267, 17781, 63896, 61841, 76799, 48517, 61493, 67614, 30391, 34502, 50531, 20039, 5233, 83107, 61987, 12187, 9403, 96846, 38880, 44828, 91629, 91208, 80081, 90763, 7249, 95777, 89497, 29116, 76600, 94643, 12326, 76625, 75881, 23050, 30468, 45453, 88407, 15429, 96318, 6172, 40396, 73077, 13028, 27079, 38572, 88290, 30139, 44971, 35113, 5412, 90546, 64910, 65944, 39482, 20262, 64509, 25555, 3098, 66786, 55452, 56063, 14780, 44113, 58608, 65068, 77184, 52197, 15677, 32255, 90286, 66651, 5027, 53441, 83447, 93175, 64776, 97540, 56244, 49105, 82832, 45056, 310, 55002, 46286, 34095, 23097, 87359, 45896, 55509, 95938, 35090, 94585, 43732, 42496, 85517, 19145, 88501, 59683, 72065, 85963, 881, 76066, 21939, 3082, 18991, 65276, 40406, 63974, 81875, 71587, 7946, 93010, 55242, 83407, 19311, 37264, 55248, 95570, 7117, 94946, 13553, 3167, 88979, 18789, 37787, 63982, 18403, 89986, 86350, 72264, 67512, 96140, 86711, 14769, 72772, 27712, 90511, 16137, 91765, 75643, 85891, 65270, 92852, 31546, 19770, 60891, 98682, 94580, 2144, 33174, 85810, 74073, 25676, 98742, 88281, 21838, 90872, 33134, 15654, 80000, 51287, 52891, 85724, 19820, 82360, 28288, 19438, 16648, 88970, 47187, 31566, 43084, 64484, 18410, 33654, 56525, 91604, 43907, 27984, 78640, 76532, 95887, 5072, 85484, 48946, 85648, 12016, 33041, 71148, 60257, 89742, 38935, 10098, 43442, 5331, 37027, 45946, 71363, 60278, 85921, 4591, 83329, 82139, 68830, 49310, 98620, 4849, 27990, 26291, 86472, 67273, 19165, 24925, 98533, 53780, 15586, 97921, 39982, 95195, 91913, 30773, 25689, 92560, 34364, 5907, 91717, 85334, 16521, 48583, 87310, 41534, 5520, 52, 50985, 76950, 70698, 63517, 95474, 1254, 417, 56229, 98388, 54351, 50557, 46125, 1351, 14316, 42940, 80808, 78484, 30094, 39911, 98310, 93129, 69679, 87846, 18033, 57023, 75157, 25873, 52285, 20761, 16508, 82357, 17619, 83135, 30355, 63575, 2139, 15328, 1264, 42439, 59937, 31352, 39077, 38336, 57518, 63597, 91582, 44510, 22873, 22608, 42921, 25176, 28940, 12213, 40896, 59484, 35362, 52971, 26460, 69720, 60578, 27355, 61622, 47732, 83802, 52863, 30185, 59835, 42326, 69967, 50431, 39936, 33806, 72774, 47772, 19957, 67641, 829, 36375, 71572, 7937, 48393, 54665, 89798, 64269, 66198, 69388, 51944, 79241, 31487, 47871, 78896, 54084, 96988, 57201, 63440, 88699, 8153, 66917, 35931, 64757, 39030, 60746, 42770, 65802, 42362, 90298, 21113, 57498, 21713, 78166, 57653, 90650, 43478, 79550, 89929, 23450, 5513, 75617, 92713, 57741, 47081, 86587, 35546, 90830, 50264, 86873, 58260, 57529, 9283, 10241, 20441, 72293, 32116, 29075, 39233, 6313, 15966, 82528, 30774, 74823, 49656, 83612, 81363, 77652, 6824, 46003, 61435, 53310, 41472, 34984, 27996, 57049, 43787, 52595, 66998, 86025, 51588, 41219, 37190, 54768, 50642, 14294, 70777, 15375, 49027, 12491, 44543, 8827, 85462, 95682, 99676, 60801, 28614, 9796, 88540, 24080, 3052, 17653, 19588, 65047, 23958, 41596, 91004, 28518, 10772, 10678, 23458, 23856, 48924, 31702, 46817, 86953, 23036, 56981, 3204, 71096, 83051, 2821, 62205, 33504, 62497, 38325, 27644, 79625, 26836, 54491, 91271, 41021, 65814, 53963, 35004, 16637, 62577, 89887, 79569, 1714, 56049, 14048, 58578, 99520, 18190, 36762, 41171, 50478, 54068, 43408, 72011, 41343, 29054, 23794, 67628, 41538, 98886, 18388, 16215, 66258, 43118, 59659, 28219, 39230, 29951, 27588, 78139, 30498, 76250, 34126, 9845, 49337, 71879, 59391, 10942, 39527, 29263, 81157, 65425, 16114, 77295, 95855, 27, 14485, 31506, 46858, 49003, 6457, 82269, 87322, 66892, 97932, 58949, 20160, 61090, 15272, 6978, 5053, 29439, 69518, 36045, 40148, 39237, 33641, 63449, 46175, 29011, 31226, 360, 42386, 68020, 53354, 46083, 68708, 92085, 1772, 89615, 54748, 12950, 45057, 21418, 69829, 5829, 83006, 40098, 17193, 55100, 69322, 13522, 34158, 83842, 92485, 1100, 56346, 72014, 33486, 12134, 17036, 90916, 39243, 39422, 37624, 84478, 77471, 93666, 99245, 27740, 71507, 21546, 80024, 93442, 13793, 38283, 10771, 13699, 57144, 21681, 45909, 76935, 33696, 21317, 4109, 99435, 28916, 4754, 60748, 72609, 33567, 32421, 25443, 49604, 78888, 48925, 50368, 72952, 62157, 5462, 81765, 34294, 22448, 18913, 12491, 11676, 60911, 91062, 55466, 39345, 92151, 33082, 51497, 57584, 55188, 18163, 92072, 77315, 70993, 1189, 94009, 90790, 86629, 113, 36782, 3944, 35594, 9876, 99613, 71267, 21279, 74437, 98398, 75234, 15731, 86735, 34635, 73407, 79657, 5094, 88985, 30975, 47868, 54855, 30488, 69500, 95069, 9078, 77620, 68918, 30964, 69271, 36201, 55942, 4862, 56648, 59367, 71983, 50639, 4715, 34680, 26831, 96659, 35272, 55237, 77985, 36985, 62009, 38312, 53360, 15023, 8593, 56463, 6581, 29493, 54587, 16194, 29384, 8914, 27244, 37470, 85128, 52100, 53676, 16670, 15078, 77721, 19838, 64335, 58027, 35954, 59327, 39989, 9175, 67553, 17462, 44021, 8201, 54357, 15971, 17444, 50033, 3483, 37155, 99199, 79270, 16882, 33614, 97128, 41526, 14913, 93374, 983, 71622, 36091, 18922, 20366, 78332, 38615, 49362, 36348, 25909, 8172, 54252, 64094, 89617, 11956, 42668, 77284, 86908, 95870, 90231, 87511, 56371, 3147, 37526, 82883, 67247, 91126, 42477, 10069, 7211, 42991, 59286, 89600, 38884, 77600, 15058, 81363, 57043, 16651, 90707, 67934, 55659, 45772, 76059, 1438, 92402, 96177, 20517, 36183, 19580, 75864, 24714, 97918, 31512, 84294, 78699, 68425, 17949, 80971, 23827, 87078, 57957, 92183, 44507, 90584, 55763, 26197, 10448, 40444, 51985, 20875, 27555, 24758, 82305, 40055, 37263, 46358, 53228, 39064, 65294, 10065, 51511, 65389, 43596, 36455, 91932, 51134, 13288, 88033, 3114, 3559, 60579, 12341, 1298, 23244, 94329, 61801, 74728, 23351, 61836, 64404, 17018, 77072, 15057, 7827, 73587, 62098, 67320, 34651, 83719, 92804, 73011, 39266, 10863, 10073, 3751, 46024, 98349, 85297, 47940, 20826, 60356, 29658, 6020, 45426, 68623, 23603, 25645, 21991, 28320, 72501, 71223, 238, 95770, 19880, 80041, 21503, 28619, 46167, 26146, 14914, 90506, 5336, 25660, 456, 15094, 20553, 84354, 95723, 34044, 31268, 44185, 19375, 73334, 43929, 50115, 98671, 15310, 5769, 38379, 21660, 22727, 12972, 55910, 10130, 58436, 94709, 52665, 83261, 76171, 14179, 62898, 56845, 24666, 33670, 17982, 79846, 46503, 13670, 5405, 23447, 58199, 29944, 94167, 5692, 86811, 59113, 61399, 42100, 25940, 30731, 77525, 71445, 47859, 51953, 64918, 85030, 46410, 23621, 60170, 74743, 46298, 67724, 65073, 51467, 64418, 4105, 86639, 23305, 422, 52145, 4740, 86421, 44460, 6301, 98525, 14829, 22528, 6261, 17985, 10936, 4853, 50513, 32140, 89836, 44026, 31534, 88693, 76625, 7955, 30524, 30650, 2131, 2275, 16101, 12957, 5754, 79427, 23028, 81583, 42257, 8335, 47083, 37793, 99925, 96875, 46785, 15809, 43457, 94121, 13434, 10443, 94845, 38997, 35132, 11240, 30564, 19536, 38991, 52974, 14522, 48083, 62737, 28593, 89716, 9182, 46563, 29161, 13027, 69692, 69944, 32931, 43558, 4114, 90004, 23465, 65550, 81104, 13443, 10098, 9585, 6649, 38696, 56771, 46395, 85538, 97649, 57608, 37285, 92912, 82391, 63488, 16455, 99673, 66000, 64378, 53940, 85519, 70554, 20900, 78399, 4336, 72805, 87910, 44690, 84760, 36626, 33389, 64596, 39683, 37458, 42894, 84312, 1065, 97436, 11855, 13513, 32394, 5555, 47343, 4975, 15301, 26790, 81512, 4664, 37497, 51346, 33966, 17456, 35980, 49339, 20502, 25113, 9968, 41052, 87487, 83613, 11243, 12843, 76721, 807, 660, 32678, 29870, 25831, 17916, 54264, 82114, 60071, 4706, 92620, 26047, 21077, 45716, 37256, 12584, 95671, 75478, 32812, 2265, 3256, 56416, 59656, 56209, 27886, 40043, 57794, 54184, 47291, 11804, 45124, 9924, 52699, 47163, 57258, 3132, 30640, 7801, 54741, 6184, 4698, 41069, 48475, 72736, 5888, 57978, 58874, 38517, 12625, 35411, 39934, 94110, 17898, 83421, 41100, 70902, 85308, 5096, 55307, 54410, 19579, 12905, 79138, 58146, 8957, 53188, 46537, 15681, 75038, 29876, 22824, 56074, 11590, 19788, 86671, 90490, 49679, 4591, 88527, 15323, 7633, 61042, 51596, 2158, 3970, 50586, 15910, 16882, 96757, 40131, 89993, 85079, 42282, 95774, 20839, 22604, 14604, 83142, 6798, 2268, 97715, 87300, 5116, 57455, 26979, 18172, 80770, 50680, 11811, 92564, 72058]
pipe_list= [[9, 1, 40027], [1, 4, 23211], [11, 2, 76779], [15, 2, 2781], [17, 2, 63691], [2, 6, 52997], [2, 5, 35396], [14, 3, 50811], [13, 4, 31554], [8, 5, 3122], [7, 16, 64676], [9, 17, 67081], [9, 12, 53644], [9, 10, 49155], [9, 16, 87182], [14, 15, 27651], [23, 18, 50747], [18, 22, 66368], [18, 19, 51842], [20, 21, 40401], [21, 22, 56138], [24, 25, 67616], [26, 27, 8645], [28, 29, 14142], [30, 44, 40700], [45, 31, 1243], [31, 32, 45477], [31, 41, 36454], [46, 32, 4852], [32, 33, 47441], [33, 44, 34920], [33, 34, 9461], [33, 39, 34331], [33, 42, 23422], [33, 37, 69520], [47, 34, 82536], [34, 40, 20645], [35, 43, 15599], [45, 36, 35510], [37, 38, 56954], [43, 44, 61022], [48, 53, 35477], [49, 52, 13974], [49, 50, 12588], [51, 52, 61310], [52, 53, 75663], [54, 139, 38015], [54, 126, 20585], [55, 139, 6831], [55, 88, 76198], [56, 87, 22576], [161, 57, 14415], [155, 58, 7480], [178, 58, 25735], [58, 75, 39351], [146, 59, 67775], [59, 63, 51479], [59, 61, 58141], [60, 71, 54150], [149, 62, 42543], [188, 62, 33619], [62, 69, 91277], [62, 76, 37590], [179, 63, 84761], [63, 71, 31649], [63, 130, 80667], [146, 64, 81886], [159, 65, 57357], [65, 84, 49095], [65, 101, 2051], [65, 142, 6062], [66, 111, 42108], [66, 106, 35456], [183, 67, 55166], [67, 70, 71708], [67, 99, 39232], [67, 114, 78688], [149, 68, 24514], [68, 129, 12819], [181, 69, 42477], [69, 79, 72908], [164, 71, 56823], [175, 71, 40310], [71, 109, 69432], [72, 76, 25143], [185, 73, 23810], [152, 74, 62543], [74, 85, 30636], [75, 143, 37723], [76, 89, 52539], [76, 122, 30270], [76, 94, 68521], [157, 77, 43418], [163, 78, 43494], [176, 78, 3365], [187, 78, 4237], [190, 78, 14071], [151, 79, 32648], [79, 83, 80191], [79, 131, 56119], [79, 100, 36121], [156, 80, 45087], [81, 107, 66486], [82, 123, 6640], [85, 132, 2145], [85, 137, 14971], [163, 86, 69480], [161, 87, 48726], [87, 104, 5637], [87, 134, 26628], [89, 139, 31314], [159, 90, 53367], [184, 90, 27877], [90, 111, 21637], [191, 91, 47526], [153, 92, 56978], [190, 93, 15369], [189, 95, 3452], [161, 96, 9603], [159, 97, 27159], [192, 98, 16711], [160, 100, 16601], [193, 101, 76476], [181, 102, 22508], [103, 126, 15072], [105, 131, 17770], [107, 129, 67685], [108, 123, 58540], [161, 110, 73716], [110, 140, 73839], [150, 111, 68982], [183, 112, 11194], [113, 128, 42940], [157, 115, 83150], [115, 121, 10071], [116, 140, 7465], [178, 117, 46331], [153, 118, 56060], [144, 119, 88518], [187, 120, 53788], [162, 123, 18764], [189, 123, 36232], [162, 124, 13914], [172, 125, 36711], [126, 141, 34045], [161, 127, 46856], [146, 128, 5314], [170, 129, 13227], [145, 130, 7405], [190, 133, 46273], [149, 135, 14423], [184, 136, 40033], [158, 138, 89686], [172, 139, 1424], [169, 141, 34638], [186, 142, 5227], [144, 146, 56293], [144, 147, 40308], [144, 156, 13648], [144, 182, 67728], [144, 154, 3867], [145, 166, 37249], [147, 149, 66572], [147, 153, 25713], [147, 192, 4787], [147, 148, 54451], [147, 171, 56329], [149, 152, 64972], [149, 157, 40618], [149, 185, 161], [152, 167, 17143], [153, 168, 1235], [153, 174, 14545], [153, 178, 60159], [156, 159, 7549], [156, 173, 57663], [157, 161, 82641], [157, 163, 87950], [157, 189, 53504], [158, 177, 71888], [161, 180, 34092], [165, 167, 36052], [177, 191, 32559], [178, 191, 59196], [182, 183, 25364], [195, 221, 67496], [196, 213, 84585], [196, 199, 22595], [197, 208, 96333], [198, 212, 35133], [198, 213, 49749], [198, 216, 75213], [200, 225, 31141], [201, 231, 64752], [201, 205, 92359], [201, 209, 77972], [201, 210, 68233], [202, 233, 23285], [203, 209, 41386], [204, 229, 64775], [204, 228, 68249], [205, 215, 3603], [206, 216, 22973], [207, 236, 31484], [207, 222, 63006], [208, 221, 66948], [211, 216, 45219], [211, 220, 77236], [214, 216, 85382], [214, 224, 49386], [215, 233, 39415], [216, 229, 85281], [216, 230, 92648], [216, 225, 72509], [216, 231, 54935], [216, 217, 19739], [216, 236, 86174], [216, 235, 29019], [218, 230, 16167], [219, 230, 19735], [221, 229, 64507], [223, 228, 11298], [225, 234, 70090], [226, 233, 36783], [227, 236, 30617], [231, 232, 88111], [238, 237, 92400], [238, 239, 75774], [238, 240, 12986], [250, 241, 17834], [249, 242, 71713], [242, 245, 1420], [246, 243, 23506], [247, 244, 4799], [248, 244, 3776], [246, 245, 3689], [247, 245, 10077], [250, 245, 9629], [251, 256, 23709], [252, 266, 27674], [253, 266, 29451], [254, 256, 80805], [254, 262, 11046], [254, 269, 24434], [255, 256, 5379], [256, 259, 10939], [257, 259, 48803], [258, 266, 17574], [258, 260, 9968], [259, 268, 34095], [259, 266, 86687], [259, 263, 28853], [261, 265, 2613], [264, 268, 43023], [265, 266, 43751], [266, 267, 71128], [270, 271, 6896], [272, 273, 37102], [274, 273, 56081], [278, 275, 59080], [275, 276, 93346], [278, 277, 91953], [279, 283, 56754], [280, 283, 59331], [281, 283, 82808], [282, 284, 64810], [283, 284, 51217], [285, 286, 694], [285, 297, 53621], [285, 298, 44244], [286, 287, 38134], [286, 294, 27141], [286, 302, 16455], [286, 336, 4094], [286, 349, 15576], [287, 293, 57468], [287, 326, 45209], [288, 311, 48215], [289, 341, 62330], [289, 353, 34317], [290, 351, 55563], [291, 323, 40097], [291, 334, 61773], [291, 352, 66756], [292, 363, 35536], [293, 304, 44932], [293, 316, 57740], [293, 318, 1991], [293, 330, 22060], [293, 341, 54520], [293, 361, 53146], [294, 296, 28292], [294, 299, 46504], [294, 306, 12824], [294, 340, 39052], [295, 336, 272], [296, 343, 40024], [296, 366, 60177], [296, 385, 17495], [297, 360, 65530], [298, 380, 55828], [299, 319, 69761], [299, 370, 4277], [299, 354, 62887], [299, 346, 52272], [300, 323, 87502], [301, 319, 26914], [302, 310, 13235], [302, 323, 1762], [302, 359, 32308], [302, 382, 63073], [303, 351, 26914], [303, 327, 39993], [304, 305, 15511], [305, 309, 64679], [305, 325, 43412], [305, 364, 44055], [306, 315, 20571], [306, 378, 76460], [307, 362, 11970], [308, 351, 55977], [309, 333, 31800], [309, 357, 38191], [309, 363, 82634], [309, 311, 67629], [309, 329, 39114], [312, 383, 13253], [313, 336, 18241], [313, 328, 26653], [314, 373, 70775], [314, 374, 75422], [315, 339, 11344], [315, 355, 87630], [316, 317, 21191], [316, 342, 33919], [318, 331, 3472], [318, 337, 72002], [319, 368, 49868], [320, 366, 85010], [321, 370, 49169], [322, 333, 10709], [323, 351, 33306], [323, 335, 13144], [324, 378, 64075], [325, 356, 19543], [325, 373, 28241], [327, 386, 4469], [331, 369, 58204], [332, 341, 27954], [334, 344, 41215], [336, 347, 47815], [336, 384, 41201], [336, 367, 9090], [336, 358, 29442], [337, 372, 25055], [338, 348, 19217], [341, 345, 19983], [343, 348, 22222], [347, 365, 52716], [350, 379, 49970], [351, 383, 67268], [355, 377, 92788], [356, 375, 40294], [356, 379, 24533], [357, 381, 38029], [361, 362, 28229], [371, 376, 13667], [376, 384, 32670], [388, 389, 29061], [390, 391, 71952], [393, 396, 71990], [394, 396, 37108], [395, 396, 52652], [397, 398, 62645], [398, 399, 52973], [400, 401, 6770], [403, 407, 13196], [404, 407, 11475], [405, 407, 33524], [406, 407, 38159], [413, 411, 8575], [412, 413, 25855], [414, 417, 64717], [414, 419, 86270], [414, 415, 14623], [416, 418, 50078], [417, 420, 3520], [418, 419, 19578], [420, 421, 23929], [426, 422, 65519], [424, 423, 16786], [424, 426, 20033], [424, 425, 61321], [427, 428, 43277], [429, 430, 3185], [432, 439, 660], [433, 447, 42670], [433, 444, 64571], [433, 460, 9265], [434, 464, 51738], [434, 454, 46944], [435, 466, 10491], [436, 469, 93642], [437, 440, 12746], [438, 466, 59566], [439, 440, 76743], [439, 446, 16644], [439, 453, 38535], [439, 466, 65964], [439, 456, 9929], [439, 457, 15251], [440, 442, 19033], [440, 447, 22983], [440, 458, 17366], [440, 472, 15119], [441, 466, 40217], [441, 461, 25434], [441, 463, 12553], [442, 470, 13038], [443, 467, 9115], [445, 482, 7643], [446, 464, 64772], [446, 478, 3972], [447, 480, 35415], [447, 469, 93811], [448, 464, 77924], [449, 482, 23953], [449, 473, 42332], [450, 474, 81572], [451, 460, 44737], [452, 477, 35681], [452, 468, 70289], [453, 467, 37501], [455, 474, 31987], [457, 481, 63915], [458, 479, 67375], [459, 464, 83702], [462, 470, 70213], [464, 482, 75959], [465, 466, 43585], [466, 477, 2318], [466, 476, 41959], [471, 480, 54643], [474, 480, 86265], [474, 475, 13272], [484, 483, 12748], [486, 488, 12510], [487, 488, 1163], [490, 494, 57458], [491, 494, 24699], [492, 496, 51540], [493, 494, 52932], [493, 496, 8285], [493, 495, 62525], [498, 501, 18530], [499, 500, 40824], [499, 501, 52911], [511, 503, 40530], [518, 503, 20182], [504, 510, 66331], [512, 505, 18345], [514, 505, 39003], [516, 505, 46491], [517, 505, 37496], [505, 507, 46281], [505, 506, 55470], [519, 506, 41287], [515, 507, 10067], [516, 508, 58858], [513, 509, 67678], [517, 509, 50378], [514, 510, 8048], [516, 518, 81356], [518, 520, 4278], [521, 522, 1843], [523, 532, 71964], [523, 544, 1848], [524, 530, 94495], [525, 535, 51195], [526, 532, 32869], [527, 533, 64222], [528, 539, 34953], [529, 533, 86479], [530, 532, 95247], [530, 538, 44964], [530, 536, 52943], [530, 543, 4251], [530, 542, 17750], [531, 532, 38528], [532, 541, 45878], [532, 540, 17222], [533, 544, 22353], [534, 535, 5018], [535, 538, 45198], [535, 539, 55888], [537, 544, 25813], [548, 546, 10299], [546, 547, 54028], [550, 551, 68976], [552, 554, 5671], [553, 555, 44442], [554, 556, 10734], [555, 556, 38471], [557, 556, 33720], [558, 560, 18435], [558, 559, 34681], [561, 562, 49097], [562, 566, 53090], [563, 565, 45121], [564, 565, 49128], [565, 566, 32426], [567, 571, 31665], [567, 576, 7520], [567, 572, 21013], [568, 576, 19323], [569, 572, 63634], [569, 573, 81563], [570, 571, 21241], [571, 575, 44602], [572, 577, 21833], [579, 574, 25696], [578, 576, 7279], [579, 576, 60405], [580, 584, 40484], [581, 584, 68228], [582, 584, 51029], [582, 585, 36996], [583, 585, 22585], [584, 586, 46318], [587, 588, 31842], [593, 589, 37992], [590, 592, 2935], [591, 592, 23096], [591, 596, 37938], [592, 593, 81426], [593, 594, 27492], [595, 596, 57282], [597, 599, 43142], [598, 600, 28044], [598, 599, 18784], [603, 614, 36080], [603, 605, 31563], [603, 606, 48816], [603, 613, 46512], [603, 615, 51804], [604, 614, 14736], [605, 609, 54720], [606, 611, 22657], [607, 613, 7354], [608, 611, 8545], [609, 610, 5425], [612, 613, 84863], [617, 616, 11792], [618, 619, 57960], [621, 622, 20603], [622, 623, 75790], [624, 627, 11008], [625, 626, 29093], [625, 627, 36480], [626, 628, 32234], [631, 629, 53114], [638, 629, 3176], [633, 630, 29162], [635, 630, 68543], [640, 630, 23868], [631, 635, 33551], [632, 635, 41786], [632, 639, 29576], [632, 637, 32439], [634, 639, 30587], [635, 636, 37152], [641, 642, 12523], [642, 644, 11827], [643, 644, 44337], [645, 647, 14508], [646, 655, 29545], [647, 648, 57472], [648, 656, 10778], [649, 656, 46431], [649, 666, 73187], [650, 655, 49107], [650, 656, 57160], [650, 651, 11042], [651, 652, 17955], [653, 664, 30335], [654, 655, 47608], [655, 668, 45172], [655, 657, 4245], [656, 664, 27413], [656, 667, 45322], [656, 663, 58668], [658, 667, 9146], [658, 665, 15604], [658, 662, 25989], [659, 661, 32376], [660, 667, 35825], [661, 667, 9043], [670, 672, 21897], [671, 672, 35012], [675, 686, 22312], [675, 682, 52219], [675, 683, 89778], [676, 682, 77376], [676, 680, 15048], [677, 686, 6772], [678, 690, 27531], [678, 679, 48526], [679, 689, 44133], [681, 684, 24267], [682, 687, 8108], [684, 690, 28453], [685, 686, 65874], [686, 688, 25460], [688, 690, 2568], [692, 691, 8951], [694, 695, 29954], [697, 698, 23657], [699, 700, 9913], [700, 701, 82236], [702, 703, 57861], [703, 705, 34510], [704, 705, 87353], [705, 706, 29650], [717, 708, 2675], [717, 709, 29752], [710, 715, 11824], [711, 717, 63303], [712, 716, 14855], [712, 713, 5458], [712, 714, 54693], [713, 715, 46831], [716, 717, 77565], [718, 722, 7335], [718, 720, 16564], [719, 722, 36393], [720, 721, 54776], [725, 730, 56377], [726, 728, 3186], [727, 730, 58025], [728, 732, 41056], [729, 730, 63609], [730, 731, 34629], [731, 732, 2757], [733, 734, 25903], [734, 735, 24792], [736, 738, 3031], [736, 737, 44635], [739, 740, 63235], [741, 762, 76403], [742, 759, 18406], [742, 748, 45827], [743, 756, 77389], [743, 758, 9984], [743, 760, 42979], [743, 744, 817], [745, 747, 42618], [745, 761, 54118], [746, 760, 6972], [746, 750, 14315], [746, 762, 10137], [746, 752, 58494], [747, 756, 2711], [747, 751, 31140], [747, 749, 3409], [750, 754, 32231], [753, 757, 40777], [755, 763, 14295], [756, 759, 13583], [757, 759, 80622], [758, 763, 65526], [764, 765, 48105], [766, 767, 43855], [766, 770, 90522], [767, 768, 4712], [767, 769, 81169], [771, 776, 46535], [772, 773, 18003], [772, 780, 51676], [773, 779, 33060], [774, 776, 73693], [774, 775, 150], [776, 779, 31217], [777, 779, 56270], [778, 779, 20935], [789, 782, 67924], [790, 782, 25948], [792, 782, 60391], [793, 782, 1750], [795, 782, 27110], [783, 784, 5945], [785, 784, 30774], [787, 784, 3167], [794, 784, 64581], [785, 792, 35099], [786, 792, 83307], [786, 788, 65791], [787, 796, 77927], [791, 794, 16006], [799, 800, 17635], [801, 800, 48859], [802, 803, 33498], [804, 806, 1667], [805, 806, 51479], [806, 807, 68874], [807, 808, 25994], [810, 811, 38994], [812, 813, 3139], [814, 816, 27063], [815, 816, 33712], [817, 822, 55968], [817, 819, 10506], [818, 824, 69172], [820, 824, 66523], [821, 822, 13992], [822, 823, 45061], [823, 824, 21895], [827, 831, 4791], [828, 831, 71732], [829, 831, 56750], [830, 831, 6131], [836, 838, 58239], [837, 841, 6732], [837, 838, 85293], [839, 840, 70709], [840, 841, 49855], [842, 843, 32385], [844, 845, 64416], [846, 852, 13280], [846, 848, 33054], [847, 851, 63782], [848, 853, 4972], [849, 851, 69408], [850, 854, 16128], [851, 852, 72979], [852, 854, 13236], [855, 856, 31035], [856, 857, 33927], [858, 860, 29179], [859, 860, 74146], [861, 862, 2626], [866, 867, 504], [868, 867, 28446], [869, 872, 28573], [869, 881, 8925], [870, 877, 31636], [871, 876, 66030], [871, 873, 68921], [871, 882, 90152], [872, 873, 1326], [872, 877, 32796], [873, 878, 6460], [874, 878, 1667], [875, 876, 25509], [876, 879, 12673], [879, 880, 5763], [887, 888, 11288], [890, 891, 77111], [892, 894, 42395], [893, 894, 24026], [897, 898, 52759], [898, 901, 17057], [898, 908, 14288], [898, 902, 8187], [899, 901, 5519], [900, 901, 7105], [900, 905, 53574], [903, 908, 31037], [904, 907, 50055], [906, 907, 27437], [907, 908, 3081], [909, 910, 49372], [911, 912, 5537], [914, 912, 63656], [914, 913, 42933], [916, 917, 88869], [916, 918, 40184], [919, 920, 43803], [920, 922, 4767], [920, 923, 62971], [920, 921, 36956], [926, 927, 26799], [926, 928, 34114], [931, 932, 69036], [933, 935, 4041], [934, 935, 3977], [938, 940, 75362], [939, 940, 64954], [940, 942, 62970], [941, 942, 3021], [944, 949, 40491], [945, 949, 48270], [946, 949, 38478], [946, 948, 37617], [947, 949, 72599], [950, 956, 61194], [950, 954, 43429], [960, 951, 15781], [951, 955, 38652], [959, 952, 68142], [952, 957, 70614], [962, 953, 1435], [953, 956, 31287], [955, 957, 46162], [955, 958, 43158], [961, 956, 34522], [956, 957, 63188], [963, 958, 50925], [965, 966, 41782], [966, 967, 77773], [966, 968, 42342], [978, 970, 56787], [971, 979, 2698], [971, 978, 29819], [971, 975, 45894], [971, 974, 49319], [972, 978, 17680], [973, 975, 7700], [975, 976, 10020], [976, 977, 5107], [981, 982, 88320], [985, 984, 60017], [987, 988, 12734], [987, 989, 32848], [1001, 995, 4586], [1002, 995, 42706], [1003, 995, 24260], [1003, 996, 63355], [1003, 997, 70782], [1002, 998, 68399], [998, 999, 25202], [1000, 1001, 41587], [1004, 1007, 31778], [1004, 1006, 52644], [1005, 1007, 91095], [1006, 1013, 47022], [1007, 1012, 7780], [1007, 1011, 44950], [1007, 1008, 16216], [1008, 1010, 66512], [1009, 1011, 70390], [1018, 1016, 48181], [1019, 1016, 51984], [1017, 1019, 21564], [1024, 1038, 3436], [1025, 1047, 83063], [1025, 1045, 74902], [1026, 1030, 8634], [1027, 1045, 16549], [1028, 1034, 31880], [1028, 1047, 18420], [1028, 1029, 21536], [1028, 1039, 19141], [1030, 1034, 51647], [1030, 1037, 68072], [1030, 1042, 21665], [1030, 1043, 34057], [1030, 1035, 37847], [1031, 1034, 46530], [1032, 1049, 22523], [1033, 1047, 13932], [1036, 1047, 41835], [1036, 1048, 47267], [1038, 1047, 8643], [1038, 1040, 66300], [1038, 1046, 60437], [1038, 1041, 17030], [1044, 1047, 68886], [1047, 1049, 9036], [1050, 1052, 33114], [1050, 1051, 42262], [1053, 1054, 1034], [1056, 1055, 15569], [1056, 1057, 14232], [1061, 1062, 62791], [1064, 1065, 35528], [1065, 1066, 22017], [1072, 1068, 5520], [1070, 1069, 19217], [1071, 1069, 36080], [1072, 1069, 22789], [1074, 1079, 43129], [1074, 1078, 9137], [1075, 1076, 47324], [1076, 1086, 48112], [1076, 1084, 23033], [1077, 1086, 88321], [1077, 1079, 4414], [1079, 1085, 74716], [1080, 1081, 19267], [1081, 1082, 32397], [1082, 1083, 51338], [1083, 1086, 4810], [1088, 1089, 69403], [1090, 1092, 30551], [1090, 1093, 5649], [1091, 1094, 12274], [1092, 1094, 32310], [1099, 1100, 49543], [1101, 1102, 93130], [1102, 1103, 76992], [1104, 1108, 63263], [1105, 1109, 6693], [1106, 1107, 15118], [1107, 1108, 33422], [1108, 1109, 35816], [1111, 1115, 14841], [1111, 1122, 52285], [1123, 1112, 81705], [1112, 1120, 68216], [1113, 1121, 48746], [1123, 1114, 44945], [1114, 1121, 5680], [1115, 1121, 2837], [1116, 1121, 38691], [1116, 1118, 2872], [1125, 1117, 59737], [1119, 1121, 68284], [1126, 1121, 66745], [1124, 1125, 10978], [1125, 1126, 67779], [1127, 1129, 30089], [1128, 1131, 62567], [1128, 1137, 28861], [1129, 1136, 9003], [1129, 1142, 31266], [1130, 1140, 9311], [1130, 1141, 23375], [1131, 1142, 39233], [1131, 1139, 23204], [1131, 1133, 21399], [1132, 1143, 63311], [1134, 1143, 59119], [1135, 1143, 15180], [1136, 1140, 38342], [1138, 1142, 47428], [1142, 1143, 3124], [1153, 1144, 75540], [1155, 1145, 17113], [1153, 1146, 70528], [1147, 1150, 78743], [1156, 1148, 26907], [1153, 1149, 45025], [1152, 1150, 58624], [1154, 1150, 62204], [1150, 1151, 46085], [1152, 1153, 57594], [1152, 1155, 19997], [1153, 1156, 54249], [1157, 1159, 8039], [1157, 1158, 84200], [1160, 1162, 82232], [1161, 1162, 31825], [1163, 1165, 8646], [1164, 1166, 15041], [1165, 1166, 77020], [1165, 1167, 63547], [1168, 1170, 13063], [1168, 1172, 25559], [1169, 1170, 13083], [1169, 1171, 17946], [1174, 1175, 54680], [1175, 1177, 2639], [1175, 1182, 24538], [1176, 1182, 37380], [1177, 1184, 40808], [1177, 1180, 73592], [1178, 1180, 75793], [1179, 1180, 61470], [1180, 1181, 70548], [1180, 1183, 1311], [1185, 1186, 11746], [1188, 1189, 9705], [1191, 1192, 71928], [1191, 1193, 77713], [1197, 1195, 15884], [1197, 1196, 10519], [1199, 1200, 29487], [1205, 1204, 4294], [1205, 1206, 7391], [1208, 1219, 17475], [1209, 1218, 64967], [1209, 1216, 39298], [1209, 1219, 29287], [1209, 1212, 41643], [1210, 1218, 50055], [1211, 1216, 34905], [1213, 1218, 52037], [1213, 1217, 74787], [1214, 1219, 35706], [1215, 1216, 1836], [1221, 1223, 78913], [1221, 1224, 6354], [1221, 1222, 10878], [1225, 1227, 34614], [1226, 1227, 3109], [1228, 1232, 64709], [1229, 1231, 44461], [1230, 1231, 77729], [1231, 1232, 35580], [1235, 1237, 13498], [1236, 1237, 35853], [1238, 1239, 60730], [1238, 1240, 22161], [1240, 1241, 10898], [1241, 1242, 34904], [1244, 1246, 54378], [1244, 1248, 4852], [1244, 1252, 18480], [1245, 1246, 76533], [1246, 1251, 16071], [1246, 1250, 29220], [1247, 1250, 52237], [1247, 1249, 35303], [1253, 1257, 3721], [1254, 1257, 46162], [1255, 1256, 34305], [1255, 1257, 45145], [1258, 1264, 79112], [1258, 1260, 30771], [1259, 1264, 85911], [1261, 1262, 32278], [1269, 1262, 42433], [1272, 1262, 9371], [1262, 1264, 85639], [1263, 1264, 55738], [1266, 1264, 43736], [1270, 1264, 94700], [1265, 1272, 84844], [1266, 1268, 64981], [1267, 1268, 19232], [1270, 1271, 73876], [1273, 1274, 2115], [1278, 1275, 72635], [1275, 1276, 14393], [1275, 1277, 18610], [1279, 1280, 47197], [1282, 1283, 1664], [1287, 1288, 32702], [1290, 1291, 11874], [1294, 1295, 23274], [1296, 1299, 42673], [1297, 1300, 29487], [1298, 1300, 19013], [1299, 1300, 55761], [1302, 1306, 15372], [1302, 1303, 44618], [1304, 1306, 46238], [1305, 1306, 18157], [1307, 1308, 33548], [1307, 1309, 49826], [1307, 1313, 59012], [1308, 1315, 33274], [1309, 1312, 1150], [1309, 1311, 7500], [1310, 1315, 20153], [1310, 1314, 46946], [1316, 1317, 28017], [1317, 1318, 66052], [1323, 1324, 15012], [1323, 1325, 31161], [1328, 1329, 533], [1335, 1336, 4644], [1336, 1337, 17296], [1338, 1339, 29731], [1340, 1342, 41230], [1341, 1342, 52331], [1343, 1346, 55775], [1343, 1344, 36010], [1343, 1347, 14100], [1345, 1346, 29597], [1348, 1350, 85100], [1349, 1350, 43503], [1353, 1356, 12931], [1353, 1354, 16541], [1353, 1355, 49797], [1365, 1357, 24786], [1358, 1359, 4230], [1360, 1359, 33687], [1361, 1359, 61457], [1364, 1359, 79416], [1365, 1359, 16961], [1361, 1362, 60592], [1363, 1364, 72975], [1366, 1367, 1608], [1367, 1369, 25217], [1367, 1368, 5780], [1369, 1370, 31411], [1371, 1372, 24445], [1375, 1376, 71936], [1377, 1378, 22527], [1381, 1383, 24494], [1382, 1384, 53844], [1383, 1384, 36501], [1385, 1389, 41754], [1385, 1387, 20474], [1386, 1387, 50950], [1387, 1388, 75435], [1389, 1390, 3702], [1391, 1393, 19413], [1392, 1393, 26246], [1394, 1397, 17674], [1394, 1396, 26458], [1395, 1397, 8313], [1397, 1398, 47005], [1399, 1400, 31787], [1401, 1404, 28122], [1402, 1404, 9332], [1402, 1403, 37605], [1409, 1410, 41648], [1411, 1412, 6679], [1413, 1418, 16516], [1413, 1419, 79561], [1414, 1415, 86604], [1415, 1417, 24223], [1416, 1418, 4160], [1416, 1420, 16773], [1417, 1418, 14405], [1423, 1424, 63974], [1425, 1426, 16627], [1426, 1427, 87018], [1432, 1433, 3757], [1435, 1437, 27299], [1435, 1438, 41551], [1436, 1437, 44613], [1439, 1441, 35956], [1440, 1441, 63588], [1443, 1444, 493], [1445, 1446, 17809], [1447, 1448, 61328], [1449, 1450, 45912], [1451, 1454, 24198], [1451, 1453, 41242], [1452, 1454, 13561], [1460, 1457, 54427], [1460, 1458, 52883], [1459, 1460, 69068], [1462, 1463, 52684], [1463, 1464, 35972], [1466, 1465, 27174], [1467, 1468, 40634], [1470, 1471, 36943], [1472, 1473, 41034], [1475, 1474, 49616], [1483, 1481, 25758], [1483, 1482, 6545], [1486, 1487, 33042], [1489, 1490, 71217], [1491, 1492, 1120], [1491, 1493, 18282], [1495, 1496, 34218], [1497, 1498, 78922], [1501, 1502, 68390], [1508, 1509, 42603], [1512, 1513, 22769], [1515, 1516, 51541], [1516, 1517, 3448], [1523, 1525, 18274], [1524, 1525, 54747], [1530, 1531, 45941], [1533, 1534, 36226], [1535, 1536, 73680], [1536, 1537, 71906], [1538, 1539, 33472], [1541, 1542, 58231], [1544, 1547, 75514], [1545, 1547, 8199], [1546, 1548, 18683], [1546, 1547, 41397], [1550, 1553, 10769], [1550, 1552, 34755], [1550, 1551, 22761], [1553, 1555, 53315], [1554, 1555, 44928], [1558, 1559, 88949], [1564, 1563, 73889], [1564, 1565, 53213], [1565, 1566, 87245], [1567, 1568, 5303], [1567, 1570, 3048], [1568, 1569, 34538], [1571, 1572, 92449], [1575, 1578, 27282], [1576, 1578, 919], [1577, 1578, 16146], [1579, 1580, 53905], [1586, 1583, 33779], [1583, 1585, 43138], [1583, 1584, 95390], [1592, 1591, 72179], [1596, 1597, 29322], [1598, 1599, 41664], [1600, 1604, 58621], [1601, 1604, 13396], [1602, 1604, 36796], [1603, 1604, 3364], [1609, 1606, 3214], [1607, 1609, 36328], [1607, 1608, 40659], [1610, 1611, 3667], [1612, 1613, 16293], [1612, 1614, 44907], [1617, 1616, 36372], [1621, 1622, 27475], [1625, 1627, 65680], [1626, 1627, 21304], [1628, 1632, 15377], [1628, 1629, 38843], [1629, 1633, 18628], [1630, 1632, 3282], [1630, 1631, 69239], [1635, 1636, 15958], [1638, 1640, 65723], [1639, 1640, 45012], [1641, 1642, 2768], [1644, 1646, 48044], [1645, 1646, 51297], [1647, 1650, 62285], [1647, 1651, 35114], [1647, 1648, 2396], [1649, 1650, 65831], [1653, 1655, 20215], [1654, 1656, 18022], [1655, 1656, 68221], [1657, 1658, 29520], [1660, 1661, 45295], [1662, 1663, 52026], [1663, 1664, 42073], [1664, 1665, 11472], [1666, 1667, 51154], [1679, 1680, 40216], [1681, 1682, 42437], [1685, 1686, 8734], [1690, 1687, 50145], [1688, 1690, 3108], [1688, 1691, 34397], [1688, 1689, 54410], [1695, 1696, 33745], [1700, 1699, 39924], [1702, 1704, 92725], [1703, 1704, 26260], [1704, 1705, 9806], [1706, 1707, 81587], [1709, 1710, 92693], [1712, 1713, 29933], [1722, 1723, 86957], [1727, 1728, 30269], [1728, 1729, 64385], [1730, 1731, 19543], [1734, 1733, 21385], [1736, 1737, 77892], [1743, 1744, 43130], [1748, 1749, 76015], [1751, 1752, 15883], [1756, 1757, 44916], [1758, 1759, 3618], [1759, 1760, 55561], [1763, 1764, 23707], [1764, 1765, 67666], [1768, 1767, 60069], [1768, 1769, 10125], [1771, 1772, 32861], [1773, 1774, 38472], [1776, 1777, 3916], [1776, 1778, 17762], [1779, 1780, 16784], [1781, 1783, 64558], [1782, 1783, 21523], [1788, 1789, 18306], [1795, 1796, 52963], [1797, 1798, 27941], [1803, 1801, 49417], [1801, 1802, 59207], [1807, 1808, 34749], [1810, 1811, 30644], [1819, 1821, 49036], [1819, 1822, 50184], [1820, 1821, 47258], [1827, 1825, 52297], [1827, 1826, 64971], [1828, 1830, 45100], [1828, 1829, 81349], [1831, 1832, 1716], [1832, 1833, 142], [1834, 1835, 26684], [1847, 1845, 75996], [1845, 1846, 50918], [1857, 1858, 95297], [1858, 1859, 88133], [1864, 1865, 18956], [1865, 1866, 84317], [1876, 1875, 58408], [1879, 1878, 53419], [1892, 1893, 20414], [1895, 1894, 17135], [1901, 1902, 3947], [1906, 1907, 34399], [1909, 1908, 1681], [1912, 1913, 27950], [1922, 1919, 52690], [1922, 1920, 3423], [1921, 1922, 32361], [1925, 1924, 91909], [1933, 1932, 20070], [1935, 1932, 35787], [1933, 1934, 6282], [1948, 1949, 38274], [1954, 1958, 26625], [1955, 1956, 52848], [1955, 1959, 54955], [1956, 1958, 45237], [1957, 1958, 7171], [1978, 1979, 33509], [1980, 1981, 35055], [1984, 1985, 11570], [1986, 1987, 24461], [2000, 2001, 45742], [2005, 2006, 57318], [2011, 2012, 17912], [2021, 2022, 65341], [2021, 2023, 38787], [2024, 2025, 77541], [2037, 2038, 1318], [2056, 2057, 17502], [2058, 2059, 47413], [2060, 2061, 44672], [2066, 2067, 12721], [2112, 2113, 73324]]
well_status= [4, 5, 10, 12, 16, 18, 21, 24, 26, 29, 36, 38, 41, 46, 47, 49, 53, 55, 60, 61, 62, 65, 72, 74, 75, 80, 81, 82, 83, 86, 89, 98, 99, 100, 102, 104, 106, 108, 109, 114, 115, 117, 125, 126, 128, 130, 131, 132, 133, 134, 135, 138, 144, 164, 167, 171, 173, 174, 175, 176, 177, 180, 183, 184, 194, 195, 197, 199, 201, 202, 203, 212, 214, 220, 223, 225, 229, 230, 232, 235, 236, 237, 239, 240, 248, 252, 253, 257, 258, 262, 265, 267, 268, 271, 272, 274, 276, 277, 279, 280, 281, 284, 288, 292, 298, 304, 307, 310, 317, 319, 320, 321, 322, 329, 330, 334, 337, 339, 340, 343, 345, 353, 354, 357, 360, 365, 369, 371, 374, 377, 378, 387, 389, 391, 392, 393, 395, 397, 400, 402, 405, 408, 409, 410, 413, 415, 418, 422, 423, 428, 430, 431, 432, 433, 434, 435, 436, 437, 441, 450, 451, 455, 459, 462, 477, 479, 480, 481, 482, 483, 485, 487, 489, 494, 495, 497, 498, 500, 502, 507, 509, 511, 514, 516, 519, 520, 522, 524, 525, 526, 527, 529, 531, 536, 538, 544, 545, 548, 549, 551, 554, 557, 559, 560, 561, 563, 572, 573, 579, 581, 583, 586, 588, 589, 592, 596, 599, 600, 601, 602, 605, 608, 610, 615, 617, 619, 620, 621, 623, 626, 635, 638, 639, 644, 645, 648, 649, 652, 661, 668, 669, 672, 673, 674, 680, 689, 690, 692, 693, 694, 696, 697, 699, 703, 704, 707, 709, 716, 720, 723, 724, 729, 732, 733, 737, 738, 740, 741, 748, 753, 754, 758, 765, 766, 768, 769, 775, 777, 778, 781, 783, 785, 788, 789, 790, 793, 794, 796, 797, 798, 799, 801, 802, 804, 808, 809, 811, 813, 816, 818, 819, 820, 825, 826, 827, 828, 829, 832, 833, 834, 835, 836, 840, 843, 845, 848, 849, 854, 855, 858, 859, 862, 863, 864, 865, 866, 872, 880, 882, 883, 884, 885, 886, 887, 889, 891, 892, 893, 895, 896, 901, 910, 913, 915, 917, 918, 919, 922, 924, 925, 926, 929, 930, 931, 935, 936, 937, 940, 943, 945, 947, 948, 953, 954, 957, 958, 959, 964, 965, 968, 969, 974, 977, 979, 980, 981, 983, 985, 986, 988, 990, 991, 992, 993, 994, 1000, 1003, 1007, 1009, 1010, 1013, 1014, 1015, 1017, 1020, 1021, 1022, 1023, 1024, 1027, 1028, 1031, 1036, 1042, 1046, 1051, 1052, 1054, 1055, 1058, 1059, 1060, 1061, 1063, 1066, 1067, 1069, 1073, 1079, 1080, 1083, 1087, 1089, 1093, 1095, 1096, 1097, 1098, 1099, 1102, 1104, 1107, 1110, 1112, 1115, 1116, 1119, 1122, 1124, 1126, 1128, 1130, 1132, 1133, 1135, 1138, 1145, 1147, 1149, 1156, 1157, 1160, 1161, 1164, 1167, 1170, 1173, 1175, 1178, 1179, 1181, 1183, 1185, 1187, 1189, 1190, 1192, 1194, 1196, 1198, 1199, 1201, 1202, 1203, 1206, 1207, 1212, 1218, 1219, 1220, 1223, 1224, 1227, 1229, 1232, 1233, 1234, 1235, 1236, 1239, 1242, 1243, 1244, 1245, 1250, 1251, 1257, 1258, 1259, 1262, 1263, 1268, 1271, 1274, 1276, 1278, 1280, 1281, 1282, 1284, 1285, 1286, 1287, 1289, 1291, 1292, 1293, 1295, 1296, 1297, 1301, 1302, 1310, 1312, 1316, 1318, 1319, 1320, 1321, 1322, 1324, 1326, 1327, 1328, 1330, 1331, 1332, 1333, 1334, 1336, 1338, 1341, 1343, 1345, 1348, 1349, 1351, 1352, 1353, 1360, 1361, 1364, 1367, 1372, 1373, 1374, 1375, 1378, 1379, 1380, 1381, 1389, 1393, 1395, 1399, 1403, 1405, 1406, 1407, 1408, 1410, 1412, 1418, 1421, 1422, 1423, 1425, 1428, 1429, 1430, 1431, 1432, 1434, 1437, 1439, 1442, 1444, 1446, 1447, 1449, 1452, 1455, 1456, 1460, 1461, 1462, 1465, 1467, 1469, 1470, 1473, 1474, 1476, 1477, 1478, 1479, 1480, 1483, 1484, 1485, 1486, 1488, 1490, 1491, 1494, 1495, 1497, 1499, 1500, 1501, 1503, 1504, 1505, 1506, 1507, 1508, 1510, 1511, 1513, 1514, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1526, 1527, 1528, 1529, 1530, 1532, 1533, 1536, 1539, 1540, 1541, 1543, 1545, 1546, 1549, 1553, 1556, 1557, 1558, 1560, 1561, 1562, 1564, 1567, 1569, 1572, 1573, 1574, 1575, 1577, 1580, 1581, 1582, 1584, 1585, 1587, 1588, 1589, 1590, 1591, 1593, 1594, 1595, 1597, 1599, 1600, 1602, 1603, 1605, 1609, 1610, 1613, 1615, 1617, 1618, 1619, 1620, 1621, 1623, 1624, 1625, 1632, 1634, 1635, 1637, 1639, 1642, 1643, 1645, 1648, 1650, 1652, 1653, 1656, 1657, 1659, 1661, 1664, 1666, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1680, 1682, 1683, 1684, 1685, 1691, 1692, 1693, 1694, 1695, 1697, 1698, 1700, 1701, 1704, 1707, 1708, 1710, 1711, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1724, 1725, 1726, 1727, 1731, 1732, 1733, 1735, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1745, 1746, 1747, 1748, 1750, 1752, 1753, 1754, 1755, 1757, 1759, 1761, 1762, 1763, 1766, 1767, 1769, 1770, 1771, 1773, 1775, 1777, 1780, 1781, 1782, 1784, 1785, 1786, 1787, 1788, 1790, 1791, 1792, 1793, 1794, 1796, 1797, 1799, 1800, 1802, 1803, 1804, 1805, 1806, 1808, 1809, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1822, 1823, 1824, 1825, 1826, 1829, 1830, 1833, 1834, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1859, 1860, 1861, 1862, 1863, 1864, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1877, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1894, 1896, 1897, 1898, 1899, 1900, 1902, 1903, 1904, 1905, 1907, 1908, 1910, 1911, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1921, 1923, 1924, 1926, 1927, 1928, 1929, 1930, 1931, 1933, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1950, 1951, 1952, 1953, 1957, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1981, 1982, 1983, 1985, 1986, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2001, 2002, 2003, 2004, 2006, 2007, 2008, 2009, 2010, 2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2023, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2059, 2061, 2062, 2063, 2064, 2065, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160]
pipe_status= [(1, 2), (3, 75), (4, 37), (4, 88), (5, 36), (6, 110), (7, 24), (7, 104), (7, 127), (8, 12), (11, 137), (12, 20), (12, 128), (13, 95), (14, 33), (14, 50), (14, 91), (15, 55), (16, 19), (16, 48), (17, 78), (17, 98), (18, 130), (20, 124), (21, 25), (22, 134), (24, 92), (25, 43), (25, 71), (26, 106), (27, 112), (27, 125), (27, 136), (27, 139), (28, 49), (28, 100), (30, 56), (31, 72), (34, 81), (34, 86), (36, 53), (39, 60), (39, 133), (40, 140), (41, 102), (42, 139), (44, 138), (45, 110), (46, 108), (47, 141), (49, 109), (50, 142), (51, 130), (52, 75), (54, 80), (59, 89), (59, 110), (60, 99), (61, 132), (62, 77), (64, 70), (65, 89), (67, 102), (68, 93), (69, 136), (72, 111), (72, 138), (73, 111), (75, 90), (76, 110), (77, 95), (78, 119), (79, 94), (84, 98), (85, 133), (88, 121), (90, 118), (91, 135), (93, 103), (93, 105), (94, 115), (96, 97), (96, 141), (98, 106), (98, 134), (101, 116), (102, 117), (102, 123), (105, 108), (107, 126), (110, 129), (114, 116), (126, 140), (131, 132), (145, 146), (146, 147), (148, 149), (150, 151), (152, 154), (153, 154), (154, 155), (156, 157), (156, 158), (159, 160), (161, 162), (162, 163), (162, 164), (164, 165), (167, 168), (170, 171), (171, 172), (173, 180), (174, 185), (174, 201), (175, 195), (175, 205), (177, 210), (178, 181), (179, 207), (180, 187), (180, 194), (180, 197), (180, 198), (181, 183), (181, 188), (181, 199), (181, 213), (182, 202), (182, 204), (183, 211), (184, 208), (186, 223), (187, 219), (189, 205), (190, 214), (190, 223), (193, 209), (193, 218), (194, 208), (196, 215), (206, 207), (207, 217), (207, 218), (212, 221), (215, 216), (224, 226), (225, 226), (226, 228), (227, 228), (229, 230), (231, 232), (231, 235), (233, 235), (234, 235), (236, 237), (238, 239), (240, 242), (244, 245), (248, 249), (248, 250), (248, 251), (252, 260), (253, 260), (253, 261), (254, 260), (255, 257), (257, 258), (258, 259), (262, 263), (264, 266), (265, 266), (268, 269), (271, 277), (271, 288), (272, 273), (272, 287), (274, 276), (274, 290), (275, 279), (275, 281), (275, 289), (275, 291), (276, 278), (276, 280), (276, 285), (279, 283), (282, 286), (284, 292), (285, 288), (287, 292), (293, 294), (293, 305), (294, 295), (294, 310), (294, 344), (294, 357), (295, 334), (296, 319), (297, 361), (298, 359), (299, 331), (299, 360), (300, 371), (301, 326), (301, 338), (302, 304), (302, 307), (302, 314), (303, 344), (304, 374), (304, 393), (306, 388), (307, 354), (307, 378), (308, 331), (309, 327), (310, 318), (310, 331), (310, 367), (310, 390), (311, 335), (311, 359), (312, 313), (313, 333), (313, 372), (314, 323), (315, 370), (316, 359), (317, 341), (320, 391), (321, 336), (321, 344), (322, 381), (323, 347), (323, 363), (324, 325), (324, 350), (326, 339), (327, 376), (330, 341), (331, 343), (331, 359), (332, 386), (333, 364), (333, 381), (335, 394), (340, 349), (342, 352), (344, 355), (344, 366), (344, 375), (345, 380), (346, 356), (349, 353), (351, 356), (358, 387), (359, 391), (364, 383), (364, 387), (365, 389), (369, 370), (379, 384), (384, 392), (396, 405), (397, 407), (398, 405), (400, 408), (401, 405), (402, 403), (402, 404), (402, 406), (404, 407), (410, 412), (411, 412), (413, 414), (414, 415), (417, 420), (419, 433), (419, 434), (421, 446), (422, 431), (423, 454), (424, 430), (425, 450), (426, 436), (427, 437), (428, 443), (428, 457), (429, 442), (432, 437), (435, 445), (436, 454), (437, 438), (437, 452), (437, 456), (439, 451), (440, 451), (442, 450), (444, 449), (446, 455), (447, 454), (448, 457), (458, 459), (460, 462), (461, 463), (462, 464), (465, 466), (467, 468), (469, 473), (469, 477), (471, 476), (472, 474), (472, 475), (479, 481), (480, 481), (481, 482), (483, 486), (483, 495), (484, 491), (485, 490), (486, 487), (486, 491), (487, 492), (488, 492), (489, 490), (490, 493), (493, 494), (497, 498), (497, 499), (501, 503), (502, 503), (504, 518), (505, 519), (506, 507), (506, 520), (507, 508), (507, 513), (507, 516), (507, 518), (508, 514), (509, 517), (510, 519), (511, 512), (517, 518), (522, 526), (523, 527), (523, 532), (524, 531), (525, 528), (525, 534), (527, 530), (528, 533), (530, 535), (538, 542), (538, 543), (538, 547), (539, 547), (540, 543), (541, 542), (542, 546), (543, 548), (545, 550), (547, 549), (551, 552), (552, 555), (552, 556), (552, 562), (553, 555), (554, 555), (554, 559), (557, 562), (558, 561), (560, 561), (561, 562), (563, 567), (564, 565), (564, 568), (565, 567), (566, 567), (570, 571), (570, 573), (572, 574), (573, 576), (574, 575), (576, 577), (578, 579), (581, 583), (582, 591), (584, 592), (585, 602), (586, 587), (587, 588), (589, 600), (590, 591), (591, 593), (591, 604), (592, 599), (592, 600), (594, 598), (594, 601), (594, 603), (595, 597), (596, 603), (597, 603), (606, 607), (609, 620), (609, 622), (610, 611), (611, 614), (612, 619), (613, 615), (618, 621), (624, 626), (625, 626), (627, 636), (628, 631), (628, 635), (629, 632), (630, 633), (630, 634), (631, 632), (631, 633), (631, 636), (638, 640), (639, 640), (641, 644), (642, 643), (645, 647), (646, 655), (647, 654), (647, 660), (648, 658), (648, 659), (649, 651), (649, 657), (652, 661), (653, 661), (660, 661), (663, 665), (666, 668), (667, 668), (669, 670), (673, 677), (674, 677), (675, 677), (676, 677), (678, 682), (680, 683), (681, 683), (682, 684), (685, 706), (688, 694), (690, 701), (692, 700), (692, 704), (692, 705), (694, 702), (694, 703), (695, 706), (696, 697), (697, 700), (697, 701), (699, 706), (708, 710), (708, 711), (709, 712), (710, 712), (713, 714), (715, 716), (717, 718), (720, 721), (723, 724), (726, 729), (726, 734), (727, 730), (727, 731), (727, 736), (727, 740), (727, 742), (728, 739), (729, 738), (730, 733), (732, 741), (739, 740), (746, 747), (746, 748), (746, 749), (749, 751), (750, 751), (752, 758), (753, 757), (754, 759), (755, 757), (756, 760), (758, 760), (761, 762), (763, 764), (764, 766), (765, 766), (768, 770), (769, 771), (772, 774), (772, 775), (773, 774), (777, 779), (780, 782), (781, 782), (783, 785), (786, 788), (786, 796), (786, 797), (787, 797), (789, 794), (790, 796), (791, 794), (792, 793), (795, 796), (799, 807), (800, 801), (801, 802), (801, 807), (803, 804), (805, 806), (810, 814), (810, 815), (811, 812), (812, 820), (812, 822), (813, 815), (815, 821), (816, 817), (817, 818), (819, 822), (823, 826), (824, 825), (824, 826), (827, 842), (828, 834), (829, 836), (829, 838), (829, 841), (830, 843), (831, 839), (832, 840), (833, 837), (834, 838), (842, 844), (848, 849), (850, 851), (851, 852), (853, 854), (853, 855), (860, 861), (862, 863), (864, 866), (867, 872), (867, 873), (868, 869), (869, 871), (870, 872), (870, 874), (871, 872), (877, 878), (880, 882), (883, 884), (884, 891), (884, 894), (885, 886), (886, 888), (887, 894), (889, 890), (892, 893), (895, 906), (896, 903), (896, 906), (897, 905), (898, 903), (900, 904), (900, 905), (901, 906), (902, 903), (907, 908), (908, 910), (910, 911), (912, 913), (912, 914), (915, 916), (915, 917), (919, 920), (920, 921), (921, 922), (923, 924), (924, 925), (926, 940), (927, 947), (928, 932), (929, 947), (930, 931), (930, 936), (930, 941), (932, 937), (932, 939), (932, 944), (932, 945), (934, 951), (935, 949), (938, 950), (940, 942), (940, 943), (940, 949), (946, 949), (949, 951), (952, 954), (954, 955), (955, 956), (959, 960), (963, 964), (964, 965), (966, 967), (966, 968), (966, 969), (970, 975), (971, 972), (971, 979), (972, 978), (973, 974), (975, 978), (977, 978), (980, 984), (981, 984), (982, 983), (982, 984), (986, 987), (988, 989), (990, 995), (993, 1001), (993, 1008), (994, 995), (995, 998), (997, 999), (997, 1005), (998, 1002), (998, 1007), (1000, 1004), (1003, 1007), (1012, 1013), (1013, 1015), (1013, 1020), (1014, 1020), (1015, 1022), (1018, 1021), (1025, 1028), (1026, 1028), (1027, 1029), (1032, 1033), (1036, 1040), (1039, 1040), (1041, 1042), (1044, 1048), (1045, 1047), (1046, 1047), (1048, 1049), (1050, 1051), (1052, 1053), (1055, 1056), (1056, 1057), (1058, 1059), (1063, 1064), (1066, 1068), (1067, 1068), (1070, 1071), (1071, 1072), (1073, 1074), (1075, 1076), (1076, 1077), (1078, 1080), (1078, 1081), (1079, 1081), (1083, 1085), (1083, 1086), (1087, 1088), (1089, 1090), (1093, 1095), (1094, 1095), (1097, 1098), (1101, 1102), (1103, 1107), (1104, 1105), (1111, 1114), (1112, 1113), (1112, 1114), (1116, 1117), (1118, 1119), (1119, 1120), (1120, 1121), (1122, 1126), (1124, 1125), (1125, 1126), (1128, 1129), (1132, 1134), (1136, 1137), (1138, 1139), (1139, 1140), (1141, 1144), (1142, 1144), (1143, 1150), (1144, 1145), (1144, 1148), (1144, 1149), (1153, 1154), (1153, 1155), (1158, 1159), (1160, 1161), (1160, 1164), (1162, 1163), (1167, 1168), (1169, 1172), (1170, 1171), (1170, 1172), (1170, 1173), (1175, 1176), (1179, 1181), (1182, 1185), (1184, 1185), (1186, 1187), (1190, 1191), (1190, 1192), (1193, 1194), (1196, 1197), (1196, 1198), (1202, 1204), (1203, 1204), (1207, 1209), (1208, 1210), (1209, 1210), (1211, 1212), (1213, 1214), (1218, 1219), (1220, 1221), (1223, 1224), (1225, 1227), (1226, 1227), (1230, 1231), (1230, 1232), (1234, 1236), (1234, 1239), (1238, 1239), (1239, 1240), (1240, 1241), (1242, 1243), (1245, 1248), (1246, 1248), (1247, 1248), (1254, 1255), (1257, 1259), (1258, 1259), (1261, 1263), (1261, 1265), (1262, 1263), (1263, 1264), (1265, 1266), (1273, 1274), (1276, 1277), (1278, 1282), (1279, 1282), (1280, 1284), (1281, 1282), (1281, 1284), (1286, 1295), (1287, 1295), (1288, 1293), (1289, 1295), (1290, 1291), (1290, 1292), (1290, 1294), (1291, 1293), (1300, 1301), (1305, 1308), (1306, 1307), (1306, 1308), (1307, 1309), (1313, 1314), (1314, 1315), (1316, 1317), (1316, 1318), (1319, 1320), (1319, 1323), (1320, 1324), (1321, 1322), (1321, 1323), (1325, 1329), (1326, 1334), (1327, 1335), (1328, 1335), (1328, 1337), (1329, 1335), (1330, 1332), (1331, 1339), (1338, 1339), (1341, 1346), (1342, 1344), (1343, 1346), (1344, 1348), (1346, 1347), (1347, 1348), (1352, 1353), (1354, 1355), (1355, 1356), (1360, 1361), (1360, 1363), (1364, 1368), (1365, 1366), (1365, 1367), (1365, 1368), (1371, 1373), (1371, 1374), (1372, 1373), (1376, 1377), (1381, 1382), (1383, 1384), (1386, 1387), (1389, 1390), (1389, 1395), (1390, 1397), (1391, 1393), (1391, 1394), (1392, 1396), (1392, 1397), (1398, 1400), (1398, 1402), (1399, 1400), (1399, 1401), (1404, 1406), (1405, 1406), (1407, 1409), (1410, 1411), (1411, 1412), (1413, 1414), (1417, 1418), (1421, 1427), (1421, 1428), (1421, 1429), (1422, 1429), (1423, 1429), (1424, 1425), (1424, 1428), (1431, 1440), (1432, 1435), (1432, 1437), (1432, 1442), (1433, 1437), (1434, 1439), (1434, 1441), (1436, 1441), (1437, 1438), (1443, 1444), (1446, 1449), (1448, 1449), (1450, 1451), (1454, 1455), (1457, 1458), (1466, 1467), (1469, 1470), (1477, 1478), (1479, 1480), (1483, 1485), (1484, 1486), (1485, 1487), (1486, 1487), (1489, 1490), (1491, 1492), (1493, 1496), (1494, 1497), (1495, 1497), (1500, 1507), (1500, 1508), (1500, 1511), (1501, 1505), (1502, 1511), (1503, 1515), (1504, 1514), (1506, 1509), (1507, 1512), (1509, 1515), (1510, 1511), (1511, 1513), (1513, 1515), (1516, 1517), (1517, 1518), (1521, 1522), (1533, 1534), (1536, 1538), (1537, 1538), (1539, 1540), (1539, 1542), (1546, 1547), (1548, 1549), (1550, 1551), (1554, 1555), (1562, 1564), (1563, 1564), (1567, 1569), (1571, 1572), (1574, 1575), (1575, 1576), (1575, 1577), (1579, 1580), (1584, 1585), (1586, 1588), (1587, 1588), (1591, 1592), (1593, 1595), (1594, 1595), (1597, 1598), (1605, 1606), (1608, 1609), (1610, 1611), (1611, 1612), (1615, 1616), (1619, 1621), (1619, 1623), (1620, 1623), (1621, 1622), (1628, 1629), (1628, 1632), (1630, 1631), (1634, 1635), (1639, 1640), (1641, 1642), (1641, 1646), (1643, 1644), (1644, 1645), (1647, 1648), (1653, 1655), (1661, 1666), (1662, 1663), (1662, 1665), (1664, 1665), (1667, 1668), (1669, 1670), (1672, 1673), (1676, 1677), (1679, 1680), (1681, 1683), (1681, 1684), (1682, 1684), (1684, 1685), (1686, 1687), (1691, 1692), (1694, 1699), (1696, 1698), (1696, 1699), (1702, 1704), (1703, 1704), (1705, 1706), (1710, 1711), (1713, 1714), (1714, 1715), (1718, 1719), (1720, 1722), (1725, 1726), (1727, 1728), (1729, 1730), (1732, 1733), (1736, 1737), (1741, 1743), (1742, 1743), (1745, 1746), (1746, 1748), (1747, 1748), (1751, 1752), (1754, 1755), (1756, 1757), (1756, 1759), (1757, 1758), (1768, 1770), (1768, 1771), (1769, 1771), (1775, 1776), (1779, 1780), (1785, 1786), (1791, 1792), (1796, 1797), (1798, 1799), (1800, 1801), (1802, 1803), (1813, 1814), (1816, 1817), (1820, 1821), (1820, 1822), (1824, 1825), (1840, 1844), (1841, 1842), (1842, 1843), (1843, 1844), (1846, 1848), (1847, 1851), (1849, 1850), (1850, 1851), (1869, 1870), (1870, 1871), (1872, 1873), (1875, 1876), (1878, 1879), (1879, 1880), (1881, 1882), (1895, 1896), (1897, 1898), (1900, 1901), (1902, 1903), (1904, 1905), (1907, 1908), (1909, 1910), (1913, 1914), (1920, 1921), (1933, 1934), (1936, 1937), (1939, 1940), (1943, 1945), (1944, 1945), (1946, 1947), (1956, 1957), (1957, 1958), (1959, 1960), (1963, 1964), (1964, 1965), (1966, 1967), (1969, 1970), (1972, 1973), (1976, 1977), (1985, 1986), (2008, 2009), (2015, 2016), (2029, 2030), (2033, 2034), (2051, 2052), (2054, 2055), (2086, 2087), (2114, 2115), (2123, 2124)]
wells_mapping= {1: 176, 2: 177, 3: 178, 4: 179, 5: 180, 6: 181, 7: 182, 8: 183, 9: 184, 10: 185, 11: 186, 12: 187, 13: 188, 14: 189, 15: 190, 16: 191, 17: 192, 18: 193, 19: 194, 20: 195, 21: 196, 22: 197, 23: 198, 24: 199, 25: 200, 26: 201, 27: 202, 28: 203, 29: 204, 30: 205, 31: 206, 32: 207, 33: 208, 34: 209, 35: 210, 36: 211, 37: 212, 38: 213, 39: 214, 40: 215, 41: 216, 42: 217, 43: 218, 44: 219, 45: 220, 46: 221, 47: 222, 48: 223, 49: 224, 50: 225, 51: 226, 52: 227, 53: 228, 54: 229, 55: 230, 56: 231, 57: 232, 58: 233, 59: 234, 60: 235, 61: 236, 62: 237, 63: 238, 64: 239, 65: 240, 66: 241, 67: 242, 68: 243, 69: 244, 70: 245, 71: 246, 72: 247, 73: 248, 74: 249, 75: 250, 76: 251, 77: 252, 78: 253, 79: 254, 80: 255, 81: 256, 82: 257, 83: 258, 84: 259, 85: 260, 86: 261, 87: 262, 88: 263, 89: 264, 90: 265, 91: 266, 92: 267, 93: 268, 94: 269, 95: 270, 96: 271, 97: 272, 98: 273, 99: 274, 100: 275, 101: 276, 102: 277, 103: 868, 104: 869, 105: 870, 106: 871, 107: 984, 108: 985, 109: 982, 110: 983, 111: 672, 112: 673, 113: 674, 114: 675, 115: 676, 116: 677, 117: 678, 118: 679, 119: 667, 120: 668, 121: 669, 122: 670, 123: 671, 124: 1609, 125: 1610, 126: 1611, 127: 942, 128: 943, 129: 944, 130: 945, 131: 946, 132: 947, 133: 948, 134: 949, 135: 950, 136: 951, 137: 952, 138: 953, 139: 954, 140: 955, 141: 956, 142: 957, 143: 1823, 144: 1443, 145: 1711, 146: 1330, 147: 1331, 148: 1332, 149: 1333, 150: 1334, 151: 1335, 152: 1596, 153: 1864, 154: 1803, 155: 1804, 156: 1805, 157: 1806, 158: 1933, 159: 1673, 160: 640, 161: 641, 162: 642, 163: 628, 164: 629, 165: 630, 166: 631, 167: 632, 168: 633, 169: 634, 170: 635, 171: 636, 172: 637, 173: 638, 174: 639, 175: 1678, 176: 832, 177: 830, 178: 831, 179: 352, 180: 353, 181: 354, 182: 355, 183: 356, 184: 357, 185: 358, 186: 344, 187: 345, 188: 346, 189: 347, 190: 348, 191: 349, 192: 350, 193: 351, 194: 2156, 195: 1367, 196: 1280, 197: 1281, 198: 1282, 199: 512, 200: 513, 201: 514, 202: 515, 203: 516, 204: 517, 205: 518, 206: 519, 207: 520, 208: 521, 209: 522, 210: 523, 211: 524, 212: 525, 213: 526, 214: 527, 215: 528, 216: 529, 217: 530, 218: 531, 219: 532, 220: 533, 221: 534, 222: 535, 223: 536, 224: 537, 225: 538, 226: 539, 227: 540, 228: 541, 229: 542, 230: 543, 231: 544, 232: 545, 233: 546, 234: 547, 235: 548, 236: 549, 237: 550, 238: 551, 239: 552, 240: 553, 241: 554, 242: 555, 243: 556, 244: 557, 245: 558, 246: 559, 247: 560, 248: 561, 249: 562, 250: 563, 251: 564, 252: 565, 253: 566, 254: 567, 255: 568, 256: 569, 257: 570, 258: 571, 259: 572, 260: 573, 261: 574, 262: 575, 263: 576, 264: 577, 265: 578, 266: 579, 267: 580, 268: 581, 269: 582, 270: 583, 271: 584, 272: 585, 273: 586, 274: 587, 275: 588, 276: 589, 277: 590, 278: 591, 279: 592, 280: 593, 281: 594, 282: 595, 283: 596, 284: 597, 285: 598, 286: 599, 287: 600, 288: 601, 289: 462, 290: 463, 291: 464, 292: 465, 293: 466, 294: 467, 295: 468, 296: 469, 297: 470, 298: 471, 299: 472, 300: 473, 301: 474, 302: 475, 303: 476, 304: 477, 305: 478, 306: 479, 307: 480, 308: 481, 309: 482, 310: 483, 311: 484, 312: 485, 313: 486, 314: 487, 315: 488, 316: 489, 317: 490, 318: 491, 319: 492, 320: 493, 321: 494, 322: 495, 323: 496, 324: 497, 325: 498, 326: 499, 327: 500, 328: 501, 329: 502, 330: 503, 331: 504, 332: 505, 333: 506, 334: 507, 335: 508, 336: 509, 337: 510, 338: 511, 339: 1226, 340: 1227, 341: 1228, 342: 1229, 343: 1594, 344: 1712, 345: 1713, 346: 1714, 347: 1715, 348: 1184, 349: 1185, 350: 1186, 351: 1187, 352: 1188, 353: 1183, 354: 1176, 355: 1177, 356: 1175, 357: 2057, 358: 680, 359: 681, 360: 682, 361: 683, 362: 684, 363: 685, 364: 686, 365: 687, 366: 1216, 367: 1217, 368: 1218, 369: 1219, 370: 1220, 371: 1212, 372: 1213, 373: 1214, 374: 1215, 375: 1017, 376: 1018, 377: 1019, 378: 1020, 379: 1817, 380: 2048, 381: 2049, 382: 2050, 383: 1869, 384: 872, 385: 873, 386: 874, 387: 1516, 388: 1517, 389: 1518, 390: 1519, 391: 1520, 392: 1521, 393: 1522, 394: 32, 395: 33, 396: 34, 397: 35, 398: 36, 399: 37, 400: 38, 401: 39, 402: 40, 403: 27, 404: 28, 405: 29, 406: 30, 407: 31, 408: 1132, 409: 1133, 410: 1134, 411: 1135, 412: 1136, 413: 968, 414: 969, 415: 977, 416: 16, 417: 14, 418: 15, 419: 2012, 420: 1287, 421: 1288, 422: 1289, 423: 1290, 424: 1291, 425: 1292, 426: 1293, 427: 1294, 428: 896, 429: 887, 430: 888, 431: 889, 432: 890, 433: 891, 434: 892, 435: 893, 436: 894, 437: 895, 438: 1396, 439: 905, 440: 906, 441: 907, 442: 908, 443: 909, 444: 910, 445: 911, 446: 912, 447: 913, 448: 914, 449: 915, 450: 916, 451: 1, 452: 2, 453: 3, 454: 4, 455: 5, 456: 6, 457: 7, 458: 8, 459: 9, 460: 10, 461: 11, 462: 12, 463: 13, 464: 902, 465: 1315, 466: 1316, 467: 1317, 468: 1318, 469: 1319, 470: 1320, 471: 1321, 472: 1322, 473: 1323, 474: 1872, 475: 1573, 476: 1637, 477: 1638, 478: 1639, 479: 1900, 480: 1041, 481: 1093, 482: 1094, 483: 1095, 484: 1409, 485: 1410, 486: 1784, 487: 1785, 488: 400, 489: 401, 490: 402, 491: 403, 492: 404, 493: 405, 494: 406, 495: 407, 496: 408, 497: 409, 498: 410, 499: 411, 500: 412, 501: 413, 502: 414, 503: 415, 504: 416, 505: 417, 506: 418, 507: 419, 508: 420, 509: 421, 510: 1360, 511: 1361, 512: 1358, 513: 1359, 514: 1037, 515: 1672, 516: 1669, 517: 1670, 518: 1671, 519: 1051, 520: 1052, 521: 1053, 522: 659, 523: 660, 524: 661, 525: 662, 526: 663, 527: 1110, 528: 1111, 529: 1112, 530: 1113, 531: 1114, 532: 1964, 533: 1965, 534: 140, 535: 141, 536: 142, 537: 143, 538: 144, 539: 145, 540: 146, 541: 147, 542: 148, 543: 149, 544: 150, 545: 151, 546: 152, 547: 153, 548: 154, 549: 155, 550: 156, 551: 2160, 552: 448, 553: 449, 554: 438, 555: 439, 556: 440, 557: 441, 558: 442, 559: 443, 560: 444, 561: 445, 562: 446, 563: 447, 564: 2144, 565: 1699, 566: 1946, 567: 1731, 568: 1732, 569: 1799, 570: 800, 571: 801, 572: 792, 573: 793, 574: 794, 575: 795, 576: 796, 577: 797, 578: 798, 579: 799, 580: 1413, 581: 1905, 582: 1822, 583: 70, 584: 71, 585: 72, 586: 73, 587: 74, 588: 75, 589: 76, 590: 77, 591: 78, 592: 79, 593: 80, 594: 81, 595: 82, 596: 83, 597: 84, 598: 85, 599: 86, 600: 87, 601: 88, 602: 89, 603: 90, 604: 91, 605: 92, 606: 93, 607: 94, 608: 95, 609: 96, 610: 97, 611: 98, 612: 99, 613: 100, 614: 101, 615: 102, 616: 103, 617: 104, 618: 105, 619: 106, 620: 107, 621: 108, 622: 109, 623: 110, 624: 111, 625: 112, 626: 113, 627: 114, 628: 115, 629: 116, 630: 117, 631: 118, 632: 119, 633: 120, 634: 2061, 635: 1648, 636: 1649, 637: 1647, 638: 1556, 639: 1268, 640: 1269, 641: 1980, 642: 1981, 643: 1982, 644: 1983, 645: 1393, 646: 1394, 647: 1395, 648: 1342, 649: 1878, 650: 1879, 651: 64, 652: 65, 653: 66, 654: 67, 655: 68, 656: 69, 657: 53, 658: 54, 659: 55, 660: 56, 661: 57, 662: 58, 663: 59, 664: 60, 665: 61, 666: 62, 667: 63, 668: 2002, 669: 2003, 670: 1700, 671: 1701, 672: 1702, 673: 776, 674: 777, 675: 778, 676: 779, 677: 780, 678: 781, 679: 782, 680: 783, 681: 784, 682: 785, 683: 1593, 684: 453, 685: 454, 686: 455, 687: 456, 688: 457, 689: 458, 690: 459, 691: 460, 692: 461, 693: 768, 694: 769, 695: 770, 696: 771, 697: 772, 698: 773, 699: 774, 700: 775, 701: 758, 702: 759, 703: 760, 704: 761, 705: 762, 706: 763, 707: 764, 708: 765, 709: 766, 710: 767, 711: 280, 712: 281, 713: 282, 714: 283, 715: 284, 716: 285, 717: 286, 718: 287, 719: 288, 720: 289, 721: 290, 722: 291, 723: 292, 724: 293, 725: 294, 726: 295, 727: 296, 728: 297, 729: 298, 730: 299, 731: 300, 732: 301, 733: 302, 734: 303, 735: 304, 736: 305, 737: 306, 738: 307, 739: 308, 740: 309, 741: 310, 742: 311, 743: 312, 744: 313, 745: 314, 746: 315, 747: 316, 748: 317, 749: 318, 750: 319, 751: 320, 752: 321, 753: 359, 754: 360, 755: 361, 756: 362, 757: 363, 758: 364, 759: 365, 760: 366, 761: 367, 762: 368, 763: 369, 764: 370, 765: 371, 766: 372, 767: 373, 768: 374, 769: 375, 770: 376, 771: 377, 772: 378, 773: 379, 774: 380, 775: 381, 776: 382, 777: 756, 778: 757, 779: 2001, 780: 1654, 781: 1655, 782: 1728, 783: 1729, 784: 1730, 785: 1115, 786: 1116, 787: 2132, 788: 325, 789: 326, 790: 327, 791: 328, 792: 329, 793: 330, 794: 331, 795: 332, 796: 333, 797: 334, 798: 335, 799: 336, 800: 337, 801: 338, 802: 339, 803: 340, 804: 341, 805: 342, 806: 343, 807: 175, 808: 1667, 809: 1875, 810: 41, 811: 42, 812: 43, 813: 44, 814: 45, 815: 46, 816: 47, 817: 48, 818: 49, 819: 50, 820: 51, 821: 1346, 822: 1347, 823: 1348, 824: 1349, 825: 1350, 826: 1351, 827: 1352, 828: 1353, 829: 1354, 830: 1355, 831: 1079, 832: 1139, 833: 1577, 834: 1937, 835: 128, 836: 129, 837: 130, 838: 131, 839: 132, 840: 133, 841: 134, 842: 135, 843: 136, 844: 137, 845: 138, 846: 126, 847: 127, 848: 1985, 849: 1986, 850: 1863, 851: 1916, 852: 813, 853: 814, 854: 815, 855: 816, 856: 817, 857: 818, 858: 819, 859: 820, 860: 821, 861: 822, 862: 823, 863: 824, 864: 825, 865: 1744, 866: 1743, 867: 1625, 868: 657, 869: 658, 870: 930, 871: 931, 872: 932, 873: 933, 874: 934, 875: 1632, 876: 1633, 877: 1634, 878: 1635, 879: 1636, 880: 2085, 881: 809, 882: 1408, 883: 1401, 884: 1402, 885: 1403, 886: 1404, 887: 1405, 888: 1406, 889: 1407, 890: 987, 891: 988, 892: 790, 893: 791, 894: 1761, 895: 1762, 896: 2155, 897: 1011, 898: 1814, 899: 1815, 900: 1232, 901: 1233, 902: 1234, 903: 1235, 904: 1236, 905: 1237, 906: 1238, 907: 1239, 908: 1240, 909: 1241, 910: 1242, 911: 1243, 912: 1244, 913: 1245, 914: 1478, 915: 690, 916: 691, 917: 692, 918: 693, 919: 694, 920: 695, 921: 696, 922: 697, 923: 1356, 924: 1602, 925: 1603, 926: 1604, 927: 1605, 928: 1368, 929: 1873, 930: 1099, 931: 1100, 932: 1101, 933: 2122, 934: 425, 935: 426, 936: 427, 937: 428, 938: 429, 939: 1012, 940: 1013, 941: 1014, 942: 1015, 943: 1016, 944: 1540, 945: 1541, 946: 1542, 947: 1300, 948: 1301, 949: 1302, 950: 1303, 951: 1304, 952: 1305, 953: 1103, 954: 965, 955: 966, 956: 1925, 957: 986, 958: 1253, 959: 1254, 960: 1255, 961: 1256, 962: 1257, 963: 1171, 964: 1172, 965: 1173, 966: 1174, 967: 430, 968: 1813, 969: 2027, 970: 1084, 971: 897, 972: 604, 973: 605, 974: 606, 975: 607, 976: 608, 977: 609, 978: 610, 979: 611, 980: 612, 981: 613, 982: 614, 983: 615, 984: 616, 985: 617, 986: 618, 987: 619, 988: 620, 989: 621, 990: 622, 991: 623, 992: 624, 993: 625, 994: 626, 995: 1780, 996: 1781, 997: 1782, 998: 833, 999: 834, 1000: 904, 1001: 903, 1002: 121, 1003: 122, 1004: 123, 1005: 124, 1006: 125, 1007: 1599, 1008: 1624, 1009: 1754, 1010: 1662, 1011: 1663, 1012: 1196, 1013: 1376, 1014: 1377, 1015: 1378, 1016: 1379, 1017: 1380, 1018: 1381, 1019: 1382, 1020: 1383, 1021: 1384, 1022: 1385, 1023: 1386, 1024: 1387, 1025: 1372, 1026: 1373, 1027: 1374, 1028: 1375, 1029: 1952, 1030: 715, 1031: 716, 1032: 717, 1033: 718, 1034: 719, 1035: 720, 1036: 721, 1037: 722, 1038: 723, 1039: 724, 1040: 725, 1041: 726, 1042: 727, 1043: 728, 1044: 729, 1045: 730, 1046: 731, 1047: 732, 1048: 733, 1049: 734, 1050: 735, 1051: 736, 1052: 737, 1053: 738, 1054: 739, 1055: 740, 1056: 827, 1057: 828, 1058: 1338, 1059: 1339, 1060: 900, 1061: 901, 1062: 1907, 1063: 2081, 1064: 2082, 1065: 1168, 1066: 1169, 1067: 1170, 1068: 1167, 1069: 2123, 1070: 1874, 1071: 2111, 1072: 841, 1073: 2080, 1074: 842, 1075: 1626, 1076: 1911, 1077: 788, 1078: 2092, 1079: 1068, 1080: 1069, 1081: 1070, 1082: 1071, 1083: 1072, 1084: 1073, 1085: 1074, 1086: 1446, 1087: 704, 1088: 705, 1089: 706, 1090: 707, 1091: 708, 1092: 699, 1093: 700, 1094: 701, 1095: 702, 1096: 703, 1097: 994, 1098: 995, 1099: 996, 1100: 997, 1101: 998, 1102: 999, 1103: 1000, 1104: 1001, 1105: 1002, 1106: 1003, 1107: 1004, 1108: 1005, 1109: 843, 1110: 844, 1111: 845, 1112: 846, 1113: 847, 1114: 848, 1115: 1038, 1116: 1039, 1117: 2031, 1118: 1357, 1119: 644, 1120: 645, 1121: 646, 1122: 647, 1123: 648, 1124: 649, 1125: 650, 1126: 651, 1127: 652, 1128: 1906, 1129: 917, 1130: 918, 1131: 919, 1132: 920, 1133: 921, 1134: 992, 1135: 993, 1136: 989, 1137: 990, 1138: 991, 1139: 1500, 1140: 2055, 1141: 1583, 1142: 741, 1143: 742, 1144: 743, 1145: 744, 1146: 745, 1147: 1123, 1148: 1124, 1149: 1125, 1150: 1126, 1151: 1127, 1152: 1128, 1153: 1129, 1154: 1130, 1155: 1531, 1156: 1208, 1157: 1209, 1158: 1484, 1159: 1640, 1160: 1641, 1161: 1642, 1162: 1643, 1163: 1768, 1164: 1769, 1165: 1770, 1166: 1779, 1167: 1200, 1168: 1201, 1169: 1202, 1170: 1880, 1171: 1881, 1172: 1056, 1173: 1057, 1174: 1949, 1175: 1840, 1176: 1841, 1177: 1842, 1178: 1843, 1179: 160, 1180: 161, 1181: 162, 1182: 163, 1183: 164, 1184: 165, 1185: 166, 1186: 167, 1187: 168, 1188: 169, 1189: 170, 1190: 171, 1191: 172, 1192: 173, 1193: 174, 1194: 157, 1195: 158, 1196: 159, 1197: 2139, 1198: 884, 1199: 885, 1200: 1523, 1201: 1524, 1202: 1846, 1203: 1260, 1204: 1261, 1205: 1262, 1206: 1263, 1207: 1264, 1208: 1152, 1209: 1148, 1210: 1149, 1211: 1150, 1212: 1151, 1213: 1707, 1214: 1501, 1215: 1502, 1216: 1552, 1217: 1550, 1218: 1551, 1219: 1574, 1220: 1575, 1221: 1192, 1222: 1193, 1223: 1597, 1224: 787, 1225: 747, 1226: 748, 1227: 749, 1228: 1120, 1229: 1121, 1230: 1122, 1231: 2113, 1232: 1204, 1233: 1205, 1234: 1206, 1235: 2125, 1236: 1755, 1237: 1008, 1238: 1009, 1239: 1007, 1240: 1434, 1241: 1435, 1242: 1436, 1243: 1437, 1244: 1438, 1245: 1107, 1246: 1108, 1247: 1109, 1248: 1990, 1249: 1848, 1250: 1849, 1251: 1850, 1252: 1425, 1253: 1426, 1254: 1427, 1255: 1428, 1256: 1429, 1257: 2089, 1258: 2090, 1259: 2091, 1260: 1914, 1261: 1915, 1262: 1096, 1263: 1097, 1264: 436, 1265: 437, 1266: 2036, 1267: 398, 1268: 1081, 1269: 2041, 1270: 2042, 1271: 1142, 1272: 1920, 1273: 1919, 1274: 2102, 1275: 664, 1276: 665, 1277: 666, 1278: 2015, 1279: 1612, 1280: 1613, 1281: 1614, 1282: 1197, 1283: 1198, 1284: 1457, 1285: 1576, 1286: 1657, 1287: 1658, 1288: 1659, 1289: 1660, 1290: 139, 1291: 1578, 1292: 1224, 1293: 1223, 1294: 1793, 1295: 1794, 1296: 1795, 1297: 1796, 1298: 1797, 1299: 21, 1300: 22, 1301: 23, 1302: 24, 1303: 25, 1304: 26, 1305: 1486, 1306: 1487, 1307: 2059, 1308: 1716, 1309: 1717, 1310: 424, 1311: 1991, 1312: 1221, 1313: 392, 1314: 393, 1315: 394, 1316: 395, 1317: 396, 1318: 397, 1319: 2103, 1320: 1810, 1321: 1811, 1322: 1158, 1323: 2094, 1324: 2069, 1325: 2098, 1326: 1630, 1327: 2079, 1328: 1748, 1329: 1749, 1330: 1474, 1331: 1475, 1332: 1476, 1333: 1466, 1334: 1467, 1335: 1468, 1336: 1058, 1337: 1507, 1338: 1508, 1339: 1581, 1340: 970, 1341: 971, 1342: 972, 1343: 973, 1344: 974, 1345: 975, 1346: 322, 1347: 323, 1348: 324, 1349: 1337, 1350: 1901, 1351: 1040, 1352: 1513, 1353: 1459, 1354: 1460, 1355: 1032, 1356: 1033, 1357: 1034, 1358: 1511, 1359: 422, 1360: 423, 1361: 786, 1362: 1266, 1363: 1267, 1364: 1993, 1365: 1526, 1366: 1527, 1367: 1493, 1368: 1494, 1369: 1495, 1370: 1787, 1371: 1546, 1372: 1547, 1373: 1548, 1374: 1549, 1375: 1586, 1376: 1587, 1377: 1896, 1378: 1897, 1379: 1895, 1380: 2137, 1381: 1572, 1382: 1816, 1383: 1858, 1384: 1859, 1385: 1935, 1386: 2120, 1387: 2121, 1388: 1584, 1389: 713, 1390: 714, 1391: 2066, 1392: 922, 1393: 923, 1394: 924, 1395: 925, 1396: 926, 1397: 927, 1398: 1207, 1399: 1752, 1400: 1753, 1401: 1751, 1402: 1876, 1403: 1877, 1404: 1674, 1405: 1536, 1406: 1537, 1407: 1538, 1408: 857, 1409: 858, 1410: 859, 1411: 860, 1412: 861, 1413: 862, 1414: 2038, 1415: 1809, 1416: 1365, 1417: 1366, 1418: 1588, 1419: 1589, 1420: 1590, 1421: 1807, 1422: 2043, 1423: 1928, 1424: 1927, 1425: 1258, 1426: 1259, 1427: 1756, 1428: 1757, 1429: 1680, 1430: 1679, 1431: 1825, 1432: 1736, 1433: 1735, 1434: 1989, 1435: 1818, 1436: 1696, 1437: 1697, 1438: 1698, 1439: 1695, 1440: 1998, 1441: 1999, 1442: 627, 1443: 2044, 1444: 1847, 1445: 1760, 1446: 1766, 1447: 1075, 1448: 1076, 1449: 864, 1450: 865, 1451: 866, 1452: 863, 1453: 1862, 1454: 1677, 1455: 1791, 1456: 1450, 1457: 2040, 1458: 1973, 1459: 1974, 1460: 1975, 1461: 602, 1462: 603, 1463: 840, 1464: 837, 1465: 838, 1466: 839, 1467: 1839, 1468: 1140, 1469: 2131, 1470: 1720, 1471: 1721, 1472: 1722, 1473: 1272, 1474: 1273, 1475: 1274, 1476: 1972, 1477: 876, 1478: 1159, 1479: 935, 1480: 936, 1481: 937, 1482: 938, 1483: 939, 1484: 1557, 1485: 959, 1486: 1819, 1487: 1950, 1488: 1951, 1489: 1021, 1490: 1022, 1491: 1571, 1492: 2056, 1493: 1035, 1494: 1090, 1495: 1091, 1496: 1092, 1497: 1719, 1498: 1043, 1499: 1044, 1500: 2004, 1501: 1085, 1502: 1802, 1503: 2070, 1504: 1143, 1505: 2022, 1506: 1059, 1507: 1060, 1508: 1061, 1509: 1062, 1510: 1063, 1511: 1064, 1512: 1618, 1513: 1844, 1514: 1845, 1515: 1440, 1516: 1439, 1517: 1800, 1518: 386, 1519: 387, 1520: 388, 1521: 1163, 1522: 1164, 1523: 1165, 1524: 1166, 1525: 1554, 1526: 1555, 1527: 434, 1528: 435, 1529: 1418, 1530: 1419, 1531: 1420, 1532: 1421, 1533: 1422, 1534: 1423, 1535: 1688, 1536: 1689, 1537: 1098, 1538: 851, 1539: 852, 1540: 853, 1541: 854, 1542: 855, 1543: 1994, 1544: 1852, 1545: 384, 1546: 385, 1547: 2095, 1548: 1045, 1549: 1046, 1550: 1047, 1551: 1048, 1552: 1049, 1553: 1910, 1554: 1966, 1555: 1967, 1556: 656, 1557: 653, 1558: 654, 1559: 655, 1560: 1909, 1561: 1248, 1562: 1249, 1563: 1250, 1564: 1251, 1565: 1252, 1566: 1247, 1567: 849, 1568: 850, 1569: 1222, 1570: 2108, 1571: 1411, 1572: 1412, 1573: 1691, 1574: 1088, 1575: 1089, 1576: 1026, 1577: 1827, 1578: 1828, 1579: 1829, 1580: 1560, 1581: 1561, 1582: 1562, 1583: 1559, 1584: 928, 1585: 1448, 1586: 1131, 1587: 1362, 1588: 1661, 1589: 1608, 1590: 1607, 1591: 1027, 1592: 278, 1593: 279, 1594: 886, 1595: 1742, 1596: 1024, 1597: 1023, 1598: 1944, 1599: 1945, 1600: 1943, 1601: 1515, 1602: 1496, 1603: 1497, 1604: 1498, 1605: 1181, 1606: 1182, 1607: 1106, 1608: 802, 1609: 803, 1610: 804, 1611: 805, 1612: 806, 1613: 1884, 1614: 1441, 1615: 1458, 1616: 1913, 1617: 2005, 1618: 2006, 1619: 2106, 1620: 1758, 1621: 2024, 1622: 2023, 1623: 1579, 1624: 1580, 1625: 1563, 1626: 1564, 1627: 1565, 1628: 1566, 1629: 1567, 1630: 1893, 1631: 1480, 1632: 1481, 1633: 1479, 1634: 1324, 1635: 1959, 1636: 1776, 1637: 1777, 1638: 712, 1639: 711, 1640: 1930, 1641: 1104, 1642: 1432, 1643: 1433, 1644: 2093, 1645: 1996, 1646: 881, 1647: 882, 1648: 883, 1649: 976, 1650: 1312, 1651: 1313, 1652: 1314, 1653: 1306, 1654: 1307, 1655: 1308, 1656: 1309, 1657: 1310, 1658: 1311, 1659: 1137, 1660: 929, 1661: 389, 1662: 390, 1663: 391, 1664: 1416, 1665: 1415, 1666: 1737, 1667: 1738, 1668: 1739, 1669: 1740, 1670: 1692, 1671: 1693, 1672: 450, 1673: 451, 1674: 1687, 1675: 1528, 1676: 1734, 1677: 2116, 1678: 1835, 1679: 1836, 1680: 1837, 1681: 1838, 1682: 688, 1683: 689, 1684: 1464, 1685: 1465, 1686: 1449, 1687: 1482, 1688: 1745, 1689: 1746, 1690: 1723, 1691: 2127, 1692: 1976, 1693: 1851, 1694: 1606, 1695: 1968, 1696: 1969, 1697: 1870, 1698: 1871, 1699: 1675, 1700: 1676, 1701: 1388, 1702: 1265, 1703: 1279, 1704: 1203, 1705: 1491, 1706: 1492, 1707: 1861, 1708: 2077, 1709: 1898, 1710: 1705, 1711: 1706, 1712: 829, 1713: 1452, 1714: 1453, 1715: 1646, 1716: 1718, 1717: 2136, 1718: 1455, 1719: 1544, 1720: 1545, 1721: 1543, 1722: 1668, 1723: 1363, 1724: 1364, 1725: 1031, 1726: 1860, 1727: 1771, 1728: 1772, 1729: 1773, 1730: 1774, 1731: 1775, 1732: 1077, 1733: 1078, 1734: 880, 1735: 877, 1736: 878, 1737: 879, 1738: 1600, 1739: 2086, 1740: 1472, 1741: 1473, 1742: 1469, 1743: 1470, 1744: 1471, 1745: 2020, 1746: 1957, 1747: 1958, 1748: 1820, 1749: 1709, 1750: 1710, 1751: 1867, 1752: 1868, 1753: 2101, 1754: 1694, 1755: 1918, 1756: 1509, 1757: 1510, 1758: 2112, 1759: 1525, 1760: 750, 1761: 751, 1762: 1325, 1763: 1832, 1764: 1833, 1765: 1834, 1766: 1997, 1767: 2034, 1768: 826, 1769: 1992, 1770: 1947, 1771: 1948, 1772: 810, 1773: 811, 1774: 812, 1775: 1953, 1776: 1763, 1777: 1006, 1778: 1955, 1779: 1664, 1780: 1665, 1781: 1144, 1782: 1145, 1783: 1146, 1784: 1147, 1785: 1430, 1786: 1431, 1787: 1962, 1788: 2109, 1789: 1666, 1790: 1054, 1791: 1055, 1792: 2062, 1793: 1194, 1794: 2147, 1795: 1620, 1796: 1190, 1797: 1650, 1798: 1651, 1799: 1652, 1800: 1246, 1801: 1656, 1802: 2126, 1803: 2051, 1804: 1340, 1805: 1341, 1806: 1902, 1807: 1903, 1808: 1798, 1809: 2033, 1810: 752, 1811: 1725, 1812: 2152, 1813: 2134, 1814: 1854, 1815: 1855, 1816: 1826, 1817: 2030, 1818: 2073, 1819: 432, 1820: 433, 1821: 431, 1822: 1923, 1823: 1924, 1824: 1050, 1825: 1328, 1826: 1326, 1827: 1327, 1828: 808, 1829: 1912, 1830: 2128, 1831: 958, 1832: 383, 1833: 1535, 1834: 2039, 1835: 1499, 1836: 2148, 1837: 2028, 1838: 2058, 1839: 1938, 1840: 1939, 1841: 1940, 1842: 2046, 1843: 2065, 1844: 2157, 1845: 1275, 1846: 1276, 1847: 1277, 1848: 2054, 1849: 1891, 1850: 899, 1851: 2158, 1852: 2097, 1853: 1601, 1854: 1830, 1855: 1831, 1856: 2078, 1857: 1160, 1858: 1161, 1859: 1162, 1860: 1296, 1861: 1987, 1862: 1988, 1863: 1616, 1864: 1447, 1865: 980, 1866: 981, 1867: 1514, 1868: 1397, 1869: 1786, 1870: 1189, 1871: 1066, 1872: 1067, 1873: 1489, 1874: 1490, 1875: 399, 1876: 1931, 1877: 1080, 1878: 1908, 1879: 1295, 1880: 1856, 1881: 2118, 1882: 898, 1883: 1036, 1884: 1456, 1885: 19, 1886: 20, 1887: 1445, 1888: 1941, 1889: 960, 1890: 961, 1891: 962, 1892: 963, 1893: 964, 1894: 1619, 1895: 2150, 1896: 835, 1897: 836, 1898: 1929, 1899: 1512, 1900: 2029, 1901: 1921, 1902: 1726, 1903: 1727, 1904: 1505, 1905: 1105, 1906: 1231, 1907: 1454, 1908: 709, 1909: 710, 1910: 1821, 1911: 1963, 1912: 2018, 1913: 2135, 1914: 1568, 1915: 1569, 1916: 1570, 1917: 2087, 1918: 979, 1919: 753, 1920: 754, 1921: 1414, 1922: 1970, 1923: 1971, 1924: 1885, 1925: 1886, 1926: 1887, 1927: 1082, 1928: 1558, 1929: 1488, 1930: 940, 1931: 1117, 1932: 1118, 1933: 1119, 1934: 2060, 1935: 2159, 1936: 2008, 1937: 1956, 1938: 1812, 1939: 1767, 1940: 1532, 1941: 1285, 1942: 1286, 1943: 1400, 1944: 1399, 1945: 1539, 1946: 1960, 1947: 1278, 1948: 2007, 1949: 1808, 1950: 1025, 1951: 1979, 1952: 1954, 1953: 1462, 1954: 1778, 1955: 1271, 1956: 1765, 1957: 1153, 1958: 1154, 1959: 1155, 1960: 1156, 1961: 1157, 1962: 1210, 1963: 1211, 1964: 1824, 1965: 1645, 1966: 1682, 1967: 1889, 1968: 1790, 1969: 2117, 1970: 1477, 1971: 1865, 1972: 2053, 1973: 1801, 1974: 2141, 1975: 2096, 1976: 2140, 1977: 1504, 1978: 1503, 1979: 1866, 1980: 2149, 1981: 1961, 1982: 1683, 1983: 1684, 1984: 1685, 1985: 1686, 1986: 1442, 1987: 2145, 1988: 1922, 1989: 1890, 1990: 2067, 1991: 2068, 1992: 967, 1993: 1270, 1994: 2076, 1995: 2146, 1996: 1617, 1997: 1984, 1998: 1936, 1999: 1750, 2000: 1028, 2001: 1029, 2002: 1030, 2003: 2133, 2004: 2115, 2005: 643, 2006: 1917, 2007: 2064, 2008: 1623, 2009: 1392, 2010: 1389, 2011: 1390, 2012: 1391, 2013: 1178, 2014: 1179, 2015: 1180, 2016: 1102, 2017: 875, 2018: 1621, 2019: 2037, 2020: 1336, 2021: 2047, 2022: 746, 2023: 1888, 2024: 698, 2025: 1615, 2026: 1788, 2027: 1195, 2028: 1582, 2029: 1444, 2030: 1789, 2031: 941, 2032: 2143, 2033: 2011, 2034: 1398, 2035: 2052, 2036: 2130, 2037: 1644, 2038: 1899, 2039: 1724, 2040: 1141, 2041: 2019, 2042: 1042, 2043: 1417, 2044: 1904, 2045: 1741, 2046: 2114, 2047: 1344, 2048: 1297, 2049: 1298, 2050: 1299, 2051: 2153, 2052: 2100, 2053: 867, 2054: 1485, 2055: 1329, 2056: 1553, 2057: 1764, 2058: 1371, 2059: 1995, 2060: 1792, 2061: 1894, 2062: 2014, 2063: 1977, 2064: 1978, 2065: 1747, 2066: 2021, 2067: 2138, 2068: 2063, 2069: 1230, 2070: 1932, 2071: 1585, 2072: 1631, 2073: 1087, 2074: 2010, 2075: 2035, 2076: 1653, 2077: 1622, 2078: 789, 2079: 2107, 2080: 1529, 2081: 1530, 2082: 1283, 2083: 2104, 2084: 2105, 2085: 1883, 2086: 1343, 2087: 2013, 2088: 2072, 2089: 2071, 2090: 2119, 2091: 1627, 2092: 1704, 2093: 1483, 2094: 2025, 2095: 1086, 2096: 2017, 2097: 2129, 2098: 1533, 2099: 1534, 2100: 1628, 2101: 1629, 2102: 1853, 2103: 2154, 2104: 1225, 2105: 1857, 2106: 1345, 2107: 755, 2108: 1733, 2109: 2083, 2110: 1708, 2111: 2124, 2112: 1591, 2113: 1703, 2114: 2151, 2115: 1065, 2116: 1463, 2117: 1592, 2118: 1369, 2119: 1370, 2120: 856, 2121: 1892, 2122: 2045, 2123: 2000, 2124: 1934, 2125: 2084, 2126: 1083, 2127: 1759, 2128: 2009, 2129: 18, 2130: 1010, 2131: 2032, 2132: 1191, 2133: 452, 2134: 2110, 2135: 1461, 2136: 17, 2137: 1451, 2138: 2074, 2139: 2075, 2140: 1424, 2141: 1681, 2142: 1882, 2143: 2099, 2144: 1690, 2145: 1783, 2146: 2016, 2147: 1598, 2148: 1926, 2149: 807, 2150: 2026, 2151: 52, 2152: 1138, 2153: 1506, 2154: 978, 2155: 1284, 2156: 2088, 2157: 2142, 2158: 1595, 2159: 1199, 2160: 1942}
res = 70052714
n,wells_dict_new,pipes_new,well_status_new,pipe_status_new = translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping)

pack2 = [wells_dict_new,pipes_new,well_status_new,pipe_status_new]


In [7]:
# # res2
# n = 2160
# wells = [42282, 2144, 33174, 85810, 86811, 5692, 30030, 28982, 69952, 93368, 97822, 58705, 64292, 12630, 37588, 71099, 91983, 68901, 54303, 16164, 27720, 84098, 70326, 56853, 62955, 88863, 70970, 73689, 34386, 98070, 84144, 70713, 86633, 97909, 86696, 58578, 53562, 73417, 94666, 24633, 92056, 63308, 53534, 25085, 64526, 53380, 20169, 58932, 55584, 3542, 15698, 19058, 38363, 67282, 97436, 28941, 37468, 77321, 21735, 39947, 45271, 75551, 22756, 94693, 74923, 63417, 66612, 36128, 85415, 74813, 28512, 26550, 67591, 47740, 68396, 15283, 15648, 23442, 95657, 99006, 53923, 40365, 93449, 64620, 98318, 80410, 88103, 13907, 62043, 21740, 36137, 79315, 86038, 19773, 82593, 43382, 34577, 23476, 27968, 58730, 60893, 48230, 32796, 82817, 48538, 6078, 95367, 84262, 88804, 69862, 69479, 60964, 38756, 25008, 7142, 23986, 84551, 7812, 79766, 21233, 86242, 25261, 74883, 5960, 66064, 77217, 53785, 46293, 83768, 25802, 54151, 80686, 14218, 90413, 97864, 73782, 94318, 67909, 84451, 91892, 12560, 17023, 52220, 2395, 80816, 18049, 40026, 29409, 14129, 19124, 32383, 59429, 76000, 10553, 35002, 85675, 78715, 60336, 91802, 56680, 44473, 15323, 72637, 60748, 28712, 57031, 39876, 51623, 95477, 50497, 43853, 38785, 8957, 7013, 86736, 46080, 16007, 3575, 73960, 22078, 60895, 46819, 23008, 78411, 12396, 38446, 35650, 86885, 7820, 66430, 70357, 55297, 39768, 20774, 39768, 74687, 95554, 48913, 64492, 99818, 92997, 82084, 91865, 42723, 7000, 56647, 68687, 94408, 78624, 79479, 75537, 36727, 69741, 86222, 1196, 59633, 48391, 95494, 54878, 15471, 8204, 4152, 91150, 4972, 69139, 98714, 2956, 12439, 69944, 32931, 45117, 17985, 67724, 65073, 23958, 41596, 91004, 29870, 76211, 2136, 63443, 87232, 99360, 64463, 4853, 38336, 57518, 97656, 92747, 60331, 63232, 34998, 63993, 58572, 22758, 85891, 65270, 92852, 31546, 19770, 86711, 14769, 72772, 27712, 90511, 16137, 91765, 75643, 9287, 60170, 23621, 59286, 37264, 55248, 95570, 7117, 94946, 13553, 3167, 88979, 18789, 37787, 18403, 86350, 89986, 63982, 72264, 67512, 96140, 37006, 81778, 68741, 81455, 23044, 33806, 72774, 47772, 50979, 9182, 16298, 21637, 34799, 93238, 55093, 99587, 31275, 89505, 99390, 95378, 69783, 31037, 57279, 50531, 82947, 84495, 70584, 88362, 98041, 1223, 43768, 1870, 2005, 46503, 34158, 83842, 92485, 1100, 56346, 72014, 39266, 25891, 319, 83518, 56033, 48482, 78600, 86706, 87729, 34123, 64805, 22640, 60603, 55603, 21563, 99566, 91655, 81295, 93129, 69679, 87846, 18033, 57023, 75157, 25873, 52285, 14316, 42940, 80808, 78484, 30094, 39911, 98310, 26188, 84762, 91156, 60839, 10191, 6178, 72845, 60727, 8605, 60336, 57466, 41129, 79672, 27170, 65879, 39237, 33641, 63449, 6313, 90437, 39107, 26877, 22849, 836, 13676, 3616, 47699, 2605, 80699, 71610, 31534, 86182, 56117, 62450, 85096, 72666, 88888, 93561, 87582, 64603, 54254, 36553, 58355, 83789, 64394, 24789, 73448, 71806, 82706, 47094, 59286, 47209, 11338, 59794, 85237, 46055, 41791, 88836, 95459, 88208, 90124, 92109, 62558, 73057, 92244, 71604, 45460, 5839, 38383, 26061, 5392, 11007, 44851, 52, 50985, 76950, 70698, 63517, 95474, 1254, 417, 56229, 85470, 46133, 77675, 8333, 97149, 77182, 47926, 66511, 45923, 86930, 58217, 44904, 84940, 85462, 95682, 99676, 60801, 28614, 9796, 88540, 24080, 49656, 83612, 81363, 77652, 6824, 46003, 4740, 86421, 422, 52145, 76625, 9283, 39083, 69931, 90410, 32374, 27588, 78139, 3609, 32872, 75207, 47343, 97932, 58949, 67553, 29263, 81157, 27644, 79625, 47083, 56077, 64099, 29307, 66597, 56082, 94769, 74838, 40201, 51050, 97608, 47250, 84274, 43415, 35040, 97305, 82722, 69731, 9150, 94333, 63698, 94971, 86961, 70040, 14837, 54135, 15162, 71747, 93408, 23362, 14048, 58578, 99520, 38312, 53360, 19788, 30524, 44543, 8827, 45426, 68623, 11657, 77471, 93666, 99245, 84478, 76935, 33696, 30943, 98760, 90108, 81437, 61468, 70688, 84354, 95723, 34044, 64378, 53940, 64776, 44697, 23103, 48570, 17009, 86346, 73416, 63076, 49260, 98661, 53309, 95999, 60577, 70634, 42417, 90525, 95924, 29440, 39994, 50494, 10641, 74709, 84196, 1750, 70151, 86337, 12960, 59063, 92622, 49872, 93661, 50762, 34059, 16868, 98740, 6993, 31362, 11019, 20304, 19372, 74905, 49212, 80230, 58367, 43291, 5776, 28588, 73898, 30214, 30923, 2637, 34549, 40675, 50209, 25622, 15280, 12805, 75216, 79715, 18537, 63277, 10582, 54956, 68340, 55974, 94261, 78109, 46787, 95059, 39538, 46154, 94704, 56472, 31165, 85879, 95116, 10144, 35958, 25520, 71432, 64569, 146, 88264, 28727, 66022, 2363, 62540, 12212, 43734, 85739, 50938, 68272, 35476, 31276, 5357, 82251, 97447, 50713, 75095, 74624, 65062, 55830, 97278, 90506, 38997, 78134, 52108, 86442, 21241, 27928, 52300, 79399, 79367, 35413, 49363, 54675, 69022, 15912, 95455, 79241, 46817, 86953, 23036, 56981, 24083, 22460, 57455, 44070, 82957, 98072, 2232, 23328, 73235, 13952, 14602, 47560, 16713, 33345, 33413, 93827, 64812, 97921, 39982, 95195, 91913, 30773, 99925, 96875, 46785, 15809, 43457, 94121, 84312, 42894, 1065, 5233, 83107, 61987, 12187, 9403, 96846, 38880, 44828, 91629, 91208, 80081, 90763, 7249, 95777, 89497, 29116, 12326, 76600, 94643, 76625, 75881, 23050, 30468, 45453, 88407, 15429, 36855, 66871, 75649, 3143, 65308, 255, 93222, 85964, 82099, 20105, 40665, 79570, 91207, 50416, 33081, 63434, 31348, 29459, 5116, 48475, 39422, 90004, 23465, 58608, 44113, 77184, 32255, 15677, 83447, 52197, 90286, 66651, 5027, 53441, 65068, 93175, 59113, 56829, 90342, 74984, 33656, 1970, 95636, 27176, 15139, 94210, 65802, 42362, 90298, 20761, 16508, 93442, 13793, 44690, 90097, 22508, 46584, 24195, 11159, 19027, 90366, 58144, 83706, 95157, 21275, 83031, 98886, 14780, 86587, 35546, 90830, 50264, 86873, 23244, 29011, 78896, 54084, 96988, 21713, 78166, 57653, 90650, 19880, 44021, 42921, 25176, 17619, 82357, 30355, 83135, 68740, 86997, 86609, 10772, 16731, 44486, 88394, 17283, 74815, 79220, 10870, 21970, 45055, 94119, 15949, 43561, 61746, 98475, 60603, 58850, 33917, 88987, 85502, 33281, 76623, 71898, 35799, 69623, 96836, 81074, 45366, 72430, 1026, 23917, 97609, 48527, 19246, 63029, 44684, 76282, 32300, 29926, 28557, 91335, 67951, 28910, 88176, 58848, 17252, 54823, 15375, 97436, 11855, 20839, 52699, 90872, 33134, 15654, 80000, 51287, 86179, 7336, 28335, 68462, 58251, 33366, 63690, 84694, 99588, 86685, 85609, 76204, 10076, 77295, 95855, 97595, 43097, 30505, 38229, 28404, 34642, 7780, 65626, 6301, 59674, 62205, 33504, 55466, 92151, 33082, 39345, 76760, 2828, 68327, 47732, 83802, 52863, 30185, 59835, 42326, 69967, 50431, 39936, 28518, 5792, 39137, 90634, 63230, 89760, 40707, 28697, 52533, 50505, 71628, 95758, 46210, 79515, 28572, 78450, 82245, 21353, 41716, 8130, 51944, 95671, 75478, 96177, 20517, 24714, 11804, 92620, 26047, 36762, 18190, 67632, 60443, 2012, 88462, 61217, 65389, 85334, 16521, 48583, 87310, 41534, 91748, 63000, 23450, 5513, 75617, 92713, 57741, 47081, 43478, 79550, 89929, 35954, 59327, 39989, 9175, 58027, 96503, 56203, 17664, 20375, 17982, 21279, 74437, 98398, 75234, 15731, 86735, 58199, 51650, 89993, 88334, 69329, 83006, 40098, 17193, 55100, 69322, 14913, 18891, 72032, 3693, 68680, 19587, 81363, 62497, 38325, 51588, 41219, 37190, 54768, 31952, 49253, 83657, 13912, 53354, 88699, 8153, 66917, 85806, 39811, 97363, 18385, 91948, 28948, 67657, 90997, 79311, 77807, 68742, 85479, 37468, 95472, 696, 29936, 86857, 26172, 87461, 71363, 45946, 60278, 68830, 85921, 4591, 83329, 82139, 49310, 98620, 4849, 27990, 52665, 25909, 51340, 79955, 1823, 26081, 22084, 74952, 40488, 64155, 4026, 92085, 89615, 1772, 14719, 32812, 12601, 73375, 25807, 35466, 6581, 91932, 51134, 80063, 33726, 3072, 61317, 97386, 963, 56416, 78884, 32294, 83093, 92659, 23372, 7513, 85680, 74823, 30774, 15966, 82528, 12957, 59391, 13027, 69692, 80770, 31226, 43408, 72011, 35931, 64757, 39030, 60746, 42770, 20082, 94607, 55850, 15632, 77489, 65853, 20450, 80456, 35530, 38572, 88290, 27079, 13670, 10678, 22528, 37793, 99673, 64485, 73304, 97540, 56244, 11240, 33357, 86465, 19348, 28559, 98798, 15834, 46537, 360, 11590, 76799, 48517, 67614, 61493, 10208, 93325, 91271, 65814, 41021, 53963, 26291, 67273, 19165, 24925, 86472, 61836, 3429, 51217, 85724, 19820, 82360, 28288, 19438, 16648, 88970, 47187, 31566, 43084, 64484, 3483, 50033, 80422, 65566, 56648, 50639, 71983, 59367, 39560, 76171, 83261, 26831, 48946, 85648, 85484, 32410, 91201, 56586, 19817, 82780, 58992, 51253, 86702, 65267, 17781, 63826, 85789, 98095, 31197, 84459, 1349, 61572, 89736, 90251, 75106, 41052, 85297, 47940, 20826, 58545, 9368, 28069, 61049, 93651, 56044, 63901, 46685, 25944, 71020, 7633, 38081, 9906, 71867, 27555, 24758, 82305, 13699, 57144, 21681, 45909, 19957, 19450, 97705, 89064, 22083, 73469, 74942, 55778, 88985, 30975, 47868, 43558, 78887, 73205, 93191, 87022, 26673, 38466, 83431, 13489, 11016, 49070, 37111, 95934, 65529, 26460, 69720, 60578, 27355, 61622, 30564, 51985, 20875, 87078, 31674, 23447, 78399, 36375, 30498, 37125, 51511, 49362, 97825, 42155, 59937, 34984, 57049, 27996, 43787, 52595, 51346, 13522, 12972, 4664, 91126, 42477, 37497, 713, 11811, 54264, 82114, 46009, 31512, 84294, 78699, 29493, 82391, 63488, 43732, 42496, 85517, 19145, 88501, 59683, 39233, 19536, 38517, 34489, 15301, 64094, 89617, 64269, 66198, 14198, 52515, 66000, 27740, 71507, 21546, 80024, 56209, 46024, 3751, 98349, 2821, 42689, 7112, 59362, 99874, 54960, 88375, 77716, 93551, 51467, 95774, 54587, 11243, 53676, 16670, 71572, 7937, 48393, 99995, 25761, 71614, 85215, 88004, 64604, 2832, 46958, 55242, 93010, 83407, 19311, 85963, 881, 76066, 65276, 21939, 3082, 18991, 63974, 40406, 81875, 71587, 7946, 63597, 8335, 55237, 62009, 77985, 36985, 17949, 23827, 80971, 46563, 88553, 73407, 79657, 39077, 77109, 41721, 84589, 98990, 28596, 69948, 36455, 83719, 92804, 73011, 39266, 25918, 94009, 27, 75038, 27984, 18388, 26493, 74140, 36594, 99796, 59057, 85989, 36091, 18922, 43442, 10098, 5331, 29944, 94167, 54748, 12950, 8593, 10130, 90707, 16651, 87487, 3204, 71096, 83051, 85446, 12947, 9924, 40148, 1189, 70993, 55763, 46167, 95069, 9078, 77620, 27244, 25831, 47871, 8201, 54357, 4109, 12625, 63977, 62761, 40131, 45015, 59629, 13220, 27122, 93374, 983, 26836, 54491, 60748, 99435, 28916, 4754, 7801, 98388, 54351, 50557, 46125, 1351, 38379, 37027, 34095, 60891, 98682, 94580, 42386, 68020, 5520, 45716, 55307, 10750, 20779, 30367, 94181, 22448, 18913, 12491, 11676, 60911, 15023, 56049, 15328, 4873, 91442, 76576, 99969, 13251, 35657, 78482, 37470, 6798, 829, 28940, 12213, 40896, 59484, 35362, 72065, 1889, 59422, 87196, 85128, 52100, 1264, 42439, 18988, 89029, 70436, 66176, 72376, 72973, 79604, 75462, 45311, 43538, 87603, 76317, 30178, 31970, 83613, 29384, 38884, 77600, 35980, 89716, 10073, 92912, 79569, 62577, 1714, 89887, 28619, 80041, 21503, 37624, 85030, 46410, 238, 9876, 99613, 71267, 20160, 46083, 68708, 23028, 81583, 20441, 72293, 15323, 48604, 45809, 33389, 10863, 21418, 69829, 13794, 65743, 17491, 97038, 93318, 95179, 90742, 20674, 51596, 34364, 5907, 91717, 54410, 95770, 74073, 21838, 98742, 88281, 25676, 42257, 56463, 75624, 73225, 57957, 92183, 1298, 60579, 12341, 86629, 33041, 71148, 63896, 92727, 69348, 23594, 72495, 49679, 89742, 86841, 66722, 36683, 56771, 65550, 81104, 30240, 29951, 95938, 35090, 94585, 48925, 71223, 6457, 82269, 28219, 43118, 59659, 4715, 34680, 30731, 33614, 79270, 16882, 99199, 60257, 81512, 60071, 4706, 32140, 89836, 44026, 50513, 45896, 55509, 94709, 58436, 50796, 23212, 66786, 55452, 56063, 25555, 3098, 19375, 6177, 80210, 75112, 25940, 78640, 76532, 95887, 16114, 17832, 22608, 14179, 19838, 64335, 45376, 54812, 11507, 1367, 70886, 78424, 70777, 20553, 15310, 54068, 21317, 48083, 44256, 31891, 91011, 5336, 61399, 73077, 13028, 61841, 14522, 94329, 10065, 2131, 77525, 71445, 62098, 36626, 32421, 65944, 39482, 20262, 98533, 53780, 15586, 17907, 26146, 5434, 5405, 20496, 40441, 92226, 28649, 18172, 49105, 82832, 45056, 310, 55002, 92564, 41538, 20572, 6020, 29658, 60356, 68918, 30964, 69271, 36201, 55942, 23856, 48924, 61435, 53310, 41472, 55659, 45772, 38696, 57043, 33966, 17456, 36348, 17916, 8172, 10942, 39527, 10178, 35004, 12134, 83421, 37526, 82883, 56525, 29439, 34502, 51497, 57584, 15078, 73334, 18410, 33654, 3559, 86908, 95870, 90231, 96757, 2268, 41343, 29054, 4105, 72952, 57258, 97715, 26979, 50586, 40444, 10448, 26197, 15058, 53207, 87344, 19588, 65047, 57608, 94174, 97128, 41526, 54665, 5072, 50531, 30391, 10443, 15971, 59656, 44185, 83142, 17462, 86671, 62629, 39984, 85482, 92560, 36045, 39243, 69394, 58260, 57529, 96318, 6172, 40396, 35113, 5096, 66892, 64404, 17018, 37458, 16215, 67641, 8914, 40055, 22873, 46175, 15910, 46358, 37285, 3114, 76059, 26790, 10771, 22824, 54741, 20039, 30640, 91590, 71113, 94173, 50298, 96619, 43929, 50115, 98671, 5555, 58874, 15272, 61090, 6978, 7955, 64918, 19580, 36183, 75864, 49027, 12491, 5888, 54706, 49932, 50642, 33567, 91582, 44510, 4114, 64910, 33203, 91604, 43907, 42991, 86639, 46286, 42662, 26649, 21991, 36782, 113, 87910, 7827, 92402, 40043, 47163, 30139, 87995, 24268, 78888, 49604, 67628, 23794, 11450, 44940, 2265, 45057, 4698, 68425, 47859, 84760, 61801, 66998, 86025, 77072, 13288, 88033, 62737, 2282, 5094, 35411, 87511, 56371, 70554, 47344, 77721, 49192, 40988, 1438, 50444, 11746, 2688, 46001, 60523, 71350, 56177, 98108, 25689, 55910, 16194, 57996, 38283, 25660, 58447, 77154, 69176, 44507, 90584, 10069, 63575, 2139, 52971, 78332, 42668, 23458, 6649, 98525, 6301, 10330, 73558, 72609, 39029, 7211, 51953, 57201, 63440, 64418, 33868, 17898, 81765, 34294, 39683, 29876, 39064, 65294, 5754, 74743, 41171, 50478, 3052, 76721, 5412, 90546, 9585, 96659, 35272, 22727, 25645, 79138, 58146, 46298, 5053, 87359, 44460, 65425, 30650, 3944, 46395, 90490, 23603, 31268, 28593, 62157, 5462, 92770, 33316, 85079, 53863, 79328, 68593, 43480, 18998, 31702, 79427, 7789, 25443, 50368, 17653, 64596, 68237, 20366, 68116, 49549, 16101, 27746, 41069, 37033, 25667, 90687, 77284, 69518, 9845, 49337, 71879, 57794, 13513, 12843, 2275, 17036, 90916, 31506, 456, 16455, 69388, 49339, 22604, 29161, 16637, 42548, 15057, 11269, 48519, 9182, 42100, 26189, 56845, 55188, 18163, 92072, 32116, 29075, 44971, 10241, 31487, 32678, 66867, 23305, 89798, 660, 54252, 4975, 74728, 23351, 69500, 79846, 157, 43596, 97918, 91062, 4591, 66258, 85519, 57498, 19579, 14294, 72736, 38615, 10098, 88527, 34651, 33486, 14604, 37155, 67934, 39230, 34635, 35594, 57978, 24666, 3132, 31534, 27886, 87300, 53228, 31352, 9494, 5769, 77315, 54855, 30488, 87066, 25425, 69204, 73587, 28320, 85538, 39934, 33670, 21660, 47291, 20502, 15681, 67320, 71622, 50680, 94845, 90790, 21113, 37263, 49003, 20900, 61042, 14829, 12016, 89600, 67247, 35132, 25113, 12584, 72501, 46858, 76250, 34126, 21077, 6261, 94110, 5829, 34148, 3147, 41100, 10936, 3970, 13443, 97649, 24482, 15094, 15812, 32394, 17444, 807, 64509, 2158, 70902, 6184, 1959, 62898, 37256, 88693, 14210, 11061, 54184, 14914, 9968, 13434, 14485, 3256, 72058, 85308, 12905, 56074, 16882, 52974, 4862, 40404, 4336, 72805, 38991, 45124, 52891, 38935, 11956, 43309, 23097, 87322, 53188]
# pipes = [[2, 4, 82232], [4, 3, 31825], [6, 5, 17135], [7, 13, 31665], [7, 18, 7520], [7, 14, 21013], [18, 8, 60405], [8, 17, 25696], [18, 9, 19323], [18, 10, 7279], [14, 11, 63634], [11, 15, 81563], [13, 12, 21241], [13, 16, 44602], [14, 19, 21833], [20, 22, 56293], [20, 23, 40308], [20, 32, 13648], [20, 58, 67728], [20, 135, 88518], [20, 30, 3867], [146, 21, 7405], [21, 42, 37249], [22, 75, 67775], [22, 144, 5314], [22, 80, 81886], [23, 25, 66572], [23, 29, 25713], [23, 68, 4787], [23, 24, 54451], [23, 47, 56329], [25, 28, 64972], [25, 33, 40618], [25, 61, 161], [25, 78, 42543], [25, 84, 24514], [25, 151, 14423], [127, 26, 68982], [95, 27, 32648], [28, 43, 17143], [28, 90, 62543], [29, 44, 1235], [29, 50, 14545], [29, 54, 60159], [29, 134, 56060], [29, 108, 56978], [74, 31, 7480], [32, 35, 7549], [32, 96, 45087], [32, 49, 57663], [33, 37, 82641], [33, 39, 87950], [33, 65, 53504], [33, 93, 43418], [33, 131, 83150], [53, 34, 71888], [34, 154, 89686], [35, 81, 57357], [35, 107, 53367], [35, 113, 27159], [116, 36, 16601], [37, 112, 9603], [37, 73, 14415], [37, 103, 48726], [37, 126, 73716], [37, 57, 34092], [37, 143, 46856], [139, 38, 18764], [38, 140, 13914], [39, 94, 43494], [39, 101, 69480], [87, 40, 56823], [43, 41, 36052], [157, 45, 34638], [145, 46, 13227], [155, 48, 1424], [48, 141, 36711], [87, 51, 40310], [94, 52, 3365], [67, 53, 32559], [54, 67, 59196], [54, 74, 25735], [54, 133, 46331], [79, 55, 84761], [85, 56, 42477], [56, 118, 22508], [58, 59, 25364], [59, 83, 55166], [59, 128, 11194], [107, 60, 27877], [60, 152, 40033], [61, 89, 23810], [158, 62, 5227], [94, 63, 4237], [63, 136, 53788], [78, 64, 33619], [65, 111, 3452], [65, 139, 36232], [94, 66, 14071], [66, 105, 15369], [66, 149, 46273], [67, 109, 47526], [68, 114, 16711], [117, 69, 76476], [155, 70, 38015], [70, 142, 20585], [155, 71, 6831], [71, 104, 76198], [103, 72, 22576], [74, 91, 39351], [75, 79, 51479], [75, 77, 58141], [87, 76, 54150], [78, 85, 91277], [78, 92, 37590], [79, 87, 31649], [79, 146, 80667], [81, 100, 49095], [81, 117, 2051], [81, 158, 6062], [127, 82, 42108], [82, 122, 35456], [83, 86, 71708], [83, 115, 39232], [83, 130, 78688], [84, 145, 12819], [85, 95, 72908], [87, 125, 69432], [92, 88, 25143], [90, 102, 30636], [91, 159, 37723], [92, 106, 52539], [92, 138, 30270], [92, 110, 68521], [95, 99, 80191], [95, 147, 56119], [95, 116, 36121], [123, 97, 66486], [139, 98, 6640], [102, 148, 2145], [102, 153, 14971], [103, 120, 5637], [103, 150, 26628], [106, 155, 31314], [107, 127, 21637], [142, 119, 15072], [147, 121, 17770], [145, 123, 67685], [139, 124, 58540], [126, 156, 73839], [144, 129, 42940], [131, 137, 10071], [156, 132, 7465], [142, 157, 34045], [160, 171, 36080], [160, 162, 31563], [160, 163, 48816], [160, 170, 46512], [160, 172, 51804], [171, 161, 14736], [162, 166, 54720], [163, 168, 22657], [170, 164, 7354], [168, 165, 8545], [166, 167, 5425], [170, 169, 84863], [176, 175, 18003], [175, 184, 51676], [181, 176, 33060], [179, 177, 73693], [177, 178, 150], [181, 179, 31217], [179, 182, 46535], [181, 180, 56270], [181, 183, 20935], [186, 185, 29954], [213, 187, 67496], [206, 188, 84585], [188, 191, 22595], [200, 189, 96333], [190, 204, 35133], [190, 206, 49749], [190, 208, 75213], [217, 192, 31141], [223, 193, 64752], [193, 197, 92359], [193, 201, 77972], [193, 202, 68233], [225, 194, 23285], [201, 195, 41386], [221, 196, 64775], [196, 220, 68249], [197, 207, 3603], [208, 198, 22973], [228, 199, 31484], [199, 214, 63006], [213, 200, 66948], [208, 203, 45219], [203, 212, 77236], [208, 205, 85382], [205, 216, 49386], [207, 225, 39415], [208, 221, 85281], [208, 222, 92648], [208, 217, 72509], [208, 223, 54935], [208, 209, 19739], [208, 228, 86174], [208, 227, 29019], [222, 210, 16167], [222, 211, 19735], [221, 213, 64507], [220, 215, 11298], [217, 226, 70090], [225, 218, 36783], [228, 219, 30617], [223, 224, 88111], [230, 229, 35055], [233, 234, 27950], [236, 235, 16627], [236, 237, 87018], [244, 239, 35477], [243, 240, 13974], [240, 241, 12588], [243, 242, 61310], [244, 243, 75663], [246, 247, 32702], [253, 248, 56377], [251, 249, 3186], [253, 250, 58025], [255, 251, 41056], [253, 252, 63609], [254, 253, 34629], [255, 254, 2757], [267, 256, 58624], [256, 257, 57594], [256, 259, 19997], [257, 260, 54249], [257, 263, 70528], [257, 261, 75540], [257, 266, 45025], [267, 258, 62204], [259, 262, 17113], [260, 265, 26907], [267, 264, 78743], [267, 268, 46085], [270, 271, 1681], [275, 273, 30089], [277, 274, 62567], [274, 286, 28861], [282, 275, 9003], [275, 288, 31266], [284, 276, 9311], [276, 287, 23375], [288, 277, 39233], [277, 285, 23204], [277, 279, 21399], [289, 278, 63311], [289, 280, 59119], [289, 281, 15180], [282, 284, 38342], [288, 283, 47428], [288, 289, 3124], [292, 290, 1667], [292, 291, 51479], [293, 292, 68874], [293, 294, 25994], [296, 295, 28017], [296, 297, 66052], [307, 298, 71964], [298, 319, 1848], [307, 299, 32869], [300, 305, 94495], [308, 301, 64222], [310, 302, 51195], [314, 303, 34953], [308, 304, 86479], [305, 307, 95247], [305, 313, 44964], [305, 311, 52943], [305, 318, 4251], [305, 317, 17750], [307, 306, 38528], [307, 316, 45878], [307, 315, 17222], [319, 308, 22353], [310, 309, 5018], [313, 310, 45198], [310, 314, 55888], [319, 312, 25813], [320, 321, 24026], [320, 322, 42395], [327, 324, 10769], [324, 326, 34755], [324, 325, 22761], [327, 329, 53315], [329, 328, 44928], [336, 330, 1243], [330, 341, 35510], [335, 331, 4852], [337, 332, 82536], [333, 334, 34920], [333, 337, 9461], [333, 335, 47441], [333, 342, 34331], [333, 345, 23422], [333, 340, 69520], [334, 347, 61022], [334, 346, 40700], [335, 336, 45477], [336, 344, 36454], [337, 343, 20645], [347, 338, 15599], [340, 339, 56954], [355, 348, 84844], [362, 349, 43736], [349, 351, 64981], [351, 350, 19232], [360, 352, 42433], [362, 353, 94700], [353, 354, 73876], [355, 360, 9371], [362, 356, 79112], [356, 358, 30771], [362, 357, 85911], [360, 359, 32278], [360, 362, 85639], [362, 361, 55738], [377, 363, 30774], [363, 370, 35099], [370, 364, 83307], [364, 366, 65791], [377, 365, 3167], [365, 374, 77927], [375, 367, 67924], [375, 368, 25948], [372, 369, 16006], [370, 375, 60391], [375, 371, 1750], [377, 372, 64581], [375, 373, 27110], [377, 376, 5945], [380, 378, 18274], [379, 380, 54747], [383, 382, 88869], [382, 384, 40184], [391, 385, 660], [400, 386, 42670], [386, 397, 64571], [386, 413, 9265], [417, 387, 51738], [387, 407, 46944], [419, 388, 10491], [421, 389, 93642], [394, 390, 12746], [391, 394, 76743], [391, 399, 16644], [391, 406, 38535], [391, 419, 65964], [391, 409, 9929], [391, 410, 15251], [416, 392, 9115], [419, 393, 40217], [393, 414, 25434], [393, 420, 12553], [394, 395, 19033], [394, 400, 22983], [394, 411, 17366], [394, 425, 15119], [395, 423, 13038], [419, 396, 59566], [435, 398, 7643], [399, 417, 64772], [399, 431, 3972], [400, 433, 35415], [400, 421, 93811], [417, 401, 77924], [435, 402, 23953], [402, 426, 42332], [427, 403, 81572], [413, 404, 44737], [430, 405, 35681], [405, 422, 70289], [406, 416, 37501], [427, 408, 31987], [410, 434, 63915], [411, 432, 67375], [417, 412, 83702], [423, 415, 70213], [417, 435, 75959], [419, 418, 43585], [419, 430, 2318], [419, 429, 41959], [433, 424, 54643], [433, 427, 86265], [427, 428, 13272], [438, 436, 54378], [436, 440, 4852], [436, 444, 18480], [438, 437, 76533], [438, 443, 16071], [438, 442, 29220], [442, 439, 52237], [439, 441, 35303], [449, 445, 40484], [449, 446, 68228], [449, 447, 51029], [447, 450, 36996], [450, 448, 22585], [451, 449, 46318], [452, 457, 40491], [457, 453, 48270], [457, 454, 38478], [454, 456, 37617], [457, 455, 72599], [463, 458, 16516], [458, 464, 79561], [460, 459, 86604], [462, 460, 24223], [463, 461, 4160], [461, 465, 16773], [463, 462, 14405], [466, 470, 41754], [466, 468, 20474], [468, 467, 50950], [468, 469, 75435], [470, 471, 3702], [473, 472, 32361], [473, 475, 3423], [473, 474, 52690], [480, 479, 20603], [481, 480, 75790], [482, 483, 33042], [484, 486, 17635], [486, 485, 48859], [489, 488, 22769], [491, 492, 78922], [493, 494, 61328], [517, 496, 76403], [514, 497, 18406], [497, 503, 45827], [498, 511, 77389], [498, 513, 9984], [498, 515, 42979], [498, 499, 817], [502, 500, 42618], [500, 516, 54118], [515, 501, 6972], [501, 505, 14315], [501, 517, 10137], [501, 507, 58494], [511, 502, 2711], [502, 506, 31140], [502, 504, 3409], [505, 509, 32231], [512, 508, 40777], [518, 510, 14295], [511, 514, 13583], [514, 512, 80622], [513, 518, 65526], [523, 519, 56754], [523, 520, 59331], [523, 521, 82808], [522, 524, 64810], [524, 523, 51217], [525, 526, 52684], [526, 527, 35972], [528, 529, 51154], [532, 533, 6679], [534, 535, 26684], [537, 538, 53213], [537, 540, 73889], [538, 539, 87245], [542, 541, 53905], [545, 543, 58239], [548, 544, 6732], [544, 545, 85293], [547, 546, 70709], [547, 548, 49855], [550, 549, 95297], [551, 550, 88133], [552, 553, 57318], [555, 556, 694], [555, 567, 53621], [555, 568, 44244], [556, 557, 38134], [556, 564, 27141], [556, 572, 16455], [556, 606, 4094], [556, 619, 15576], [557, 563, 57468], [557, 596, 45209], [581, 558, 48215], [611, 559, 62330], [559, 623, 34317], [621, 560, 55563], [593, 561, 40097], [561, 604, 61773], [561, 622, 66756], [633, 562, 35536], [563, 574, 44932], [563, 586, 57740], [563, 588, 1991], [563, 600, 22060], [563, 611, 54520], [563, 631, 53146], [564, 566, 28292], [564, 569, 46504], [564, 576, 12824], [564, 610, 39052], [606, 565, 272], [566, 613, 40024], [566, 636, 60177], [566, 655, 17495], [567, 630, 65530], [568, 650, 55828], [569, 589, 69761], [569, 640, 4277], [569, 624, 62887], [569, 616, 52272], [593, 570, 87502], [589, 571, 26914], [572, 580, 13235], [572, 593, 1762], [572, 629, 32308], [572, 652, 63073], [621, 573, 26914], [573, 597, 39993], [574, 575, 15511], [575, 579, 64679], [575, 595, 43412], [575, 634, 44055], [576, 585, 20571], [576, 648, 76460], [632, 577, 11970], [621, 578, 55977], [579, 603, 31800], [579, 627, 38191], [579, 633, 82634], [579, 581, 67629], [579, 599, 39114], [653, 582, 13253], [606, 583, 18241], [583, 598, 26653], [643, 584, 70775], [584, 644, 75422], [585, 609, 11344], [585, 625, 87630], [586, 587, 21191], [586, 612, 33919], [588, 601, 3472], [588, 608, 72002], [589, 638, 49868], [636, 590, 85010], [640, 591, 49169], [603, 592, 10709], [593, 621, 33306], [593, 605, 13144], [648, 594, 64075], [595, 626, 19543], [595, 643, 28241], [597, 656, 4469], [601, 639, 58204], [611, 602, 27954], [604, 614, 41215], [606, 617, 47815], [606, 654, 41201], [606, 637, 9090], [606, 628, 29442], [618, 607, 19217], [608, 642, 25055], [611, 615, 19983], [613, 618, 22222], [617, 635, 52716], [649, 620, 49970], [621, 653, 67268], [625, 647, 92788], [626, 645, 40294], [626, 649, 24533], [627, 651, 38029], [631, 632, 28229], [646, 641, 13667], [654, 646, 32670], [668, 659, 3689], [659, 666, 23506], [668, 660, 10077], [660, 667, 4799], [665, 661, 71713], [667, 662, 3776], [668, 663, 9629], [663, 664, 17834], [668, 665, 1420], [669, 672, 37108], [672, 670, 71990], [672, 671, 52652], [676, 674, 27299], [674, 677, 41551], [676, 675, 44613], [679, 678, 3139], [684, 681, 28573], [681, 693, 8925], [691, 682, 31636], [688, 683, 66030], [683, 685, 68921], [683, 694, 90152], [685, 684, 1326], [684, 691, 32796], [685, 689, 6460], [688, 686, 25509], [690, 687, 5763], [688, 690, 12673], [689, 692, 1667], [699, 695, 64709], [698, 696, 44461], [698, 697, 77729], [699, 698, 35580], [704, 700, 26625], [702, 701, 52848], [701, 705, 54955], [704, 702, 45237], [704, 703, 7171], [707, 706, 65341], [707, 708, 38787], [723, 709, 3436], [732, 710, 83063], [710, 730, 74902], [715, 711, 8634], [730, 712, 16549], [713, 719, 31880], [713, 732, 18420], [713, 714, 21536], [713, 724, 19141], [719, 715, 51647], [715, 722, 68072], [715, 725, 21665], [715, 728, 34057], [715, 720, 37847], [719, 716, 46530], [734, 717, 22523], [732, 718, 13932], [732, 721, 41835], [721, 733, 47267], [732, 723, 8643], [723, 726, 66300], [723, 731, 60437], [723, 727, 17030], [732, 729, 68886], [732, 734, 9036], [745, 735, 40530], [747, 736, 18345], [751, 737, 67678], [747, 738, 39003], [738, 741, 8048], [749, 739, 10067], [747, 740, 46491], [740, 743, 81356], [740, 750, 58858], [741, 746, 66331], [747, 742, 37496], [742, 751, 50378], [743, 744, 4278], [743, 745, 20182], [749, 747, 46281], [747, 748, 55470], [748, 752, 41287], [757, 756, 11570], [769, 758, 47324], [762, 759, 43129], [759, 764, 9137], [770, 760, 88321], [760, 762, 4414], [765, 761, 19267], [762, 763, 74716], [766, 765, 32397], [767, 766, 51338], [767, 770, 4810], [769, 768, 23033], [770, 769, 48112], [772, 774, 63656], [772, 775, 42933], [774, 773, 5537], [777, 776, 43855], [776, 780, 90522], [778, 777, 4712], [777, 779, 81169], [783, 781, 85100], [782, 783, 43503], [784, 785, 2115], [787, 786, 92449], [790, 789, 52759], [793, 790, 17057], [790, 800, 14288], [790, 794, 8187], [793, 791, 5519], [793, 792, 7105], [792, 797, 53574], [800, 795, 31037], [799, 796, 50055], [799, 798, 27437], [800, 799, 3081], [804, 803, 1608], [804, 806, 25217], [804, 805, 5780], [806, 807, 31411], [811, 810, 4644], [811, 812, 17296], [813, 816, 12931], [813, 814, 16541], [813, 815, 49797], [820, 819, 23274], [822, 821, 14393], [823, 822, 72635], [822, 824, 18610], [825, 826, 504], [826, 827, 28446], [829, 830, 10299], [830, 831, 54028], [834, 832, 14508], [842, 833, 29545], [835, 834, 57472], [843, 835, 10778], [843, 836, 46431], [836, 853, 73187], [837, 842, 49107], [837, 843, 57160], [837, 838, 11042], [838, 839, 17955], [851, 840, 30335], [842, 841, 47608], [842, 855, 45172], [842, 844, 4245], [843, 851, 27413], [843, 854, 45322], [843, 850, 58668], [854, 845, 9146], [845, 852, 15604], [845, 849, 25989], [848, 846, 32376], [854, 847, 35825], [854, 848, 9043], [862, 856, 23709], [871, 857, 27674], [871, 858, 29451], [862, 859, 80805], [859, 867, 11046], [859, 874, 24434], [862, 860, 5379], [871, 861, 17574], [861, 865, 9968], [862, 864, 10939], [864, 863, 48803], [864, 873, 34095], [864, 871, 86687], [864, 868, 28853], [870, 866, 2613], [873, 869, 43023], [871, 870, 43751], [871, 872, 71128], [877, 876, 77541], [882, 880, 13063], [880, 884, 25559], [882, 881, 13083], [881, 883, 17946], [885, 886, 57960], [888, 887, 41587], [891, 888, 4586], [891, 889, 42706], [889, 894, 68399], [891, 890, 24260], [890, 893, 70782], [890, 892, 63355], [894, 895, 25202], [897, 896, 38994], [899, 898, 68390], [908, 900, 2698], [900, 907, 29819], [900, 904, 45894], [900, 903, 49319], [907, 901, 17680], [904, 902, 7700], [904, 905, 10020], [905, 906, 5107], [907, 909, 56787], [910, 911, 493], [914, 912, 3214], [915, 913, 40659], [914, 915, 36328], [916, 917, 12734], [916, 918, 32848], [919, 920, 33548], [919, 921, 49826], [919, 925, 59012], [920, 927, 33274], [921, 924, 1150], [921, 923, 7500], [927, 922, 20153], [922, 926, 46946], [930, 929, 31035], [930, 931, 33927], [942, 932, 22312], [932, 943, 52219], [932, 939, 89778], [943, 933, 77376], [933, 937, 15048], [942, 934, 6772], [947, 935, 27531], [935, 936, 48526], [936, 946, 44133], [940, 938, 24267], [947, 940, 28453], [942, 941, 65874], [945, 942, 25460], [943, 944, 8108], [945, 947, 2568], [950, 949, 12721], [952, 951, 44916], [956, 955, 44672], [958, 957, 27174], [962, 959, 11008], [961, 960, 29093], [960, 962, 36480], [961, 963, 32234], [965, 966, 60730], [965, 967, 22161], [967, 968, 10898], [968, 969, 34904], [971, 970, 77111], [980, 972, 33687], [980, 973, 61457], [973, 974, 60592], [976, 975, 72975], [980, 976, 79416], [977, 980, 16961], [977, 978, 24786], [980, 979, 4230], [983, 981, 3108], [981, 984, 34397], [981, 982, 54410], [983, 985, 50145], [988, 986, 43142], [987, 989, 28044], [987, 988, 18784], [995, 991, 15377], [991, 992, 38843], [992, 996, 18628], [993, 995, 3282], [993, 994, 69239], [1001, 1000, 64416], [1005, 1002, 75514], [1005, 1003, 8199], [1006, 1004, 18683], [1004, 1005, 41397], [1012, 1008, 7335], [1008, 1010, 16564], [1012, 1009, 36393], [1010, 1011, 54776], [1015, 1014, 17809], [1019, 1016, 28122], [1019, 1017, 9332], [1017, 1018, 37605], [1021, 1020, 41782], [1021, 1022, 77773], [1021, 1023, 42342], [1027, 1025, 41230], [1026, 1027, 52331], [1037, 1028, 64676], [1043, 1029, 3122], [1038, 1030, 67081], [1030, 1039, 40027], [1030, 1033, 53644], [1030, 1031, 49155], [1030, 1037, 87182], [1040, 1032, 76779], [1042, 1034, 31554], [1036, 1035, 27651], [1035, 1041, 50811], [1040, 1036, 2781], [1038, 1040, 63691], [1039, 1042, 23211], [1040, 1044, 52997], [1040, 1043, 35396], [1045, 1046, 33498], [1047, 1057, 64967], [1047, 1055, 39298], [1047, 1058, 29287], [1047, 1052, 41643], [1058, 1048, 17475], [1057, 1049, 50055], [1055, 1050, 1836], [1055, 1051, 34905], [1057, 1053, 52037], [1053, 1056, 74787], [1058, 1054, 35706], [1067, 1061, 13280], [1061, 1063, 33054], [1066, 1062, 63782], [1063, 1068, 4972], [1069, 1064, 16128], [1066, 1065, 69408], [1067, 1066, 72979], [1067, 1069, 13236], [1072, 1070, 73680], [1072, 1071, 71906], [1075, 1076, 67616], [1078, 1077, 8951], [1081, 1080, 52963], [1086, 1082, 13196], [1086, 1083, 11475], [1086, 1084, 33524], [1085, 1086, 38159], [1093, 1089, 57458], [1093, 1090, 24699], [1095, 1091, 51540], [1093, 1092, 52932], [1092, 1095, 8285], [1092, 1094, 62525], [1096, 1097, 36501], [1096, 1099, 53844], [1097, 1098, 24494], [1102, 1103, 33509], [1107, 1106, 36943], [1111, 1108, 55775], [1108, 1109, 36010], [1108, 1112, 14100], [1111, 1110, 29597], [1118, 1113, 33720], [1116, 1114, 5671], [1117, 1115, 44442], [1116, 1118, 10734], [1118, 1117, 38471], [1120, 1119, 26799], [1119, 1121, 34114], [1122, 1123, 14232], [1122, 1124, 15569], [1130, 1131, 6770], [1133, 1132, 69403], [1137, 1136, 3031], [1136, 1138, 44635], [1140, 1139, 32385], [1146, 1144, 51984], [1144, 1147, 48181], [1146, 1145, 21564], [1149, 1148, 88320], [1153, 1150, 24198], [1150, 1151, 41242], [1153, 1152, 13561], [1158, 1155, 10878], [1156, 1158, 78913], [1158, 1157, 6354], [1160, 1161, 23657], [1163, 1162, 54680], [1163, 1165, 2639], [1163, 1170, 24538], [1170, 1164, 37380], [1165, 1172, 40808], [1165, 1168, 73592], [1168, 1166, 75793], [1168, 1167, 61470], [1168, 1169, 70548], [1168, 1171, 1311], [1173, 1174, 39924], [1176, 1175, 6896], [1179, 1177, 20215], [1180, 1178, 18022], [1178, 1179, 68221], [1183, 1182, 53419], [1185, 1186, 10519], [1186, 1187, 15884], [1191, 1188, 31778], [1188, 1190, 52644], [1191, 1189, 91095], [1190, 1197, 47022], [1191, 1196, 7780], [1191, 1195, 44950], [1191, 1192, 16216], [1192, 1194, 66512], [1195, 1193, 70390], [1204, 1198, 11824], [1205, 1199, 63303], [1203, 1200, 14855], [1200, 1201, 5458], [1200, 1202, 54693], [1201, 1204, 46831], [1203, 1205, 77565], [1205, 1207, 29752], [1205, 1206, 2675], [1211, 1209, 45100], [1209, 1210, 81349], [1213, 1212, 57861], [1216, 1213, 34510], [1216, 1214, 87353], [1216, 1215, 29650], [1218, 1217, 12523], [1218, 1220, 11827], [1220, 1219, 44337], [1223, 1225, 56081], [1225, 1224, 37102], [1228, 1226, 64558], [1227, 1228, 21523], [1229, 1232, 27282], [1232, 1230, 919], [1232, 1231, 16146], [1235, 1234, 43803], [1237, 1235, 4767], [1235, 1238, 62971], [1235, 1236, 36956], [1239, 1240, 49372], [1243, 1241, 65723], [1243, 1242, 45012], [1245, 1249, 33551], [1245, 1255, 53114], [1249, 1246, 41786], [1246, 1253, 29576], [1246, 1251, 32439], [1253, 1247, 30587], [1256, 1248, 29162], [1249, 1250, 37152], [1249, 1256, 68543], [1255, 1252, 3176], [1256, 1254, 23868], [1262, 1258, 15372], [1258, 1259, 44618], [1262, 1260, 46238], [1262, 1261, 18157], [1265, 1264, 16784], [1276, 1275, 3185], [1281, 1278, 17674], [1278, 1279, 26458], [1281, 1280, 8313], [1281, 1282, 47005], [1288, 1287, 77892], [1292, 1293, 17502], [1296, 1295, 23707], [1296, 1297, 67666], [1299, 1300, 45742], [1305, 1301, 63263], [1306, 1302, 6693], [1304, 1303, 15118], [1305, 1304, 33422], [1306, 1305, 35816], [1312, 1313, 86957], [1314, 1315, 533], [1316, 1317, 11288], [1319, 1320, 5303], [1319, 1322, 3048], [1320, 1321, 34538], [1326, 1324, 64971], [1325, 1326, 52297], [1328, 1331, 64717], [1328, 1333, 86270], [1328, 1329, 14623], [1332, 1330, 50078], [1331, 1334, 3520], [1333, 1332, 19578], [1334, 1335, 23929], [1340, 1341, 42437], [1343, 1342, 15012], [1342, 1344, 31161], [1346, 1345, 2935], [1346, 1348, 81426], [1346, 1347, 23096], [1347, 1351, 37938], [1348, 1349, 27492], [1348, 1352, 37992], [1351, 1350, 57282], [1355, 1353, 10978], [1361, 1354, 44945], [1354, 1358, 81705], [1356, 1355, 67779], [1355, 1360, 59737], [1367, 1356, 66745], [1362, 1357, 14841], [1357, 1368, 52285], [1358, 1366, 68216], [1359, 1367, 48746], [1367, 1361, 5680], [1367, 1362, 2837], [1367, 1363, 38691], [1363, 1365, 2872], [1367, 1364, 68284], [1373, 1371, 52026], [1372, 1374, 11472], [1374, 1373, 42073], [1375, 1377, 60069], [1377, 1376, 10125], [1380, 1381, 15958], [1384, 1383, 50747], [1388, 1384, 66368], [1384, 1385, 51842], [1387, 1386, 40401], [1387, 1388, 56138], [1392, 1390, 49036], [1390, 1393, 50184], [1392, 1391, 47258], [1401, 1400, 49097], [1401, 1405, 53090], [1404, 1402, 45121], [1404, 1403, 49128], [1405, 1404, 32426], [1406, 1407, 29933], [1410, 1408, 7391], [1408, 1409, 4294], [1411, 1412, 20414], [1414, 1413, 33472], [1418, 1417, 76015], [1422, 1420, 35956], [1421, 1422, 63588], [1424, 1423, 68976], [1427, 1428, 36372], [1433, 1431, 48044], [1433, 1432, 51297], [1437, 1438, 33745], [1441, 1442, 63235], [1445, 1446, 18435], [1445, 1447, 34681], [1449, 1448, 92693], [1451, 1450, 45912], [1452, 1453, 33779], [1453, 1455, 43138], [1453, 1454, 95390], [1461, 1457, 3721], [1461, 1458, 46162], [1460, 1459, 34305], [1459, 1461, 45145], [1465, 1467, 8039], [1465, 1466, 84200], [1469, 1468, 45941], [1474, 1475, 9913], [1475, 1476, 82236], [1481, 1477, 58621], [1478, 1481, 13396], [1481, 1479, 36796], [1481, 1480, 3364], [1485, 1486, 62645], [1486, 1487, 52973], [1491, 1488, 92400], [1488, 1489, 75774], [1488, 1490, 12986], [1498, 1495, 42673], [1499, 1496, 29487], [1499, 1497, 19013], [1499, 1498, 55761], [1501, 1502, 35012], [1501, 1503, 21897], [1505, 1504, 40216], [1506, 1507, 1664], [1514, 1508, 68142], [1516, 1509, 1435], [1519, 1510, 34522], [1515, 1511, 15781], [1521, 1512, 50925], [1519, 1513, 61194], [1513, 1517, 43429], [1520, 1514, 70614], [1518, 1515, 38652], [1516, 1519, 31287], [1520, 1518, 46162], [1518, 1521, 43158], [1519, 1520, 63188], [1524, 1525, 43130], [1532, 1530, 69068], [1532, 1531, 54427], [1532, 1533, 52883], [1535, 1534, 75996], [1536, 1535, 50918], [1538, 1539, 34399], [1543, 1541, 65680], [1543, 1542, 21304], [1545, 1546, 36226], [1548, 1547, 38274], [1549, 1550, 71936], [1552, 1553, 29061], [1557, 1556, 58231], [1561, 1558, 18530], [1559, 1560, 40824], [1559, 1561, 52911], [1563, 1562, 59080], [1562, 1565, 91953], [1563, 1564, 93346], [1569, 1567, 13498], [1568, 1569, 35853], [1574, 1572, 8646], [1574, 1573, 63547], [1575, 1574, 77020], [1575, 1576, 15041], [1579, 1580, 2626], [1581, 1582, 32861], [1584, 1583, 49417], [1585, 1584, 59207], [1588, 1587, 29487], [1591, 1590, 43277], [1592, 1593, 60017], [1598, 1596, 4041], [1597, 1598, 3977], [1601, 1600, 24461], [1605, 1604, 93130], [1605, 1606, 76992], [1609, 1610, 42603], [1613, 1611, 6545], [1611, 1612, 25758], [1614, 1615, 29520], [1617, 1619, 9806], [1619, 1618, 26260], [1619, 1620, 92725], [1624, 1623, 47413], [1625, 1628, 20070], [1625, 1626, 6282], [1628, 1627, 35787], [1630, 1629, 49543], [1632, 1631, 58408], [1639, 1635, 19217], [1639, 1636, 36080], [1639, 1637, 22789], [1637, 1638, 5520], [1641, 1642, 11792], [1646, 1645, 71928], [1645, 1647, 77713], [1653, 1652, 8734], [1659, 1655, 4791], [1656, 1659, 71732], [1659, 1657, 56750], [1659, 1658, 6131], [1668, 1666, 29179], [1667, 1668, 74146], [1672, 1671, 1034], [1678, 1679, 3947], [1684, 1683, 35528], [1685, 1684, 22017], [1688, 1686, 34614], [1688, 1687, 3109], [1694, 1696, 27063], [1696, 1695, 33712], [1700, 1698, 30551], [1698, 1701, 5649], [1702, 1699, 12274], [1700, 1702, 32310], [1707, 1706, 142], [1707, 1708, 1716], [1712, 1709, 62285], [1709, 1713, 35114], [1709, 1710, 2396], [1712, 1711, 65831], [1715, 1714, 3757], [1718, 1716, 19413], [1718, 1717, 26246], [1719, 1720, 15883], [1723, 1724, 1318], [1728, 1729, 34218], [1734, 1735, 21385], [1740, 1739, 3667], [1743, 1744, 11746], [1747, 1746, 30269], [1747, 1748, 64385], [1752, 1751, 41034], [1761, 1759, 17762], [1761, 1760, 3916], [1765, 1766, 63974], [1770, 1769, 81587], [1784, 1782, 12510], [1784, 1783, 1163], [1789, 1790, 24445], [1791, 1793, 33114], [1791, 1792, 42262], [1798, 1797, 30644], [1820, 1817, 75362], [1820, 1818, 64954], [1820, 1819, 62970], [1819, 1821, 3021], [1823, 1822, 18956], [1823, 1824, 84317], [1829, 1827, 3448], [1827, 1828, 51541], [1832, 1834, 55561], [1832, 1833, 3618], [1836, 1835, 41648], [1839, 1838, 12748], [1843, 1842, 11874], [1848, 1847, 9705], [1853, 1852, 1843], [1856, 1855, 27475], [1864, 1863, 14142], [1866, 1865, 72179], [1868, 1867, 49616], [1870, 1869, 8645], [1878, 1879, 31787], [1881, 1882, 27941], [1888, 1887, 19543], [1892, 1893, 48105], [1900, 1895, 55968], [1895, 1897, 10506], [1902, 1896, 69172], [1898, 1902, 66523], [1900, 1899, 13992], [1901, 1900, 45061], [1902, 1901, 21895], [1910, 1909, 25903], [1910, 1911, 24792], [1912, 1913, 38472], [1916, 1915, 47197], [1923, 1922, 91909], [1924, 1925, 69036], [1930, 1931, 29731], [1936, 1935, 41664], [1939, 1940, 18306], [1944, 1943, 40634], [1947, 1948, 62791], [1950, 1951, 45295], [1955, 1954, 73324], [1969, 1968, 29322], [1975, 1973, 20033], [1973, 1977, 16786], [1973, 1974, 61321], [1975, 1976, 65519], [1988, 1987, 31842], [1992, 1993, 25855], [1993, 1994, 8575], [1997, 1998, 1120], [1997, 1999, 18282], [2004, 2005, 88949], [2022, 2023, 16293], [2022, 2024, 44907], [2026, 2025, 22527], [2038, 2037, 34749], [2075, 2074, 2768], [2076, 2077, 71952], [2108, 2107, 71217], [2151, 2150, 17912]]

# well_status= [3, 5, 6, 9, 10, 15, 16, 18, 20, 21, 41, 44, 48, 50, 51, 52, 53, 54, 58, 60, 61, 72, 77, 78, 79, 82, 89, 91, 92, 97, 98, 99, 100, 102, 107, 115, 116, 117, 119, 121, 123, 125, 126, 131, 132, 134, 142, 143, 145, 147, 148, 149, 150, 151, 152, 155, 161, 163, 164, 165, 166, 168, 170, 175, 177, 183, 184, 187, 188, 189, 193, 196, 198, 200, 203, 210, 212, 215, 218, 221, 222, 227, 228, 230, 231, 234, 235, 236, 239, 240, 242, 244, 246, 247, 250, 253, 256, 262, 269, 274, 275, 281, 285, 291, 297, 300, 303, 310, 312, 313, 314, 315, 322, 323, 327, 331, 332, 333, 336, 338, 346, 347, 350, 353, 358, 362, 364, 367, 370, 371, 380, 382, 383, 386, 388, 392, 393, 394, 398, 402, 405, 407, 409, 410, 414, 416, 417, 420, 421, 422, 426, 428, 431, 433, 434, 437, 438, 439, 440, 444, 447, 448, 451, 456, 460, 466, 470, 471, 472, 473, 478, 481, 482, 483, 486, 487, 489, 490, 492, 493, 494, 496, 497, 500, 502, 506, 509, 511, 512, 514, 517, 523, 524, 529, 531, 533, 534, 538, 540, 544, 549, 550, 557, 561, 562, 564, 565, 566, 567, 568, 569, 570, 571, 574, 584, 585, 589, 593, 596, 611, 613, 614, 615, 616, 618, 619, 621, 624, 626, 627, 632, 634, 636, 639, 644, 647, 648, 649, 655, 657, 659, 661, 665, 666, 667, 672, 673, 678, 680, 681, 683, 684, 691, 692, 697, 699, 700, 705, 707, 708, 710, 712, 716, 717, 720, 724, 725, 729, 731, 732, 733, 736, 737, 740, 749, 756, 757, 761, 763, 764, 765, 766, 767, 770, 775, 776, 780, 782, 784, 785, 787, 789, 794, 796, 797, 798, 799, 802, 805, 807, 809, 811, 812, 813, 822, 823, 830, 833, 835, 839, 840, 842, 845, 846, 847, 851, 854, 861, 862, 866, 869, 873, 875, 878, 880, 885, 892, 894, 896, 898, 900, 903, 904, 905, 907, 914, 915, 917, 921, 923, 925, 926, 933, 935, 938, 942, 943, 945, 946, 947, 953, 962, 963, 966, 967, 968, 969, 972, 973, 976, 977, 979, 981, 983, 988, 989, 991, 996, 998, 999, 1001, 1002, 1003, 1004, 1005, 1006, 1008, 1011, 1012, 1014, 1016, 1018, 1019, 1020, 1021, 1025, 1027, 1030, 1033, 1035, 1040, 1041, 1042, 1043, 1044, 1045, 1047, 1048, 1050, 1054, 1057, 1058, 1059, 1065, 1068, 1069, 1070, 1073, 1074, 1075, 1078, 1079, 1080, 1081, 1082, 1088, 1091, 1097, 1100, 1103, 1105, 1107, 1108, 1111, 1112, 1113, 1115, 1116, 1119, 1122, 1123, 1124, 1130, 1133, 1135, 1139, 1140, 1143, 1145, 1147, 1148, 1149, 1151, 1156, 1157, 1158, 1159, 1160, 1161, 1163, 1164, 1170, 1171, 1174, 1178, 1179, 1181, 1183, 1185, 1187, 1188, 1192, 1194, 1198, 1203, 1205, 1206, 1207, 1211, 1214, 1215, 1216, 1217, 1219, 1221, 1222, 1223, 1225, 1228, 1230, 1231, 1234, 1238, 1239, 1241, 1243, 1244, 1247, 1248, 1249, 1251, 1254, 1255, 1259, 1260, 1263, 1264, 1266, 1268, 1270, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1282, 1283, 1285, 1289, 1290, 1293, 1294, 1295, 1297, 1298, 1302, 1303, 1304, 1307, 1308, 1310, 1313, 1323, 1325, 1328, 1332, 1334, 1337, 1340, 1341, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1351, 1352, 1353, 1354, 1358, 1359, 1360, 1361, 1363, 1365, 1366, 1367, 1370, 1372, 1373, 1374, 1376, 1377, 1378, 1381, 1383, 1385, 1388, 1390, 1393, 1397, 1400, 1402, 1403, 1405, 1407, 1412, 1414, 1419, 1420, 1422, 1423, 1424, 1426, 1427, 1433, 1438, 1439, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1453, 1454, 1456, 1458, 1464, 1471, 1473, 1475, 1476, 1478, 1480, 1483, 1484, 1486, 1487, 1488, 1489, 1490, 1492, 1493, 1496, 1498, 1500, 1501, 1502, 1503, 1506, 1507, 1509, 1511, 1512, 1514, 1515, 1517, 1518, 1520, 1522, 1523, 1526, 1527, 1528, 1530, 1531, 1532, 1533, 1535, 1536, 1538, 1539, 1540, 1542, 1543, 1544, 1545, 1546, 1548, 1549, 1550, 1552, 1553, 1554, 1556, 1557, 1558, 1559, 1561, 1562, 1563, 1566, 1567, 1575, 1576, 1579, 1580, 1581, 1584, 1586, 1587, 1589, 1590, 1591, 1592, 1593, 1596, 1598, 1599, 1600, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1613, 1614, 1615, 1616, 1617, 1618, 1621, 1624, 1626, 1627, 1628, 1630, 1631, 1632, 1634, 1636, 1637, 1639, 1640, 1642, 1643, 1645, 1647, 1648, 1649, 1651, 1652, 1653, 1655, 1656, 1658, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1672, 1675, 1676, 1678, 1679, 1680, 1681, 1682, 1683, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1705, 1707, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1718, 1719, 1722, 1723, 1725, 1726, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1744, 1745, 1747, 1748, 1750, 1752, 1753, 1756, 1757, 1759, 1760, 1761, 1763, 1764, 1765, 1767, 1768, 1770, 1772, 1773, 1774, 1775, 1776, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1789, 1792, 1794, 1795, 1798, 1799, 1800, 1801, 1804, 1806, 1807, 1808, 1809, 1811, 1812, 1813, 1814, 1815, 1816, 1819, 1820, 1821, 1822, 1824, 1825, 1828, 1829, 1830, 1831, 1832, 1835, 1836, 1837, 1838, 1840, 1841, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1869, 1872, 1874, 1875, 1876, 1877, 1880, 1881, 1882, 1884, 1886, 1889, 1892, 1893, 1894, 1896, 1897, 1898, 1900, 1901, 1902, 1903, 1905, 1906, 1907, 1908, 1909, 1911, 1913, 1914, 1916, 1917, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1951, 1952, 1953, 1954, 1955, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1966, 1967, 1968, 1969, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1980, 1982, 1983, 1984, 1985, 1986, 1987, 1990, 1991, 1992, 1993, 1994, 1997, 1998, 2000, 2001, 2002, 2004, 2005, 2007, 2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2026, 2027, 2028, 2029, 2030, 2031, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2061, 2062, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2081, 2082, 2083, 2084, 2085, 2086, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160]
# pipe_status= [(1, 9), (2, 3), (4, 9), (4, 12), (5, 7), (8, 9), (10, 11), (11, 13), (14, 15), (17, 18), (19, 20), (21, 31), (21, 33), (21, 136), (22, 43), (22, 147), (23, 81), (23, 145), (24, 25), (24, 69), (26, 34), (26, 62), (26, 85), (26, 152), (27, 128), (28, 96), (29, 44), (30, 45), (30, 51), (30, 109), (30, 135), (32, 75), (33, 36), (34, 94), (35, 54), (36, 114), (37, 117), (38, 58), (38, 74), (38, 113), (38, 127), (38, 144), (39, 140), (39, 141), (40, 95), (42, 44), (46, 158), (47, 146), (49, 156), (52, 88), (53, 95), (54, 68), (55, 75), (56, 80), (57, 86), (57, 119), (59, 60), (60, 129), (61, 108), (61, 153), (62, 90), (63, 159), (64, 95), (64, 137), (65, 79), (66, 112), (66, 140), (67, 95), (67, 106), (68, 110), (69, 115), (70, 118), (71, 143), (72, 105), (72, 156), (73, 104), (75, 92), (76, 80), (80, 88), (82, 101), (82, 118), (82, 159), (83, 123), (84, 87), (84, 116), (85, 146), (89, 93), (92, 160), (93, 111), (93, 139), (96, 117), (98, 124), (99, 140), (103, 149), (103, 154), (104, 121), (108, 128), (120, 143), (122, 148), (127, 157), (130, 145), (132, 138), (133, 157), (143, 158), (162, 183), (163, 171), (167, 178), (169, 177), (169, 181), (169, 182), (171, 179), (171, 180), (172, 183), (173, 174), (174, 177), (174, 178), (176, 183), (184, 185), (186, 188), (190, 191), (190, 192), (190, 193), (193, 195), (194, 195), (196, 197), (199, 200), (200, 201), (202, 203), (203, 204), (205, 207), (205, 209), (205, 210), (206, 208), (208, 209), (211, 213), (211, 214), (212, 213), (215, 216), (215, 217), (218, 219), (218, 220), (221, 225), (221, 229), (223, 228), (224, 226), (224, 227), (231, 238), (232, 234), (233, 245), (237, 240), (239, 243), (241, 243), (244, 245), (247, 248), (248, 249), (251, 252), (251, 253), (251, 254), (255, 274), (256, 261), (256, 264), (257, 263), (257, 277), (258, 267), (259, 266), (260, 267), (261, 276), (263, 269), (264, 265), (266, 268), (267, 271), (267, 273), (268, 270), (268, 272), (268, 277), (271, 275), (278, 279), (278, 290), (279, 280), (279, 295), (279, 329), (279, 342), (280, 319), (281, 304), (282, 346), (283, 344), (284, 316), (284, 345), (285, 356), (286, 311), (286, 323), (287, 289), (287, 292), (287, 299), (288, 329), (289, 359), (289, 378), (291, 373), (292, 339), (292, 363), (293, 316), (294, 312), (295, 303), (295, 316), (295, 352), (295, 375), (296, 320), (296, 344), (297, 298), (298, 318), (298, 357), (299, 308), (300, 355), (301, 344), (302, 326), (305, 376), (306, 321), (306, 329), (307, 366), (308, 332), (308, 348), (309, 310), (309, 335), (311, 324), (312, 361), (315, 326), (316, 328), (316, 344), (317, 371), (318, 349), (318, 366), (320, 379), (325, 334), (327, 337), (329, 340), (329, 351), (329, 360), (330, 341), (331, 365), (334, 338), (336, 341), (343, 372), (344, 376), (349, 368), (349, 372), (350, 374), (354, 355), (364, 369), (369, 377), (380, 381), (383, 385), (384, 391), (385, 390), (387, 392), (388, 396), (389, 397), (391, 397), (392, 397), (393, 395), (399, 406), (399, 407), (400, 406), (401, 403), (403, 404), (404, 405), (406, 408), (411, 414), (412, 414), (413, 420), (414, 415), (414, 418), (414, 419), (423, 424), (423, 432), (424, 429), (425, 426), (427, 429), (427, 430), (429, 431), (434, 435), (435, 436), (440, 441), (441, 442), (442, 443), (444, 458), (445, 465), (446, 450), (447, 465), (448, 449), (448, 454), (448, 459), (450, 455), (450, 457), (450, 460), (450, 463), (452, 469), (453, 467), (456, 468), (458, 461), (458, 462), (458, 467), (464, 467), (467, 469), (473, 474), (473, 477), (475, 477), (476, 477), (479, 480), (479, 482), (483, 484), (485, 488), (487, 488), (491, 492), (495, 501), (498, 506), (498, 513), (499, 501), (500, 504), (500, 510), (501, 503), (503, 507), (503, 512), (505, 509), (508, 512), (514, 515), (516, 522), (516, 523), (516, 527), (517, 526), (518, 527), (519, 527), (520, 523), (521, 522), (522, 525), (523, 528), (529, 530), (532, 533), (535, 544), (536, 550), (537, 546), (539, 547), (541, 549), (542, 543), (542, 548), (543, 547), (545, 547), (546, 549), (547, 550), (547, 551), (552, 558), (553, 559), (554, 555), (554, 556), (554, 557), (555, 558), (559, 560), (559, 561), (562, 563), (566, 572), (567, 578), (567, 594), (568, 588), (568, 598), (570, 602), (571, 575), (572, 580), (572, 587), (572, 590), (572, 591), (573, 597), (574, 595), (574, 601), (575, 576), (575, 581), (575, 592), (575, 606), (576, 604), (577, 600), (579, 616), (580, 612), (582, 598), (583, 607), (583, 616), (586, 603), (586, 611), (587, 597), (589, 608), (599, 600), (600, 610), (600, 611), (605, 614), (608, 609), (617, 618), (620, 622), (621, 622), (623, 626), (625, 626), (627, 628), (628, 629), (630, 632), (630, 633), (631, 632), (634, 637), (635, 636), (638, 639), (639, 640), (639, 641), (641, 642), (643, 645), (644, 645), (646, 647), (649, 650), (651, 654), (651, 662), (652, 656), (652, 658), (652, 661), (653, 662), (654, 657), (655, 660), (662, 663), (664, 666), (664, 669), (668, 669), (669, 670), (670, 671), (673, 674), (673, 676), (674, 675), (677, 682), (679, 681), (679, 682), (685, 686), (686, 687), (686, 688), (687, 697), (689, 692), (689, 694), (690, 692), (691, 695), (692, 693), (695, 696), (698, 699), (700, 701), (701, 702), (701, 703), (703, 704), (705, 706), (708, 709), (710, 711), (712, 713), (713, 714), (715, 718), (716, 718), (717, 719), (721, 722), (721, 723), (721, 725), (726, 730), (727, 731), (728, 729), (729, 730), (733, 735), (734, 743), (736, 744), (737, 754), (738, 739), (739, 740), (741, 752), (742, 743), (743, 745), (743, 756), (744, 751), (744, 752), (746, 750), (746, 753), (746, 755), (747, 749), (748, 755), (749, 755), (758, 760), (758, 761), (759, 762), (760, 762), (766, 774), (767, 768), (769, 770), (771, 772), (771, 774), (773, 774), (777, 778), (777, 781), (778, 786), (778, 787), (779, 781), (780, 792), (781, 788), (782, 790), (782, 793), (783, 785), (783, 791), (794, 795), (800, 804), (801, 802), (802, 803), (803, 804), (806, 809), (808, 822), (808, 824), (810, 835), (811, 820), (812, 843), (813, 819), (814, 839), (815, 825), (816, 826), (817, 832), (817, 846), (818, 831), (821, 826), (823, 834), (825, 843), (826, 827), (826, 841), (826, 845), (828, 840), (829, 840), (831, 839), (833, 838), (835, 844), (836, 843), (837, 846), (848, 851), (848, 860), (849, 858), (850, 855), (851, 852), (851, 858), (852, 856), (853, 855), (854, 857), (855, 857), (856, 859), (862, 863), (863, 864), (865, 866), (866, 867), (866, 868), (870, 871), (870, 872), (870, 873), (872, 874), (875, 877), (876, 878), (876, 879), (877, 881), (882, 889), (882, 891), (883, 890), (883, 891), (884, 888), (885, 890), (886, 887), (886, 891), (888, 891), (892, 893), (895, 897), (896, 897), (899, 900), (901, 903), (902, 903), (906, 907), (906, 908), (907, 909), (908, 910), (911, 912), (912, 914), (913, 914), (915, 916), (917, 919), (918, 919), (919, 920), (922, 923), (924, 930), (924, 935), (925, 929), (927, 928), (927, 929), (927, 931), (927, 936), (927, 939), (928, 940), (928, 941), (931, 937), (932, 941), (933, 934), (944, 945), (948, 955), (948, 958), (948, 959), (949, 953), (950, 958), (951, 963), (952, 962), (954, 956), (956, 963), (957, 958), (958, 961), (959, 960), (961, 963), (964, 966), (965, 966), (969, 970), (971, 972), (974, 976), (974, 978), (975, 976), (980, 981), (980, 984), (982, 983), (985, 987), (986, 988), (987, 988), (989, 990), (989, 993), (991, 992), (994, 996), (995, 996), (996, 997), (1000, 1002), (1007, 1008), (1008, 1010), (1008, 1015), (1009, 1015), (1010, 1017), (1013, 1016), (1022, 1025), (1023, 1025), (1024, 1025), (1024, 1026), (1028, 1030), (1028, 1038), (1028, 1039), (1029, 1039), (1031, 1036), (1032, 1038), (1033, 1036), (1034, 1035), (1037, 1038), (1045, 1046), (1048, 1049), (1050, 1051), (1050, 1053), (1052, 1053), (1055, 1056), (1055, 1057), (1059, 1060), (1061, 1066), (1062, 1064), (1063, 1066), (1064, 1068), (1066, 1067), (1067, 1068), (1071, 1072), (1072, 1073), (1076, 1077), (1076, 1078), (1081, 1084), (1082, 1085), (1083, 1085), (1086, 1090), (1087, 1090), (1088, 1090), (1089, 1090), (1092, 1097), (1092, 1098), (1093, 1094), (1094, 1096), (1095, 1097), (1095, 1099), (1096, 1097), (1100, 1101), (1101, 1102), (1103, 1104), (1105, 1106), (1108, 1109), (1110, 1112), (1113, 1114), (1115, 1117), (1118, 1126), (1119, 1120), (1119, 1127), (1121, 1128), (1122, 1125), (1123, 1128), (1123, 1129), (1124, 1126), (1131, 1137), (1132, 1136), (1133, 1138), (1134, 1139), (1135, 1136), (1137, 1139), (1140, 1141), (1141, 1142), (1144, 1145), (1146, 1147), (1150, 1151), (1151, 1152), (1153, 1154), (1153, 1157), (1155, 1156), (1161, 1162), (1164, 1165), (1166, 1170), (1167, 1170), (1168, 1169), (1168, 1170), (1172, 1174), (1173, 1175), (1174, 1176), (1175, 1176), (1177, 1178), (1180, 1182), (1181, 1182), (1184, 1185), (1184, 1186), (1188, 1189), (1190, 1201), (1191, 1205), (1192, 1195), (1192, 1201), (1193, 1203), (1194, 1204), (1195, 1200), (1196, 1201), (1197, 1198), (1197, 1199), (1202, 1206), (1208, 1212), (1208, 1213), (1209, 1210), (1209, 1211), (1211, 1213), (1217, 1218), (1219, 1220), (1223, 1224), (1225, 1226), (1226, 1227), (1229, 1230), (1232, 1234), (1232, 1236), (1233, 1234), (1233, 1235), (1237, 1238), (1239, 1240), (1241, 1242), (1245, 1247), (1246, 1247), (1250, 1254), (1252, 1253), (1253, 1254), (1255, 1257), (1256, 1257), (1258, 1259), (1261, 1262), (1262, 1263), (1264, 1265), (1267, 1268), (1268, 1269), (1270, 1271), (1279, 1280), (1280, 1281), (1284, 1289), (1285, 1286), (1285, 1288), (1287, 1288), (1291, 1292), (1291, 1294), (1295, 1296), (1299, 1301), (1299, 1302), (1300, 1301), (1303, 1305), (1304, 1306), (1308, 1309), (1311, 1314), (1312, 1314), (1312, 1317), (1313, 1314), (1313, 1315), (1313, 1316), (1317, 1318), (1319, 1321), (1319, 1323), (1320, 1321), (1321, 1322), (1323, 1324), (1326, 1328), (1326, 1330), (1327, 1330), (1328, 1329), (1331, 1332), (1332, 1333), (1334, 1335), (1334, 1336), (1337, 1338), (1338, 1339), (1341, 1342), (1350, 1351), (1352, 1356), (1355, 1356), (1357, 1358), (1361, 1362), (1364, 1365), (1368, 1370), (1369, 1370), (1370, 1371), (1374, 1375), (1379, 1381), (1380, 1382), (1381, 1382), (1383, 1384), (1385, 1386), (1386, 1387), (1389, 1390), (1389, 1392), (1391, 1393), (1392, 1395), (1393, 1394), (1395, 1396), (1397, 1398), (1397, 1399), (1401, 1402), (1403, 1404), (1405, 1406), (1407, 1408), (1409, 1410), (1409, 1415), (1410, 1417), (1411, 1413), (1411, 1414), (1412, 1416), (1412, 1417), (1418, 1419), (1421, 1422), (1425, 1434), (1426, 1430), (1426, 1434), (1427, 1428), (1427, 1429), (1427, 1435), (1428, 1436), (1431, 1436), (1432, 1436), (1437, 1438), (1440, 1441), (1440, 1442), (1449, 1453), (1450, 1453), (1451, 1455), (1452, 1453), (1452, 1455), (1456, 1457), (1459, 1461), (1459, 1466), (1459, 1470), (1460, 1461), (1461, 1464), (1461, 1465), (1462, 1464), (1463, 1464), (1463, 1468), (1467, 1470), (1469, 1470), (1471, 1472), (1473, 1474), (1476, 1477), (1479, 1481), (1481, 1482), (1482, 1483), (1485, 1486), (1490, 1491), (1493, 1494), (1493, 1495), (1496, 1497), (1497, 1499), (1503, 1504), (1505, 1506), (1507, 1508), (1510, 1511), (1513, 1514), (1515, 1516), (1518, 1519), (1521, 1524), (1523, 1524), (1525, 1526), (1528, 1529), (1533, 1534), (1535, 1537), (1541, 1543), (1546, 1547), (1550, 1551), (1554, 1555), (1558, 1560), (1564, 1566), (1565, 1566), (1567, 1573), (1568, 1575), (1569, 1578), (1570, 1574), (1571, 1580), (1572, 1576), (1574, 1577), (1575, 1578), (1577, 1580), (1581, 1582), (1581, 1583), (1585, 1586), (1588, 1589), (1593, 1594), (1595, 1597), (1595, 1598), (1600, 1601), (1612, 1613), (1618, 1619), (1618, 1620), (1621, 1622), (1623, 1624), (1625, 1626), (1629, 1630), (1633, 1635), (1634, 1635), (1638, 1639), (1640, 1644), (1641, 1642), (1646, 1647), (1649, 1650), (1654, 1655), (1657, 1658), (1659, 1663), (1660, 1663), (1661, 1662), (1661, 1663), (1671, 1672), (1673, 1674), (1673, 1675), (1676, 1677), (1683, 1684), (1694, 1695), (1701, 1703), (1704, 1705), (1706, 1707), (1707, 1708), (1715, 1716), (1716, 1717), (1719, 1720), (1721, 1722), (1723, 1724), (1726, 1727), (1735, 1736), (1746, 1747), (1749, 1750), (1751, 1752), (1753, 1754), (1755, 1756), (1757, 1758), (1762, 1763), (1766, 1767), (1769, 1770), (1771, 1772), (1776, 1778), (1777, 1778), (1779, 1780), (1788, 1790), (1788, 1791), (1789, 1790), (1793, 1794), (1795, 1796), (1796, 1797), (1802, 1803), (1802, 1805), (1803, 1804), (1810, 1811), (1817, 1818), (1817, 1819), (1823, 1824), (1825, 1826), (1825, 1827), (1833, 1834), (1834, 1836), (1838, 1839), (1842, 1843), (1852, 1853), (1853, 1854), (1867, 1868), (1869, 1870), (1871, 1872), (1873, 1874), (1877, 1878), (1879, 1881), (1882, 1883), (1884, 1885), (1887, 1890), (1888, 1889), (1888, 1890), (1891, 1892), (1895, 1896), (1899, 1900), (1904, 1905), (1910, 1911), (1910, 1912), (1915, 1916), (1917, 1918), (1919, 1920), (1920, 1921), (1929, 1930), (1949, 1950), (1955, 1956), (1957, 1958), (1965, 1966), (1970, 1971), (1978, 1979), (1981, 1982), (1987, 1988), (1988, 1989), (1994, 1996), (1995, 1996), (1998, 1999), (2003, 2004), (2005, 2006), (2008, 2009), (2024, 2026), (2025, 2026), (2032, 2033), (2060, 2061), (2063, 2064), (2079, 2080), (2087, 2088)]
# wells_mapping= {1: 352, 2: 353, 3: 354, 4: 355, 5: 356, 6: 357, 7: 358, 8: 359, 9: 360, 10: 361, 11: 362, 12: 348, 13: 349, 14: 350, 15: 351, 16: 1919, 17: 786, 18: 787, 19: 1835, 20: 1836, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 121, 123: 122, 124: 123, 125: 124, 126: 125, 127: 126, 128: 127, 129: 128, 130: 129, 131: 130, 132: 131, 133: 132, 134: 133, 135: 134, 136: 135, 137: 136, 138: 137, 139: 138, 140: 139, 141: 140, 142: 141, 143: 142, 144: 143, 145: 144, 146: 145, 147: 146, 148: 147, 149: 148, 150: 149, 151: 150, 152: 151, 153: 152, 154: 153, 155: 154, 156: 155, 157: 156, 158: 157, 159: 158, 160: 159, 161: 1774, 162: 298, 163: 299, 164: 300, 165: 301, 166: 302, 167: 303, 168: 304, 169: 305, 170: 306, 171: 307, 172: 308, 173: 309, 174: 310, 175: 311, 176: 312, 177: 313, 178: 314, 179: 315, 180: 316, 181: 317, 182: 318, 183: 319, 184: 1624, 185: 1623, 186: 480, 187: 481, 188: 479, 189: 1184, 190: 324, 191: 325, 192: 326, 193: 327, 194: 328, 195: 329, 196: 1939, 197: 1940, 198: 1884, 199: 1383, 200: 1384, 201: 1385, 202: 1386, 203: 1387, 204: 1388, 205: 704, 206: 705, 207: 700, 208: 701, 209: 702, 210: 703, 211: 1392, 212: 1393, 213: 1390, 214: 1391, 215: 1501, 216: 1502, 217: 1503, 218: 1611, 219: 1612, 220: 1613, 221: 436, 222: 437, 223: 438, 224: 439, 225: 440, 226: 441, 227: 442, 228: 443, 229: 444, 230: 1269, 231: 363, 232: 364, 233: 365, 234: 366, 235: 367, 236: 368, 237: 369, 238: 370, 239: 371, 240: 372, 241: 373, 242: 374, 243: 375, 244: 376, 245: 377, 246: 1291, 247: 1909, 248: 1910, 249: 1911, 250: 1234, 251: 1235, 252: 1236, 253: 1237, 254: 1238, 255: 512, 256: 513, 257: 514, 258: 515, 259: 516, 260: 517, 261: 518, 262: 496, 263: 497, 264: 498, 265: 499, 266: 500, 267: 501, 268: 502, 269: 503, 270: 504, 271: 505, 272: 506, 273: 507, 274: 508, 275: 509, 276: 510, 277: 511, 278: 555, 279: 556, 280: 557, 281: 558, 282: 559, 283: 560, 284: 561, 285: 562, 286: 563, 287: 564, 288: 565, 289: 566, 290: 567, 291: 568, 292: 569, 293: 570, 294: 571, 295: 572, 296: 573, 297: 574, 298: 575, 299: 576, 300: 577, 301: 578, 302: 579, 303: 580, 304: 581, 305: 582, 306: 583, 307: 584, 308: 585, 309: 586, 310: 587, 311: 588, 312: 589, 313: 590, 314: 591, 315: 592, 316: 593, 317: 594, 318: 595, 319: 596, 320: 597, 321: 598, 322: 599, 323: 600, 324: 601, 325: 602, 326: 603, 327: 604, 328: 605, 329: 606, 330: 607, 331: 608, 332: 609, 333: 610, 334: 611, 335: 612, 336: 613, 337: 614, 338: 615, 339: 616, 340: 617, 341: 618, 342: 619, 343: 620, 344: 621, 345: 622, 346: 623, 347: 624, 348: 625, 349: 626, 350: 627, 351: 628, 352: 629, 353: 630, 354: 631, 355: 632, 356: 633, 357: 634, 358: 635, 359: 636, 360: 637, 361: 638, 362: 639, 363: 640, 364: 641, 365: 642, 366: 643, 367: 644, 368: 645, 369: 646, 370: 647, 371: 648, 372: 649, 373: 650, 374: 651, 375: 652, 376: 653, 377: 654, 378: 655, 379: 656, 380: 1720, 381: 1719, 382: 1721, 383: 1353, 384: 1354, 385: 1355, 386: 1356, 387: 1357, 388: 1358, 389: 1359, 390: 1360, 391: 1361, 392: 1362, 393: 1363, 394: 1364, 395: 1365, 396: 1366, 397: 1367, 398: 1368, 399: 900, 400: 901, 401: 902, 402: 903, 403: 904, 404: 905, 405: 906, 406: 907, 407: 908, 408: 909, 409: 1389, 410: 1777, 411: 1188, 412: 1189, 413: 1190, 414: 1191, 415: 1192, 416: 1193, 417: 1194, 418: 1195, 419: 1196, 420: 1197, 421: 1154, 422: 1726, 423: 175, 424: 176, 425: 177, 426: 178, 427: 179, 428: 180, 429: 181, 430: 182, 431: 183, 432: 184, 433: 1785, 434: 1185, 435: 1186, 436: 1187, 437: 1764, 438: 2058, 439: 1957, 440: 1152, 441: 1153, 442: 1150, 443: 1151, 444: 709, 445: 710, 446: 711, 447: 712, 448: 713, 449: 714, 450: 715, 451: 716, 452: 717, 453: 718, 454: 719, 455: 720, 456: 721, 457: 722, 458: 723, 459: 724, 460: 725, 461: 726, 462: 727, 463: 728, 464: 729, 465: 730, 466: 731, 467: 732, 468: 733, 469: 734, 470: 1270, 471: 1554, 472: 2111, 473: 1258, 474: 1259, 475: 1260, 476: 1261, 477: 1262, 478: 1906, 479: 672, 480: 669, 481: 670, 482: 671, 483: 1176, 484: 1175, 485: 1155, 486: 1156, 487: 1157, 488: 1158, 489: 1589, 490: 549, 491: 550, 492: 551, 493: 1336, 494: 1773, 495: 856, 496: 857, 497: 858, 498: 859, 499: 860, 500: 861, 501: 862, 502: 863, 503: 864, 504: 865, 505: 866, 506: 867, 507: 868, 508: 869, 509: 870, 510: 871, 511: 872, 512: 873, 513: 874, 514: 1424, 515: 1423, 516: 7, 517: 8, 518: 9, 519: 10, 520: 11, 521: 12, 522: 13, 523: 14, 524: 15, 525: 16, 526: 17, 527: 18, 528: 19, 529: 1641, 530: 1642, 531: 1737, 532: 1417, 533: 1418, 534: 1860, 535: 1028, 536: 1029, 537: 1030, 538: 1031, 539: 1032, 540: 1033, 541: 1034, 542: 1035, 543: 1036, 544: 1037, 545: 1038, 546: 1039, 547: 1040, 548: 1041, 549: 1042, 550: 1043, 551: 1044, 552: 1198, 553: 1199, 554: 1200, 555: 1201, 556: 1202, 557: 1203, 558: 1204, 559: 1205, 560: 1206, 561: 1207, 562: 1856, 563: 1855, 564: 1967, 565: 1494, 566: 385, 567: 386, 568: 387, 569: 388, 570: 389, 571: 390, 572: 391, 573: 392, 574: 393, 575: 394, 576: 395, 577: 396, 578: 397, 579: 398, 580: 399, 581: 400, 582: 401, 583: 402, 584: 403, 585: 404, 586: 405, 587: 406, 588: 407, 589: 408, 590: 409, 591: 410, 592: 411, 593: 412, 594: 413, 595: 414, 596: 415, 597: 416, 598: 417, 599: 418, 600: 419, 601: 420, 602: 421, 603: 422, 604: 423, 605: 424, 606: 425, 607: 426, 608: 427, 609: 428, 610: 429, 611: 430, 612: 431, 613: 432, 614: 433, 615: 434, 616: 435, 617: 1987, 618: 1988, 619: 1762, 620: 1025, 621: 1026, 622: 1027, 623: 1320, 624: 1321, 625: 1322, 626: 1319, 627: 1485, 628: 1486, 629: 1487, 630: 674, 631: 675, 632: 676, 633: 677, 634: 1560, 635: 1561, 636: 1558, 637: 1559, 638: 803, 639: 804, 640: 805, 641: 806, 642: 807, 643: 1572, 644: 1573, 645: 1574, 646: 1575, 647: 1576, 648: 1792, 649: 1793, 650: 1791, 651: 256, 652: 257, 653: 258, 654: 259, 655: 260, 656: 261, 657: 262, 658: 263, 659: 264, 660: 265, 661: 266, 662: 267, 663: 268, 664: 1895, 665: 1896, 666: 1897, 667: 1898, 668: 1899, 669: 1900, 670: 1901, 671: 1902, 672: 1607, 673: 537, 674: 538, 675: 539, 676: 540, 677: 452, 678: 453, 679: 454, 680: 455, 681: 456, 682: 457, 683: 2071, 684: 1877, 685: 768, 686: 769, 687: 770, 688: 758, 689: 759, 690: 760, 691: 761, 692: 762, 693: 763, 694: 764, 695: 765, 696: 766, 697: 767, 698: 1080, 699: 1081, 700: 1280, 701: 1281, 702: 1282, 703: 1278, 704: 1279, 705: 1148, 706: 1149, 707: 2146, 708: 1106, 709: 1107, 710: 1888, 711: 1887, 712: 235, 713: 236, 714: 237, 715: 1002, 716: 1003, 717: 1004, 718: 1005, 719: 1006, 720: 1644, 721: 1973, 722: 1974, 723: 1975, 724: 1976, 725: 1977, 726: 544, 727: 545, 728: 546, 729: 547, 730: 548, 731: 543, 732: 2010, 733: 832, 734: 833, 735: 834, 736: 835, 737: 836, 738: 837, 739: 838, 740: 839, 741: 840, 742: 841, 743: 842, 744: 843, 745: 844, 746: 845, 747: 846, 748: 847, 749: 848, 750: 849, 751: 850, 752: 851, 753: 852, 754: 853, 755: 854, 756: 855, 757: 2116, 758: 1698, 759: 1699, 760: 1700, 761: 1701, 762: 1702, 763: 2032, 764: 1566, 765: 1395, 766: 972, 767: 973, 768: 974, 769: 975, 770: 976, 771: 977, 772: 978, 773: 979, 774: 980, 775: 1745, 776: 2120, 777: 288, 778: 289, 779: 273, 780: 274, 781: 275, 782: 276, 783: 277, 784: 278, 785: 279, 786: 280, 787: 281, 788: 282, 789: 283, 790: 284, 791: 285, 792: 286, 793: 287, 794: 1075, 795: 1076, 796: 1616, 797: 878, 798: 1693, 799: 1301, 800: 1302, 801: 1303, 802: 1304, 803: 1305, 804: 1306, 805: 187, 806: 188, 807: 189, 808: 190, 809: 191, 810: 192, 811: 193, 812: 194, 813: 195, 814: 196, 815: 197, 816: 198, 817: 199, 818: 200, 819: 201, 820: 202, 821: 203, 822: 204, 823: 205, 824: 206, 825: 207, 826: 208, 827: 209, 828: 210, 829: 211, 830: 212, 831: 213, 832: 214, 833: 215, 834: 216, 835: 217, 836: 218, 837: 219, 838: 220, 839: 221, 840: 222, 841: 223, 842: 224, 843: 225, 844: 226, 845: 227, 846: 228, 847: 1128, 848: 681, 849: 682, 850: 683, 851: 684, 852: 685, 853: 686, 854: 687, 855: 688, 856: 689, 857: 690, 858: 691, 859: 692, 860: 693, 861: 694, 862: 1474, 863: 1475, 864: 1476, 865: 816, 866: 813, 867: 814, 868: 815, 869: 1837, 870: 981, 871: 982, 872: 983, 873: 984, 874: 985, 875: 448, 876: 449, 877: 450, 878: 451, 879: 445, 880: 446, 881: 447, 882: 659, 883: 660, 884: 661, 885: 662, 886: 663, 887: 664, 888: 665, 889: 666, 890: 667, 891: 668, 892: 185, 893: 186, 894: 1876, 895: 1241, 896: 1242, 897: 1243, 898: 1074, 899: 1954, 900: 1955, 901: 1120, 902: 1121, 903: 1119, 904: 2155, 905: 1673, 906: 960, 907: 961, 908: 962, 909: 963, 910: 959, 911: 1217, 912: 1218, 913: 1219, 914: 1220, 915: 1400, 916: 1401, 917: 1402, 918: 1403, 919: 1404, 920: 1405, 921: 1753, 922: 233, 923: 234, 924: 330, 925: 331, 926: 332, 927: 333, 928: 334, 929: 335, 930: 336, 931: 337, 932: 338, 933: 339, 934: 340, 935: 341, 936: 342, 937: 343, 938: 344, 939: 345, 940: 346, 941: 347, 942: 1654, 943: 801, 944: 1136, 945: 1137, 946: 1138, 947: 1, 948: 932, 949: 933, 950: 934, 951: 935, 952: 936, 953: 937, 954: 938, 955: 939, 956: 940, 957: 941, 958: 942, 959: 943, 960: 944, 961: 945, 962: 946, 963: 947, 964: 1596, 965: 1597, 966: 1598, 967: 1733, 968: 1324, 969: 1325, 970: 1326, 971: 541, 972: 542, 973: 2118, 974: 1216, 975: 1212, 976: 1213, 977: 1214, 978: 1215, 979: 232, 980: 1709, 981: 1710, 982: 1711, 983: 1712, 984: 1713, 985: 1371, 986: 1372, 987: 1373, 988: 1374, 989: 1108, 990: 1109, 991: 1110, 992: 1111, 993: 1112, 994: 1617, 995: 1618, 996: 1619, 997: 1620, 998: 1523, 999: 1435, 1000: 1209, 1001: 1210, 1002: 1211, 1003: 477, 1004: 2029, 1005: 1772, 1006: 1013, 1007: 1162, 1008: 1163, 1009: 1164, 1010: 1165, 1011: 1166, 1012: 1167, 1013: 1168, 1014: 1169, 1015: 1170, 1016: 1171, 1017: 1172, 1018: 1941, 1019: 1980, 1020: 1858, 1021: 2128, 1022: 1817, 1023: 1818, 1024: 1819, 1025: 1820, 1026: 1821, 1027: 1849, 1028: 160, 1029: 161, 1030: 162, 1031: 163, 1032: 164, 1033: 165, 1034: 166, 1035: 167, 1036: 168, 1037: 169, 1038: 170, 1039: 171, 1040: 172, 1041: 990, 1042: 1267, 1043: 1908, 1044: 2001, 1045: 246, 1046: 247, 1047: 1978, 1048: 1848, 1049: 1847, 1050: 1625, 1051: 1626, 1052: 1627, 1053: 1628, 1054: 1603, 1055: 1827, 1056: 1828, 1057: 1829, 1058: 1339, 1059: 491, 1060: 492, 1061: 248, 1062: 249, 1063: 250, 1064: 251, 1065: 252, 1066: 253, 1067: 254, 1068: 255, 1069: 2094, 1070: 2073, 1071: 1683, 1072: 1684, 1073: 1685, 1074: 1755, 1075: 1725, 1076: 1296, 1077: 1297, 1078: 1295, 1079: 1780, 1080: 1555, 1081: 1495, 1082: 1496, 1083: 1497, 1084: 1498, 1085: 1499, 1086: 1082, 1087: 1083, 1088: 1084, 1089: 1085, 1090: 1086, 1091: 1274, 1092: 458, 1093: 459, 1094: 460, 1095: 461, 1096: 462, 1097: 463, 1098: 464, 1099: 465, 1100: 1760, 1101: 1761, 1102: 1759, 1103: 1468, 1104: 1469, 1105: 1376, 1106: 1377, 1107: 1375, 1108: 1869, 1109: 1870, 1110: 296, 1111: 297, 1112: 295, 1113: 952, 1114: 951, 1115: 1666, 1116: 1667, 1117: 1668, 1118: 1248, 1119: 1249, 1120: 1250, 1121: 1251, 1122: 1252, 1123: 1253, 1124: 1254, 1125: 1255, 1126: 1256, 1127: 1245, 1128: 1246, 1129: 1247, 1130: 2112, 1131: 1061, 1132: 1062, 1133: 1063, 1134: 1064, 1135: 1065, 1136: 1066, 1137: 1067, 1138: 1068, 1139: 1069, 1140: 1706, 1141: 1707, 1142: 1708, 1143: 1586, 1144: 1744, 1145: 1743, 1146: 2107, 1147: 2108, 1148: 1141, 1149: 1661, 1150: 1345, 1151: 1346, 1152: 1347, 1153: 1348, 1154: 1349, 1155: 1350, 1156: 1351, 1157: 1352, 1158: 1929, 1159: 1934, 1160: 1126, 1161: 1224, 1162: 1225, 1163: 1223, 1164: 1592, 1165: 1593, 1166: 1635, 1167: 1636, 1168: 1637, 1169: 1638, 1170: 1639, 1171: 1113, 1172: 1114, 1173: 1115, 1174: 1116, 1175: 1117, 1176: 1118, 1177: 552, 1178: 553, 1179: 1965, 1180: 824, 1181: 821, 1182: 822, 1183: 823, 1184: 916, 1185: 917, 1186: 918, 1187: 1914, 1188: 1450, 1189: 1451, 1190: 736, 1191: 737, 1192: 738, 1193: 739, 1194: 740, 1195: 741, 1196: 742, 1197: 743, 1198: 744, 1199: 745, 1200: 746, 1201: 747, 1202: 748, 1203: 749, 1204: 750, 1205: 751, 1206: 752, 1207: 735, 1208: 992, 1209: 993, 1210: 994, 1211: 995, 1212: 996, 1213: 991, 1214: 1986, 1215: 1741, 1216: 1722, 1217: 1314, 1218: 1315, 1219: 1924, 1220: 1925, 1221: 1104, 1222: 1440, 1223: 1912, 1224: 1913, 1225: 1020, 1226: 1021, 1227: 1022, 1228: 1023, 1229: 756, 1230: 757, 1231: 1871, 1232: 880, 1233: 881, 1234: 882, 1235: 883, 1236: 884, 1237: 2025, 1238: 2026, 1239: 1173, 1240: 1174, 1241: 898, 1242: 899, 1243: 2064, 1244: 2103, 1245: 986, 1246: 987, 1247: 988, 1248: 989, 1249: 1284, 1250: 695, 1251: 696, 1252: 697, 1253: 698, 1254: 699, 1255: 1541, 1256: 1542, 1257: 1543, 1258: 532, 1259: 533, 1260: 2121, 1261: 1344, 1262: 1342, 1263: 1343, 1264: 1739, 1265: 1740, 1266: 1221, 1267: 1992, 1268: 1993, 1269: 1994, 1270: 1077, 1271: 1078, 1272: 1825, 1273: 1439, 1274: 1273, 1275: 1874, 1276: 1763, 1277: 1669, 1278: 1621, 1279: 829, 1280: 830, 1281: 831, 1282: 1338, 1283: 964, 1284: 239, 1285: 240, 1286: 241, 1287: 242, 1288: 243, 1289: 244, 1290: 1932, 1291: 1562, 1292: 1563, 1293: 1564, 1294: 1565, 1295: 1524, 1296: 1525, 1297: 2138, 1298: 2070, 1299: 772, 1300: 773, 1301: 774, 1302: 775, 1303: 1177, 1304: 1178, 1305: 1179, 1306: 1180, 1307: 1926, 1308: 1437, 1309: 1438, 1310: 887, 1311: 888, 1312: 889, 1313: 890, 1314: 891, 1315: 892, 1316: 893, 1317: 894, 1318: 895, 1319: 466, 1320: 467, 1321: 468, 1322: 469, 1323: 470, 1324: 471, 1325: 2063, 1326: 1008, 1327: 1009, 1328: 1010, 1329: 1011, 1330: 1012, 1331: 810, 1332: 811, 1333: 812, 1334: 1997, 1335: 1998, 1336: 1999, 1337: 1432, 1338: 1433, 1339: 1431, 1340: 2093, 1341: 1600, 1342: 1601, 1343: 1972, 1344: 2089, 1345: 928, 1346: 2157, 1347: 1268, 1348: 1682, 1349: 2119, 1350: 784, 1351: 785, 1352: 1655, 1353: 1656, 1354: 1657, 1355: 1658, 1356: 1659, 1357: 1838, 1358: 1839, 1359: 1704, 1360: 2156, 1361: 1160, 1362: 1161, 1363: 2085, 1364: 1445, 1365: 1446, 1366: 1447, 1367: 2129, 1368: 1530, 1369: 1531, 1370: 1532, 1371: 1533, 1372: 1327, 1373: 2000, 1374: 1316, 1375: 1317, 1376: 536, 1377: 771, 1378: 1382, 1379: 912, 1380: 913, 1381: 914, 1382: 915, 1383: 1609, 1384: 1610, 1385: 825, 1386: 826, 1387: 827, 1388: 1444, 1389: 1328, 1390: 1329, 1391: 1330, 1392: 1331, 1393: 1332, 1394: 1333, 1395: 1334, 1396: 1335, 1397: 1072, 1398: 1070, 1399: 1071, 1400: 1568, 1401: 1569, 1402: 1567, 1403: 2004, 1404: 2005, 1405: 1728, 1406: 1729, 1407: 1545, 1408: 1546, 1409: 919, 1410: 920, 1411: 921, 1412: 922, 1413: 923, 1414: 924, 1415: 925, 1416: 926, 1417: 927, 1418: 1678, 1419: 1679, 1420: 1804, 1421: 955, 1422: 956, 1423: 1811, 1424: 1285, 1425: 1056, 1426: 1057, 1427: 1058, 1428: 1047, 1429: 1048, 1430: 1049, 1431: 1050, 1432: 1051, 1433: 1052, 1434: 1053, 1435: 1054, 1436: 1055, 1437: 910, 1438: 911, 1439: 2059, 1440: 1122, 1441: 1123, 1442: 1124, 1443: 755, 1444: 1862, 1445: 1795, 1446: 1779, 1447: 1995, 1448: 1464, 1449: 1089, 1450: 1090, 1451: 1091, 1452: 1092, 1453: 1093, 1454: 1094, 1455: 1095, 1456: 528, 1457: 529, 1458: 1928, 1459: 800, 1460: 789, 1461: 790, 1462: 791, 1463: 792, 1464: 793, 1465: 794, 1466: 795, 1467: 796, 1468: 797, 1469: 798, 1470: 799, 1471: 1936, 1472: 1935, 1473: 1556, 1474: 1557, 1475: 1263, 1476: 1632, 1477: 1631, 1478: 1134, 1479: 965, 1480: 966, 1481: 967, 1482: 968, 1483: 969, 1484: 2027, 1485: 876, 1486: 877, 1487: 2003, 1488: 2125, 1489: 2152, 1490: 1765, 1491: 1766, 1492: 1754, 1493: 1696, 1494: 1694, 1495: 1695, 1496: 472, 1497: 473, 1498: 474, 1499: 475, 1500: 2159, 1501: 2013, 1502: 1266, 1503: 1878, 1504: 1879, 1505: 1340, 1506: 1341, 1507: 1581, 1508: 1582, 1509: 2067, 1510: 1915, 1511: 1916, 1512: 1677, 1513: 1441, 1514: 1442, 1515: 1380, 1516: 1381, 1517: 1958, 1518: 1947, 1519: 1948, 1520: 1477, 1521: 1478, 1522: 1479, 1523: 1480, 1524: 1481, 1525: 885, 1526: 886, 1527: 1394, 1528: 1672, 1529: 1671, 1530: 1808, 1531: 2054, 1532: 1599, 1533: 1102, 1534: 1103, 1535: 378, 1536: 379, 1537: 380, 1538: 1953, 1539: 1775, 1540: 809, 1541: 1488, 1542: 1489, 1543: 1490, 1544: 1491, 1545: 781, 1546: 782, 1547: 783, 1548: 1426, 1549: 2097, 1550: 1587, 1551: 1588, 1552: 1222, 1553: 381, 1554: 2150, 1555: 2151, 1556: 1483, 1557: 530, 1558: 1536, 1559: 1534, 1560: 1535, 1561: 519, 1562: 520, 1563: 521, 1564: 522, 1565: 523, 1566: 524, 1567: 1508, 1568: 1509, 1569: 1510, 1570: 1511, 1571: 1512, 1572: 1513, 1573: 1514, 1574: 1515, 1575: 1516, 1576: 1517, 1577: 1518, 1578: 1519, 1579: 1520, 1580: 1521, 1581: 1832, 1582: 1833, 1583: 1834, 1584: 2114, 1585: 5, 1586: 6, 1587: 2039, 1588: 1797, 1589: 1798, 1590: 2041, 1591: 1942, 1592: 2147, 1593: 1045, 1594: 1046, 1595: 1232, 1596: 1229, 1597: 1230, 1598: 1231, 1599: 1803, 1600: 1288, 1601: 1287, 1602: 1470, 1603: 1456, 1604: 1757, 1605: 1415, 1606: 1840, 1607: 2132, 1608: 2056, 1609: 1419, 1610: 2062, 1611: 1810, 1612: 320, 1613: 321, 1614: 322, 1615: 2068, 1616: 1996, 1617: 1800, 1618: 1688, 1619: 1686, 1620: 1687, 1621: 1240, 1622: 1239, 1623: 896, 1624: 897, 1625: 2076, 1626: 2077, 1627: 2047, 1628: 1294, 1629: 1842, 1630: 1843, 1631: 1857, 1632: 2136, 1633: 1824, 1634: 1822, 1635: 1823, 1636: 1396, 1637: 1778, 1638: 1139, 1639: 1140, 1640: 776, 1641: 777, 1642: 778, 1643: 779, 1644: 780, 1645: 1105, 1646: 1299, 1647: 1300, 1648: 948, 1649: 384, 1650: 382, 1651: 383, 1652: 2083, 1653: 495, 1654: 1275, 1655: 1276, 1656: 2143, 1657: 1264, 1658: 1265, 1659: 1457, 1660: 1458, 1661: 1459, 1662: 1460, 1663: 1461, 1664: 2078, 1665: 1662, 1666: 1436, 1667: 808, 1668: 1736, 1669: 1917, 1670: 1946, 1671: 1892, 1672: 1893, 1673: 1408, 1674: 1409, 1675: 1410, 1676: 1292, 1677: 1293, 1678: 1522, 1679: 1983, 1680: 2133, 1681: 1676, 1682: 1528, 1683: 1427, 1684: 1428, 1685: 818, 1686: 1781, 1687: 1794, 1688: 1379, 1689: 2043, 1690: 1809, 1691: 2149, 1692: 2160, 1693: 2110, 1694: 482, 1695: 483, 1696: 2019, 1697: 999, 1698: 1873, 1699: 2042, 1700: 2021, 1701: 484, 1702: 485, 1703: 486, 1704: 1922, 1705: 1923, 1706: 1604, 1707: 1605, 1708: 1606, 1709: 2051, 1710: 1007, 1711: 1675, 1712: 1079, 1713: 2134, 1714: 1991, 1715: 929, 1716: 930, 1717: 931, 1718: 1989, 1719: 1312, 1720: 1313, 1721: 1406, 1722: 1407, 1723: 1752, 1724: 1751, 1725: 2012, 1726: 1930, 1727: 1931, 1728: 2135, 1729: 1208, 1730: 1689, 1731: 1060, 1732: 1816, 1733: 2086, 1734: 2007, 1735: 1950, 1736: 1951, 1737: 2098, 1738: 2033, 1739: 1845, 1740: 1493, 1741: 2080, 1742: 2126, 1743: 2008, 1744: 1945, 1745: 2072, 1746: 1000, 1747: 1001, 1748: 2099, 1749: 678, 1750: 679, 1751: 949, 1752: 950, 1753: 1714, 1754: 1715, 1755: 229, 1756: 230, 1757: 1549, 1758: 1550, 1759: 1758, 1760: 2084, 1761: 2002, 1762: 1852, 1763: 1853, 1764: 1434, 1765: 1370, 1766: 1968, 1767: 1969, 1768: 1959, 1769: 1014, 1770: 1015, 1771: 1590, 1772: 1591, 1773: 1430, 1774: 1551, 1775: 879, 1776: 290, 1777: 291, 1778: 292, 1779: 293, 1780: 294, 1781: 1649, 1782: 2046, 1783: 1463, 1784: 1738, 1785: 1962, 1786: 1690, 1787: 2105, 1788: 1144, 1789: 1145, 1790: 1146, 1791: 1147, 1792: 2154, 1793: 1769, 1794: 1770, 1795: 1746, 1796: 1747, 1797: 1748, 1798: 2044, 1799: 2053, 1800: 1577, 1801: 954, 1802: 1096, 1803: 1097, 1804: 1098, 1805: 1099, 1806: 1471, 1807: 1127, 1808: 1890, 1809: 1323, 1810: 1579, 1811: 1580, 1812: 2113, 1813: 828, 1814: 817, 1815: 1544, 1816: 487, 1817: 1784, 1818: 1782, 1819: 1783, 1820: 1872, 1821: 1088, 1822: 1664, 1823: 2074, 1824: 2075, 1825: 1465, 1826: 1466, 1827: 1467, 1828: 1425, 1829: 2087, 1830: 2101, 1831: 1814, 1832: 1971, 1833: 1452, 1834: 1453, 1835: 1454, 1836: 1455, 1837: 1271, 1838: 493, 1839: 494, 1840: 2030, 1841: 2131, 1842: 957, 1843: 958, 1844: 2158, 1845: 1952, 1846: 1537, 1847: 1602, 1848: 998, 1849: 2124, 1850: 2017, 1851: 1272, 1852: 2024, 1853: 2022, 1854: 2023, 1855: 2036, 1856: 1859, 1857: 1875, 1858: 2130, 1859: 1907, 1860: 1622, 1861: 2102, 1862: 1830, 1863: 2145, 1864: 1787, 1865: 1732, 1866: 2057, 1867: 1881, 1868: 1882, 1869: 1130, 1870: 1131, 1871: 2037, 1872: 2038, 1873: 1865, 1874: 1866, 1875: 658, 1876: 1309, 1877: 1506, 1878: 1507, 1879: 1584, 1880: 1585, 1881: 1583, 1882: 1864, 1883: 1863, 1884: 1734, 1885: 1735, 1886: 2081, 1887: 1016, 1888: 1017, 1889: 1018, 1890: 1019, 1891: 1413, 1892: 1414, 1893: 1851, 1894: 1807, 1895: 970, 1896: 971, 1897: 1799, 1898: 1937, 1899: 1723, 1900: 1724, 1901: 2092, 1902: 2104, 1903: 2049, 1904: 1504, 1905: 1505, 1906: 1527, 1907: 231, 1908: 2031, 1909: 478, 1910: 1645, 1911: 1646, 1912: 1647, 1913: 1850, 1914: 1981, 1915: 1552, 1916: 1553, 1917: 1629, 1918: 1630, 1919: 706, 1920: 707, 1921: 708, 1922: 1938, 1923: 1961, 1924: 1979, 1925: 1640, 1926: 1378, 1927: 2117, 1928: 1788, 1929: 1652, 1930: 1653, 1931: 1482, 1932: 2069, 1933: 1087, 1934: 1181, 1935: 1660, 1936: 2028, 1937: 1861, 1938: 1416, 1939: 1705, 1940: 2123, 1941: 1920, 1942: 2055, 1943: 1307, 1944: 1703, 1945: 1960, 1946: 269, 1947: 1742, 1948: 1529, 1949: 534, 1950: 535, 1951: 2034, 1952: 1886, 1953: 1984, 1954: 2, 1955: 3, 1956: 4, 1957: 1867, 1958: 1868, 1959: 754, 1960: 1571, 1961: 1608, 1962: 1159, 1963: 1634, 1964: 875, 1965: 270, 1966: 271, 1967: 1142, 1968: 1670, 1969: 1397, 1970: 819, 1971: 820, 1972: 1462, 1973: 1927, 1974: 1956, 1975: 1883, 1976: 1918, 1977: 657, 1978: 1411, 1979: 1412, 1980: 2065, 1981: 1448, 1982: 1449, 1983: 1310, 1984: 2006, 1985: 1595, 1986: 1802, 1987: 525, 1988: 526, 1989: 527, 1990: 2088, 1991: 531, 1992: 1805, 1993: 1059, 1994: 1420, 1995: 1421, 1996: 1422, 1997: 1643, 1998: 1182, 1999: 1183, 2000: 1277, 2001: 1125, 2002: 1337, 2003: 1132, 2004: 1133, 2005: 1614, 2006: 1615, 2007: 1226, 2008: 1227, 2009: 1228, 2010: 2052, 2011: 2066, 2012: 1680, 2013: 1257, 2014: 1813, 2015: 2091, 2016: 2137, 2017: 2045, 2018: 1949, 2019: 245, 2020: 2009, 2021: 2011, 2022: 2141, 2023: 2048, 2024: 1716, 2025: 1717, 2026: 1718, 2027: 1233, 2028: 2115, 2029: 1964, 2030: 2127, 2031: 2015, 2032: 488, 2033: 489, 2034: 1841, 2035: 1844, 2036: 1891, 2037: 2018, 2038: 1289, 2039: 1697, 2040: 1921, 2041: 673, 2042: 1730, 2043: 1526, 2044: 1880, 2045: 1885, 2046: 2100, 2047: 1674, 2048: 1812, 2049: 1100, 2050: 1933, 2051: 1500, 2052: 1776, 2053: 2020, 2054: 2140, 2055: 323, 2056: 1826, 2057: 1768, 2058: 1318, 2059: 1540, 2060: 1789, 2061: 1790, 2062: 680, 2063: 1944, 2064: 1943, 2065: 1283, 2066: 1484, 2067: 1985, 2068: 1771, 2069: 1749, 2070: 1650, 2071: 1135, 2072: 953, 2073: 1796, 2074: 1831, 2075: 1398, 2076: 1815, 2077: 1308, 2078: 1904, 2079: 1547, 2080: 1548, 2081: 1894, 2082: 1578, 2083: 1854, 2084: 1727, 2085: 1101, 2086: 2014, 2087: 1538, 2088: 1539, 2089: 1970, 2090: 1073, 2091: 1963, 2092: 1429, 2093: 2016, 2094: 997, 2095: 1731, 2096: 2060, 2097: 2153, 2098: 1756, 2099: 2040, 2100: 1286, 2101: 2122, 2102: 1806, 2103: 1472, 2104: 2144, 2105: 1786, 2106: 2148, 2107: 1767, 2108: 1290, 2109: 1473, 2110: 2139, 2111: 2109, 2112: 476, 2113: 1399, 2114: 1903, 2115: 1990, 2116: 1692, 2117: 1129, 2118: 2061, 2119: 1905, 2120: 1369, 2121: 788, 2122: 1298, 2123: 1024, 2124: 1594, 2125: 1750, 2126: 174, 2127: 1691, 2128: 1648, 2129: 1143, 2130: 1889, 2131: 1801, 2132: 1681, 2133: 554, 2134: 2079, 2135: 490, 2136: 1570, 2137: 2106, 2138: 2095, 2139: 2050, 2140: 173, 2141: 1651, 2142: 1982, 2143: 1443, 2144: 1244, 2145: 1663, 2146: 2096, 2147: 2090, 2148: 1665, 2149: 1311, 2150: 2035, 2151: 1633, 2152: 1846, 2153: 1492, 2154: 802, 2155: 2142, 2156: 272, 2157: 753, 2158: 2082, 2159: 1966, 2160: 238}
# res = 70033241

# n,wells_dict_new,pipes_new,well_status_new,pipe_status_new = translate_back(n,wells,pipes,well_status,pipe_status,wells_mapping)

# pack2 = [wells_dict_new,pipes_new,well_status_new,pipe_status_new]


In [97]:
sorted(pack1[2])

[1,
 3,
 4,
 9,
 13,
 19,
 22,
 25,
 30,
 43,
 45,
 52,
 54,
 59,
 62,
 63,
 68,
 69,
 71,
 74,
 77,
 82,
 84,
 85,
 86,
 87,
 88,
 91,
 92,
 94,
 98,
 101,
 102,
 105,
 108,
 114,
 118,
 122,
 124,
 132,
 133,
 134,
 137,
 138,
 139,
 142,
 150,
 152,
 153,
 154,
 155,
 157,
 158,
 161,
 170,
 174,
 176,
 177,
 181,
 184,
 185,
 188,
 190,
 192,
 193,
 194,
 196,
 204,
 207,
 212,
 214,
 216,
 221,
 222,
 223,
 224,
 225,
 226,
 228,
 231,
 233,
 237,
 244,
 247,
 248,
 249,
 250,
 251,
 253,
 254,
 256,
 257,
 260,
 264,
 268,
 269,
 271,
 272,
 273,
 275,
 276,
 278,
 280,
 281,
 282,
 283,
 284,
 286,
 287,
 290,
 291,
 292,
 294,
 295,
 299,
 300,
 302,
 303,
 304,
 308,
 312,
 313,
 316,
 317,
 318,
 320,
 324,
 325,
 328,
 329,
 331,
 333,
 334,
 335,
 337,
 338,
 339,
 340,
 342,
 344,
 345,
 346,
 348,
 351,
 352,
 358,
 361,
 363,
 364,
 367,
 368,
 371,
 373,
 376,
 377,
 378,
 380,
 384,
 386,
 388,
 390,
 392,
 393,
 395,
 396,
 397,
 400,
 405,
 407,
 408,
 409,
 411,
 41

In [98]:
sorted(pack2[2])

[1,
 5,
 9,
 12,
 15,
 17,
 18,
 19,
 20,
 23,
 24,
 29,
 33,
 35,
 38,
 40,
 42,
 44,
 47,
 49,
 50,
 51,
 52,
 57,
 65,
 70,
 73,
 75,
 76,
 79,
 83,
 86,
 87,
 88,
 89,
 92,
 95,
 97,
 102,
 104,
 106,
 107,
 108,
 110,
 113,
 122,
 128,
 129,
 133,
 136,
 138,
 142,
 144,
 150,
 151,
 154,
 155,
 157,
 159,
 160,
 162,
 164,
 166,
 168,
 170,
 171,
 173,
 179,
 180,
 185,
 187,
 191,
 193,
 196,
 199,
 201,
 204,
 211,
 213,
 216,
 221,
 222,
 224,
 228,
 230,
 235,
 236,
 237,
 240,
 247,
 249,
 250,
 255,
 256,
 257,
 258,
 261,
 264,
 273,
 274,
 275,
 277,
 279,
 285,
 289,
 292,
 293,
 298,
 301,
 302,
 306,
 307,
 309,
 310,
 317,
 324,
 325,
 326,
 327,
 330,
 331,
 333,
 334,
 335,
 336,
 338,
 339,
 341,
 353,
 356,
 357,
 359,
 360,
 364,
 371,
 372,
 374,
 375,
 381,
 384,
 385,
 386,
 387,
 388,
 389,
 395,
 397,
 399,
 401,
 406,
 407,
 409,
 410,
 412,
 414,
 419,
 421,
 423,
 424,
 426,
 427,
 428,
 434,
 435,
 438,
 441,
 443,
 444,
 445,
 447,
 450,
 451,
 452,
 45

In [99]:
len(set(pack1[2])-set(pack2[2]))

545

In [100]:
len(set(pack2[2])-set(pack1[2]))

544

In [101]:
len(set(pack0[2])-set(pack2[2]))

483

In [102]:
len(set(pack2[2])-set(pack0[2]))

482

In [33]:
sorted(pack0[2])

[1,
 2,
 5,
 7,
 11,
 15,
 16,
 20,
 21,
 22,
 24,
 27,
 28,
 31,
 33,
 35,
 38,
 42,
 44,
 45,
 48,
 49,
 50,
 55,
 56,
 58,
 60,
 62,
 65,
 67,
 68,
 75,
 77,
 80,
 88,
 89,
 90,
 93,
 98,
 101,
 102,
 103,
 104,
 105,
 106,
 110,
 119,
 120,
 124,
 128,
 131,
 146,
 148,
 149,
 150,
 151,
 153,
 154,
 156,
 162,
 166,
 172,
 178,
 181,
 184,
 191,
 193,
 194,
 195,
 196,
 203,
 204,
 208,
 211,
 213,
 214,
 217,
 219,
 227,
 228,
 231,
 234,
 239,
 243,
 245,
 248,
 251,
 252,
 261,
 268,
 277,
 278,
 280,
 283,
 285,
 286,
 306,
 309,
 313,
 315,
 316,
 317,
 318,
 319,
 322,
 325,
 326,
 337,
 342,
 343,
 344,
 347,
 354,
 356,
 357,
 362,
 363,
 364,
 365,
 368,
 371,
 380,
 381,
 382,
 384,
 386,
 388,
 390,
 391,
 396,
 397,
 399,
 407,
 408,
 410,
 412,
 413,
 414,
 415,
 416,
 417,
 420,
 426,
 429,
 430,
 433,
 438,
 444,
 448,
 452,
 453,
 459,
 462,
 466,
 470,
 474,
 477,
 479,
 481,
 488,
 489,
 492,
 494,
 497,
 498,
 503,
 505,
 509,
 512,
 513,
 514,
 517,
 519,
 523,

In [34]:
sorted(pack2[2])

[1,
 2,
 3,
 6,
 8,
 14,
 15,
 20,
 40,
 43,
 47,
 49,
 50,
 51,
 52,
 53,
 57,
 59,
 60,
 71,
 76,
 77,
 78,
 81,
 88,
 90,
 91,
 96,
 97,
 98,
 99,
 101,
 106,
 114,
 115,
 116,
 118,
 120,
 122,
 124,
 125,
 130,
 131,
 133,
 141,
 142,
 144,
 146,
 147,
 148,
 149,
 150,
 151,
 154,
 162,
 165,
 167,
 172,
 173,
 174,
 178,
 180,
 183,
 185,
 187,
 189,
 191,
 193,
 194,
 195,
 204,
 205,
 212,
 215,
 217,
 221,
 222,
 224,
 227,
 228,
 230,
 231,
 232,
 234,
 235,
 238,
 240,
 244,
 245,
 246,
 252,
 255,
 260,
 262,
 264,
 266,
 269,
 271,
 272,
 274,
 276,
 278,
 279,
 281,
 283,
 290,
 294,
 295,
 297,
 299,
 300,
 301,
 302,
 304,
 306,
 311,
 313,
 319,
 321,
 322,
 323,
 327,
 331,
 332,
 339,
 341,
 344,
 351,
 354,
 356,
 357,
 360,
 361,
 363,
 366,
 367,
 368,
 371,
 372,
 374,
 376,
 378,
 379,
 381,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 393,
 403,
 404,
 408,
 412,
 415,
 430,
 432,
 433,
 434,
 435,
 436,
 437,
 442,
 443,
 446,
 448,
 451,
 453,
 455,
 456

In [13]:
import json
def dump_json(var,filename='tmp.json'):
    with open(filename, "w") as outfile:
        json.dump(var, outfile)
       
       
def read_json(filename):
    with open(filename,'r') as f:
        data = json.load(f)
        return data


## Archive

In [ ]:
# Node to Json
if type(wells)==list:
    wells_new = [int(round(x/1000,0)) for x in wells]
elif type(wells)==dict:
    wells_new ={k:int(round(v/1000,0)) for k,v in wells.items()}
pipes_new = [(x[0],x[1], int(round(x[2]/1000,0))) for x in pipes]



# Init
node_dict = {}
link_dict = {}

if type(wells_new)==list:
    wells_new = {i+1:cost for i,cost in enumerate(wells_new)}


for i,cost in wells_new.items():
    node_dict[i] = {}
    node_dict[i]['id'] = str(i)
    node_dict[i]['info'] = str(i)
    node_dict[i]['label'] = str(cost)
    node_dict[i]['fillColor'] = '#999999'
    node_dict[i]['radius'] = 10
    node_dict[i]['opacity'] = 0.7

for i,j,cost in pipes_new:
    key = tuple(sorted((i,j)))
    link_dict[key] = {}
    link_dict[key]['from'] = str(i)
    link_dict[key]['to'] = str(j)
    link_dict[key]['label'] = str(cost)
    link_dict[key]['fillColor'] = '#999999'
    
    
for i in well_status:
    node_dict[i]['fillColor'] = '#ff6500'
    
for key in pipe_status:
    link_dict[key]['fillColor'] = '#ff6500'

    
for i in res:
    if i in well_status:
        node_dict[i]['fillColor'] = '#800080'
    
    else:
        node_dict[i]['fillColor'] = '#0000ff'
    
    
    
    
    
    
nodes = list(node_dict.values())
links = list(link_dict.values())


output_json = {'version': '1',
     'nodes': nodes,
     'links': links,
     'title': 'test'
    }
dump_json(output_json,'test.json')
